In [1]:
import pandas as pd 
import numpy as np
import sklearn
import copy
import seaborn as sn
import os
import warnings
import datetime as dt 
import statsmodels.api as sm
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import chi2_contingency
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", 500)

In [2]:
cd "D:\Python\Scorecard_Python"

D:\Python\Scorecard_Python


In [3]:
#data=pd.read_csv("sample_scrub.csv")
data=pd.read_excel("sample_scrub.xlsx")

data.columns=data.columns.str.lower()
data['val']=data.groupby(by='customerid').cumcount()+1
data['tgt_key']=data['customerid'].astype('str')+"_"+data['val'].astype('str')

#data['balance_dt']=pd.to_datetime(data['balance_dt'])
#data[['closed_dt']].head()

In [4]:
data[['balance_dt','open_dt','closed_dt']].head(20)

,balance_dt,open_dt,closed_dt
0,2016-09-30,2016-03-11 00:00:00,2016-09-01 00:00:00
1,2016-09-30,2016-04-04 00:00:00,2016-09-05 00:00:00
2,2019-03-31,2016-03-11 00:00:00,2019-03-31 00:00:00
3,2019-03-31,2016-03-11 00:00:00,2019-03-31 00:00:00
4,2020-03-31,2020-02-20 00:00:00,NaN
5,2019-09-30,2017-06-30 00:00:00,NaN
6,2019-09-30,2017-08-29 00:00:00,2019-12-24 00:00:00
7,2020-03-31,2019-02-04 00:00:00,NaN
8,2016-10-31,2007-11-08 00:00:00,2011-02-12 00:00:00
9,2020-03-31,2019-09-30 00:00:00,NaN


In [5]:
data['balance_dt']=pd.to_datetime(data['balance_dt'],errors='coerce')
#data['balance_dt']=pd.to_datetime(data['balance_dt'],errors='coerce')
data['balance_dt1']=data['balance_dt']+pd.offsets.MonthEnd(0)
#data['open_dt']=pd.to_datetime(data['open_dt'],format='%d-%m-%Y',errors='coerce')
data['open_dt']=pd.to_datetime(data['open_dt'],errors='coerce') #,format='%Y-%m-%d'
#data['closed_dt']=pd.to_datetime(data['closed_dt'],format='%d-%m-%Y',errors='coerce')
data['closed_dt']=pd.to_datetime(data['closed_dt'],errors='coerce')
data=data.drop(['unnamed: 0','actual_payment_am', 'balance_am', 'credit_limit_am', 'days_past_due', 'past_due_am'],axis=1)


In [6]:
data

,customerid,account_nb,m_sub_id,acct_type_cd,open_dt,asset_class_cd,balance_dt,charge_off_am,closed_dt,dflt_status_dt,...,days_past_due_42,days_past_due_43,days_past_due_44,days_past_due_45,days_past_due_46,days_past_due_47,days_past_due_48,val,tgt_key,balance_dt1
0,1000002,XXXX,PUB,121,2016-03-11,S,2016-09-30,NaN,2016-09-01,NaT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1000002_1,2016-09-30
1,1000002,XXXX,PUB,121,2016-04-04,S,2016-09-30,NaN,2016-09-05,NaT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1000002_2,2016-09-30
2,1000002,XXXX,NBF,223,2016-03-11,NaN,2019-03-31,NaN,2019-03-31,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1000002_3,2019-03-31
3,1000002,XXXX,NBF,223,2016-03-11,NaN,2019-03-31,NaN,2019-03-31,NaT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1000002_4,2019-03-31
4,1000002,XXXX,NBF,222,2020-02-20,NaN,2020-03-31,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1000002_5,2020-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43823,1010206,XXXX,PVT,189,2019-11-02,NaN,2020-05-31,NaN,2020-03-23,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1010206_1,2020-05-31
43824,1010206,XXXX,PVT,189,2019-11-02,NaN,2020-05-31,NaN,2020-03-26,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1010206_2,2020-05-31
43825,1010206,XXXX,PUB,227,2016-06-14,S,2020-05-30,0.0,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1010206_3,2020-05-31
43826,1010206,XXXX,NBF,173,2019-01-28,S,2020-04-30,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1010206_4,2020-04-30


In [7]:
df_transpose=pd.wide_to_long(data,['actual_payment_am', 'balance_am', 'credit_limit_am', 
                                   'days_past_due', 'past_due_am','payment_rating_cd'],
                            i=['customerid','tgt_key','balance_dt1'],
                            j='step',
                            sep='_')
df_transpose=df_transpose.reset_index()
df_transpose['mon']=df_transpose['step']-1
df_transpose=df_transpose.drop('step',axis=1)

In [8]:
df_transpose

,customerid,tgt_key,balance_dt1,acct_type_cd,dflt_status_dt,account_nb,val,orig_loan_am,portfolio_rating_type_cd,suit_filed_willful_dflt,...,balance_dt,written_off_and_settled_status,charge_off_am,actual_payment_am,balance_am,credit_limit_am,days_past_due,past_due_am,payment_rating_cd,mon
0,1000002,1000002_1,2016-09-30,121,NaT,XXXX,1,125000.0,I,NaN,...,2016-09-30,NaN,NaN,NaN,0.0,125000.0,NaN,NaN,S,0
1,1000002,1000002_1,2016-09-30,121,NaT,XXXX,1,125000.0,I,NaN,...,2016-09-30,NaN,NaN,NaN,130559.0,125000.0,NaN,5559.0,S,1
2,1000002,1000002_1,2016-09-30,121,NaT,XXXX,1,125000.0,I,NaN,...,2016-09-30,NaN,NaN,NaN,129525.0,125000.0,NaN,4525.0,S,2
3,1000002,1000002_1,2016-09-30,121,NaT,XXXX,1,125000.0,I,NaN,...,2016-09-30,NaN,NaN,NaN,128499.0,125000.0,NaN,3499.0,S,3
4,1000002,1000002_1,2016-09-30,121,NaT,XXXX,1,125000.0,I,NaN,...,2016-09-30,NaN,NaN,NaN,127514.0,125000.0,NaN,2514.0,S,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103739,1010206,1010206_5,2020-05-31,47,NaT,XXXX,5,500000.0,I,NaN,...,2020-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43
2103740,1010206,1010206_5,2020-05-31,47,NaT,XXXX,5,500000.0,I,NaN,...,2020-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44
2103741,1010206,1010206_5,2020-05-31,47,NaT,XXXX,5,500000.0,I,NaN,...,2020-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45
2103742,1010206,1010206_5,2020-05-31,47,NaT,XXXX,5,500000.0,I,NaN,...,2020-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46


In [9]:
import datetime
from dateutil.relativedelta import *

df_transpose['monthly_dt']=df_transpose['balance_dt1'].values.astype('datetime64[M]')-df_transpose['mon'].values.astype('timedelta64[M]')
df_transpose['monthly_dt']=pd.to_datetime(df_transpose['monthly_dt'])+MonthEnd(0)

In [10]:
df_transpose=df_transpose.drop(['account_nb','payment_history_grid','asset_class_cd','portfolio_rating_type_cd','val'],axis=1)
#df_transpose.head()

In [11]:
#-No. Of unsecured account taken in last 12months, Pvt bnk
#-No. Of all closed accounts excluding cc
#-Max DPD on HL LAP in last 24 months
#-Current utilisation on cc
#-Avg utilisation on all products
#11#-Max consecutive delinquency in last 12 months
#-Current overdue (this is not balance)on unsecure products
#-Max current dpd all live account
#-No of products zero+ in last 3 month's

In [12]:
df_transpose.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2103744 entries, 0 to 2103743
Data columns (total 24 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   customerid                      int64         
 1   tgt_key                         object        
 2   balance_dt1                     datetime64[ns]
 3   acct_type_cd                    int64         
 4   dflt_status_dt                  datetime64[ns]
 5   orig_loan_am                    float64       
 6   suit_filed_willful_dflt         float64       
 7   last_payment_dt                 object        
 8   write_off_status_dt             datetime64[ns]
 9   m_sub_id                        object        
 10  open_dt                         datetime64[ns]
 11  responsibility_cd               float64       
 12  closed_dt                       datetime64[ns]
 13  balance_dt                      datetime64[ns]
 14  written_off_and_settled_status  float64       
 15

In [13]:
# #-No. Of unsecured account taken in last 12months, Pvt bnk
# df_transpose['var1']=np.where(((df_transpose['acct_type_cd']== 123)|(df_transpose['acct_type_cd']== 5)
#                                |(df_transpose['acct_type_cd']== 214)|
#                                (df_transpose['acct_type_cd']== 189)|(df_transpose['acct_type_cd']== 130)|
#                                (df_transpose['acct_type_cd']== 213))  &
#                               (df_transpose['m_sub_id']=="PVT") & (df_transpose['mon'] < 12)
#                               ,1,0)
 
# df_transpose['var11']=df_transpose.groupby(['tgt_key'])['var1'].transform(max)
# df_transpose['var111']=df_transpose.groupby(['customerid'])['var11'].transform(sum)
# df_transpose['var1111']=df_transpose['var111']/48

# df_transpose['no_uns_pvt_l12m']=df_transpose['var1111']



In [14]:
# df_transpose['no_uns_pvt_l12m'].value_counts()

In [15]:
# ##-No. Of all closed accounts excluding cc
# #213,224,220,5,214
# df_transpose['var2']=np.where(((df_transpose['acct_type_cd'] != 213) | (df_transpose['acct_type_cd'] != 224)
#                                | (df_transpose['acct_type_cd'] != 220) |
#                                (df_transpose['acct_type_cd'] != 5) | (df_transpose['acct_type_cd'] != 214)
#                                ) & df_transpose['closed_dt'].notnull(),1,0)
# df_transpose['var22']=df_transpose.groupby(['tgt_key'])['var2'].transform(max)
# df_transpose['var222']=df_transpose.groupby(['customerid'])['var22'].transform(sum)
# df_transpose['var2222']=df_transpose['var222']/48

# df_transpose['no_closed_accts_exc_cc']=df_transpose['var2222']


In [16]:
# #Max DPD on HL LAP in last 24 months

# df_transpose['var3']=np.where(((df_transpose['acct_type_cd'] == 195) | (df_transpose['acct_type_cd'] == 58) )
#                                  & (df_transpose['mon'] < 24),df_transpose['days_past_due'],0)
# df_transpose['var33']=df_transpose.groupby(['tgt_key'])['var3'].transform(max)
# df_transpose['var333']=df_transpose.groupby(['customerid'])['var33'].transform(max)

# df_transpose['max_dpd_hllap_l24m']=df_transpose['var333']

In [17]:
# ##Current utilisation on cc

# df_transpose['var4']=np.where(((df_transpose['acct_type_cd'] == 213) | (df_transpose['acct_type_cd'] == 224)
#                                | (df_transpose['acct_type_cd'] == 220) |
#                                (df_transpose['acct_type_cd'] == 5) | (df_transpose['acct_type_cd'] == 214)
#                                ) & (df_transpose['closed_dt'].isnull()) & (df_transpose['mon']==0),
#                               df_transpose['balance_am']/df_transpose[['orig_loan_am','balance_am']].max(axis=1),0)

# df_transpose['var44']=df_transpose.groupby(['tgt_key'])['var4'].transform(max)
# df_transpose['var444']=df_transpose.groupby(['customerid'])['var44'].transform(max)

# df_transpose['curr_util_cc']=df_transpose['var444']

In [18]:
# #-Max current dpd all live account


# df_transpose['var5']=np.where(df_transpose['closed_dt'].isnull() & df_transpose['mon']==0,df_transpose['days_past_due'],-1)
# df_transpose['var55']=df_transpose.groupby(['tgt_key'])['var5'].transform(max)

# df_transpose['max_curr_dpd_live_all']=df_transpose['var55']


In [19]:
# #-No of products zero+ in last 3 month's

# df_transpose['var6']=np.where((df_transpose['days_past_due'] > 0) & (df_transpose['mon'] < 3),1,0)
# df_transpose['var66']=df_transpose.groupby(['tgt_key'])['var6'].transform(max)
# df_transpose['var666']=df_transpose.groupby(['customerid'])['var66'].transform(sum)
# df_transpose['var6666']=df_transpose['var666']/48

# df_transpose['no_prod_0p_l3m']=df_transpose['var6666']


In [20]:
# #-Avg utilisation on all products

# df_transpose['var7']=np.where(df_transpose['closed_dt'].isnull() 
#                               & df_transpose['mon']==0 ,
#                               df_transpose['balance_am']/df_transpose[['orig_loan_am','balance_am']].max(axis=1),0)
# df_transpose['var77']=df_transpose.groupby(['tgt_key'])['var7'].transform(max)
# df_transpose['var777']=df_transpose.groupby(['customerid'])['var77'].transform('mean')

# df_transpose['avg_util_all_prods']=df_transpose['var777']

In [21]:
# #-Current overdue (this is not balance)on unsecure products

# df_transpose['var8']=np.where(df_transpose['mon']==0 &((df_transpose['acct_type_cd']== 123)|(df_transpose['acct_type_cd']== 5)
#                                |(df_transpose['acct_type_cd']== 214)|
#                                (df_transpose['acct_type_cd']== 189)|(df_transpose['acct_type_cd']== 130)|
#                                (df_transpose['acct_type_cd']== 213))& df_transpose['closed_dt'].isnull() ,df_transpose['past_due_am'],0)
# df_transpose['var88']=df_transpose.groupby(['tgt_key'])['var8'].transform(max)

# df_transpose['curr_overdue_uns_prods']=df_transpose['var88']

In [22]:
# df_transpose=df_transpose.drop(['var1111','var2222','var333','var444','var55','var6666','var777','var88'
#                                ],axis=1)

In [13]:
df_transpose.shape

(2103744, 24)

In [14]:
df_transpose = df_transpose.rename(columns = {"open_dt": "open_date", 
                                       "closed_dt":"closed_date", 
                                       "balance_dt": "balance_date",
                                       "customerid":"cust_id",
                                       "actual_payment_am" : "actual_payment_amt",
                                       "balance_am" : "balance_amt",
                                       "past_due_am":"past_due_amt",
                                       "credit_limit_am" : "credit_limit_amt",
                                       "orig_loan_am" : "orig_loan_amt",
                                       "acct_type_cd":"account_type_cd",
                                       "responsibility_cd":"resp_code",
                                       "m_sub_id" : "sub_id",
                                       "suit_filed_willful_dflt" : "suitfiled_wilfuldefault"})

#df.drop(['month_lag', 'id' ,'rep_month'], axis=1, inplace=True)

In [15]:
df_transpose[(df_transpose['account_type_cd']==189)]['balance_dt1'].value_counts()
#df_transpose['open_date'].value_counts()
#df_transpose

2020-04-30    268416
2019-07-31     46992
2019-06-30     27552
2020-05-31     26880
2020-03-31     25056
2020-02-29     16800
2017-05-31     16656
2018-06-30     11856
2019-02-28     10800
2017-11-30     10464
2019-09-30     10176
2017-03-31      8544
2020-01-31      8208
2019-12-31      7776
2019-01-31      6336
2019-08-31      6048
2019-05-31      5280
2019-11-30      3600
2018-04-30      3552
2019-03-31      3408
2019-10-31      3360
2019-04-30      3072
2018-05-31      2832
2018-12-31      2592
2018-02-28      2496
2017-12-31      2352
2017-06-30      2112
2016-03-31      1968
2018-09-30      1968
2018-01-31      1968
2017-09-30      1728
2018-11-30      1392
2017-10-31      1344
2017-07-31      1344
2018-03-31      1248
2017-08-31      1104
2017-01-31      1104
2017-04-30      1104
2018-10-31      1104
2016-12-31       960
2014-07-31       768
2016-04-30       768
2018-07-31       720
2016-05-31       720
2017-02-28       672
2016-08-31       672
2016-09-30       624
2014-09-30   

In [16]:
pl=df_transpose[(df_transpose['open_date'] <= datetime.datetime(2019,4,30) )
               & (df_transpose['balance_dt1']==datetime.datetime(2020,4,30)) & (df_transpose['account_type_cd']==189)]
pl

,cust_id,tgt_key,balance_dt1,account_type_cd,dflt_status_dt,orig_loan_amt,suitfiled_wilfuldefault,last_payment_dt,write_off_status_dt,sub_id,...,written_off_and_settled_status,charge_off_am,actual_payment_amt,balance_amt,credit_limit_amt,days_past_due,past_due_amt,payment_rating_cd,mon,monthly_dt
1584,1000009,1000009_2,2020-04-30,189,NaT,35250.0,NaN,2018-05-05 00:00:00,NaT,NBF,...,NaN,NaN,NaN,0.0,35250.0,0.0,0.0,0,0,2020-04-30
1585,1000009,1000009_2,2020-04-30,189,NaT,35250.0,NaN,2018-05-05 00:00:00,NaT,NBF,...,NaN,NaN,NaN,0.0,35250.0,0.0,0.0,0,1,2020-03-31
1586,1000009,1000009_2,2020-04-30,189,NaT,35250.0,NaN,2018-05-05 00:00:00,NaT,NBF,...,NaN,NaN,NaN,0.0,35250.0,0.0,0.0,0,2,2020-02-29
1587,1000009,1000009_2,2020-04-30,189,NaT,35250.0,NaN,2018-05-05 00:00:00,NaT,NBF,...,NaN,NaN,NaN,0.0,35250.0,0.0,0.0,0,3,2020-01-31
1588,1000009,1000009_2,2020-04-30,189,NaT,35250.0,NaN,2018-05-05 00:00:00,NaT,NBF,...,NaN,NaN,NaN,0.0,35250.0,0.0,0.0,0,4,2019-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103451,1010201,1010201_3,2020-04-30,189,NaT,30000.0,NaN,2020-03-12 00:00:00,NaT,NBF,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,43,2016-09-30
2103452,1010201,1010201_3,2020-04-30,189,NaT,30000.0,NaN,2020-03-12 00:00:00,NaT,NBF,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,44,2016-08-31
2103453,1010201,1010201_3,2020-04-30,189,NaT,30000.0,NaN,2020-03-12 00:00:00,NaT,NBF,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,45,2016-07-31
2103454,1010201,1010201_3,2020-04-30,189,NaT,30000.0,NaN,2020-03-12 00:00:00,NaT,NBF,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,46,2016-06-30


In [17]:
pl.shape

(147456, 24)

In [18]:
pl_d=pd.DataFrame(pl['cust_id'].unique(),columns=['ids'])

In [19]:
pl_d.shape

(1388, 1)

In [20]:
pl_f=pd.merge(pl_d,df_transpose,how="left",left_on="ids",right_on="cust_id")

In [21]:
pl_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 902064 entries, 0 to 902063
Data columns (total 25 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ids                             902064 non-null  int64         
 1   cust_id                         902064 non-null  int64         
 2   tgt_key                         902064 non-null  object        
 3   balance_dt1                     902064 non-null  datetime64[ns]
 4   account_type_cd                 902064 non-null  int64         
 5   dflt_status_dt                  96 non-null      datetime64[ns]
 6   orig_loan_amt                   888288 non-null  float64       
 7   suitfiled_wilfuldefault         101664 non-null  float64       
 8   last_payment_dt                 756288 non-null  object        
 9   write_off_status_dt             192 non-null     datetime64[ns]
 10  sub_id                          902064 non-null  object 

In [22]:
pl_f=pl_f.drop('ids',axis=1)
pl_f.shape

(902064, 24)

In [23]:
pl_f.shape

(902064, 24)

In [24]:
c=pl_f[pl_f.cust_id.isin([1000009])]

In [25]:
c[['resp_code','tgt_key']]

,resp_code,tgt_key
0,1.0,1000009_1
1,1.0,1000009_1
2,1.0,1000009_1
3,1.0,1000009_1
4,1.0,1000009_1
5,1.0,1000009_1
6,1.0,1000009_1
7,1.0,1000009_1
8,1.0,1000009_1
9,1.0,1000009_1


# Base Creation

In [26]:
bureau_base_0=copy.deepcopy(pl_f)

In [27]:
bureau_base_0.shape

(902064, 24)

In [28]:
bureau_base_0['suitfiledwillfuldefault'] = bureau_base_0['suitfiled_wilfuldefault']
bureau_base_0=bureau_base_0.loc[bureau_base_0['resp_code'].isin([1,2,4])]
bureau_base_0.shape

(890304, 25)

In [29]:
import datetime as dt

bureau_base_0['open_month']=np.where(bureau_base_0['open_date'].dt.day==1,bureau_base_0['open_date'],bureau_base_0['open_date']+pd.tseries.offsets.MonthBegin(-1)) 
bureau_base_0['closed_month']=np.where(bureau_base_0['closed_date'].dt.day==1,bureau_base_0['closed_date'],bureau_base_0['closed_date']+pd.tseries.offsets.MonthBegin(-1)) 
bureau_base_0['balanced_month']=np.where(bureau_base_0['monthly_dt'].dt.day==1,bureau_base_0['monthly_dt'],bureau_base_0['monthly_dt']+pd.tseries.offsets.MonthBegin(-1)) 


In [30]:
conditions = [bureau_base_0.loc[ : ,'suitfiled_wilfuldefault'].isin([1,2,3]) | bureau_base_0.loc[ : ,'suitfiledwillfuldefault'].isin([201,202,203,204,205,206,207,208,209]),
             bureau_base_0.loc[ : , 'written_off_and_settled_status'].isin([2,3,4,5,6,7,8,9,10]),
             (bureau_base_0.loc[ : , 'written_off_and_settled_status'].isin([0,1])) | (bureau_base_0['suitfiledwillfuldefault']==200),
             (bureau_base_0.loc[ : , 'payment_rating_cd'].isin(['D','L','6'])) | (bureau_base_0['days_past_due'] >= 180),
             (bureau_base_0.loc[ : , 'payment_rating_cd'].isin(['5'])) | (bureau_base_0['days_past_due'] >= 150),
             (bureau_base_0.loc[ : , 'payment_rating_cd'].isin(['4'])) | (bureau_base_0['days_past_due'] >= 120),
             (bureau_base_0.loc[ : , 'payment_rating_cd'].isin(['B','3'])) | (bureau_base_0['days_past_due'] >= 90),
             (bureau_base_0.loc[ : , 'payment_rating_cd'].isin(['M','2'])) | (bureau_base_0['days_past_due'] >= 60),
             (bureau_base_0.loc[ : , 'payment_rating_cd'].isin(['1'])) | (bureau_base_0['days_past_due'] >= 30),
             bureau_base_0['days_past_due'] > 0,
             bureau_base_0['past_due_amt'] > 0,
             (np.isnan(bureau_base_0['days_past_due'])==True) | (bureau_base_0['days_past_due'] < 0)]


choices = [180, 180, 180, 180, 150, 120, 90, 60, 30, 29, 29, 0]


In [31]:
bureau_base_0['dpd_new']=np.select(conditions,choices,default=bureau_base_0['days_past_due'])
bureau_base_0['cust_id'] = bureau_base_0['cust_id'].astype(str)
bureau_base_0['account_type_cd'] = bureau_base_0['account_type_cd'].astype(str)
#Credit Card Account Types 
# 


In [32]:
bureau_base_0['dpd_new_2']=np.where((bureau_base_0['account_type_cd'].isin(['5','220','214'])) & (bureau_base_0['balance_amt'] <= 3000),0,bureau_base_0['dpd_new'] )

In [33]:
bureau_base_0=bureau_base_0[(bureau_base_0['open_date'] <= pd.to_datetime('2019-4-30'))&
                            (bureau_base_0['monthly_dt'] <= pd.to_datetime('2020-4-30'))]

In [34]:
bureau_base_0.head()

,cust_id,tgt_key,balance_dt1,account_type_cd,dflt_status_dt,orig_loan_amt,suitfiled_wilfuldefault,last_payment_dt,write_off_status_dt,sub_id,...,past_due_amt,payment_rating_cd,mon,monthly_dt,suitfiledwillfuldefault,open_month,closed_month,balanced_month,dpd_new,dpd_new_2
0,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,S,0,2020-04-30,0.0,2017-09-01,NaT,2020-04-01,0.0,0.0
1,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,S,1,2020-03-31,0.0,2017-09-01,NaT,2020-03-01,0.0,0.0
2,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,S,2,2020-02-29,0.0,2017-09-01,NaT,2020-02-01,0.0,0.0
3,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,S,3,2020-01-31,0.0,2017-09-01,NaT,2020-01-01,0.0,0.0
4,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,S,4,2019-12-31,0.0,2017-09-01,NaT,2019-12-01,0.0,0.0


In [35]:
bureau_base_0.sort_values(by=['cust_id', 'tgt_key', 'balanced_month', 'resp_code'],inplace=True)

In [36]:
bureau_base_0.shape

(659396, 30)

In [37]:
#a=q['tgt_key']
#b=bureau_base_0[bureau_base_0.loc[:,'tgt_key'].isin(['1004879_2','1008179_1'])]
#b.shape

In [38]:
#b.to_csv('date.csv')

In [39]:
bureau_base_1=copy.deepcopy(bureau_base_0)

In [40]:
mod_date = pd.to_datetime('2019-4-30')
last_date = pd.to_datetime('2020-4-30')

In [41]:
bureau_base_1=bureau_base_1.drop_duplicates(['cust_id', 'tgt_key', 'balanced_month'])

In [42]:
bureau_base_1.shape

(659396, 30)

In [43]:
closed_date_1=copy.deepcopy(bureau_base_1)

In [44]:
#closed_date_1['last_6'] = np.where((bureau_base_1['balanced_month'] <= mod_date) & (bureau_base_1['balanced_month'] >= mod_date+pd.offsets.MonthBegin(-6)), 1, 0)
#l6 is l12

bureau_base_1['last_6_no_acct_type_cd']=np.where((bureau_base_1['balanced_month'] <= mod_date) &
                                                 (bureau_base_1['balanced_month'] >= mod_date+pd.offsets.MonthBegin(-6)) & 
                                 (~bureau_base_1.loc[:,'account_type_cd'].isin(['5','220','214'])), 1, 0)
#closed_date_1['l6_bal_amt'] = np.where(bureau_base_1['last_6']==1, bureau_base_1['balance_amt'], 0)
bureau_base_1['l6_bal_amt_no_acct_type_cd']=np.where(bureau_base_1['last_6_no_acct_type_cd']==1 &
                                                     (~bureau_base_1.loc[:,'account_type_cd'].isin(['5','220','214'])) , bureau_base_1['balance_amt'], 0)

In [45]:
bureau_base_1['max_balance_amt_l6m']=bureau_base_1.groupby(['tgt_key'])['l6_bal_amt_no_acct_type_cd'].transform(max)
bureau_base_1['min_balance_amt_l6m']=bureau_base_1.groupby(['tgt_key'])['l6_bal_amt_no_acct_type_cd'].transform(min)

In [46]:
bureau_base_1['closed_flag_1'] = np.where(((bureau_base_1['max_balance_amt_l6m']==0) & (bureau_base_1['min_balance_amt_l6m']==0)), 1, 0)

In [47]:
cond=[bureau_base_1['closed_month'].isnull(), 
bureau_base_1.loc[:,'closed_month'].dt.year.isin(["2001"]),   #????
bureau_base_1['closed_month'] <= mod_date] 
ch=[0,1,1]
bureau_base_1['closed_flag_date']=np.select(cond,ch)

In [48]:
bureau_base_1['closure_status'] = np.where(((bureau_base_1['closed_flag_1']==1) | (bureau_base_1['closed_flag_date']==1)),
                                           1, 0)

In [49]:
acct_bal_month = bureau_base_1[(bureau_base_1['balanced_month'] >= pd.to_datetime('2019-5-1')) & (bureau_base_1['balanced_month'] < pd.to_datetime('2020-5-01'))]

In [50]:
acct_bal_month['last_bal_month'] = acct_bal_month.groupby('tgt_key')['balanced_month'].transform(max)

In [51]:
acct_bal_month = acct_bal_month.sort_values(by=['tgt_key'])

In [52]:
acct_bal_month.head(10)

,cust_id,tgt_key,balance_dt1,account_type_cd,dflt_status_dt,orig_loan_amt,suitfiled_wilfuldefault,last_payment_dt,write_off_status_dt,sub_id,...,dpd_new,dpd_new_2,last_6_no_acct_type_cd,l6_bal_amt_no_acct_type_cd,max_balance_amt_l6m,min_balance_amt_l6m,closed_flag_1,closed_flag_date,closure_status,last_bal_month
11,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01
0,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01
1,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01
2,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01
4,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01
5,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01
3,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01
7,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01
8,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01
9,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,0.0,0,0.0,463731.0,0.0,0,0,0,2020-04-01


In [53]:
temp_df = copy.deepcopy(acct_bal_month)

In [54]:
acct_dpd_month = temp_df.groupby(['tgt_key']).agg({'dpd_new_2' : ['max']})   #?????1-48 instead of 1-12
acct_dpd_month.columns = ['acct_max_dpd_l6m']
acct_dpd_month = acct_dpd_month.reset_index()

In [55]:
temp_df['acct_max_dpd_l'] = np.where((temp_df['balanced_month'] == temp_df['last_bal_month']), temp_df['dpd_new_2'], 0)

In [56]:
temp = temp_df.groupby(['tgt_key']).agg({'acct_max_dpd_l' : ['max']})

In [57]:
temp.columns = ['acct_max_dpd_l']
temp = temp.reset_index()
temp.head()

,tgt_key,acct_max_dpd_l
0,1000009_1,0.0
1,1000009_2,0.0
2,1000009_3,0.0
3,1000027_10,0.0
4,1000027_11,150.0


In [58]:
acct_dpd_month['acct_max_dpd_l'] =temp['acct_max_dpd_l']

In [59]:
acct_dpd_month.shape

(8808, 3)

In [60]:
#acct_dpd_month.loc[acct_dpd_month['acct_max_dpd_l6m'] != acct_dpd_month['acct_max_dpd_l']].shape

In [61]:
acct_dpd_month=acct_dpd_month.sort_values(by=['tgt_key'])
acct_dpd_month.head(10)

,tgt_key,acct_max_dpd_l6m,acct_max_dpd_l
0,1000009_1,0.0,0.0
1,1000009_2,0.0,0.0
2,1000009_3,0.0,0.0
3,1000027_10,0.0,0.0
4,1000027_11,150.0,150.0
5,1000027_12,180.0,30.0
6,1000027_3,0.0,0.0
7,1000027_5,29.0,29.0
8,1000027_7,29.0,0.0
9,1000027_8,30.0,30.0


In [62]:
base_3 = pd.merge(bureau_base_1, acct_dpd_month, on = 'tgt_key',how='left')

In [63]:
base_3['def_90'] = np.where((base_3['acct_max_dpd_l6m'] >= 90), 1, 0)
base_3['def_max_90'] = np.where((base_3['acct_max_dpd_l'] >= 90), 1, 0)

In [64]:
#len(base_3['tgt_key'].unique())
len(base_3['tgt_key'].unique())

13760

In [65]:
base_4=base_3.assign(target_90=np.where(base_3['closure_status']<= 0,base_3['def_90'],np.nan),
                    target_end_90=np.where(base_3['closure_status']<= 0,base_3['def_max_90'],np.nan)
                    ).groupby('cust_id').agg({'target_90' : 'max',
                                             'target_end_90' : 'max'})
print(base_4)
base_4.reset_index(inplace=True)
base_4.sort_values(by='cust_id',inplace=True)

         target_90  target_end_90
cust_id                          
1000009        0.0            0.0
1000027        1.0            1.0
1000028        0.0            0.0
1000033        1.0            1.0
1000037        1.0            1.0
...            ...            ...
1010163        0.0            0.0
1010176        0.0            0.0
1010180        1.0            1.0
1010183        0.0            0.0
1010201        1.0            1.0

[1388 rows x 2 columns]


In [66]:
base_3

,cust_id,tgt_key,balance_dt1,account_type_cd,dflt_status_dt,orig_loan_amt,suitfiled_wilfuldefault,last_payment_dt,write_off_status_dt,sub_id,...,l6_bal_amt_no_acct_type_cd,max_balance_amt_l6m,min_balance_amt_l6m,closed_flag_1,closed_flag_date,closure_status,acct_max_dpd_l6m,acct_max_dpd_l,def_90,def_max_90
0,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,463731.0,0.0,0,0,0,0.0,0.0,0,0
1,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,463731.0,0.0,0,0,0,0.0,0.0,0,0
2,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,463731.0,0.0,0,0,0,0.0,0.0,0,0
3,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,463731.0,0.0,0,0,0,0.0,0.0,0,0
4,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,0.0,463731.0,0.0,0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659391,1010201,1010201_3,2020-04-30,189,NaT,30000.0,NaN,2020-03-12 00:00:00,NaT,NBF,...,0.0,21000.0,0.0,0,0,0,180.0,180.0,1,1
659392,1010201,1010201_3,2020-04-30,189,NaT,30000.0,NaN,2020-03-12 00:00:00,NaT,NBF,...,0.0,21000.0,0.0,0,0,0,180.0,180.0,1,1
659393,1010201,1010201_3,2020-04-30,189,NaT,30000.0,NaN,2020-03-12 00:00:00,NaT,NBF,...,0.0,21000.0,0.0,0,0,0,180.0,180.0,1,1
659394,1010201,1010201_3,2020-04-30,189,NaT,30000.0,NaN,2020-03-12 00:00:00,NaT,NBF,...,0.0,21000.0,0.0,0,0,0,180.0,180.0,1,1


# Factor creation

In [67]:
def nan_sum(series):
    if (series.isnull().all() == True):
        return np.nan
    else:
        return np.sum(series)

In [68]:
factor_0 = base_3[(base_3['monthly_dt'] <= mod_date) & (base_3['monthly_dt']>= mod_date + pd.offsets.MonthEnd(-36))]  #???36 or 48

In [69]:
cond1=[factor_0.loc[:,'account_type_cd'].isin(['176','177','178','179','197','198','199','200','175','172','214','224']),
 factor_0.loc[:,'account_type_cd'].isin(['999','47','58','195','185','123','189','191','130','187','5','219','121','173','181','184' ,'196'])]
cho1=['com','cons']
#com =commercial
#cons = consumer

factor_0['con_com']=np.select(cond1,cho1,default='ot')




cond2=[factor_0.loc[:,'account_type_cd'].isin(['176','177','178','179']) ,
     factor_0.loc[:,'account_type_cd'].isin(['999','47','58','195','185','123','189','191','130','187','5','219','121','173','181','184','196'])]

cho2=['bus_1','cons']

factor_0['agri']=np.select(cond2,cho2,default='bus_2')



cond3=[factor_0.loc[:,'account_type_cd'].isin(['123','5','214','189','169','130','213']) ,
 factor_0.loc[:,'account_type_cd'].isin(['47','58','195','185','191','219','173','184','172','220','221','222','223','224','170'])]
cho3=['uns','sec']
#uns = Unsecured Loans
#sec = Secured Loans


factor_0['sec_unsec']=np.select(cond3,cho3,default='oth')
factor_0['ref_date']=pd.to_datetime("2019-4-30")

In [70]:
# t=last_date-mod_date
# round(t.days/30,0)

In [71]:
# mob=mod_date-factor_0['open_date']
# l=list()
# for i in mob:
#     k=round(i.days/30,0)
#     l.append(k) 

In [72]:
# len(l)

In [73]:
factor_0['mob']=((factor_0['ref_date']-factor_0['open_date'])/np.timedelta64(1,'M'))
factor_0['mob']=factor_0['mob'].astype('int')

In [74]:
factor_0['max_bal_amt'] = factor_0.groupby('tgt_key')['balance_amt'].transform(max)
factor_0['loan_amt'] = factor_0.groupby('tgt_key')['orig_loan_amt'].transform(max)
factor_0['credit_limit'] = factor_0.groupby('tgt_key')['credit_limit_amt'].transform(max)

In [75]:
pd.set_option("display.max_rows",400)
f0=factor_0[['tgt_key','loan_amt','orig_loan_amt','balance_amt']].drop_duplicates(['tgt_key'])   #???Need for drop
f0

,tgt_key,loan_amt,orig_loan_amt,balance_amt
0,1000009_1,525000.0,525000.0,NaN
48,1000009_2,35250.0,35250.0,NaN
104,1000009_3,9500.0,9500.0,NaN
165,1000027_1,30500.0,30500.0,NaN
192,1000027_10,1590.0,1590.0,NaN
...,...,...,...,...
659160,1010183_8,1849.0,1849.0,NaN
659204,1010183_9,16800.0,16800.0,NaN
659252,1010201_1,67782.0,67782.0,NaN
659300,1010201_2,18500.0,18500.0,NaN


In [76]:
f0.to_csv('bal.csv')

In [77]:
''''''
InquiryMapping3=pd.read_table(r"InquiryMapping3.txt", sep='|', encoding='cp1252')
InquiryMapping3=InquiryMapping3.rename(columns={'BUREAU_MBR_ID':'m_sub_id',
                                                'BM_NM':'bank_name',
                                                'LEGACY_CUST_NB':'subscriber_id',
                                                'Category':'bank_type'})
InquiryMapping3['m_sub_id']=InquiryMapping3['m_sub_id'].astype('str')


In [78]:
factor_1=pd.merge(factor_0,InquiryMapping3,left_on='sub_id',right_on='m_sub_id',how="left")

#factor_1=factor_0   #########

In [79]:
factor_1.shape

(401334, 53)

In [80]:
factor_2=copy.deepcopy(factor_1)

In [81]:
factor_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401334 entries, 0 to 401333
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   cust_id                         401334 non-null  object        
 1   tgt_key                         401334 non-null  object        
 2   balance_dt1                     401334 non-null  datetime64[ns]
 3   account_type_cd                 401334 non-null  object        
 4   dflt_status_dt                  0 non-null       datetime64[ns]
 5   orig_loan_amt                   393320 non-null  float64       
 6   suitfiled_wilfuldefault         45289 non-null   float64       
 7   last_payment_dt                 346947 non-null  object        
 8   write_off_status_dt             0 non-null       datetime64[ns]
 9   sub_id                          401334 non-null  object        
 10  open_date                       401334 non-null  datetim

In [82]:
factor_2['count']=factor_2.groupby('tgt_key').cumcount()+1
factor_2['max_diff']=factor_2['count']-1
factor_2['bal_amt']=factor_2['balance_amt']
factor_2['modified_limit']=factor_2[['max_bal_amt','loan_amt']].max(axis=1)
#????
factor_2.loc[(factor_2.account_type_cd.isin(['5','220','214'])),'modified_limit']=factor_2[['max_bal_amt','credit_limit']].max(axis=1)

In [83]:
factor_2['credit_limit'].value_counts()

5000.0      7002
50000.0     6501
100000.0    5806
20000.0     4619
0.0         4533
            ... 
26901.0        1
40081.0        1
73300.0        1
164801.0       1
5057.0         1
Name: credit_limit, Length: 3381, dtype: int64

In [84]:
#factor_2[['tgt_key','modified_limit']].head(50)

In [85]:
# fac=factor_2[factor_2.modified_limit==0]
# fac.groupby('tgt_key').agg({'modified_limit':[min]})

In [86]:
# factor_2['ref_date']=pd.to_datetime("2019-10-31")
#app_diff = MOB till ref_date
factor_2['app_diff']=((factor_2['ref_date']-factor_2['monthly_dt'])/np.timedelta64(1,'M'))
factor_2['app_diff']=factor_2['app_diff'].astype('int')

In [87]:
factor_3=copy.deepcopy(factor_2)

In [88]:
factor_3['mnth_since_last_reported']=factor_2.groupby(['cust_id','tgt_key'])['app_diff'].transform(min)
factor_3['mnth_earliest_reported']=factor_2.groupby(['cust_id','tgt_key'])['app_diff'].transform(max)
factor_3['closed_with_0_bal_flag_2']=factor_2.groupby(['cust_id','tgt_key'])['closure_status'].transform(max)
factor_3['written_off_amt']=0
factor_3.loc[factor_3.suitfiledwillfuldefault.isin([1,2,3,201,202,203,204,205,206,207,208,209]),'written_off_amt']=factor_3.bal_amt

In [89]:
factor_3.head(50)

,cust_id,tgt_key,balance_dt1,account_type_cd,dflt_status_dt,orig_loan_amt,suitfiled_wilfuldefault,last_payment_dt,write_off_status_dt,sub_id,...,bank_type,count,max_diff,bal_amt,modified_limit,app_diff,mnth_since_last_reported,mnth_earliest_reported,closed_with_0_bal_flag_2,written_off_amt
0,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,1,0,NaN,525000.0,34,0,34,0,0.0
1,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,2,1,NaN,525000.0,33,0,34,0,0.0
2,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,3,2,NaN,525000.0,32,0,34,0,0.0
3,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,4,3,NaN,525000.0,31,0,34,0,0.0
4,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,5,4,NaN,525000.0,30,0,34,0,0.0
5,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,6,5,NaN,525000.0,29,0,34,0,0.0
6,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,7,6,NaN,525000.0,28,0,34,0,0.0
7,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,8,7,NaN,525000.0,27,0,34,0,0.0
8,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,9,8,NaN,525000.0,26,0,34,0,0.0
9,1000009,1000009_1,2020-04-30,47,NaT,525000.0,0.0,2020-03-05 00:00:00,NaT,PUB,...,NaN,10,9,NaN,525000.0,25,0,34,0,0.0


In [90]:
factor_3.mob.value_counts()

0       19676
5       19474
3       17982
6       17299
4       16939
2       16878
1       14865
9       13934
8       13805
10      12913
7       12871
11      11637
14       9721
13       9600
18       9572
12       9426
16       9033
15       8626
17       8005
22       7866
19       7037
20       7032
21       6512
23       6444
25       6221
24       6122
30       5537
26       5129
27       4846
29       4322
28       4314
34       4214
32       3966
31       3621
33       3533
38       2942
41       2699
36       2683
35       2632
37       2466
39       2222
42       2148
40       2050
48       1974
43       1932
46       1871
54       1850
50       1567
44       1560
53       1557
45       1514
52       1508
51       1439
47       1374
49       1178
56        967
59        927
58        904
63        841
60        802
57        775
55        767
62        731
64        610
76        575
61        542
65        542
80        481
66        462
68        371
79        358
67    

In [91]:
factor_31=copy.deepcopy(factor_3)

In [92]:
factor_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401334 entries, 0 to 401333
Data columns (total 62 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   cust_id                         401334 non-null  object        
 1   tgt_key                         401334 non-null  object        
 2   balance_dt1                     401334 non-null  datetime64[ns]
 3   account_type_cd                 401334 non-null  object        
 4   dflt_status_dt                  0 non-null       datetime64[ns]
 5   orig_loan_amt                   393320 non-null  float64       
 6   suitfiled_wilfuldefault         45289 non-null   float64       
 7   last_payment_dt                 346947 non-null  object        
 8   write_off_status_dt             0 non-null       datetime64[ns]
 9   sub_id                          401334 non-null  object        
 10  open_date                       401334 non-null  datetim

In [229]:
# factor_4=factor_3.assign(nbr_mnth_info=factor_3['tgt_key'],
#                          nbr_mnth_info_36m=np.where(factor_3['app_diff'] <= 36 ,1,0),min_mob=factor_3.mob
#                    ,dpd_0m=np.where(factor_3['app_diff']==1,factor_3['dpd_new_2'],np.nan),
#                     max_dpd_3m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3),
#                                         factor_3['dpd_new_2'],0)
#                    ,max_dpd_6m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6),
#                                        factor_3['dpd_new_2'],0),
#                    max_dpd_12m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12),
#                                        factor_3['dpd_new_2'],0)
#                    ,max_dpd_24m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24),
#                                        factor_3['dpd_new_2'],0),
#                    max_dpd_36m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=36),
#                                       factor_3['dpd_new_2'],0),
#                     max_dpd_life=np.where((factor_3['app_diff']>=1),factor_3['dpd_new_2'],0)
#                   ,nbr_0_3m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3) &
#                                        (factor_3['dpd_new_2'] > 0),1,0),
#                    nbr_0_3m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3)
#                                        ,1,0),
#                    nbr_0_6m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6) &
#                                        (factor_3['dpd_new_2'] > 0),1,0),
#                    nbr_0_6m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6)
#                                        ,1,0)
#                    ,nbr_0_12m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12) &
#                                        (factor_3['dpd_new_2'] > 0),1,0),
#                    nbr_0_12m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12)
#                                        ,1,0),
#                    nbr_0_24m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24) &
#                                        (factor_3['dpd_new_2'] > 0),1,0),
#                    nbr_0_24m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24)
#                                        ,1,0)

                   
#                    ,nbr_30_3m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3) &
#                                        (factor_3['dpd_new_2'] > 30),1,0),
#                    nbr_30_3m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3)
#                                        ,1,0),
#                    nbr_30_6m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6) &
#                                        (factor_3['dpd_new_2'] > 30),1,0),
#                    nbr_30_6m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6)
#                                        ,1,0),
#                    nbr_30_12m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12) &
#                                        (factor_3['dpd_new_2'] > 30),1,0),
#                    nbr_30_12m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12)
#                                        ,1,0),
#                    nbr_30_24m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24) &
#                                        (factor_3['dpd_new_2'] > 30),1,0),
#                    nbr_30_24m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24)
#                                        ,1,0),
                   
                   
                   
#                    nbr_90_3m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3) &
#                                        (factor_3['dpd_new_2'] > 90),1,0),
#                    nbr_90_3m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3)
#                                        ,1,0),
#                    nbr_90_6m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6) &
#                                        (factor_3['dpd_new_2'] > 90),1,0),
#                    nbr_90_6m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6)
#                                        ,1,0),
#                    nbr_90_12m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12) &
#                                        (factor_3['dpd_new_2'] > 90),1,0),
#                    nbr_90_12m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12)
#                                        ,1,0),
#                    nbr_90_24m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24) &
#                                        (factor_3['dpd_new_2'] > 90),1,0),
#                    nbr_90_24m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24)
#                                        ,1,0)
                   
                   
#                    ,month_since_0=np.where(factor_3['dpd_new_2'] > 0,factor_3['app_diff'],np.nan),
#                    month_since_30=np.where(factor_3['dpd_new_2'] >= 30,factor_3['app_diff'],np.nan),
#                    month_since_60=np.where(factor_3['dpd_new_2'] >= 60,factor_3['app_diff'],np.nan),
#                    month_since_90=np.where(factor_3['dpd_new_2'] >= 90,factor_3['app_diff'],np.nan),
#                    month_since_180=np.where(factor_3['dpd_new_2'] >= 180,factor_3['app_diff'],np.nan),
                   
                   
#         a_write_off_month=np.where((factor_3['written_off_and_settled_status'].isin([2,6,8])) | (factor_3['suitfiledwillfuldefault'].isin([204,205])),factor_3['monthly_dt'],pd.NaT),
                   
#         a_settlement_month=np.where((factor_3['written_off_and_settled_status'].isin([3,4,9])) |
#                                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([208,209])),
#                                               factor_3['monthly_dt'],pd.NaT),
                   
#         a_restructure_month=np.where((factor_3['written_off_and_settled_status'].isin([0,1,10])) |
#                                                (factor_3['suitfiledwillfuldefault'].isin([200])),
#                                               factor_3['monthly_dt'],pd.NaT),
                   
#         a_suitfiled_month=np.where((factor_3['suitfiledwillfuldefault'].isin([201,203,1,3])),
#                                               factor_3['monthly_dt'],pd.NaT),
        
#         a_wilful_default_month=np.where((factor_3['suitfiledwillfuldefault'].isin([201,203,2,3])),
#                                               factor_3['monthly_dt'],pd.NaT),
                   
#         a_suit_wilful_def_month=np.where((factor_3['suitfiledwillfuldefault'].isin([203,3])),
#                                               factor_3['monthly_dt'],pd.NaT),
                   
# a_write_off_month_12m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
#                                (factor_3['written_off_and_settled_status'].isin([2,6,8])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([204,205]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
                   
# a_settlement_month_12m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
#                                (factor_3.loc[:,'written_off_and_settled_status'].isin([3,4,9])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([208,209]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_restructure_month_12m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
#                                (factor_3.loc[:,'written_off_and_settled_status'].isin([0,1,10])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([200]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_suitfiled_month_12m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([1,3])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([201,203]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_wilful_default_month_12m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([2,3])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([202,203]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_suit_wilful_def_month_12m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([3])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([203]))),
#                               factor_3['monthly_dt'],pd.NaT),


# ##last 24 months
# a_write_off_month_24m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
#                                (factor_3.loc[:,'written_off_and_settled_status'].isin([2,6,8])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([204,205]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
                   
# a_settlement_month_24m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
#                                (factor_3.loc[:,'written_off_and_settled_status'].isin([3,4,9])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([208,209]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_restructure_month_24m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
#                                (factor_3.loc[:,'written_off_and_settled_status'].isin([0,1,10])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([200]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_suitfiled_month_24m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([1,3])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([201,203]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_wilful_default_month_24m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([2,3])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([202,203]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_suit_wilful_def_month_24m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([3])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([203]))),
#                               factor_3['monthly_dt'],pd.NaT),


# ##last 36 months
                   
# a_write_off_month_36m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
#                                (factor_3.loc[:,'written_off_and_settled_status'].isin([2,6,8])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([204,205]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
                   
# a_settlement_month_36m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
#                                (factor_3.loc[:,'written_off_and_settled_status'].isin([3,4,9])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([208,209]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_restructure_month_36m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
#                                (factor_3.loc[:,'written_off_and_settled_status'].isin([0,1,10])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([200]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_suitfiled_month_36m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([1,3])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([201,203]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_wilful_default_month_36m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([2,3])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([202,203]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
# a_suit_wilful_def_month_36m=np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([3])))|
#                               ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
#                                (factor_3.loc[:,'suitfiledwillfuldefault'].isin([203]))),
#                               factor_3['monthly_dt'],pd.NaT),
                   
                   
        
# ##active credit card lines
            
# max_bal_act=np.where(factor_3['mnth_since_last_reported'] <= 12,factor_3['bal_amt'],np.nan),
# min_bal_act=np.where(factor_3['mnth_since_last_reported'] <= 12,factor_3['bal_amt'],np.nan),
                   
                   
# ##utilization
            
# curr_bal_active=np.where((factor_3['mnth_since_last_reported'] <= 12) & (factor_3['app_diff']==1)
#                          ,factor_3['bal_amt'],0),
# bal_l3m_active=np.where((factor_3['mnth_since_last_reported'] <= 12) & (factor_3['app_diff']>=1) & 
#                         (factor_3['app_diff']<=3),factor_3['bal_amt'],0),
# bal_l6m_active=np.where((factor_3['mnth_since_last_reported'] <= 12) & (factor_3['app_diff']>=1) & 
#                         (factor_3['app_diff']<=6),factor_3['bal_amt'],0),
# bal_l12m_active=np.where((factor_3['mnth_since_last_reported'] <= 12) & (factor_3['app_diff']>=1) & 
#                         (factor_3['app_diff']<=12),factor_3['bal_amt'],0),
                   
                   
                   
# curr_bal_live=np.where((factor_3['closed_with_0_bal_flag_2'] <= 0) & (factor_3['app_diff']==1)
#                          ,factor_3['bal_amt'],0),
# bal_l3m_live=np.where((factor_3['closed_with_0_bal_flag_2'] <= 0) & (factor_3['app_diff']>=1) & 
#                         (factor_3['app_diff']<=3),factor_3['bal_amt'],0),
# bal_l6m_live=np.where((factor_3['closed_with_0_bal_flag_2'] <= 0) & (factor_3['app_diff']>=1) & 
#                         (factor_3['app_diff']<=6),factor_3['bal_amt'],0),
# bal_l12m_live=np.where((factor_3['closed_with_0_bal_flag_2'] <= 0) & (factor_3['app_diff']>=1) & 
#                         (factor_3['app_diff']<=12),factor_3['bal_amt'],0),

                   
                   
# curr_bal_total=np.where(factor_3['app_diff']==1,factor_3['bal_amt'],0),
# bal_l3m_Total=np.where((factor_3['app_diff']>=1) & 
#                         (factor_3['app_diff']<=3),factor_3['bal_amt'],0),
# bal_l6m_Total=np.where((factor_3['app_diff']>=1) & 
#                         (factor_3['app_diff']<=6),factor_3['bal_amt'],0),

# bal_l12m_Total=np.where((factor_3['app_diff']>=1) & 
#                         (factor_3['app_diff']<=12),factor_3['bal_amt'],0),
                   
                   

# sec_uns_product=np.where(factor_3.loc[:,'account_type_cd'].isin(['123','5','214','189','169','130','213']),1,
#                         np.where(factor_3.loc[:,'account_type_cd'].isin(['47','58','195','185','191','219','173','184','172','220','221','222','223','224','170']),
#                                  2,4)),
                   
                   
#  con_com_product=np.where(factor_3.loc[:,'account_type_cd'].isin(['176','177','178','179','197','198','199','200','175','172','214','224']),1,
#                          np.where(factor_3.loc[:,'account_type_cd'].isin(['999','47','58','195','185','123','189','191','130','187','5',
#                                                                           '219','121','173','181','184','196']),2,3)),
                                      
#  bus_product=np.where(factor_3.loc[:,'account_type_cd'].isin(['176','177','178','179']),1,
#                          np.where(factor_3.loc[:,'account_type_cd'].isin(['999','47','58','195','185','123','189','191','130','187','5','219','121','173','181','184' ,'196']),
#                                  3,2))

# ).groupby(['cust_id','tgt_key','open_date','sub_id','account_type_cd','closed_with_0_bal_flag_2','mnth_since_last_reported','modified_limit','bank_type'])

In [93]:
#import numpy as np
# 
factor_4=factor_3.assign(nbr_mnth_info=factor_3['tgt_key'],
                         nbr_mnth_info_36m=np.where(factor_3['app_diff'] <= 36 ,1,0),min_mob=factor_3.mob
                   ,dpd_0m=np.where(factor_3['app_diff']==1,factor_3['dpd_new_2'],np.nan),
                    max_dpd_3m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3),
                                        factor_3['dpd_new_2'],0)
                   ,max_dpd_6m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6),
                                       factor_3['dpd_new_2'],0),
                   max_dpd_12m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12),
                                       factor_3['dpd_new_2'],0)
                   ,max_dpd_24m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24),
                                       factor_3['dpd_new_2'],0),
                   max_dpd_36m=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=36),
                                      factor_3['dpd_new_2'],0),
                    max_dpd_life=np.where((factor_3['app_diff']>=1),factor_3['dpd_new_2'],0)
                  ,nbr_0_3m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3) &
                                       (factor_3['dpd_new_2'] > 0),1,0),
                   nbr_0_3m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3)
                                       ,1,0),
                   nbr_0_6m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6) &
                                       (factor_3['dpd_new_2'] > 0),1,0),
                   nbr_0_6m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6)
                                       ,1,0)
                   ,nbr_0_12m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12) &
                                       (factor_3['dpd_new_2'] > 0),1,0),
                   nbr_0_12m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12)
                                       ,1,0),
                   nbr_0_24m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24) &
                                       (factor_3['dpd_new_2'] > 0),1,0),
                   nbr_0_24m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24)
                                       ,1,0)

                   
                   ,nbr_30_3m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3) &
                                       (factor_3['dpd_new_2'] > 30),1,0),
                   nbr_30_3m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3)
                                       ,1,0),
                   nbr_30_6m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6) &
                                       (factor_3['dpd_new_2'] > 30),1,0),
                   nbr_30_6m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6)
                                       ,1,0),
                   nbr_30_12m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12) &
                                       (factor_3['dpd_new_2'] > 30),1,0),
                   nbr_30_12m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12)
                                       ,1,0),
                   nbr_30_24m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24) &
                                       (factor_3['dpd_new_2'] > 30),1,0),
                   nbr_30_24m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24)
                                       ,1,0),
                   
                   
                   
                   nbr_90_3m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3) &
                                       (factor_3['dpd_new_2'] > 90),1,0),
                   nbr_90_3m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=3)
                                       ,1,0),
                   nbr_90_6m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6) &
                                       (factor_3['dpd_new_2'] > 90),1,0),
                   nbr_90_6m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=6)
                                       ,1,0),
                   nbr_90_12m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12) &
                                       (factor_3['dpd_new_2'] > 90),1,0),
                   nbr_90_12m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=12)
                                       ,1,0),
                   nbr_90_24m_num=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24) &
                                       (factor_3['dpd_new_2'] > 90),1,0),
                   nbr_90_24m_den=np.where((factor_3['app_diff'] >=1) & (factor_3['app_diff'] <=24)
                                       ,1,0)
                   
                   
                   ,month_since_0=np.where(factor_3['dpd_new_2'] > 0,factor_3['app_diff'],np.nan),
                   month_since_30=np.where(factor_3['dpd_new_2'] >= 30,factor_3['app_diff'],np.nan),
                   month_since_60=np.where(factor_3['dpd_new_2'] >= 60,factor_3['app_diff'],np.nan),
                   month_since_90=np.where(factor_3['dpd_new_2'] >= 90,factor_3['app_diff'],np.nan),
                   month_since_180=np.where(factor_3['dpd_new_2'] >= 180,factor_3['app_diff'],np.nan),
                   
                   
        a_write_off_month=pd.to_datetime(np.where((factor_3['written_off_and_settled_status'].isin([2,6,8])) | (factor_3['suitfiledwillfuldefault'].isin([204,205])),factor_3['monthly_dt'],pd.NaT)),
                   
        a_settlement_month=pd.to_datetime(np.where((factor_3['written_off_and_settled_status'].isin([3,4,9])) |
                                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([208,209])),
                                              factor_3['monthly_dt'],pd.NaT)),
                   
        a_restructure_month=pd.to_datetime(np.where((factor_3['written_off_and_settled_status'].isin([0,1,10])) |
                                               (factor_3['suitfiledwillfuldefault'].isin([200])),
                                              factor_3['monthly_dt'],pd.NaT)),
                   
        a_suitfiled_month=pd.to_datetime(np.where((factor_3['suitfiledwillfuldefault'].isin([201,203,1,3])),
                                              factor_3['monthly_dt'],pd.NaT)),
        
        a_wilful_default_month=pd.to_datetime(np.where((factor_3['suitfiledwillfuldefault'].isin([201,203,2,3])),
                                              factor_3['monthly_dt'],pd.NaT)),
                   
        a_suit_wilful_def_month=pd.to_datetime(np.where((factor_3['suitfiledwillfuldefault'].isin([203,3])),
                                              factor_3['monthly_dt'],pd.NaT)),
                   
a_write_off_month_12m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
                               (factor_3['written_off_and_settled_status'].isin([2,6,8])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([204,205]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
                   
a_settlement_month_12m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
                               (factor_3.loc[:,'written_off_and_settled_status'].isin([3,4,9])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([208,209]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_restructure_month_12m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
                               (factor_3.loc[:,'written_off_and_settled_status'].isin([0,1,10])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([200]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_suitfiled_month_12m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([1,3])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([201,203]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_wilful_default_month_12m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([2,3])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([202,203]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_suit_wilful_def_month_12m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) & 
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([3])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 12) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([203]))),
                              factor_3['monthly_dt'],pd.NaT)),


##last 24 months
a_write_off_month_24m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
                               (factor_3.loc[:,'written_off_and_settled_status'].isin([2,6,8])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([204,205]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
                   
a_settlement_month_24m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
                               (factor_3.loc[:,'written_off_and_settled_status'].isin([3,4,9])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([208,209]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_restructure_month_24m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
                               (factor_3.loc[:,'written_off_and_settled_status'].isin([0,1,10])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([200]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_suitfiled_month_24m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([1,3])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([201,203]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_wilful_default_month_24m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([2,3])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([202,203]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_suit_wilful_def_month_24m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) & 
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([3])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 24) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([203]))),
                              factor_3['monthly_dt'],pd.NaT)),


##last 36 months
                   
a_write_off_month_36m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
                               (factor_3.loc[:,'written_off_and_settled_status'].isin([2,6,8])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([204,205]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
                   
a_settlement_month_36m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
                               (factor_3.loc[:,'written_off_and_settled_status'].isin([3,4,9])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([208,209]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_restructure_month_36m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
                               (factor_3.loc[:,'written_off_and_settled_status'].isin([0,1,10])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([200]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_suitfiled_month_36m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([1,3])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([201,203]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_wilful_default_month_36m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([2,3])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([202,203]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
a_suit_wilful_def_month_36m=pd.to_datetime(np.where(((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) & 
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([3])))|
                              ((factor_3['app_diff']>=1) & (factor_3['app_diff'] <= 36) &
                               (factor_3.loc[:,'suitfiledwillfuldefault'].isin([203]))),
                              factor_3['monthly_dt'],pd.NaT)),
                   
                   
        
##active credit lines
            
max_bal_act=np.where(factor_3['mnth_since_last_reported'] <= 12,factor_3['bal_amt'],np.nan),
min_bal_act=np.where(factor_3['mnth_since_last_reported'] <= 12,factor_3['bal_amt'],np.nan),
                   
                   
##utilization
            
curr_bal_active=np.where((factor_3['mnth_since_last_reported'] <= 12) & (factor_3['app_diff']==1)
                         ,factor_3['bal_amt'],0),
bal_l3m_active=np.where((factor_3['mnth_since_last_reported'] <= 12) & (factor_3['app_diff']>=1) & 
                        (factor_3['app_diff']<=3),factor_3['bal_amt'],0),
bal_l6m_active=np.where((factor_3['mnth_since_last_reported'] <= 12) & (factor_3['app_diff']>=1) & 
                        (factor_3['app_diff']<=6),factor_3['bal_amt'],0),
bal_l12m_active=np.where((factor_3['mnth_since_last_reported'] <= 12) & (factor_3['app_diff']>=1) & 
                        (factor_3['app_diff']<=12),factor_3['bal_amt'],0),
                   
                   
                   
curr_bal_live=np.where((factor_3['closed_with_0_bal_flag_2'] <= 0) & (factor_3['app_diff']==1)
                         ,factor_3['bal_amt'],0),
bal_l3m_live=np.where((factor_3['closed_with_0_bal_flag_2'] <= 0) & (factor_3['app_diff']>=1) & 
                        (factor_3['app_diff']<=3),factor_3['bal_amt'],0),
bal_l6m_live=np.where((factor_3['closed_with_0_bal_flag_2'] <= 0) & (factor_3['app_diff']>=1) & 
                        (factor_3['app_diff']<=6),factor_3['bal_amt'],0),
bal_l12m_live=np.where((factor_3['closed_with_0_bal_flag_2'] <= 0) & (factor_3['app_diff']>=1) & 
                        (factor_3['app_diff']<=12),factor_3['bal_amt'],0),

                   
                   
curr_bal_total=np.where(factor_3['app_diff']==1,factor_3['bal_amt'],0),
bal_l3m_total=np.where((factor_3['app_diff']>=1) & 
                        (factor_3['app_diff']<=3),factor_3['bal_amt'],0),
bal_l6m_total=np.where((factor_3['app_diff']>=1) & 
                        (factor_3['app_diff']<=6),factor_3['bal_amt'],0),

bal_l12m_total=np.where((factor_3['app_diff']>=1) & 
                        (factor_3['app_diff']<=12),factor_3['bal_amt'],0),
                   
                   

sec_uns_product=np.where(factor_3.loc[:,'account_type_cd'].isin(['123','5','214','189','169','130','213']),1,
                        np.where(factor_3.loc[:,'account_type_cd'].isin(['47','58','195','185','191','219','173','184','172','220','221','222','223','224','170']),
                                 2,4)),
                   
                   
 con_com_product=np.where(factor_3.loc[:,'account_type_cd'].isin(['176','177','178','179','197','198','199','200','175','172','214','224']),1,
                         np.where(factor_3.loc[:,'account_type_cd'].isin(['999','47','58','195','185','123','189','191','130','187','5',
                                                                          '219','121','173','181','184','196']),2,3)),
                                      
 bus_product=np.where(factor_3.loc[:,'account_type_cd'].isin(['176','177','178','179']),1,
                         np.where(factor_3.loc[:,'account_type_cd'].isin(['999','47','58','195','185','123','189','191','130','187','5','219','121','173','181','184' ,'196']),
                                 3,2)),
    a_max_written_off_amt=np.where(factor_3.written_off_amt >= 0,factor_3.written_off_amt,0),
    util_l12m_active_den=np.where(factor_3.mnth_since_last_reported <= 12,factor_3.modified_limit,0)
                         
).groupby(['cust_id','tgt_key','open_date','sub_id','account_type_cd','closed_with_0_bal_flag_2','mnth_since_last_reported','modified_limit']).agg({'nbr_mnth_info':'count',
                                 'nbr_mnth_info_36m':'sum'
                                  ,'min_mob':'min','dpd_0m':'max'
                                  ,'max_dpd_3m':'max','max_dpd_6m':'max','max_dpd_12m':'max'
                                  ,'max_dpd_24m':'max','max_dpd_life':'max'
                                  ,'nbr_0_3m_num':'sum','nbr_0_3m_den':'sum'
                                  ,'nbr_0_6m_num':'sum','nbr_0_6m_den':'sum'
                                  ,'nbr_0_12m_num':'sum','nbr_0_12m_den':'sum',
                                  'nbr_0_24m_num':'sum','nbr_0_24m_den':'sum'
                                  ,'nbr_30_3m_num':'sum','nbr_30_3m_den':'sum',
                                  'nbr_30_6m_num':'sum','nbr_30_6m_den':'sum',
                                  'nbr_30_12m_num':'sum','nbr_30_12m_den':'sum',
                                  'nbr_30_24m_num':'sum','nbr_30_24m_den':'sum',
                                  'nbr_90_3m_num':'sum','nbr_90_3m_den':'sum',
                                  'nbr_90_6m_num':'sum','nbr_90_6m_den':'sum',
                                  'nbr_90_12m_num':'sum','nbr_90_12m_den':'sum',
                                  'nbr_90_24m_num':'sum','nbr_90_24m_den':'sum'
                                  ,'month_since_0':'min','month_since_30':'min',
                                 'month_since_60':'min','month_since_90':'min',
                                 'month_since_180':'min','a_write_off_month':'min',
                                 'a_settlement_month':'min','a_restructure_month':'min',
                                 'a_suitfiled_month':'min','a_wilful_default_month':'min',
                                 'a_suit_wilful_def_month':'min','a_write_off_month_12m':'min',
                                 'a_settlement_month_12m':'min','a_restructure_month_12m':'min',
                                 'a_suitfiled_month_12m':'min','a_wilful_default_month_12m':'min',
                                 'a_suit_wilful_def_month_12m':'min','a_write_off_month_24m':'min',
                                 'a_settlement_month_24m':'min','a_restructure_month_24m':'min',
                                 'a_suitfiled_month_24m':'min','a_wilful_default_month_24m':'min',
                                 'a_suit_wilful_def_month_24m':'min','a_write_off_month_36m':'min',
                                 'a_settlement_month_36m':'min','a_restructure_month_36m':'min',
                                 'a_suitfiled_month_36m':'min','a_wilful_default_month_36m':'min',
                                 'a_suit_wilful_def_month_36m':'min','max_bal_act':'max',
                                 'min_bal_act':'min','curr_bal_active':'sum',
                                 'bal_l3m_active':'sum','bal_l6m_active':'sum','bal_l12m_active':'sum',
                                 'curr_bal_live':'sum',
                                 'bal_l3m_live':'sum','bal_l6m_live':'sum','bal_l12m_live':'sum',
                                 'curr_bal_total':'sum',
                                 'bal_l3m_total':'sum','bal_l6m_total':'sum','bal_l12m_total':'sum',
                                 'sec_uns_product':'max','con_com_product':'max','bus_product':'max'
                                  ,'a_max_written_off_amt':'max','util_l12m_active_den':'sum'})
                               


In [94]:
factor_4=factor_4.reset_index()
factor_4.head()

,cust_id,tgt_key,open_date,sub_id,account_type_cd,closed_with_0_bal_flag_2,mnth_since_last_reported,modified_limit,nbr_mnth_info,nbr_mnth_info_36m,...,bal_l12m_live,curr_bal_total,bal_l3m_total,bal_l6m_total,bal_l12m_total,sec_uns_product,con_com_product,bus_product,a_max_written_off_amt,util_l12m_active_den
0,1000009,1000009_1,2017-09-21,PUB,47,0,0,525000.0,36,36,...,5254676.0,0.0,907509.0,2313376.0,5254676.0,2,2,3,0.0,18900000.0
1,1000009,1000009_2,2017-04-24,NBF,189,1,0,35250.0,36,36,...,0.0,0.0,0.0,0.0,7050.0,1,2,3,0.0,1269000.0
2,1000009,1000009_3,2018-04-08,NBF,189,1,0,9500.0,37,37,...,0.0,0.0,0.0,2850.0,20900.0,1,2,3,0.0,351500.0
3,1000027,1000027_1,2016-07-25,PUB,191,1,9,36739.0,27,27,...,0.0,0.0,0.0,0.0,142444.0,2,2,3,0.0,991953.0
4,1000027,1000027_10,2019-01-05,NBF,189,0,0,1590.0,37,37,...,2981.0,0.0,2981.0,2981.0,2981.0,1,2,3,0.0,58830.0


In [95]:
factor_4['curr_util_total']=np.where(factor_4.modified_limit==0,0,(factor_4.curr_bal_total/factor_4.modified_limit))
factor_4['util_l3m_total']=np.where(factor_4.modified_limit==0,0,(factor_4.bal_l3m_total/factor_4.modified_limit))
factor_4['util_l6m_total']=np.where(factor_4.modified_limit==0,0,(factor_4.bal_l6m_total/factor_4.modified_limit))
factor_4['util_l12m_total']=np.where(factor_4.modified_limit==0,0,(factor_4.bal_l12m_total/factor_4.modified_limit))


factor_4['nbr_0_3m']=factor_4.nbr_0_3m_num/factor_4.nbr_0_3m_den
factor_4['nbr_0_6m']=factor_4.nbr_0_6m_num/factor_4.nbr_0_6m_den
factor_4['nbr_0_12m']=factor_4.nbr_0_12m_num/factor_4.nbr_0_12m_den
factor_4['nbr_0_24m']=factor_4.nbr_0_12m_num/factor_4.nbr_0_12m_den

factor_4['nbr_30_3m']=factor_4.nbr_30_3m_num/factor_4.nbr_30_3m_den
factor_4['nbr_30_6m']=factor_4.nbr_30_6m_num/factor_4.nbr_30_6m_den
factor_4['nbr_30_12m']=factor_4.nbr_30_12m_num/factor_4.nbr_30_12m_den
factor_4['nbr_30_24m']=factor_4.nbr_30_12m_num/factor_4.nbr_30_12m_den

factor_4['nbr_90_3m']=factor_4.nbr_90_3m_num/factor_4.nbr_90_3m_den
factor_4['nbr_90_6m']=factor_4.nbr_90_6m_num/factor_4.nbr_90_6m_den
factor_4['nbr_90_12m']=factor_4.nbr_90_12m_num/factor_4.nbr_90_12m_den
factor_4['nbr_90_24m']=factor_4.nbr_90_12m_num/factor_4.nbr_90_12m_den

factor_4['curr_util_active']=factor_4.curr_bal_active/factor_4.modified_limit
factor_4['util_l3m_active']=factor_4.bal_l3m_active/factor_4.modified_limit
factor_4['util_l6m_active']=factor_4.bal_l6m_active/factor_4.modified_limit
factor_4['util_l12m_active']=factor_4.bal_l6m_active/factor_4.util_l12m_active_den

factor_4['curr_util_live']=factor_4.curr_bal_live/factor_4.modified_limit   
factor_4['util_l3m_live']=factor_4.bal_l3m_live/factor_4.modified_limit
factor_4['util_l6m_live']=factor_4.bal_l6m_live/factor_4.modified_limit
factor_4['util_l12m_live']=factor_4.bal_l12m_live/factor_4.modified_limit




In [96]:
#factor_4[['nbr_0_3m','nbr_90_3m']].head()

In [97]:
temp=factor_2.sort_values(by=['tgt_key','monthly_dt'])
temp=factor_2.drop_duplicates(['tgt_key','monthly_dt'])
temp=temp[['tgt_key','monthly_dt','balance_amt']]

In [98]:
temp2=pd.merge(factor_4,temp,how='left',on='tgt_key')

In [99]:
temp2['a_ful_write_off_life_flag']=np.where((temp2.a_write_off_month == temp2.monthly_dt) & (temp2.a_max_written_off_amt >= temp2.balance_amt),1,0)
temp2['a_ful_write_off_12m_flag']=np.where((temp2.a_write_off_month_12m == temp2.monthly_dt) & (temp2.a_max_written_off_amt >= temp2.balance_amt),1,0)
temp2['a_ful_write_off_24m_flag']=np.where(((temp2.a_write_off_month_24m == temp2.monthly_dt) & (temp2.a_max_written_off_amt >= temp2.balance_amt)),1,0)
temp2['a_ful_write_off_36m_flag']=np.where((temp2.a_write_off_month_36m == temp2.monthly_dt) & (temp2.a_max_written_off_amt >= temp2.balance_amt),1,0)


In [100]:
temp3=temp2.assign(aa_ful_write_off_life_flag=temp2.a_ful_write_off_life_flag,
                  aa_ful_write_off_12m_flag=temp2.a_ful_write_off_12m_flag,
                  aa_ful_write_off_24m_flag=temp2.a_ful_write_off_24m_flag,
                  aa_ful_write_off_36m_flag=temp2.a_ful_write_off_36m_flag,
                  a_writeoff_amt_life=temp2.a_max_written_off_amt).groupby(['tgt_key']).agg({'aa_ful_write_off_life_flag':'max',
                                    'aa_ful_write_off_12m_flag':'max',
                                    'aa_ful_write_off_24m_flag': 'max',
                                    'aa_ful_write_off_36m_flag':'max',
                                     'a_writeoff_amt_life':'max'})                                                            
                                   

In [101]:
temp3=temp3.rename(columns={'aa_ful_write_off_life_flag':'a_ful_write_off_life_flag','aa_ful_write_off_12m_flag':'a_ful_write_off_12m_flag','aa_ful_write_off_24m_flag':'a_ful_write_off_24m_flag','aa_ful_write_off_36m_flag':'a_ful_write_off_36m_flag'})

In [102]:
temp3=temp3.reset_index()

In [103]:
factor_5=pd.merge(factor_4,temp3,how='left',on='tgt_key')

In [104]:
factor_5.head()

,cust_id,tgt_key,open_date,sub_id,account_type_cd,closed_with_0_bal_flag_2,mnth_since_last_reported,modified_limit,nbr_mnth_info,nbr_mnth_info_36m,...,util_l12m_active,curr_util_live,util_l3m_live,util_l6m_live,util_l12m_live,a_ful_write_off_life_flag,a_ful_write_off_12m_flag,a_ful_write_off_24m_flag,a_ful_write_off_36m_flag,a_writeoff_amt_life
0,1000009,1000009_1,2017-09-21,PUB,47,0,0,525000.0,36,36,...,0.122401,0.0,1.728589,4.406430,10.008907,0,0,0,0,0.0
1,1000009,1000009_2,2017-04-24,NBF,189,1,0,35250.0,36,36,...,0.000000,0.0,0.000000,0.000000,0.000000,0,0,0,0,0.0
2,1000009,1000009_3,2018-04-08,NBF,189,1,0,9500.0,37,37,...,0.008108,0.0,0.000000,0.000000,0.000000,0,0,0,0,0.0
3,1000027,1000027_1,2016-07-25,PUB,191,1,9,36739.0,27,27,...,0.000000,0.0,0.000000,0.000000,0.000000,0,0,0,0,0.0
4,1000027,1000027_10,2019-01-05,NBF,189,0,0,1590.0,37,37,...,0.050671,0.0,1.874843,1.874843,1.874843,0,0,0,0,0.0


In [105]:
factor_5.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12457 entries, 0 to 12456
Data columns (total 118 columns):
 #    Column                       Dtype         
---   ------                       -----         
 0    cust_id                      object        
 1    tgt_key                      object        
 2    open_date                    datetime64[ns]
 3    sub_id                       object        
 4    account_type_cd              object        
 5    closed_with_0_bal_flag_2     int64         
 6    mnth_since_last_reported     int64         
 7    modified_limit               float64       
 8    nbr_mnth_info                int64         
 9    nbr_mnth_info_36m            int32         
 10   min_mob                      int32         
 11   dpd_0m                       float64       
 12   max_dpd_3m                   float64       
 13   max_dpd_6m                   float64       
 14   max_dpd_12m                  float64       
 15   max_dpd_24m                  float

In [106]:
f5=copy.deepcopy(factor_5)

In [107]:
# factor_61=f5.assign(nbr_live_accts=np.where(f5.closed_with_0_bal_flag_2 <= 0 ,1,np.nan),
#                   nbr_closed_accts=np.where(f5.closed_with_0_bal_flag_2 > 0 ,1,np.nan),
#                   nbr_tot_accts=f5.tgt_key,
                   
#                   nbr_hl_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
#                   nbr_hl_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
#                   nbr_hl_tot_accts=np.where((f5.account_type_cd.isin(['58','195'])),1,np.nan),
                  
#                   nbr_pl_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['123'])),1,np.nan),
#                   nbr_pl_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['123'])),1,np.nan),
#                   nbr_pl_tot_accts=np.where((f5.account_type_cd.isin(['123'])),1,np.nan),
                   
                   
#                   nbr_al_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
#                   nbr_al_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
#                   nbr_al_tot_accts=np.where((f5.account_type_cd.isin(['47','221'])),1,np.nan),
                   
                  
#                   nbr_cc_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['5','214','220'])),1,np.nan),
#                   nbr_cc_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['5','214','220'])),1,np.nan),
#                   nbr_cc_tot_accts=np.where((f5.account_type_cd.isin(['5','214','220'])),1,np.nan),
                   
                   
#                   nbr_gl_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['191'])),1,np.nan),
#                   nbr_gl_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['191'])),1,np.nan),
#                   nbr_gl_tot_accts=np.where((f5.account_type_cd.isin(['191'])),1,np.nan),
                  
                  
#                   nbr_tw_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['173'])),1,np.nan),
#                   nbr_tw_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['173'])),1,np.nan),
#                   nbr_tw_tot_accts=np.where((f5.account_type_cd.isin(['173'])),1,np.nan),
                   
                   
#                   nbr_cl_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['189'])),1,np.nan),
#                   nbr_cl_closed_accts=np.where((f5.closed_with_0_bal_flag_2 ==1) & (f5.account_type_cd.isin(['189'])),1,np.nan),
#                   nbr_cl_tot_accts=np.where((f5.account_type_cd.isin(['189'])),1,np.nan),
                   
                   
#                   nbr_oth_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product == 3),1,np.nan),
#                   nbr_oth_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.sec_uns_product == 3),1,np.nan),
#                   nbr_oth_tot_accts=np.where((f5.sec_uns_product == 3),1,np.nan),
                   
                   
#                   nbr_uns_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product == 1),1,np.nan),
#                   nbr_uns_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.sec_uns_product == 1),1,np.nan),
#                   nbr_uns_tot_accts=np.where((f5.sec_uns_product == 1),1,np.nan), 
                   
                   
#                   nbr_sec_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product == 2),1,np.nan),
#                   nbr_sec_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.sec_uns_product == 2),1,np.nan),
#                   nbr_sec_tot_accts=np.where((f5.sec_uns_product == 2),1,np.nan), 
                   
                   
#                   nbr_com_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product == 1),1,np.nan),
#                   nbr_com_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.con_com_product == 1),1,np.nan),
#                   nbr_com_tot_accts=np.where((f5.con_com_product == 1),1,np.nan),                        
     
                   
#                   nbr_con_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product == 2),1,np.nan),
#                   nbr_con_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.con_com_product == 2),1,np.nan),
#                   nbr_con_tot_accts=np.where((f5.con_com_product == 2),1,np.nan),                         
                   
                   
#                   nbr_com_agri_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product == 1),1,np.nan),
#                   nbr_com_agri_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.bus_product == 1),1,np.nan),
#                   nbr_com_agri_tot_accts=np.where((f5.bus_product == 1),1,np.nan),                        

                   
#                   nbr_com_oth_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product == 2),1,np.nan),
#                   nbr_com_oth_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.bus_product == 2),1,np.nan),
#                   nbr_com_oth_tot_accts=np.where((f5.bus_product == 2),1,np.nan),                       
                       
                   
#                   ##no bank type var in dataset 412-439
# #                   based on last statement in last 12 months 
#                   nbr_live_active_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.mnth_since_last_reported <= 12),1,np.nan),
#                   nbr_closed_active_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.mnth_since_last_reported <= 12),1,np.nan),
#                   nbr_tot_active_accts=np.where((f5.mnth_since_last_reported <= 12),1,np.nan),                       
                   
#                   nbr_live_inactive_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.mnth_since_last_reported > 12),1,np.nan),
#                   nbr_closed_inactive_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.mnth_since_last_reported > 12),1,np.nan),
#                   nbr_tot_inactive_accts=np.where((f5.mnth_since_last_reported > 12),1,np.nan),                        
                   
                   
#                   nbr_cc_live_active_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['5','214','220'])) & (~f5.max_bal_act.isin([0,np.nan])) & (~f5.min_bal_act.isin([0,np.nan])),1,np.nan), 
                   
# #                 No. of clean relationship
#                   nbr_clean_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.dpd_0m <= 0),1,np.nan),
                   
# #               number of accounts opened
                   
#                    nbr_accts_open_l3m=np.where(f5.min_mob < 3,1,np.nan),
#                    nbr_accts_open_l6m=np.where(f5.min_mob < 6,1,np.nan),
#                    nbr_accts_open_l9m=np.where(f5.min_mob < 9,1,np.nan),
#                    nbr_accts_open_l12m=np.where(f5.min_mob < 12,1,np.nan),
                   
#                    nbr_accts_open_l3m_hl=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
#                    nbr_accts_open_l6m_hl=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
#                    nbr_accts_open_l9m_hl=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
#                    nbr_accts_open_l12m_hl=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_pl=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['123'])),1,np.nan),
#                    nbr_accts_open_l6m_pl=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['123'])),1,np.nan),
#                    nbr_accts_open_l9m_pl=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['123'])),1,np.nan),
#                    nbr_accts_open_l12m_pl=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['123'])),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_cc=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['5','220','214'])),1,np.nan),
#                    nbr_accts_open_l6m_cc=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['5','220','214'])),1,np.nan),
#                    nbr_accts_open_l9m_cc=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['5','220','214'])),1,np.nan),
#                    nbr_accts_open_l12m_cc=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['5','220','214'])),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_al=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
#                    nbr_accts_open_l6m_al=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
#                    nbr_accts_open_l9m_al=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
#                    nbr_accts_open_l12m_al=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_gl=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['191'])),1,np.nan),
#                    nbr_accts_open_l6m_gl=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['191'])),1,np.nan),
#                    nbr_accts_open_l9m_gl=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['191'])),1,np.nan),
#                    nbr_accts_open_l12m_gl=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['191'])),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_tw=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['173'])),1,np.nan),
#                    nbr_accts_open_l6m_tw=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['173'])),1,np.nan),
#                    nbr_accts_open_l9m_tw=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['173'])),1,np.nan),
#                    nbr_accts_open_l12m_tw=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['173'])),1,np.nan),
                   
#                    nbr_accts_open_l3m_oth=np.where((f5.min_mob < 3) & (f5.sec_uns_product==3),1,np.nan),
#                    nbr_accts_open_l6m_oth=np.where((f5.min_mob < 6) & (f5.sec_uns_product==3),1,np.nan),
#                    nbr_accts_open_l9m_oth=np.where((f5.min_mob < 9) & (f5.sec_uns_product==3),1,np.nan),
#                    nbr_accts_open_l12m_oth=np.where((f5.min_mob < 12) & (f5.sec_uns_product==3),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_uns=np.where((f5.min_mob < 3) & (f5.sec_uns_product==1),1,np.nan),
#                    nbr_accts_open_l6m_uns=np.where((f5.min_mob < 6) & (f5.sec_uns_product==1),1,np.nan),
#                    nbr_accts_open_l9m_uns=np.where((f5.min_mob < 9) & (f5.sec_uns_product==1),1,np.nan),
#                    nbr_accts_open_l12m_uns=np.where((f5.min_mob < 12) & (f5.sec_uns_product==1),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_sec=np.where((f5.min_mob < 3) & (f5.sec_uns_product==2),1,np.nan),
#                    nbr_accts_open_l6m_sec=np.where((f5.min_mob < 6) & (f5.sec_uns_product==2),1,np.nan),
#                    nbr_accts_open_l9m_sec=np.where((f5.min_mob < 9) & (f5.sec_uns_product==2),1,np.nan),
#                    nbr_accts_open_l12m_sec=np.where((f5.min_mob < 12) & (f5.sec_uns_product==2),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_com=np.where((f5.min_mob < 3) & (f5.con_com_product==1),1,np.nan),
#                    nbr_accts_open_l6m_com=np.where((f5.min_mob < 6) & (f5.con_com_product==1),1,np.nan),
#                    nbr_accts_open_l9m_com=np.where((f5.min_mob < 9) & (f5.con_com_product==1),1,np.nan),
#                    nbr_accts_open_l12m_com=np.where((f5.min_mob < 12) & (f5.con_com_product==1),1,np.nan),

#                    nbr_accts_open_l3m_con=np.where((f5.min_mob < 3) & (f5.con_com_product==2),1,np.nan),
#                    nbr_accts_open_l6m_con=np.where((f5.min_mob < 6) & (f5.con_com_product==2),1,np.nan),
#                    nbr_accts_open_l9m_con=np.where((f5.min_mob < 9) & (f5.con_com_product==2),1,np.nan),
#                    nbr_accts_open_l12m_con=np.where((f5.min_mob < 12) & (f5.con_com_product==2),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_com_agri=np.where((f5.min_mob < 3) & (f5.bus_product==1),1,np.nan),
#                    nbr_accts_open_l6m_com_agri=np.where((f5.min_mob < 6) & (f5.bus_product==1),1,np.nan),
#                    nbr_accts_open_l9m_com_agri=np.where((f5.min_mob < 9) & (f5.bus_product==1),1,np.nan),
#                    nbr_accts_open_l12m_com_agri=np.where((f5.min_mob < 12) & (f5.bus_product==1),1,np.nan),
                   
                   
#                    nbr_accts_open_l3m_com_oth=np.where((f5.min_mob < 3) & (f5.bus_product==2),1,np.nan),
#                    nbr_accts_open_l6m_com_oth=np.where((f5.min_mob < 6) & (f5.bus_product==2),1,np.nan),
#                    nbr_accts_open_l9m_com_oth=np.where((f5.min_mob < 9) & (f5.bus_product==2),1,np.nan),
#                    nbr_accts_open_l12m_com_oth=np.where((f5.min_mob < 12) & (f5.bus_product==2),1,np.nan),
                   
                   
                   
# #                  length of history
                   
#                    max_length_hist=f5.nbr_mnth_info,
#                    min_length_hist=f5.nbr_mnth_info,
#                    max_length_hist_36m=f5.nbr_mnth_info_36m,
#                    max_mob_all=f5.min_mob,
#                    max_live_mob_all=np.where(f5.closed_with_0_bal_flag_2 <= 0 ,f5.min_mob,np.nan),
#                    max_live_mob_hl=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['58','195'])) ,f5.min_mob,np.nan),
#                    max_live_mob_al=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['47','221'])) ,f5.min_mob,np.nan),
#                    max_live_mob_pl=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['123'])) ,f5.min_mob,np.nan),
#                    max_live_mob_cc=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['5','220','214'])) ,f5.min_mob,np.nan),
#                    max_live_mob_gl=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['191'])) ,f5.min_mob,np.nan),
#                    max_live_mob_tw=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['173'])) ,f5.min_mob,np.nan),
#                    max_live_mob_sec=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product==2),1,np.nan),
#                    max_live_mob_uns=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product==1),1,np.nan),
#                    max_live_mob_oth=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product==3),1,np.nan),
#                    max_live_mob_com=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product==1),1,np.nan),
#                    max_live_mob_con=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product==2),1,np.nan),
#                    max_live_mob_com_agri=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product==1),1,np.nan),
#                    max_live_mob_com_oth=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product==2),1,np.nan),
#                    max_live_mob_active=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.mnth_since_last_reported<=12),1,np.nan),
                   
#                    min_mob_all=f5.min_mob,
                   
#                    max_mob_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.min_mob,np.nan),
#                    min_mob_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.min_mob,np.nan),
                   
#                    max_mob_pl=np.where(f5.account_type_cd.isin(['123']),f5.min_mob,np.nan),
#                    min_mob_pl=np.where(f5.account_type_cd.isin(['123']),f5.min_mob,np.nan),
                   
#                    max_mob_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.min_mob,np.nan),
#                    min_mob_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.min_mob,np.nan),
                   
#                    max_mob_al=np.where(f5.account_type_cd.isin(['47','221']),f5.min_mob,np.nan),
#                    min_mob_al=np.where(f5.account_type_cd.isin(['47','221']),f5.min_mob,np.nan),
                   
#                    max_mob_gl=np.where(f5.account_type_cd.isin(['191']),f5.min_mob,np.nan),
#                    min_mob_gl=np.where(f5.account_type_cd.isin(['191']),f5.min_mob,np.nan),
                   
#                    max_mob_tw=np.where(f5.account_type_cd.isin(['173']),f5.min_mob,np.nan),
#                    min_mob_tw=np.where(f5.account_type_cd.isin(['173']),f5.min_mob,np.nan),
                   
                   
#                    max_mob_oth=np.where(f5.sec_uns_product==3,f5.min_mob,np.nan),
#                    min_mob_oth=np.where(f5.sec_uns_product==3,f5.min_mob,np.nan),
                   
#                    max_mob_uns=np.where(f5.sec_uns_product==1,f5.min_mob,np.nan),
#                    min_mob_uns=np.where(f5.sec_uns_product==1,f5.min_mob,np.nan),
                   
#                    max_mob_sec=np.where(f5.sec_uns_product==2,f5.min_mob,np.nan),
#                    min_mob_sec=np.where(f5.sec_uns_product==2,f5.min_mob,np.nan),
                   
#                    max_mob_com=np.where(f5.con_com_product==1,f5.min_mob,np.nan),
#                    min_mob_com=np.where(f5.con_com_product==1,f5.min_mob,np.nan),
                   
#                    max_mob_con=np.where(f5.con_com_product==2,f5.min_mob,np.nan),
#                    min_mob_con=np.where(f5.con_com_product==2,f5.min_mob,np.nan),
                   
#                    max_mob_com_agri=np.where(f5.bus_product==1,f5.min_mob,np.nan),
#                    min_mob_com_agri=np.where(f5.bus_product==1,f5.min_mob,np.nan),
                   
#                    max_mob_com_oth=np.where(f5.bus_product==2,f5.min_mob,np.nan),
#                    min_mob_com_oth=np.where(f5.bus_product==2,f5.min_mob,np.nan),
                   
# #                    maximum delinquency history
                   
                   
#                    max_dpd_0m_all=f5.dpd_0m,
#                    max_dpd_3m_all=f5.max_dpd_3m,
#                    max_dpd_6m_all=f5.max_dpd_6m,
#                    max_dpd_12m_all=f5.max_dpd_12m,
#                    max_dpd_24m_all=f5.max_dpd_24m,
#                    max_dpd_life_all=f5.max_dpd_life,
                   
#                    max_dpd_0m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.dpd_0m,np.nan),
#                    max_dpd_3m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.max_dpd_24m,np.nan),
                   
#                    max_dpd_0m_pl=np.where(f5.account_type_cd.isin(['123']),f5.dpd_0m,np.nan),
#                    max_dpd_3m_pl=np.where(f5.account_type_cd.isin(['123']),f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_pl=np.where(f5.account_type_cd.isin(['123']),f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_pl=np.where(f5.account_type_cd.isin(['123']),f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_pl=np.where(f5.account_type_cd.isin(['123']),f5.max_dpd_24m,np.nan),
                   
#                    max_dpd_0m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.dpd_0m,np.nan),
#                    max_dpd_3m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.max_dpd_24m,np.nan),
                   
#                    max_dpd_0m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.dpd_0m,np.nan),
#                    max_dpd_3m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.max_dpd_24m,np.nan),
                   
#                    max_dpd_0m_gl=np.where(f5.account_type_cd.isin(['191']),f5.dpd_0m,np.nan),
#                    max_dpd_3m_gl=np.where(f5.account_type_cd.isin(['191']),f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_gl=np.where(f5.account_type_cd.isin(['191']),f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_gl=np.where(f5.account_type_cd.isin(['191']),f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_gl=np.where(f5.account_type_cd.isin(['191']),f5.max_dpd_24m,np.nan),
                   
#                    max_dpd_0m_tw=np.where(f5.account_type_cd.isin(['173']),f5.dpd_0m,np.nan),
#                    max_dpd_3m_tw=np.where(f5.account_type_cd.isin(['173']),f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_tw=np.where(f5.account_type_cd.isin(['173']),f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_tw=np.where(f5.account_type_cd.isin(['173']),f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_tw=np.where(f5.account_type_cd.isin(['173']),f5.max_dpd_24m,np.nan),
                   
#                    max_dpd_0m_uns=np.where(f5.sec_uns_product==1,f5.dpd_0m,np.nan),
#                    max_dpd_3m_uns=np.where(f5.sec_uns_product==1,f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_uns=np.where(f5.sec_uns_product==1,f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_uns=np.where(f5.sec_uns_product==1,f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_uns=np.where(f5.sec_uns_product==1,f5.max_dpd_24m,np.nan),
                   
                   
#                    max_dpd_0m_sec=np.where(f5.sec_uns_product==2,f5.dpd_0m,np.nan),
#                    max_dpd_3m_sec=np.where(f5.sec_uns_product==2,f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_sec=np.where(f5.sec_uns_product==2,f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_sec=np.where(f5.sec_uns_product==2,f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_sec=np.where(f5.sec_uns_product==2,f5.max_dpd_24m,np.nan),
                   
                   
#                    max_dpd_0m_oth=np.where(f5.sec_uns_product==3,f5.dpd_0m,np.nan),
#                    max_dpd_3m_oth=np.where(f5.sec_uns_product==3,f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_oth=np.where(f5.sec_uns_product==3,f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_oth=np.where(f5.sec_uns_product==3,f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_oth=np.where(f5.sec_uns_product==3,f5.max_dpd_24m,np.nan),
                   
                   
#                    max_dpd_0m_com=np.where(f5.con_com_product==1,f5.dpd_0m,np.nan),
#                    max_dpd_3m_com=np.where(f5.con_com_product==1,f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_com=np.where(f5.con_com_product==1,f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_com=np.where(f5.con_com_product==1,f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_com=np.where(f5.con_com_product==1,f5.max_dpd_24m,np.nan),
                   
                   
#                    max_dpd_0m_con=np.where(f5.con_com_product==2,f5.dpd_0m,np.nan),
#                    max_dpd_3m_con=np.where(f5.con_com_product==2,f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_con=np.where(f5.con_com_product==2,f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_con=np.where(f5.con_com_product==2,f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_con=np.where(f5.con_com_product==2,f5.max_dpd_24m,np.nan),
                   
                   
#                    max_dpd_0m_com_agri=np.where(f5.bus_product==1,f5.dpd_0m,np.nan),
#                    max_dpd_3m_com_agri=np.where(f5.bus_product==1,f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_com_agri=np.where(f5.bus_product==1,f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_com_agri=np.where(f5.bus_product==1,f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_com_agri=np.where(f5.bus_product==1,f5.max_dpd_24m,np.nan),
                   
#                    max_dpd_0m_com_oth=np.where(f5.bus_product==2,f5.dpd_0m,np.nan),
#                    max_dpd_3m_com_oth=np.where(f5.bus_product==2,f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_com_oth=np.where(f5.bus_product==2,f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_com_oth=np.where(f5.bus_product==2,f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_com_oth=np.where(f5.bus_product==2,f5.max_dpd_24m,np.nan),
                   
                   
#                    max_dpd_0m_com_act=np.where(f5.mnth_since_last_reported<=12,f5.dpd_0m,np.nan),
#                    max_dpd_3m_com_act=np.where(f5.mnth_since_last_reported<=12,f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_com_act=np.where(f5.mnth_since_last_reported<=12,f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_com_act=np.where(f5.mnth_since_last_reported<=12,f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_com_act=np.where(f5.mnth_since_last_reported<=12,f5.max_dpd_24m,np.nan),
                   
                   
#                    max_dpd_0m_com_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.dpd_0m,np.nan),
#                    max_dpd_3m_com_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.max_dpd_3m,np.nan),
#                    max_dpd_6m_com_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.max_dpd_6m,np.nan),
#                    max_dpd_12m_com_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.max_dpd_12m,np.nan),
#                    max_dpd_24m_com_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.max_dpd_24m,np.nan),
                   
                   
# #                  number of times delinquency 0 history-693
                   
                   
#                   nbr_0_3m_all=f5.nbr_0_3m,
#                   nbr_0_6m_all=f5.nbr_0_6m, 
#                   nbr_0_12m_all=f5.nbr_0_12m, 
#                   nbr_0_24m_all=f5.nbr_0_24m, 
                   
#                   max_nbr_0_3m_all=f5.nbr_0_3m,
#                   max_nbr_0_6m_all=f5.nbr_0_6m, 
#                   max_nbr_0_12m_all=f5.nbr_0_12m, 
#                   max_nbr_0_24m_all=f5.nbr_0_24m,  
                   
#                   nbr_0_3m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_3m,np.nan), 
#                   nbr_0_6m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_6m,np.nan),  
#                   nbr_0_12m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_12m,np.nan),  
#                   nbr_0_24m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_24m,np.nan),
                   
#                   max_nbr_0_3m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_3m,np.nan), 
#                   max_nbr_0_6m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_6m,np.nan),  
#                   max_nbr_0_12m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_12m,np.nan),  
#                   max_nbr_0_24m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_24m,np.nan), 
                   
                   
#                   nbr_prod_0_3m_all=np.where(f5.nbr_0_3m > 0,1,np.nan), 
#                   nbr_prod_0_6m_all=np.where(f5.nbr_0_6m > 0,1,np.nan),  
#                   nbr_prod_0_12m_all=np.where(f5.nbr_0_12m > 0,1,np.nan),  
#                   nbr_prod_0_24m_all=np.where(f5.nbr_0_24m > 0,1,np.nan),  
                   
#                   nbr_prod_0_3m_hl=np.where((f5.nbr_0_3m > 0) & (f5.account_type_cd.isin(['58','195'])),1,np.where((f5.nbr_0_3m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)),
#                   nbr_prod_0_6m_hl=np.where((f5.nbr_0_6m > 0) & (f5.account_type_cd.isin(['58','195'])),1,np.where((f5.nbr_0_6m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)), 
#                   nbr_prod_0_12m_hl=np.where((f5.nbr_0_12m > 0) & (f5.account_type_cd.isin(['58','195'])),1,np.where((f5.nbr_0_12m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)), 
#                   nbr_prod_0_24m_hl=np.where((f5.nbr_0_24m > 0) & (f5.account_type_cd.isin(['58','195'])),1,np.where((f5.nbr_0_24m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)) 
                   
#                   )

In [108]:
factor_61=f5.assign(nbr_live_accts=np.where(f5.closed_with_0_bal_flag_2 <= 0 ,1,np.nan),
                  nbr_closed_accts=np.where(f5.closed_with_0_bal_flag_2 > 0 ,1,np.nan),
                  nbr_tot_accts=f5.tgt_key,
                   
                  nbr_hl_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
                  nbr_hl_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
                  nbr_hl_tot_accts=np.where((f5.account_type_cd.isin(['58','195'])),1,np.nan),
                  
                  nbr_pl_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['123'])),1,np.nan),
                  nbr_pl_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['123'])),1,np.nan),
                  nbr_pl_tot_accts=np.where((f5.account_type_cd.isin(['123'])),1,np.nan),
                   
                   
                  nbr_al_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
                  nbr_al_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
                  nbr_al_tot_accts=np.where((f5.account_type_cd.isin(['47','221'])),1,np.nan),
                   
                  
                  nbr_cc_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['5','214','220'])),1,np.nan),
                  nbr_cc_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['5','214','220'])),1,np.nan),
                  nbr_cc_tot_accts=np.where((f5.account_type_cd.isin(['5','214','220'])),1,np.nan),
                   
                   
                  nbr_gl_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['191'])),1,np.nan),
                  nbr_gl_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['191'])),1,np.nan),
                  nbr_gl_tot_accts=np.where((f5.account_type_cd.isin(['191'])),1,np.nan),
                  
                  
                  nbr_tw_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['173'])),1,np.nan),
                  nbr_tw_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.account_type_cd.isin(['173'])),1,np.nan),
                  nbr_tw_tot_accts=np.where((f5.account_type_cd.isin(['173'])),1,np.nan),
                   
                   
                  nbr_cl_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['189'])),1,np.nan),
                  nbr_cl_closed_accts=np.where((f5.closed_with_0_bal_flag_2 ==1) & (f5.account_type_cd.isin(['189'])),1,np.nan),
                  nbr_cl_tot_accts=np.where((f5.account_type_cd.isin(['189'])),1,np.nan),
                   
                   
                  nbr_oth_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product == 3),1,np.nan),
                  nbr_oth_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.sec_uns_product == 3),1,np.nan),
                  nbr_oth_tot_accts=np.where((f5.sec_uns_product == 3),1,np.nan),
                   
                   
                  nbr_uns_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product == 1),1,np.nan),
                  nbr_uns_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.sec_uns_product == 1),1,np.nan),
                  nbr_uns_tot_accts=np.where((f5.sec_uns_product == 1),1,np.nan)
                   
                   
                  ,nbr_sec_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product == 2),1,np.nan),
                  nbr_sec_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.sec_uns_product == 2),1,np.nan),
                  nbr_sec_tot_accts=np.where((f5.sec_uns_product == 2),1,np.nan), 
                   
                   
                  nbr_com_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product == 1),1,np.nan),
                  nbr_com_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.con_com_product == 1),1,np.nan),
                  nbr_com_tot_accts=np.where((f5.con_com_product == 1),1,np.nan),                        
     
                   
                  nbr_con_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product == 2),1,np.nan),
                  nbr_con_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.con_com_product == 2),1,np.nan),
                  nbr_con_tot_accts=np.where((f5.con_com_product == 2),1,np.nan),                         
                   
                   
                  nbr_com_agri_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product == 1),1,np.nan),
                  nbr_com_agri_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.bus_product == 1),1,np.nan),
                  nbr_com_agri_tot_accts=np.where((f5.bus_product == 1),1,np.nan),                        

                   
                  nbr_com_oth_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product == 2),1,np.nan),
                  nbr_com_oth_closed_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.bus_product == 2),1,np.nan),
                  nbr_com_oth_tot_accts=np.where((f5.bus_product == 2),1,np.nan)                     
                       
                   
#                   ##no bank type var in dataset 412-439
# #                   based on last statement in last 12 months 
                  ,nbr_live_active_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.mnth_since_last_reported <= 12),1,np.nan),
                  nbr_closed_active_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.mnth_since_last_reported <= 12),1,np.nan),
                  nbr_tot_active_accts=np.where((f5.mnth_since_last_reported <= 12),1,np.nan),                       
                   
                  nbr_live_inactive_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.mnth_since_last_reported > 12),1,np.nan),
                  nbr_closed_inactive_accts=np.where((f5.closed_with_0_bal_flag_2 == 1) & (f5.mnth_since_last_reported > 12),1,np.nan),
                  nbr_tot_inactive_accts=np.where((f5.mnth_since_last_reported > 12),1,np.nan),                        
                   
                   
                  nbr_cc_live_active_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['5','214','220'])) & (~f5.max_bal_act.isin([0,np.nan])) & (~f5.min_bal_act.isin([0,np.nan])),1,np.nan), 
                   
#                 No. of clean relationship
                  nbr_clean_live_accts=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.dpd_0m <= 0),1,np.nan),
                   
#               number of accounts opened
                   
                   nbr_accts_open_l3m=np.where(f5.min_mob < 3,1,np.nan),
                   nbr_accts_open_l6m=np.where(f5.min_mob < 6,1,np.nan),
                   nbr_accts_open_l9m=np.where(f5.min_mob < 9,1,np.nan),
                   nbr_accts_open_l12m=np.where(f5.min_mob < 12,1,np.nan),
                   
                   nbr_accts_open_l3m_hl=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
                   nbr_accts_open_l6m_hl=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
                   nbr_accts_open_l9m_hl=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
                   nbr_accts_open_l12m_hl=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['58','195'])),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_pl=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['123'])),1,np.nan),
                   nbr_accts_open_l6m_pl=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['123'])),1,np.nan),
                   nbr_accts_open_l9m_pl=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['123'])),1,np.nan),
                   nbr_accts_open_l12m_pl=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['123'])),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_cc=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['5','220','214'])),1,np.nan),
                   nbr_accts_open_l6m_cc=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['5','220','214'])),1,np.nan),
                   nbr_accts_open_l9m_cc=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['5','220','214'])),1,np.nan),
                   nbr_accts_open_l12m_cc=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['5','220','214'])),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_al=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
                   nbr_accts_open_l6m_al=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
                   nbr_accts_open_l9m_al=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
                   nbr_accts_open_l12m_al=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['47','221'])),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_gl=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['191'])),1,np.nan),
                   nbr_accts_open_l6m_gl=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['191'])),1,np.nan),
                   nbr_accts_open_l9m_gl=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['191'])),1,np.nan),
                   nbr_accts_open_l12m_gl=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['191'])),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_tw=np.where((f5.min_mob < 3) & (f5.account_type_cd.isin(['173'])),1,np.nan),
                   nbr_accts_open_l6m_tw=np.where((f5.min_mob < 6) & (f5.account_type_cd.isin(['173'])),1,np.nan),
                   nbr_accts_open_l9m_tw=np.where((f5.min_mob < 9) & (f5.account_type_cd.isin(['173'])),1,np.nan),
                   nbr_accts_open_l12m_tw=np.where((f5.min_mob < 12) & (f5.account_type_cd.isin(['173'])),1,np.nan),
                   
                   nbr_accts_open_l3m_oth=np.where((f5.min_mob < 3) & (f5.sec_uns_product==3),1,np.nan),
                   nbr_accts_open_l6m_oth=np.where((f5.min_mob < 6) & (f5.sec_uns_product==3),1,np.nan),
                   nbr_accts_open_l9m_oth=np.where((f5.min_mob < 9) & (f5.sec_uns_product==3),1,np.nan),
                   nbr_accts_open_l12m_oth=np.where((f5.min_mob < 12) & (f5.sec_uns_product==3),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_uns=np.where((f5.min_mob < 3) & (f5.sec_uns_product==1),1,np.nan),
                   nbr_accts_open_l6m_uns=np.where((f5.min_mob < 6) & (f5.sec_uns_product==1),1,np.nan),
                   nbr_accts_open_l9m_uns=np.where((f5.min_mob < 9) & (f5.sec_uns_product==1),1,np.nan),
                   nbr_accts_open_l12m_uns=np.where((f5.min_mob < 12) & (f5.sec_uns_product==1),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_sec=np.where((f5.min_mob < 3) & (f5.sec_uns_product==2),1,np.nan),
                   nbr_accts_open_l6m_sec=np.where((f5.min_mob < 6) & (f5.sec_uns_product==2),1,np.nan),
                   nbr_accts_open_l9m_sec=np.where((f5.min_mob < 9) & (f5.sec_uns_product==2),1,np.nan),
                   nbr_accts_open_l12m_sec=np.where((f5.min_mob < 12) & (f5.sec_uns_product==2),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_com=np.where((f5.min_mob < 3) & (f5.con_com_product==1),1,np.nan),
                   nbr_accts_open_l6m_com=np.where((f5.min_mob < 6) & (f5.con_com_product==1),1,np.nan),
                   nbr_accts_open_l9m_com=np.where((f5.min_mob < 9) & (f5.con_com_product==1),1,np.nan),
                   nbr_accts_open_l12m_com=np.where((f5.min_mob < 12) & (f5.con_com_product==1),1,np.nan),

                   nbr_accts_open_l3m_con=np.where((f5.min_mob < 3) & (f5.con_com_product==2),1,np.nan),
                   nbr_accts_open_l6m_con=np.where((f5.min_mob < 6) & (f5.con_com_product==2),1,np.nan),
                   nbr_accts_open_l9m_con=np.where((f5.min_mob < 9) & (f5.con_com_product==2),1,np.nan),
                   nbr_accts_open_l12m_con=np.where((f5.min_mob < 12) & (f5.con_com_product==2),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_com_agri=np.where((f5.min_mob < 3) & (f5.bus_product==1),1,np.nan),
                   nbr_accts_open_l6m_com_agri=np.where((f5.min_mob < 6) & (f5.bus_product==1),1,np.nan),
                   nbr_accts_open_l9m_com_agri=np.where((f5.min_mob < 9) & (f5.bus_product==1),1,np.nan),
                   nbr_accts_open_l12m_com_agri=np.where((f5.min_mob < 12) & (f5.bus_product==1),1,np.nan),
                   
                   
                   nbr_accts_open_l3m_com_oth=np.where((f5.min_mob < 3) & (f5.bus_product==2),1,np.nan),
                   nbr_accts_open_l6m_com_oth=np.where((f5.min_mob < 6) & (f5.bus_product==2),1,np.nan),
                   nbr_accts_open_l9m_com_oth=np.where((f5.min_mob < 9) & (f5.bus_product==2),1,np.nan),
                   nbr_accts_open_l12m_com_oth=np.where((f5.min_mob < 12) & (f5.bus_product==2),1,np.nan)
                   
                   
                   
# #                  length of history
                   
                   ,max_length_hist=f5.nbr_mnth_info,
                   min_length_hist=f5.nbr_mnth_info,
                   max_length_hist_36m=f5.nbr_mnth_info_36m,
                   max_mob_all=f5.min_mob,
                   max_live_mob_all=np.where(f5.closed_with_0_bal_flag_2 <= 0 ,f5.min_mob,np.nan),
                   max_live_mob_hl=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['58','195'])) ,f5.min_mob,np.nan),
                   max_live_mob_al=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['47','221'])) ,f5.min_mob,np.nan),
                   max_live_mob_pl=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['123'])) ,f5.min_mob,np.nan),
                   max_live_mob_cc=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['5','220','214'])) ,f5.min_mob,np.nan),
                   max_live_mob_gl=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['191'])) ,f5.min_mob,np.nan),
                   max_live_mob_tw=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['173'])) ,f5.min_mob,np.nan),
                   max_live_mob_sec=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product==2),f5.min_mob,np.nan),
                   max_live_mob_uns=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product==1),f5.min_mob,np.nan),
                   max_live_mob_oth=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product==3),f5.min_mob,np.nan),
                   max_live_mob_com=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product==1),f5.min_mob,np.nan),
                   max_live_mob_con=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product==2),f5.min_mob,np.nan),
                   max_live_mob_com_agri=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product==1),f5.min_mob,np.nan),
                   max_live_mob_com_oth=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product==2),f5.min_mob,np.nan),
                   max_live_mob_active=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.mnth_since_last_reported<=12),1,np.nan),
                   
                   min_mob_all=f5.min_mob,
                   
                   max_mob_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.min_mob,np.nan),
                   min_mob_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.min_mob,np.nan),
                   
                   max_mob_pl=np.where(f5.account_type_cd.isin(['123']),f5.min_mob,np.nan),
                   min_mob_pl=np.where(f5.account_type_cd.isin(['123']),f5.min_mob,np.nan),
                   
                   max_mob_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.min_mob,np.nan),
                   min_mob_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.min_mob,np.nan),
                   
                   max_mob_al=np.where(f5.account_type_cd.isin(['47','221']),f5.min_mob,np.nan),
                   min_mob_al=np.where(f5.account_type_cd.isin(['47','221']),f5.min_mob,np.nan),
                   
                   max_mob_gl=np.where(f5.account_type_cd.isin(['191']),f5.min_mob,np.nan),
                   min_mob_gl=np.where(f5.account_type_cd.isin(['191']),f5.min_mob,np.nan),
                   
                   max_mob_tw=np.where(f5.account_type_cd.isin(['173']),f5.min_mob,np.nan),
                   min_mob_tw=np.where(f5.account_type_cd.isin(['173']),f5.min_mob,np.nan),
                   
                   
                   max_mob_oth=np.where(f5.sec_uns_product==3,f5.min_mob,np.nan),
                   min_mob_oth=np.where(f5.sec_uns_product==3,f5.min_mob,np.nan),
                   
                   max_mob_uns=np.where(f5.sec_uns_product==1,f5.min_mob,np.nan),
                   min_mob_uns=np.where(f5.sec_uns_product==1,f5.min_mob,np.nan),
                   
                   max_mob_sec=np.where(f5.sec_uns_product==2,f5.min_mob,np.nan),
                   min_mob_sec=np.where(f5.sec_uns_product==2,f5.min_mob,np.nan),
                   
                   max_mob_com=np.where(f5.con_com_product==1,f5.min_mob,np.nan),
                   min_mob_com=np.where(f5.con_com_product==1,f5.min_mob,np.nan),
                   
                   max_mob_con=np.where(f5.con_com_product==2,f5.min_mob,np.nan),
                   min_mob_con=np.where(f5.con_com_product==2,f5.min_mob,np.nan),
                   
                   max_mob_com_agri=np.where(f5.bus_product==1,f5.min_mob,np.nan),
                   min_mob_com_agri=np.where(f5.bus_product==1,f5.min_mob,np.nan),
                   
                   max_mob_com_oth=np.where(f5.bus_product==2,f5.min_mob,np.nan),
                   min_mob_com_oth=np.where(f5.bus_product==2,f5.min_mob,np.nan)
                   
# #                    maximum delinquency history
                   
                   
                   ,max_dpd_0m_all=f5.dpd_0m,
                   max_dpd_3m_all=f5.max_dpd_3m,
                   max_dpd_6m_all=f5.max_dpd_6m,
                   max_dpd_12m_all=f5.max_dpd_12m,
                   max_dpd_24m_all=f5.max_dpd_24m,
                   max_dpd_life_all=f5.max_dpd_life,
                   
                   max_dpd_0m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.dpd_0m,np.nan),
                   max_dpd_3m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.max_dpd_3m,np.nan),
                   max_dpd_6m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.max_dpd_6m,np.nan),
                   max_dpd_12m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.max_dpd_12m,np.nan),
                   max_dpd_24m_hl=np.where(f5.account_type_cd.isin(['58','195']),f5.max_dpd_24m,np.nan),
                   
                   max_dpd_0m_pl=np.where(f5.account_type_cd.isin(['123']),f5.dpd_0m,np.nan),
                   max_dpd_3m_pl=np.where(f5.account_type_cd.isin(['123']),f5.max_dpd_3m,np.nan),
                   max_dpd_6m_pl=np.where(f5.account_type_cd.isin(['123']),f5.max_dpd_6m,np.nan),
                   max_dpd_12m_pl=np.where(f5.account_type_cd.isin(['123']),f5.max_dpd_12m,np.nan),
                   max_dpd_24m_pl=np.where(f5.account_type_cd.isin(['123']),f5.max_dpd_24m,np.nan),
                   
                   max_dpd_0m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.dpd_0m,np.nan),
                   max_dpd_3m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.max_dpd_3m,np.nan),
                   max_dpd_6m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.max_dpd_6m,np.nan),
                   max_dpd_12m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.max_dpd_12m,np.nan),
                   max_dpd_24m_cc=np.where(f5.account_type_cd.isin(['5','220','214']),f5.max_dpd_24m,np.nan),
                   
                   max_dpd_0m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.dpd_0m,np.nan),
                   max_dpd_3m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.max_dpd_3m,np.nan),
                   max_dpd_6m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.max_dpd_6m,np.nan),
                   max_dpd_12m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.max_dpd_12m,np.nan),
                   max_dpd_24m_al=np.where(f5.account_type_cd.isin(['47','221']),f5.max_dpd_24m,np.nan),
                   
                   max_dpd_0m_gl=np.where(f5.account_type_cd.isin(['191']),f5.dpd_0m,np.nan),
                   max_dpd_3m_gl=np.where(f5.account_type_cd.isin(['191']),f5.max_dpd_3m,np.nan),
                   max_dpd_6m_gl=np.where(f5.account_type_cd.isin(['191']),f5.max_dpd_6m,np.nan),
                   max_dpd_12m_gl=np.where(f5.account_type_cd.isin(['191']),f5.max_dpd_12m,np.nan),
                   max_dpd_24m_gl=np.where(f5.account_type_cd.isin(['191']),f5.max_dpd_24m,np.nan),
                   
                   max_dpd_0m_tw=np.where(f5.account_type_cd.isin(['173']),f5.dpd_0m,np.nan),
                   max_dpd_3m_tw=np.where(f5.account_type_cd.isin(['173']),f5.max_dpd_3m,np.nan),
                   max_dpd_6m_tw=np.where(f5.account_type_cd.isin(['173']),f5.max_dpd_6m,np.nan),
                   max_dpd_12m_tw=np.where(f5.account_type_cd.isin(['173']),f5.max_dpd_12m,np.nan),
                   max_dpd_24m_tw=np.where(f5.account_type_cd.isin(['173']),f5.max_dpd_24m,np.nan),
                   
                   max_dpd_0m_uns=np.where(f5.sec_uns_product==1,f5.dpd_0m,np.nan),
                   max_dpd_3m_uns=np.where(f5.sec_uns_product==1,f5.max_dpd_3m,np.nan),
                   max_dpd_6m_uns=np.where(f5.sec_uns_product==1,f5.max_dpd_6m,np.nan),
                   max_dpd_12m_uns=np.where(f5.sec_uns_product==1,f5.max_dpd_12m,np.nan),
                   max_dpd_24m_uns=np.where(f5.sec_uns_product==1,f5.max_dpd_24m,np.nan),
                   
                   
                   max_dpd_0m_sec=np.where(f5.sec_uns_product==2,f5.dpd_0m,np.nan),
                   max_dpd_3m_sec=np.where(f5.sec_uns_product==2,f5.max_dpd_3m,np.nan),
                   max_dpd_6m_sec=np.where(f5.sec_uns_product==2,f5.max_dpd_6m,np.nan),
                   max_dpd_12m_sec=np.where(f5.sec_uns_product==2,f5.max_dpd_12m,np.nan),
                   max_dpd_24m_sec=np.where(f5.sec_uns_product==2,f5.max_dpd_24m,np.nan),
                   
                   
                   max_dpd_0m_oth=np.where(f5.sec_uns_product==3,f5.dpd_0m,np.nan),
                   max_dpd_3m_oth=np.where(f5.sec_uns_product==3,f5.max_dpd_3m,np.nan),
                   max_dpd_6m_oth=np.where(f5.sec_uns_product==3,f5.max_dpd_6m,np.nan),
                   max_dpd_12m_oth=np.where(f5.sec_uns_product==3,f5.max_dpd_12m,np.nan),
                   max_dpd_24m_oth=np.where(f5.sec_uns_product==3,f5.max_dpd_24m,np.nan),
                   
                   
                   max_dpd_0m_com=np.where(f5.con_com_product==1,f5.dpd_0m,np.nan),
                   max_dpd_3m_com=np.where(f5.con_com_product==1,f5.max_dpd_3m,np.nan),
                   max_dpd_6m_com=np.where(f5.con_com_product==1,f5.max_dpd_6m,np.nan),
                   max_dpd_12m_com=np.where(f5.con_com_product==1,f5.max_dpd_12m,np.nan),
                   max_dpd_24m_com=np.where(f5.con_com_product==1,f5.max_dpd_24m,np.nan),
                   
                   
                   max_dpd_0m_con=np.where(f5.con_com_product==2,f5.dpd_0m,np.nan),
                   max_dpd_3m_con=np.where(f5.con_com_product==2,f5.max_dpd_3m,np.nan),
                   max_dpd_6m_con=np.where(f5.con_com_product==2,f5.max_dpd_6m,np.nan),
                   max_dpd_12m_con=np.where(f5.con_com_product==2,f5.max_dpd_12m,np.nan),
                   max_dpd_24m_con=np.where(f5.con_com_product==2,f5.max_dpd_24m,np.nan),
                   
                   
                   max_dpd_0m_com_agri=np.where(f5.bus_product==1,f5.dpd_0m,np.nan),
                   max_dpd_3m_com_agri=np.where(f5.bus_product==1,f5.max_dpd_3m,np.nan),
                   max_dpd_6m_com_agri=np.where(f5.bus_product==1,f5.max_dpd_6m,np.nan),
                   max_dpd_12m_com_agri=np.where(f5.bus_product==1,f5.max_dpd_12m,np.nan),
                   max_dpd_24m_com_agri=np.where(f5.bus_product==1,f5.max_dpd_24m,np.nan),
                   
                   max_dpd_0m_com_oth=np.where(f5.bus_product==2,f5.dpd_0m,np.nan),
                   max_dpd_3m_com_oth=np.where(f5.bus_product==2,f5.max_dpd_3m,np.nan),
                   max_dpd_6m_com_oth=np.where(f5.bus_product==2,f5.max_dpd_6m,np.nan),
                   max_dpd_12m_com_oth=np.where(f5.bus_product==2,f5.max_dpd_12m,np.nan),
                   max_dpd_24m_com_oth=np.where(f5.bus_product==2,f5.max_dpd_24m,np.nan)
                   
                   
                   ,max_dpd_0m_act=np.where(f5.mnth_since_last_reported<=12,f5.dpd_0m,np.nan),
                   max_dpd_3m_act=np.where(f5.mnth_since_last_reported<=12,f5.max_dpd_3m,np.nan),
                   max_dpd_6m_act=np.where(f5.mnth_since_last_reported<=12,f5.max_dpd_6m,np.nan),
                   max_dpd_12m_act=np.where(f5.mnth_since_last_reported<=12,f5.max_dpd_12m,np.nan),
                   max_dpd_24m_act=np.where(f5.mnth_since_last_reported<=12,f5.max_dpd_24m,np.nan),
                   
                   
                   max_dpd_0m_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.dpd_0m,np.nan),
                   max_dpd_3m_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.max_dpd_3m,np.nan),
                   max_dpd_6m_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.max_dpd_6m,np.nan),
                   max_dpd_12m_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.max_dpd_12m,np.nan),
                   max_dpd_24m_live=np.where(f5.closed_with_0_bal_flag_2<=0,f5.max_dpd_24m,np.nan)
                   
                   
# #                  number of times delinquency 0 history-693
                   
                   
                  ,nbr_0_3m_all=f5.nbr_0_3m,
                  nbr_0_6m_all=f5.nbr_0_6m, 
                  nbr_0_12m_all=f5.nbr_0_12m, 
                  nbr_0_24m_all=f5.nbr_0_24m, 
                   
                  max_nbr_0_3m_all=f5.nbr_0_3m,
                  max_nbr_0_6m_all=f5.nbr_0_6m, 
                  max_nbr_0_12m_all=f5.nbr_0_12m, 
                  max_nbr_0_24m_all=f5.nbr_0_24m,  
                   
                  nbr_0_3m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_3m,np.nan), 
                  nbr_0_6m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_6m,np.nan),  
                  nbr_0_12m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_12m,np.nan),  
                  nbr_0_24m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_24m,np.nan),
                   
                  max_nbr_0_3m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_3m,np.nan), 
                  max_nbr_0_6m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_6m,np.nan),  
                  max_nbr_0_12m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_12m,np.nan),  
                  max_nbr_0_24m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_0_24m,np.nan), 
                   
                   
                  nbr_prod_0_3m_all=np.where(f5.nbr_0_3m > 0,1,np.nan), 
                  nbr_prod_0_6m_all=np.where(f5.nbr_0_6m > 0,1,np.nan),  
                  nbr_prod_0_12m_all=np.where(f5.nbr_0_12m > 0,1,np.nan),  
                  nbr_prod_0_24m_all=np.where(f5.nbr_0_24m > 0,1,np.nan),  
                   
                  nbr_prod_0_3m_hl=np.where((f5.nbr_0_3m > 0) & (f5.account_type_cd.isin(['58','195'])),1,np.where((f5.nbr_0_3m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)),
                  nbr_prod_0_6m_hl=np.where((f5.nbr_0_6m > 0) & (f5.account_type_cd.isin(['58','195'])),1,np.where((f5.nbr_0_6m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)), 
                  nbr_prod_0_12m_hl=np.where((f5.nbr_0_12m > 0) & (f5.account_type_cd.isin(['58','195'])),1,np.where((f5.nbr_0_12m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)), 
                  nbr_prod_0_24m_hl=np.where((f5.nbr_0_24m > 0) & (f5.account_type_cd.isin(['58','195'])),1,np.where((f5.nbr_0_24m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)) 
                   
                  ).groupby('cust_id').agg({'nbr_live_accts':'sum','nbr_closed_accts':'sum',
                                                       'nbr_tot_accts':'count',
                                           'nbr_hl_live_accts':'sum',
                                           'nbr_hl_closed_accts':'sum',
                                           'nbr_hl_tot_accts':'sum',
                                           'nbr_pl_live_accts':'sum',
                                           'nbr_pl_closed_accts':'sum',
                                           'nbr_pl_tot_accts':'sum',
                                           'nbr_al_live_accts':'sum',
                                           'nbr_al_closed_accts':'sum',
                                           'nbr_al_tot_accts':'sum',
                                           'nbr_cc_live_accts':'sum',
                                           'nbr_cc_closed_accts':'sum',
                                           'nbr_cc_tot_accts':'sum',
                                           'nbr_gl_live_accts':'sum',
                                           'nbr_gl_closed_accts':'sum',
                                           'nbr_gl_tot_accts':'sum',
                                           'nbr_tw_live_accts':'sum',
                                           'nbr_tw_closed_accts':'sum',
                                           'nbr_tw_tot_accts':'sum',
                                           'nbr_cl_live_accts':'sum',
                                           'nbr_cl_closed_accts':'sum',
                                           'nbr_cl_tot_accts':'sum',
                                           'nbr_oth_live_accts':'sum',
                                           'nbr_oth_closed_accts':'sum',
                                           'nbr_oth_tot_accts':'sum',
                                           'nbr_uns_live_accts':'sum','nbr_uns_closed_accts':'sum','nbr_uns_tot_accts':'sum'
    ,'nbr_sec_live_accts':'sum','nbr_sec_closed_accts':'sum','nbr_sec_tot_accts':'sum','nbr_com_live_accts':'sum','nbr_com_closed_accts':'sum',
    'nbr_com_tot_accts':'sum','nbr_con_live_accts':'sum','nbr_con_closed_accts':'sum','nbr_con_tot_accts':'sum','nbr_com_agri_live_accts':'sum',
    'nbr_com_agri_closed_accts':'sum','nbr_com_agri_tot_accts':'sum','nbr_com_oth_live_accts':'sum','nbr_com_oth_closed_accts':'sum',
    'nbr_com_oth_tot_accts':'sum'
     ,'nbr_live_active_accts':'sum','nbr_closed_active_accts':'sum','nbr_tot_active_accts':'sum','nbr_live_inactive_accts':'sum',
    'nbr_closed_inactive_accts':'sum','nbr_tot_inactive_accts':'sum','nbr_cc_live_active_accts':'sum','nbr_clean_live_accts':'sum',
    'nbr_accts_open_l3m':'sum','nbr_accts_open_l6m':'sum','nbr_accts_open_l9m':'sum','nbr_accts_open_l12m':'sum',
    'nbr_accts_open_l3m_hl':'sum','nbr_accts_open_l6m_hl':'sum','nbr_accts_open_l9m_hl':'sum','nbr_accts_open_l12m_hl':'sum',
    'nbr_accts_open_l3m_pl':'sum','nbr_accts_open_l6m_pl':'sum','nbr_accts_open_l9m_pl':'sum','nbr_accts_open_l12m_pl':'sum',
    'nbr_accts_open_l3m_cc':'sum','nbr_accts_open_l6m_cc':'sum','nbr_accts_open_l9m_cc':'sum','nbr_accts_open_l12m_cc':'sum',
    'nbr_accts_open_l3m_al':'sum','nbr_accts_open_l6m_al':'sum','nbr_accts_open_l9m_al':'sum','nbr_accts_open_l12m_al':'sum',
    'nbr_accts_open_l3m_gl':'sum','nbr_accts_open_l6m_gl':'sum','nbr_accts_open_l9m_gl':'sum','nbr_accts_open_l12m_gl':'sum',
    'nbr_accts_open_l3m_tw':'sum','nbr_accts_open_l6m_tw':'sum','nbr_accts_open_l9m_tw':'sum','nbr_accts_open_l12m_tw':'sum',
                                           
    'nbr_accts_open_l3m_oth':'sum','nbr_accts_open_l6m_oth':'sum','nbr_accts_open_l9m_oth':'sum','nbr_accts_open_l12m_oth':'sum',
    'nbr_accts_open_l3m_uns':'sum','nbr_accts_open_l6m_uns':'sum','nbr_accts_open_l9m_uns':'sum','nbr_accts_open_l12m_uns':'sum',
    'nbr_accts_open_l3m_sec':'sum','nbr_accts_open_l6m_sec':'sum','nbr_accts_open_l9m_sec':'sum','nbr_accts_open_l12m_sec':'sum',
    'nbr_accts_open_l3m_com':'sum','nbr_accts_open_l6m_com':'sum','nbr_accts_open_l9m_com':'sum','nbr_accts_open_l12m_com':'sum',
    'nbr_accts_open_l3m_con':'sum','nbr_accts_open_l6m_con':'sum','nbr_accts_open_l9m_con':'sum','nbr_accts_open_l12m_con':'sum',
    'nbr_accts_open_l3m_com_agri':'sum','nbr_accts_open_l6m_com_agri':'sum','nbr_accts_open_l9m_com_agri':'sum','nbr_accts_open_l12m_com_agri':'sum',
    'nbr_accts_open_l3m_com_oth':'sum','nbr_accts_open_l6m_com_oth':'sum','nbr_accts_open_l9m_com_oth':'sum','nbr_accts_open_l12m_com_oth':'sum'

                                           
    ,'max_length_hist':'max','min_length_hist':'min','max_length_hist_36m':'max','max_mob_all':'max',
    'max_live_mob_all':'max','max_live_mob_hl':'max','max_live_mob_al':'max','max_live_mob_pl':'max',
    'max_live_mob_cc':'max','max_live_mob_gl':'max','max_live_mob_tw':'max','max_live_mob_sec':'max',
    'max_live_mob_uns':'max','max_live_mob_oth':'max','max_live_mob_com':'max','max_live_mob_con':'max',
    'max_live_mob_com_agri':'max','max_live_mob_com_oth':'max','max_live_mob_active':'max','min_mob_all':'min',
    'max_mob_hl':'max','min_mob_hl':'min','max_mob_pl':'max','min_mob_pl':'min','max_mob_cc':'max','min_mob_cc':'min','max_mob_al':'max','min_mob_al':'min','max_mob_gl':'max','min_mob_gl':'min','max_mob_tw':'max',
    'min_mob_tw':'min',
    'max_mob_oth':'max','min_mob_oth':'min','max_mob_uns':'max','min_mob_uns':'min','max_mob_sec':'max','min_mob_uns':'min',
    'max_mob_com':'min','min_mob_com':'min','max_mob_con':'max','min_mob_con':'min','max_mob_com_agri':'max','min_mob_com_agri':'min',
    'max_mob_com_oth':'max','min_mob_com_oth':'min'
,'max_dpd_0m_all':'max','max_dpd_3m_all':'max','max_dpd_6m_all':'max','max_dpd_12m_all':'max','max_dpd_24m_all':'max',
     'max_dpd_life_all':'max','max_dpd_0m_hl':'max','max_dpd_3m_hl':'max','max_dpd_6m_hl':'max','max_dpd_12m_hl':'max',
     'max_dpd_24m_hl':'max','max_dpd_0m_pl':'max','max_dpd_3m_pl':'max','max_dpd_6m_pl':'max','max_dpd_12m_pl':'max',
     'max_dpd_24m_pl':'max','max_dpd_0m_cc':'max','max_dpd_3m_cc':'max','max_dpd_6m_cc':'max','max_dpd_12m_cc':'max',
     'max_dpd_24m_cc':'max','max_dpd_0m_al':'max','max_dpd_3m_al':'max','max_dpd_6m_al':'max','max_dpd_12m_al':'max',
     'max_dpd_24m_al':'max','max_dpd_0m_gl':'max','max_dpd_3m_gl':'max','max_dpd_6m_gl':'max','max_dpd_12m_gl':'max',
     'max_dpd_24m_gl':'max','max_dpd_0m_tw':'max','max_dpd_3m_tw':'max','max_dpd_6m_tw':'max','max_dpd_12m_tw':'max',
     'max_dpd_24m_tw':'max','max_dpd_0m_uns':'max','max_dpd_3m_uns':'max','max_dpd_6m_uns':'max','max_dpd_12m_uns':'max',
     'max_dpd_24m_uns':'max','max_dpd_0m_sec':'max','max_dpd_3m_sec':'max','max_dpd_6m_sec':'max','max_dpd_12m_sec':'max',
     'max_dpd_24m_sec':'max','max_dpd_0m_oth':'max','max_dpd_3m_oth':'max','max_dpd_6m_oth':'max','max_dpd_12m_oth':'max',
     'max_dpd_24m_oth':'max','max_dpd_0m_com':'max','max_dpd_3m_com':'max','max_dpd_6m_com':'max','max_dpd_12m_com':'max',
     'max_dpd_24m_com':'max','max_dpd_0m_con':'max','max_dpd_3m_con':'max','max_dpd_6m_con':'max','max_dpd_12m_con':'max',
     'max_dpd_24m_con':'max','max_dpd_0m_com_agri':'max','max_dpd_3m_com_agri':'max','max_dpd_6m_com_agri':'max','max_dpd_12m_com_agri':'max',
     'max_dpd_24m_com_agri':'max','max_dpd_0m_com_oth':'max','max_dpd_3m_com_oth':'max','max_dpd_6m_com_oth':'max','max_dpd_12m_com_oth':'max',
     'max_dpd_24m_com_oth':'max'
                                         
    ,'max_dpd_0m_act':'max','max_dpd_3m_act':'max','max_dpd_6m_act':'max','max_dpd_12m_act':'max','max_dpd_24m_act':'max',
     'max_dpd_0m_live':'max','max_dpd_3m_live':'max','max_dpd_6m_live':'max','max_dpd_12m_live':'max','max_dpd_24m_live':'max'
     ,'nbr_0_3m_all':'sum','nbr_0_6m_all':'sum','nbr_0_12m_all':'sum','nbr_0_24m_all':'sum','max_nbr_0_3m_all':'max',
     'max_nbr_0_6m_all':'max','max_nbr_0_12m_all':'max','max_nbr_0_24m_all':'max','nbr_0_3m_act':'sum','nbr_0_6m_act':'sum','nbr_0_12m_act':'sum',
     'nbr_0_24m_act':'sum','max_nbr_0_3m_act':'max','max_nbr_0_6m_act':'max','max_nbr_0_12m_act':'max','max_nbr_0_24m_act':'max',
     'nbr_prod_0_3m_all': nan_sum,'nbr_prod_0_6m_all': nan_sum,'nbr_prod_0_12m_all': nan_sum,'nbr_prod_0_24m_all': nan_sum,
     'nbr_prod_0_3m_hl':nan_sum,'nbr_prod_0_6m_hl': nan_sum,'nbr_prod_0_12m_hl': nan_sum,'nbr_prod_0_24m_hl': nan_sum})                                           

In [109]:
factor_62=f5.assign(nbr_prod_0_3m_pl=np.where((f5.nbr_0_3m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)),
                  nbr_prod_0_6m_pl=np.where((f5.nbr_0_6m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)), 
                  nbr_prod_0_12m_pl=np.where((f5.nbr_0_12m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)), 
                  nbr_prod_0_24m_pl=np.where((f5.nbr_0_24m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)),
                   
                   
                  nbr_prod_0_3m_cc=np.where((f5.nbr_0_3m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)),
                  nbr_prod_0_6m_cc=np.where((f5.nbr_0_6m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)), 
                  nbr_prod_0_12m_cc=np.where((f5.nbr_0_12m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)), 
                  nbr_prod_0_24m_cc=np.where((f5.nbr_0_24m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)),  
                   
                    
                  nbr_prod_0_3m_al=np.where((f5.nbr_0_3m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)),
                  nbr_prod_0_6m_al=np.where((f5.nbr_0_6m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)), 
                  nbr_prod_0_12m_al=np.where((f5.nbr_0_12m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)), 
                  nbr_prod_0_24m_al=np.where((f5.nbr_0_24m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)),    
                   
                    
                   nbr_prod_0_3m_gl=np.where((f5.nbr_0_3m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)),
                  nbr_prod_0_6m_gl=np.where((f5.nbr_0_6m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)), 
                  nbr_prod_0_12m_gl=np.where((f5.nbr_0_12m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)), 
                  nbr_prod_0_24m_gl=np.where((f5.nbr_0_24m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)), 
                   
                   
                  nbr_prod_0_3m_tw=np.where((f5.nbr_0_3m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)),
                  nbr_prod_0_6m_tw=np.where((f5.nbr_0_6m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)), 
                  nbr_prod_0_12m_tw=np.where((f5.nbr_0_12m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)), 
                  nbr_prod_0_24m_tw=np.where((f5.nbr_0_24m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)), 
                   
                   
                  nbr_prod_0_3m_oth=np.where((f5.nbr_0_3m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.sec_uns_product==3),0,np.nan)),
                  nbr_prod_0_6m_oth=np.where((f5.nbr_0_6m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.sec_uns_product==3),0,np.nan)), 
                  nbr_prod_0_12m_oth=np.where((f5.nbr_0_12m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.sec_uns_product==3),0,np.nan)), 
                  nbr_prod_0_24m_oth=np.where((f5.nbr_0_24m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.sec_uns_product==3),0,np.nan)), 
                   
                   
                  nbr_prod_0_3m_uns=np.where((f5.nbr_0_3m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.sec_uns_product==1),0,np.nan)),
                  nbr_prod_0_6m_uns=np.where((f5.nbr_0_6m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.sec_uns_product==1),0,np.nan)), 
                  nbr_prod_0_12m_uns=np.where((f5.nbr_0_12m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.sec_uns_product==1),0,np.nan)), 
                  nbr_prod_0_24m_uns=np.where((f5.nbr_0_24m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.sec_uns_product==1),0,np.nan)),  
                   
                   
                  nbr_prod_0_3m_sec=np.where((f5.nbr_0_3m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.sec_uns_product==2),0,np.nan)),
                  nbr_prod_0_6m_sec=np.where((f5.nbr_0_6m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.sec_uns_product==2),0,np.nan)), 
                  nbr_prod_0_12m_sec=np.where((f5.nbr_0_12m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.sec_uns_product==2),0,np.nan)), 
                  nbr_prod_0_24m_sec=np.where((f5.nbr_0_24m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.sec_uns_product==2),0,np.nan)),   
                   
                   
                  nbr_prod_0_3m_con=np.where((f5.nbr_0_3m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.con_com_product==2),0,np.nan)),
                  nbr_prod_0_6m_con=np.where((f5.nbr_0_6m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.con_com_product==2),0,np.nan)), 
                  nbr_prod_0_12m_con=np.where((f5.nbr_0_12m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.con_com_product==2),0,np.nan)), 
                  nbr_prod_0_24m_con=np.where((f5.nbr_0_24m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.con_com_product==2),0,np.nan)),   
                   
                   
                  nbr_prod_0_3m_com=np.where((f5.nbr_0_3m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.con_com_product==1),0,np.nan)),
                  nbr_prod_0_6m_com=np.where((f5.nbr_0_6m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.con_com_product==1),0,np.nan)), 
                  nbr_prod_0_12m_com=np.where((f5.nbr_0_12m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.con_com_product==1),0,np.nan)), 
                  nbr_prod_0_24m_com=np.where((f5.nbr_0_24m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.con_com_product==1),0,np.nan)),    
                   
                   
                  nbr_prod_0_3m_com_oth=np.where((f5.nbr_0_3m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.bus_product==2),0,np.nan)),
                  nbr_prod_0_6m_com_oth=np.where((f5.nbr_0_6m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.bus_product==2),0,np.nan)), 
                  nbr_prod_0_12m_com_oth=np.where((f5.nbr_0_12m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.bus_product==2),0,np.nan)), 
                  nbr_prod_0_24m_com_oth=np.where((f5.nbr_0_24m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.bus_product==2),0,np.nan)),
                    
                    
                  nbr_prod_0_3m_com_agri=np.where((f5.nbr_0_3m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.bus_product==1),0,np.nan)),
                  nbr_prod_0_6m_com_agri=np.where((f5.nbr_0_6m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.bus_product==1),0,np.nan)), 
                  nbr_prod_0_12m_com_agri=np.where((f5.nbr_0_12m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.bus_product==1),0,np.nan)), 
                  nbr_prod_0_24m_com_agri=np.where((f5.nbr_0_24m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.bus_product==1),0,np.nan)),
                    
                   
                  nbr_prod_0_3m_act=np.where((f5.nbr_0_3m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)),
                  nbr_prod_0_6m_act=np.where((f5.nbr_0_6m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)), 
                  nbr_prod_0_12m_act=np.where((f5.nbr_0_12m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)), 
                  nbr_prod_0_24m_act=np.where((f5.nbr_0_24m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)),  
                   
                   
                  nbr_prod_0_3m_live=np.where((f5.nbr_0_3m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_0_3m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)),
                  nbr_prod_0_6m_live=np.where((f5.nbr_0_6m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_0_6m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)), 
                  nbr_prod_0_12m_live=np.where((f5.nbr_0_12m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_0_12m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)), 
                  nbr_prod_0_24m_live=np.where((f5.nbr_0_24m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_0_24m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)),   
                   
#                    number of times delinquency 30 history
                    
                  nbr_30_3m_all=f5.nbr_30_3m,
                  nbr_30_6m_all=f5.nbr_30_6m, 
                  nbr_30_12m_all=f5.nbr_30_12m, 
                  nbr_30_24m_all=f5.nbr_30_24m, 
                   
                  max_nbr_30_3m_all=f5.nbr_30_3m,
                  max_nbr_30_6m_all=f5.nbr_30_6m, 
                  max_nbr_30_12m_all=f5.nbr_30_12m, 
                  max_nbr_30_24m_all=f5.nbr_30_24m,  
                   
                  nbr_30_3m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_30_3m,np.nan), 
                  nbr_30_6m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_30_6m,np.nan),  
                  nbr_30_12m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_30_12m,np.nan),  
                  nbr_30_24m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_30_24m,np.nan),
                   
                  max_nbr_30_3m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_30_3m,np.nan), 
                  max_nbr_30_6m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_30_6m,np.nan),  
                  max_nbr_30_12m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_30_12m,np.nan),  
                  max_nbr_30_24m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_30_24m,np.nan), 
                   
                   
                  nbr_prod_30_3m_all=np.where(f5.nbr_30_3m > 0,1,np.nan), 
                  nbr_prod_30_6m_all=np.where(f5.nbr_30_6m > 0,1,np.nan),  
                  nbr_prod_30_12m_all=np.where(f5.nbr_30_12m > 0,1,np.nan),  
                  nbr_prod_30_24m_all=np.where(f5.nbr_30_24m > 0,1,np.nan),  
                   
                  nbr_prod_30_3m_hl=np.where((f5.nbr_30_3m > 0) & (f5.account_type_cd.isin(['58','195'])),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)),
                  nbr_prod_30_6m_hl=np.where((f5.nbr_30_6m > 0) & (f5.account_type_cd.isin(['58','195'])),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)), 
                  nbr_prod_30_12m_hl=np.where((f5.nbr_30_12m > 0) & (f5.account_type_cd.isin(['58','195'])),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)), 
                  nbr_prod_30_24m_hl=np.where((f5.nbr_30_24m > 0) & (f5.account_type_cd.isin(['58','195'])),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)),  
                   
                   
                  nbr_prod_30_3m_pl=np.where((f5.nbr_30_3m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)),
                  nbr_prod_30_6m_pl=np.where((f5.nbr_30_6m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)), 
                  nbr_prod_30_12m_pl=np.where((f5.nbr_30_12m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)), 
                  nbr_prod_30_24m_pl=np.where((f5.nbr_30_24m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)),
                   
                   
                  nbr_prod_30_3m_cc=np.where((f5.nbr_30_3m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)),
                  nbr_prod_30_6m_cc=np.where((f5.nbr_30_6m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)), 
                  nbr_prod_30_12m_cc=np.where((f5.nbr_30_12m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)), 
                  nbr_prod_30_24m_cc=np.where((f5.nbr_30_24m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)),  
                   
                    
                  nbr_prod_30_3m_al=np.where((f5.nbr_30_3m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)),
                  nbr_prod_30_6m_al=np.where((f5.nbr_30_6m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)), 
                  nbr_prod_30_12m_al=np.where((f5.nbr_30_12m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)), 
                  nbr_prod_30_24m_al=np.where((f5.nbr_30_24m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)),    
                   
                    
                   nbr_prod_30_3m_gl=np.where((f5.nbr_30_3m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)),
                  nbr_prod_30_6m_gl=np.where((f5.nbr_30_6m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)), 
                  nbr_prod_30_12m_gl=np.where((f5.nbr_30_12m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)), 
                  nbr_prod_30_24m_gl=np.where((f5.nbr_30_24m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)), 
                   
                   
                  nbr_prod_30_3m_tw=np.where((f5.nbr_30_3m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)),
                  nbr_prod_30_6m_tw=np.where((f5.nbr_30_6m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)), 
                  nbr_prod_30_12m_tw=np.where((f5.nbr_30_12m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)), 
                  nbr_prod_30_24m_tw=np.where((f5.nbr_30_24m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)), 
                   
                   
                  nbr_prod_30_3m_oth=np.where((f5.nbr_30_3m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.sec_uns_product==3),0,np.nan)),
                  nbr_prod_30_6m_oth=np.where((f5.nbr_30_6m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.sec_uns_product==3),0,np.nan)), 
                  nbr_prod_30_12m_oth=np.where((f5.nbr_30_12m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.sec_uns_product==3),0,np.nan)), 
                  nbr_prod_30_24m_oth=np.where((f5.nbr_30_24m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.sec_uns_product==3),0,np.nan)), 
                   
                   
                  nbr_prod_30_3m_uns=np.where((f5.nbr_30_3m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.sec_uns_product==1),0,np.nan)),
                  nbr_prod_30_6m_uns=np.where((f5.nbr_30_6m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.sec_uns_product==1),0,np.nan)), 
                  nbr_prod_30_12m_uns=np.where((f5.nbr_30_12m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.sec_uns_product==1),0,np.nan)), 
                  nbr_prod_30_24m_uns=np.where((f5.nbr_30_24m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.sec_uns_product==1),0,np.nan)),  
                   
                   
                  nbr_prod_30_3m_sec=np.where((f5.nbr_30_3m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.sec_uns_product==2),0,np.nan)),
                  nbr_prod_30_6m_sec=np.where((f5.nbr_30_6m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.sec_uns_product==2),0,np.nan)), 
                  nbr_prod_30_12m_sec=np.where((f5.nbr_30_12m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.sec_uns_product==2),0,np.nan)), 
                  nbr_prod_30_24m_sec=np.where((f5.nbr_30_24m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.sec_uns_product==2),0,np.nan)),   
                   
                   
                  nbr_prod_30_3m_con=np.where((f5.nbr_30_3m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.con_com_product==2),0,np.nan)),
                  nbr_prod_30_6m_con=np.where((f5.nbr_30_6m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.con_com_product==2),0,np.nan)), 
                  nbr_prod_30_12m_con=np.where((f5.nbr_30_12m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.con_com_product==2),0,np.nan)), 
                  nbr_prod_30_24m_con=np.where((f5.nbr_30_24m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.con_com_product==2),0,np.nan)),   
                   
                   
                  nbr_prod_30_3m_com=np.where((f5.nbr_30_3m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.con_com_product==1),0,np.nan)),
                  nbr_prod_30_6m_com=np.where((f5.nbr_30_6m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.con_com_product==1),0,np.nan)), 
                  nbr_prod_30_12m_com=np.where((f5.nbr_30_12m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.con_com_product==1),0,np.nan)), 
                  nbr_prod_30_24m_com=np.where((f5.nbr_30_24m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.con_com_product==1),0,np.nan)),    
                   
                   
                  nbr_prod_30_3m_com_oth=np.where((f5.nbr_30_3m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.bus_product==2),0,np.nan)),
                  nbr_prod_30_6m_com_oth=np.where((f5.nbr_30_6m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.bus_product==2),0,np.nan)), 
                  nbr_prod_30_12m_com_oth=np.where((f5.nbr_30_12m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.bus_product==2),0,np.nan)), 
                  nbr_prod_30_24m_com_oth=np.where((f5.nbr_30_24m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.bus_product==2),0,np.nan)),
                    
                    
                  nbr_prod_30_3m_com_agri=np.where((f5.nbr_30_3m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.bus_product==1),0,np.nan)),
                  nbr_prod_30_6m_com_agri=np.where((f5.nbr_30_6m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.bus_product==1),0,np.nan)), 
                  nbr_prod_30_12m_com_agri=np.where((f5.nbr_30_12m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.bus_product==1),0,np.nan)), 
                  nbr_prod_30_24m_com_agri=np.where((f5.nbr_30_24m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.bus_product==1),0,np.nan)),
                    
                   
                  nbr_prod_30_3m_act=np.where((f5.nbr_30_3m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)),
                  nbr_prod_30_6m_act=np.where((f5.nbr_30_6m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)), 
                  nbr_prod_30_12m_act=np.where((f5.nbr_30_12m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)), 
                  nbr_prod_30_24m_act=np.where((f5.nbr_30_24m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)),  
                   
                   
                  nbr_prod_30_3m_live=np.where((f5.nbr_30_3m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_30_3m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)),
                  nbr_prod_30_6m_live=np.where((f5.nbr_30_6m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_30_6m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)), 
                  nbr_prod_30_12m_live=np.where((f5.nbr_30_12m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_30_12m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)), 
                  nbr_prod_30_24m_live=np.where((f5.nbr_30_24m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_30_24m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)),    
                   
                   
#                   number of times delinquency 90 history 
                   
                   nbr_90_3m_all=f5.nbr_90_3m,
                  nbr_90_6m_all=f5.nbr_90_6m, 
                  nbr_90_12m_all=f5.nbr_90_12m, 
                  nbr_90_24m_all=f5.nbr_90_24m, 
                   
                  max_nbr_90_3m_all=f5.nbr_90_3m,
                  max_nbr_90_6m_all=f5.nbr_90_6m, 
                  max_nbr_90_12m_all=f5.nbr_90_12m, 
                  max_nbr_90_24m_all=f5.nbr_90_24m,  
                   
                  nbr_90_3m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_90_3m,np.nan), 
                  nbr_90_6m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_90_6m,np.nan),  
                  nbr_90_12m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_90_12m,np.nan),  
                  nbr_90_24m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_90_24m,np.nan),
                   
                  max_nbr_90_3m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_90_3m,np.nan), 
                  max_nbr_90_6m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_90_6m,np.nan),  
                  max_nbr_90_12m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_90_12m,np.nan),  
                  max_nbr_90_24m_act=np.where(f5.mnth_since_last_reported <= 12,f5.nbr_90_24m,np.nan), 
                   
                   
                  nbr_prod_90_3m_all=np.where(f5.nbr_90_3m > 0,1,np.nan), 
                  nbr_prod_90_6m_all=np.where(f5.nbr_90_6m > 0,1,np.nan),  
                  nbr_prod_90_12m_all=np.where(f5.nbr_90_12m > 0,1,np.nan),  
                  nbr_prod_90_24m_all=np.where(f5.nbr_90_24m > 0,1,np.nan),  
                   
                  nbr_prod_90_3m_hl=np.where((f5.nbr_90_3m > 0) & (f5.account_type_cd.isin(['58','195'])),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)),
                  nbr_prod_90_6m_hl=np.where((f5.nbr_90_6m > 0) & (f5.account_type_cd.isin(['58','195'])),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)), 
                  nbr_prod_90_12m_hl=np.where((f5.nbr_90_12m > 0) & (f5.account_type_cd.isin(['58','195'])),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)), 
                  nbr_prod_90_24m_hl=np.where((f5.nbr_90_24m > 0) & (f5.account_type_cd.isin(['58','195'])),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.account_type_cd.isin(['58','195'])),0,np.nan)),  
                   
                   
                  nbr_prod_90_3m_pl=np.where((f5.nbr_90_3m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)),
                  nbr_prod_90_6m_pl=np.where((f5.nbr_90_6m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)), 
                  nbr_prod_90_12m_pl=np.where((f5.nbr_90_12m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)), 
                  nbr_prod_90_24m_pl=np.where((f5.nbr_90_24m > 0) & (f5.account_type_cd.isin(['123'])),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.account_type_cd.isin(['123'])),0,np.nan)),
                   
                   
                  nbr_prod_90_3m_cc=np.where((f5.nbr_90_3m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)),
                  nbr_prod_90_6m_cc=np.where((f5.nbr_90_6m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)), 
                  nbr_prod_90_12m_cc=np.where((f5.nbr_90_12m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)), 
                  nbr_prod_90_24m_cc=np.where((f5.nbr_90_24m > 0) & (f5.account_type_cd.isin(['5','220','214'])),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.account_type_cd.isin(['5','220','214'])),0,np.nan)),  
                   
                    
                  nbr_prod_90_3m_al=np.where((f5.nbr_90_3m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)),
                  nbr_prod_90_6m_al=np.where((f5.nbr_90_6m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)), 
                  nbr_prod_90_12m_al=np.where((f5.nbr_90_12m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)), 
                  nbr_prod_90_24m_al=np.where((f5.nbr_90_24m > 0) & (f5.account_type_cd.isin(['47','221'])),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.account_type_cd.isin(['47','221'])),0,np.nan)),    
                   
                    
                   nbr_prod_90_3m_gl=np.where((f5.nbr_90_3m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)),
                  nbr_prod_90_6m_gl=np.where((f5.nbr_90_6m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)), 
                  nbr_prod_90_12m_gl=np.where((f5.nbr_90_12m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)), 
                  nbr_prod_90_24m_gl=np.where((f5.nbr_90_24m > 0) & (f5.account_type_cd.isin(['191'])),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.account_type_cd.isin(['191'])),0,np.nan)), 
                   
                   
                  nbr_prod_90_3m_tw=np.where((f5.nbr_90_3m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)),
                  nbr_prod_90_6m_tw=np.where((f5.nbr_90_6m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)), 
                  nbr_prod_90_12m_tw=np.where((f5.nbr_90_12m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)), 
                  nbr_prod_90_24m_tw=np.where((f5.nbr_90_24m > 0) & (f5.account_type_cd.isin(['173'])),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.account_type_cd.isin(['173'])),0,np.nan)), 
                   
                   
                  nbr_prod_90_3m_oth=np.where((f5.nbr_90_3m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.sec_uns_product==3),0,np.nan)),
                  nbr_prod_90_6m_oth=np.where((f5.nbr_90_6m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.sec_uns_product==3),0,np.nan)), 
                  nbr_prod_90_12m_oth=np.where((f5.nbr_90_12m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.sec_uns_product==3),0,np.nan)), 
                  nbr_prod_90_24m_oth=np.where((f5.nbr_90_24m > 0) & (f5.sec_uns_product==3),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.sec_uns_product==3),0,np.nan)), 
                   
                   
                  nbr_prod_90_3m_uns=np.where((f5.nbr_90_3m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.sec_uns_product==1),0,np.nan)),
                  nbr_prod_90_6m_uns=np.where((f5.nbr_90_6m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.sec_uns_product==1),0,np.nan)), 
                  nbr_prod_90_12m_uns=np.where((f5.nbr_90_12m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.sec_uns_product==1),0,np.nan)), 
                  nbr_prod_90_24m_uns=np.where((f5.nbr_90_24m > 0) & (f5.sec_uns_product==1),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.sec_uns_product==1),0,np.nan)),  
                   
                   
                  nbr_prod_90_3m_sec=np.where((f5.nbr_90_3m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.sec_uns_product==2),0,np.nan)),
                  nbr_prod_90_6m_sec=np.where((f5.nbr_90_6m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.sec_uns_product==2),0,np.nan)), 
                  nbr_prod_90_12m_sec=np.where((f5.nbr_90_12m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.sec_uns_product==2),0,np.nan)), 
                  nbr_prod_90_24m_sec=np.where((f5.nbr_90_24m > 0) & (f5.sec_uns_product==2),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.sec_uns_product==2),0,np.nan)),   
                   
                   
                  nbr_prod_90_3m_con=np.where((f5.nbr_90_3m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.con_com_product==2),0,np.nan)),
                  nbr_prod_90_6m_con=np.where((f5.nbr_90_6m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.con_com_product==2),0,np.nan)), 
                  nbr_prod_90_12m_con=np.where((f5.nbr_90_12m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.con_com_product==2),0,np.nan)), 
                  nbr_prod_90_24m_con=np.where((f5.nbr_90_24m > 0) & (f5.con_com_product==2),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.con_com_product==2),0,np.nan)),   
                   
                   
                  nbr_prod_90_3m_com=np.where((f5.nbr_90_3m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.con_com_product==1),0,np.nan)),
                  nbr_prod_90_6m_com=np.where((f5.nbr_90_6m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.con_com_product==1),0,np.nan)), 
                  nbr_prod_90_12m_com=np.where((f5.nbr_90_12m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.con_com_product==1),0,np.nan)), 
                  nbr_prod_90_24m_com=np.where((f5.nbr_90_24m > 0) & (f5.con_com_product==1),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.con_com_product==1),0,np.nan)),    
                   
                   
                  nbr_prod_90_3m_com_oth=np.where((f5.nbr_90_3m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.bus_product==2),0,np.nan)),
                  nbr_prod_90_6m_com_oth=np.where((f5.nbr_90_6m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.bus_product==2),0,np.nan)), 
                  nbr_prod_90_12m_com_oth=np.where((f5.nbr_90_12m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.bus_product==2),0,np.nan)), 
                  nbr_prod_90_24m_com_oth=np.where((f5.nbr_90_24m > 0) & (f5.bus_product==2),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.bus_product==2),0,np.nan)),
                    
                    
                  nbr_prod_90_3m_com_agri=np.where((f5.nbr_90_3m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.bus_product==1),0,np.nan)),
                  nbr_prod_90_6m_com_agri=np.where((f5.nbr_90_6m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.bus_product==1),0,np.nan)), 
                  nbr_prod_90_12m_com_agri=np.where((f5.nbr_90_12m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.bus_product==1),0,np.nan)), 
                  nbr_prod_90_24m_com_agri=np.where((f5.nbr_90_24m > 0) & (f5.bus_product==1),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.bus_product==1),0,np.nan)),
                    
                   
                  nbr_prod_90_3m_act=np.where((f5.nbr_90_3m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)),
                  nbr_prod_90_6m_act=np.where((f5.nbr_90_6m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)), 
                  nbr_prod_90_12m_act=np.where((f5.nbr_90_12m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)), 
                  nbr_prod_90_24m_act=np.where((f5.nbr_90_24m > 0) & (f5.mnth_since_last_reported <= 12),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.mnth_since_last_reported <= 12),0,np.nan)),  
                   
                   
                  nbr_prod_90_3m_live=np.where((f5.nbr_90_3m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_90_3m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)),
                  nbr_prod_90_6m_live=np.where((f5.nbr_90_6m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_90_6m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)), 
                  nbr_prod_90_12m_live=np.where((f5.nbr_90_12m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_90_12m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)), 
                  nbr_prod_90_24m_live=np.where((f5.nbr_90_24m > 0) & (f5.closed_with_0_bal_flag_2 <= 0),1,
                                           np.where((f5.nbr_90_24m == 0) & (f5.closed_with_0_bal_flag_2 <= 0),0,np.nan)),    
                   
#                 months since delinqeunt history
                    
                  mon_since0_all=f5.month_since_0,
                  mon_since30_all=f5.month_since_30,  
                  mon_since60_all=f5.month_since_60,  
                  mon_since90_all=f5.month_since_90,  
                  mon_since180_all=f5.month_since_180,
                    
                  
                  mon_since0_hl=np.where((f5.account_type_cd.isin(['58','195'])),f5.month_since_0,np.nan),
                  mon_since30_hl=np.where((f5.account_type_cd.isin(['58','195'])),f5.month_since_30,np.nan),  
                  mon_since60_hl=np.where((f5.account_type_cd.isin(['58','195'])),f5.month_since_60,np.nan),  
                  mon_since90_hl=np.where((f5.account_type_cd.isin(['58','195'])),f5.month_since_90,np.nan),  
                  mon_since180_hl=np.where((f5.account_type_cd.isin(['58','195'])),f5.month_since_180,np.nan),
                    
                  mon_since0_pl=np.where((f5.account_type_cd.isin(['123'])),f5.month_since_0,np.nan),
                  mon_since30_pl=np.where((f5.account_type_cd.isin(['123'])),f5.month_since_30,np.nan),  
                  mon_since60_pl=np.where((f5.account_type_cd.isin(['123'])),f5.month_since_60,np.nan),  
                  mon_since90_pl=np.where((f5.account_type_cd.isin(['123'])),f5.month_since_90,np.nan),  
                  mon_since180_pl=np.where((f5.account_type_cd.isin(['123'])),f5.month_since_180,np.nan),  
                    
                  mon_since0_cc=np.where((f5.account_type_cd.isin(['5','220','214'])),f5.month_since_0,np.nan),
                  mon_since30_cc=np.where((f5.account_type_cd.isin(['5','220','214'])),f5.month_since_30,np.nan),  
                  mon_since60_cc=np.where((f5.account_type_cd.isin(['5','220','214'])),f5.month_since_60,np.nan),  
                  mon_since90_cc=np.where((f5.account_type_cd.isin(['5','220','214'])),f5.month_since_90,np.nan),  
                  mon_since180_cc=np.where((f5.account_type_cd.isin(['5','220','214'])),f5.month_since_180,np.nan),  
                    
                  mon_since0_al=np.where((f5.account_type_cd.isin(['47','221'])),f5.month_since_0,np.nan),
                  mon_since30_al=np.where((f5.account_type_cd.isin(['47','221'])),f5.month_since_30,np.nan),  
                  mon_since60_al=np.where((f5.account_type_cd.isin(['47','221'])),f5.month_since_60,np.nan),  
                  mon_since90_al=np.where((f5.account_type_cd.isin(['47','221'])),f5.month_since_90,np.nan),  
                  mon_since180_al=np.where((f5.account_type_cd.isin(['47','221'])),f5.month_since_180,np.nan),    
                    
                  
                  mon_since0_gl=np.where((f5.account_type_cd.isin(['191'])),f5.month_since_0,np.nan),
                  mon_since30_gl=np.where((f5.account_type_cd.isin(['191'])),f5.month_since_30,np.nan),  
                  mon_since60_gl=np.where((f5.account_type_cd.isin(['191'])),f5.month_since_60,np.nan),  
                  mon_since90_gl=np.where((f5.account_type_cd.isin(['191'])),f5.month_since_90,np.nan),  
                  mon_since180_gl=np.where((f5.account_type_cd.isin(['191'])),f5.month_since_180,np.nan)   
                    
                                    
                   ).groupby('cust_id').agg({'nbr_prod_0_3m_pl': nan_sum,'nbr_prod_0_6m_pl': nan_sum,'nbr_prod_0_12m_pl': nan_sum,'nbr_prod_0_24m_pl': nan_sum,
     'nbr_prod_0_3m_cc': nan_sum,'nbr_prod_0_6m_cc': nan_sum,'nbr_prod_0_12m_cc': nan_sum,'nbr_prod_0_24m_cc': nan_sum,
     'nbr_prod_0_3m_al': nan_sum,'nbr_prod_0_6m_al': nan_sum,'nbr_prod_0_12m_al': nan_sum,'nbr_prod_0_24m_al': nan_sum,
     'nbr_prod_0_3m_gl': nan_sum,'nbr_prod_0_6m_gl': nan_sum,'nbr_prod_0_12m_gl': nan_sum,'nbr_prod_0_24m_gl': nan_sum,
     'nbr_prod_0_3m_tw': nan_sum,'nbr_prod_0_6m_tw': nan_sum,'nbr_prod_0_12m_tw': nan_sum,'nbr_prod_0_24m_tw': nan_sum,
     'nbr_prod_0_3m_uns': nan_sum,'nbr_prod_0_6m_uns': nan_sum,'nbr_prod_0_12m_uns': nan_sum,'nbr_prod_0_24m_uns': nan_sum,
     'nbr_prod_0_3m_sec': nan_sum,'nbr_prod_0_6m_sec': nan_sum,'nbr_prod_0_12m_sec': nan_sum,'nbr_prod_0_24m_sec': nan_sum,
     'nbr_prod_0_3m_oth': nan_sum ,'nbr_prod_0_6m_oth': nan_sum,'nbr_prod_0_12m_oth': nan_sum,'nbr_prod_0_24m_oth': nan_sum,
     'nbr_prod_0_3m_com': nan_sum,'nbr_prod_0_6m_com': nan_sum,'nbr_prod_0_12m_com': nan_sum,'nbr_prod_0_24m_com': nan_sum,
     'nbr_prod_0_3m_con': nan_sum,'nbr_prod_0_6m_con': nan_sum,'nbr_prod_0_12m_con': nan_sum,'nbr_prod_0_24m_con': nan_sum,
                                            
      'nbr_prod_0_3m_com_agri': nan_sum,'nbr_prod_0_6m_com_agri': nan_sum,'nbr_prod_0_12m_com_agri': nan_sum,'nbr_prod_0_24m_com_agri': nan_sum,
     'nbr_prod_0_3m_com_oth': nan_sum,'nbr_prod_0_6m_com_oth': nan_sum,'nbr_prod_0_12m_com_oth': nan_sum,'nbr_prod_0_24m_com_oth': nan_sum,
     'nbr_prod_0_3m_act': nan_sum,'nbr_prod_0_6m_act': nan_sum,'nbr_prod_0_12m_act': nan_sum,'nbr_prod_0_24m_act': nan_sum,
     'nbr_prod_0_3m_live':nan_sum,'nbr_prod_0_6m_live': nan_sum,'nbr_prod_0_12m_live':nan_sum,'nbr_prod_0_24m_live': nan_sum,
     'nbr_30_3m_all': nan_sum,'nbr_30_6m_all': nan_sum,'nbr_30_12m_all': nan_sum,'nbr_30_24m_all': nan_sum,'max_nbr_30_3m_all': nan_sum,
     'max_nbr_30_6m_all':'max','max_nbr_30_12m_all':'max','max_nbr_30_24m_all':'max','nbr_30_3m_act': nan_sum,'nbr_30_6m_act': nan_sum,'nbr_30_12m_act': nan_sum,
     'nbr_30_24m_act': nan_sum,'max_nbr_30_3m_act':'max','max_nbr_30_6m_act':'max','max_nbr_30_12m_act':'max','max_nbr_30_24m_act':'max',
                                            
       'nbr_prod_30_3m_all': nan_sum,'nbr_prod_30_6m_all': nan_sum,'nbr_prod_30_12m_all': nan_sum,'nbr_prod_30_24m_all': nan_sum,
     'nbr_prod_30_3m_hl': nan_sum,'nbr_prod_30_6m_hl': nan_sum,'nbr_prod_30_12m_hl': nan_sum,'nbr_prod_30_24m_hl': nan_sum,
     'nbr_prod_30_3m_pl': nan_sum,'nbr_prod_30_6m_pl': nan_sum,'nbr_prod_30_12m_pl': nan_sum,'nbr_prod_30_24m_pl': nan_sum,
     'nbr_prod_30_3m_cc': nan_sum,'nbr_prod_30_6m_cc': nan_sum,'nbr_prod_30_12m_cc': nan_sum,'nbr_prod_30_24m_cc': nan_sum,
     'nbr_prod_30_3m_al': nan_sum,'nbr_prod_30_6m_al': nan_sum,'nbr_prod_30_12m_al': nan_sum,'nbr_prod_30_24m_al': nan_sum,
     'nbr_prod_30_3m_gl': nan_sum,'nbr_prod_30_6m_gl': nan_sum,'nbr_prod_30_12m_gl': nan_sum,'nbr_prod_30_24m_gl': nan_sum,
     'nbr_prod_30_3m_tw': nan_sum,'nbr_prod_30_6m_tw': nan_sum,'nbr_prod_30_12m_tw': nan_sum,'nbr_prod_30_24m_tw': nan_sum,
     'nbr_prod_30_3m_uns': nan_sum,'nbr_prod_30_6m_uns': nan_sum,'nbr_prod_30_12m_uns': nan_sum,'nbr_prod_30_24m_uns': nan_sum,
     'nbr_prod_30_3m_sec': nan_sum,'nbr_prod_30_6m_sec': nan_sum,'nbr_prod_30_12m_sec': nan_sum,'nbr_prod_30_24m_sec': nan_sum,
     'nbr_prod_30_3m_oth': nan_sum,'nbr_prod_30_6m_oth': nan_sum,'nbr_prod_30_12m_oth': nan_sum,'nbr_prod_30_24m_oth': nan_sum,
     'nbr_prod_30_3m_com': nan_sum,'nbr_prod_30_6m_com': nan_sum,'nbr_prod_30_12m_com': nan_sum,'nbr_prod_30_24m_com': nan_sum,
     'nbr_prod_30_3m_con': nan_sum,'nbr_prod_30_6m_con': nan_sum,'nbr_prod_30_12m_con': nan_sum,'nbr_prod_30_24m_con': nan_sum,
                                            
     'nbr_prod_30_3m_com_agri': nan_sum,'nbr_prod_30_6m_com_agri': nan_sum,'nbr_prod_30_12m_com_agri': nan_sum,'nbr_prod_30_24m_com_agri': nan_sum,
     'nbr_prod_30_3m_com_oth': nan_sum,'nbr_prod_30_6m_com_oth': nan_sum,'nbr_prod_30_12m_com_oth': nan_sum,'nbr_prod_30_24m_com_oth': nan_sum,
     'nbr_prod_30_3m_act': nan_sum ,'nbr_prod_30_6m_act': nan_sum,'nbr_prod_30_12m_act': nan_sum,'nbr_prod_30_24m_act': nan_sum,
     'nbr_prod_30_3m_live': nan_sum,'nbr_prod_30_6m_live': nan_sum,'nbr_prod_30_12m_live': nan_sum,'nbr_prod_30_24m_live': nan_sum,
     'nbr_90_3m_all': nan_sum,'nbr_90_6m_all': nan_sum,'nbr_90_12m_all': nan_sum,'nbr_90_24m_all': nan_sum,
                                             
     'max_nbr_90_3m_all':'max', 'max_nbr_90_6m_all':'max','max_nbr_90_12m_all':'max','max_nbr_90_24m_all':'max',
                                             
     'nbr_90_3m_act': nan_sum,'nbr_90_6m_act': nan_sum,'nbr_90_12m_act': nan_sum, 'nbr_90_24m_act': nan_sum,
                                             
     'max_nbr_90_3m_act':'max','max_nbr_90_6m_act':'max','max_nbr_90_12m_act':'max','max_nbr_90_24m_act':'max',
                                             
     'nbr_prod_90_3m_all': nan_sum,'nbr_prod_90_6m_all': nan_sum,'nbr_prod_90_12m_all': nan_sum,'nbr_prod_90_24m_all': nan_sum,
     'nbr_prod_90_3m_hl': nan_sum,'nbr_prod_90_6m_hl': nan_sum,'nbr_prod_90_12m_hl': nan_sum,'nbr_prod_90_24m_hl': nan_sum,
     'nbr_prod_90_3m_pl': nan_sum,'nbr_prod_90_6m_pl': nan_sum,'nbr_prod_90_12m_pl': nan_sum,'nbr_prod_90_24m_pl': nan_sum,
     'nbr_prod_90_3m_cc': nan_sum,'nbr_prod_90_6m_cc': nan_sum,'nbr_prod_90_12m_cc': nan_sum,'nbr_prod_90_24m_cc': nan_sum,
     'nbr_prod_90_3m_al': nan_sum,'nbr_prod_90_6m_al': nan_sum,'nbr_prod_90_12m_al': nan_sum,'nbr_prod_90_24m_al': nan_sum,
     'nbr_prod_90_3m_gl': nan_sum,'nbr_prod_90_6m_gl': nan_sum,'nbr_prod_90_12m_gl': nan_sum,'nbr_prod_90_24m_gl': nan_sum,
     'nbr_prod_90_3m_tw': nan_sum,'nbr_prod_90_6m_tw':nan_sum,'nbr_prod_90_12m_tw': nan_sum,'nbr_prod_90_24m_tw': nan_sum,
     'nbr_prod_90_3m_uns': nan_sum,'nbr_prod_90_6m_uns': nan_sum,'nbr_prod_90_12m_uns': nan_sum,'nbr_prod_90_24m_uns': nan_sum,
     'nbr_prod_90_3m_sec': nan_sum,'nbr_prod_90_6m_sec': nan_sum,'nbr_prod_90_12m_sec': nan_sum,'nbr_prod_90_24m_sec': nan_sum,
     'nbr_prod_90_3m_oth': nan_sum,'nbr_prod_90_6m_oth': nan_sum,'nbr_prod_90_12m_oth': nan_sum,'nbr_prod_90_24m_oth': nan_sum,
     'nbr_prod_90_3m_com': nan_sum,'nbr_prod_90_6m_com': nan_sum,'nbr_prod_90_12m_com': nan_sum,'nbr_prod_90_24m_com': nan_sum,
     'nbr_prod_90_3m_con': nan_sum,'nbr_prod_90_6m_con': nan_sum,'nbr_prod_90_12m_con': nan_sum,'nbr_prod_90_24m_con': nan_sum,
     'nbr_prod_90_3m_com_agri': nan_sum,'nbr_prod_90_6m_com_agri': nan_sum,'nbr_prod_90_12m_com_agri': nan_sum,'nbr_prod_90_24m_com_agri': nan_sum,
     'nbr_prod_90_3m_com_oth': nan_sum,'nbr_prod_90_6m_com_oth': nan_sum,'nbr_prod_90_12m_com_oth': nan_sum,'nbr_prod_90_24m_com_oth': nan_sum,
     'nbr_prod_90_3m_act': nan_sum,'nbr_prod_90_6m_act': nan_sum,'nbr_prod_90_12m_act': nan_sum,'nbr_prod_90_24m_act': nan_sum,
     'nbr_prod_90_3m_live': nan_sum,'nbr_prod_90_6m_live': nan_sum,'nbr_prod_90_12m_live': nan_sum,'nbr_prod_90_24m_live': nan_sum,
                                            
     'mon_since0_all':'min','mon_since30_all':'min','mon_since60_all':'min','mon_since90_all':'min','mon_since180_all':'min',
     'mon_since0_hl':'min','mon_since30_hl':'min','mon_since60_hl':'min','mon_since90_hl':'min','mon_since180_hl':'min',
     'mon_since0_pl':'min','mon_since30_pl':'min','mon_since60_pl':'min','mon_since90_pl':'min','mon_since180_pl':'min',
     'mon_since0_cc':'min','mon_since30_cc':'min','mon_since60_cc':'min','mon_since90_cc':'min','mon_since180_cc':'min',
     'mon_since0_al':'min','mon_since30_al':'min','mon_since60_al':'min','mon_since90_al':'min','mon_since180_al':'min',
     'mon_since0_gl':'min','mon_since30_gl':'min','mon_since60_gl':'min','mon_since90_gl':'min','mon_since180_gl':'min'})

In [110]:
factor_63=f5.assign(
    
                  mon_since0_tw=np.where(f5.account_type_cd.isin(['173']),f5.month_since_0,np.nan),
                  mon_since30_tw=np.where((f5.account_type_cd.isin(['173'])),f5.month_since_30,np.nan),  
                  mon_since60_tw=np.where((f5.account_type_cd.isin(['173'])),f5.month_since_60,np.nan),  
                  mon_since90_tw=np.where((f5.account_type_cd.isin(['173'])),f5.month_since_90,np.nan),  
                  mon_since180_tw=np.where((f5.account_type_cd.isin(['173'])),f5.month_since_180,np.nan),    
                    
                    
                  mon_since0_oth=np.where(f5.sec_uns_product == 3,f5.month_since_0,np.nan),
                  mon_since30_oth=np.where(f5.sec_uns_product == 3,f5.month_since_30,np.nan),  
                  mon_since60_oth=np.where(f5.sec_uns_product == 3,f5.month_since_60,np.nan),  
                  mon_since90_oth=np.where(f5.sec_uns_product == 3,f5.month_since_90,np.nan),  
                  mon_since180_oth=np.where(f5.sec_uns_product == 3,f5.month_since_180,np.nan),      
                    
                  mon_since0_uns=np.where(f5.sec_uns_product == 1,f5.month_since_0,np.nan),
                  mon_since30_uns=np.where(f5.sec_uns_product == 1,f5.month_since_30,np.nan),  
                  mon_since60_uns=np.where(f5.sec_uns_product == 1,f5.month_since_60,np.nan),  
                  mon_since90_uns=np.where(f5.sec_uns_product == 1,f5.month_since_90,np.nan),  
                  mon_since180_uns=np.where(f5.sec_uns_product == 1,f5.month_since_180,np.nan),        
                    
                  mon_since0_sec=np.where(f5.sec_uns_product == 2,f5.month_since_0,np.nan),
                  mon_since30_sec=np.where(f5.sec_uns_product == 2,f5.month_since_30,np.nan),  
                  mon_since60_sec=np.where(f5.sec_uns_product == 2,f5.month_since_60,np.nan),  
                  mon_since90_sec=np.where(f5.sec_uns_product == 2,f5.month_since_90,np.nan),  
                  mon_since180_sec=np.where(f5.sec_uns_product == 2,f5.month_since_180,np.nan),
                    
                  mon_since0_com=np.where(f5.con_com_product == 1,f5.month_since_0,np.nan),
                  mon_since30_com=np.where(f5.con_com_product == 1,f5.month_since_30,np.nan),  
                  mon_since60_com=np.where(f5.con_com_product == 1,f5.month_since_60,np.nan),  
                  mon_since90_com=np.where(f5.con_com_product == 1,f5.month_since_90,np.nan),  
                  mon_since180_com=np.where(f5.con_com_product == 1,f5.month_since_180,np.nan),  
                    
                  mon_since0_con=np.where(f5.con_com_product == 2,f5.month_since_0,np.nan),
                  mon_since30_con=np.where(f5.con_com_product == 2,f5.month_since_30,np.nan),  
                  mon_since60_con=np.where(f5.con_com_product == 2,f5.month_since_60,np.nan),  
                  mon_since90_con=np.where(f5.con_com_product == 2,f5.month_since_90,np.nan),  
                  mon_since180_con=np.where(f5.con_com_product == 2,f5.month_since_180,np.nan),    
                  
                  mon_since0_com_agri=np.where(f5.bus_product == 1,f5.month_since_0,np.nan),
                  mon_since30_com_agri=np.where(f5.bus_product == 1,f5.month_since_30,np.nan),  
                  mon_since60_com_agri=np.where(f5.bus_product == 1,f5.month_since_60,np.nan),  
                  mon_since90_com_agri=np.where(f5.bus_product == 1,f5.month_since_90,np.nan),  
                  mon_since180_com_agri=np.where(f5.bus_product == 1,f5.month_since_180,np.nan),
                     
                  mon_since0_com_oth=np.where(f5.bus_product == 2,f5.month_since_0,np.nan),
                  mon_since30_com_oth=np.where(f5.bus_product == 2,f5.month_since_30,np.nan),  
                  mon_since60_com_oth=np.where(f5.bus_product == 2,f5.month_since_60,np.nan),  
                  mon_since90_com_oth=np.where(f5.bus_product == 2,f5.month_since_90,np.nan),  
                  mon_since180_com_oth=np.where(f5.bus_product == 2,f5.month_since_180,np.nan),   
                    
                  mon_since0_act=np.where(f5.mnth_since_last_reported <= 12,f5.month_since_0,np.nan),
                  mon_since30_act=np.where(f5.mnth_since_last_reported <= 12,f5.month_since_30,np.nan),  
                  mon_since60_act=np.where(f5.mnth_since_last_reported <= 12,f5.month_since_60,np.nan),  
                  mon_since90_act=np.where(f5.mnth_since_last_reported <= 12,f5.month_since_90,np.nan),  
                  mon_since180_act=np.where(f5.mnth_since_last_reported <= 12,f5.month_since_180,np.nan),
                    
                  mon_since0_live=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.month_since_0,np.nan),
                  mon_since30_live=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.month_since_30,np.nan),  
                  mon_since60_live=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.month_since_60,np.nan),  
                  mon_since90_live=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.month_since_90,np.nan),  
                  mon_since180_live=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.month_since_180,np.nan),  
                    
#                   writeoff variables
                   
                  min_write_off_month_life=pd.to_datetime(f5.a_write_off_month),
                  max_write_off_month_life=pd.to_datetime(f5.a_write_off_month),
                  count_write_off_life=f5.a_write_off_month,
                     
                  min_settlement_month_life=pd.to_datetime(f5.a_settlement_month),
                  max_settlement_month_life=pd.to_datetime(f5.a_settlement_month),
                  count_settlement_life=f5.a_settlement_month,   
                    
                  min_restructure_month_life=pd.to_datetime(f5.a_restructure_month),
                  max_restructure_month_life=pd.to_datetime(f5.a_restructure_month),
                  count_restructure_life=f5.a_restructure_month,     
                    
                  min_suitfiled_month_life=pd.to_datetime(f5.a_suitfiled_month),
                  max_suitfiled_month_life=pd.to_datetime(f5.a_suitfiled_month),
                  count_suitfiled_life=f5.a_suitfiled_month,       
                    
                  min_wilful_default_month_life=pd.to_datetime(f5.a_wilful_default_month),
                  max_wilful_default_month_life=pd.to_datetime(f5.a_wilful_default_month),
                  count_wilful_default_life=f5.a_wilful_default_month,         
                
                
                  min_suit_wilful_def_month_life=pd.to_datetime(f5.a_suit_wilful_def_month),
                  max_suit_wilful_def_month_life=pd.to_datetime(f5.a_suit_wilful_def_month),
                  count_suit_wilful_def_life=f5.a_suit_wilful_def_month,           
                    
                  count_ful_write_off_life=f5.a_ful_write_off_life_flag,
                     
#                   12 month 
                     
                  min_write_off_month_12m=pd.to_datetime(f5.a_write_off_month_12m),
                  max_write_off_month_12m=pd.to_datetime(f5.a_write_off_month_12m),
                  count_write_off_12m=f5.a_write_off_month_12m,
                     
                  min_settlement_month_12m=pd.to_datetime(f5.a_settlement_month_12m),
                  max_settlement_month_12m=pd.to_datetime(f5.a_settlement_month_12m),
                  count_settlement_12m=f5.a_settlement_month_12m,   
                    
                  min_restructure_month_12m=pd.to_datetime(f5.a_restructure_month_12m),
                  max_restructure_month_12m=pd.to_datetime(f5.a_restructure_month_12m),
                  count_restructure_12m=f5.a_restructure_month_12m,     
                    
                  min_suitfiled_month_12m=pd.to_datetime(f5.a_suitfiled_month_12m),
                  max_suitfiled_month_12m=pd.to_datetime(f5.a_suitfiled_month_12m),
                  count_suitfiled_12m=f5.a_suitfiled_month_12m,       
                   
                    
                  min_wilful_default_month_12m=pd.to_datetime(f5.a_wilful_default_month_12m),
                  max_wilful_default_month_12m=pd.to_datetime(f5.a_wilful_default_month_12m),
                  count_wilful_default_12m=f5.a_wilful_default_month_12m,         
                    
                  min_suit_wilful_def_month_12m=pd.to_datetime(f5.a_suit_wilful_def_month_12m),
                  max_suit_wilful_def_month_12m=pd.to_datetime(f5.a_suit_wilful_def_month_12m),
                  count_suit_wilful_def_12m=f5.a_suit_wilful_def_month_12m,           
                    
                  count_ful_write_off_12m=f5.a_ful_write_off_12m_flag,
                     
#                    24 month  
                    
                  min_write_off_month_24m=pd.to_datetime(f5.a_write_off_month_24m),
                  max_write_off_month_24m=pd.to_datetime(f5.a_write_off_month_24m),
                  count_write_off_24m=f5.a_write_off_month_24m,
                     
                  min_settlement_month_24m=pd.to_datetime(f5.a_settlement_month_24m),
                  max_settlement_month_24m=pd.to_datetime(f5.a_settlement_month_24m),
                  count_settlement_24m=f5.a_settlement_month_24m,   
                    
                  min_restructure_month_24m=pd.to_datetime(f5.a_restructure_month_24m),
                  max_restructure_month_24m=pd.to_datetime(f5.a_restructure_month_24m),
                  count_restructure_24m=f5.a_restructure_month_24m,     
                    
                  min_suitfiled_month_24m=pd.to_datetime(f5.a_suitfiled_month_24m),
                  max_suitfiled_month_24m=pd.to_datetime(f5.a_suitfiled_month_24m),
                  count_suitfiled_24m=f5.a_suitfiled_month_24m,       
      
                    

                  min_wilful_default_month_24m=pd.to_datetime(f5.a_wilful_default_month_24m),
                  max_wilful_default_month_24m=pd.to_datetime(f5.a_wilful_default_month_24m),
                  count_wilful_default_24m=f5.a_wilful_default_month_24m,         
                    
                  min_suit_wilful_def_month_24m=pd.to_datetime(f5.a_suit_wilful_def_month_24m),
                  max_suit_wilful_def_month_24m=pd.to_datetime(f5.a_suit_wilful_def_month_24m),
                  count_suit_wilful_def_24m=f5.a_suit_wilful_def_month_24m,           
                    
                  count_ful_write_off_24m=f5.a_ful_write_off_24m_flag,
                     
#                    36 month 
                     
                  min_write_off_month_36m=pd.to_datetime(f5.a_write_off_month_36m),
                  max_write_off_month_36m=pd.to_datetime(f5.a_write_off_month_36m),
                  count_write_off_36m=f5.a_write_off_month_36m,
                     
                  min_settlement_month_36m=pd.to_datetime(f5.a_settlement_month_36m),
                  max_settlement_month_36m=pd.to_datetime(f5.a_settlement_month_36m),
                  count_settlement_36m=f5.a_settlement_month_36m,   
                    
                  min_restructure_month_36m=pd.to_datetime(f5.a_restructure_month_36m),
                  max_restructure_month_36m=pd.to_datetime(f5.a_restructure_month_36m),
                  count_restructure_36m=f5.a_restructure_month_36m,     
                    
                  min_suitfiled_month_36m=pd.to_datetime(f5.a_suitfiled_month_36m),
                  max_suitfiled_month_36m=pd.to_datetime(f5.a_suitfiled_month_36m),
                  count_suitfiled_36m=f5.a_suitfiled_month_36m,       
                    
                  min_wilful_default_month_36m=pd.to_datetime(f5.a_wilful_default_month_36m),
                  max_wilful_default_month_36m=pd.to_datetime(f5.a_wilful_default_month_36m),
                  count_wilful_default_36m=f5.a_wilful_default_month_36m,         
                    
                  min_suit_wilful_def_month_36m=pd.to_datetime(f5.a_suit_wilful_def_month_36m),
                  max_suit_wilful_def_month_36m=pd.to_datetime(f5.a_suit_wilful_def_month_36m),
                  count_suit_wilful_def_36m=f5.a_suit_wilful_def_month_36m,           
                    
                  count_ful_write_off_36m=f5.a_ful_write_off_36m_flag,   
                    
#                   utilization variables
                     
                  util_curr_all_act_num=f5.curr_bal_active,
                  util_curr_all_act_den=np.where(f5.mnth_since_last_reported <= 12,f5.modified_limit,0),
                  util_l3m_all_act_num=f5.bal_l3m_active,
                  util_l3m_all_act_den=np.where(f5.mnth_since_last_reported <= 12,f5.modified_limit,0),
                  util_l6m_all_act_num=f5.bal_l6m_active,   
                  util_l6m_all_act_den=np.where(f5.mnth_since_last_reported <= 12,f5.modified_limit,0),   
                  util_l12m_all_act_num=f5.bal_l12m_active,
                  util_l12m_all_act_den=np.where(f5.mnth_since_last_reported <= 12,f5.modified_limit,0),      
                     
                     
                  util_curr_all_live_num=f5.curr_bal_live,
                  util_curr_all_live_den=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.modified_limit,0),
                  util_l3m_all_live_num=f5.bal_l3m_live,
                  util_l3m_all_live_den=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.modified_limit,0),
                  util_l6m_all_live_num=f5.bal_l6m_live,   
                  util_l6m_all_live_den=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.modified_limit,0),   
                  util_l12m_all_live_num=f5.bal_l12m_live,
                  util_l12m_all_live_den=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.modified_limit,0),         
                     
                     
                  util_curr_all_tot_num=f5.curr_bal_total,
                  util_curr_all_tot_den=f5.modified_limit,
                  util_l3m_all_tot_num=f5.bal_l3m_total,
                  util_l3m_all_tot_den=f5.modified_limit,  
                  util_l6m_all_tot_num=f5.bal_l6m_total,
                  util_l6m_all_tot_den=f5.modified_limit ,     
                  util_l12m_all_tot_num=f5.bal_l12m_total,
                  util_l12m_all_tot_den=f5.modified_limit ,     
                     
                     
                  avg_util_curr_all_act=f5.curr_util_active,
                  avg_util_l3m_all_act=f5.util_l3m_active,
                  avg_util_l6m_all_act=f5.util_l6m_active,
                  avg_util_l12m_all_act=f5.util_l12m_active,
                     
                  avg_util_curr_all_live=f5.curr_util_live,
                  avg_util_l3m_all_live=f5.util_l3m_live,
                  avg_util_l6m_all_live=f5.util_l6m_live,
                  avg_util_l12m_all_live=f5.util_l12m_live,
                     
                     
                  avg_util_curr_all_tot=f5.curr_util_total,
                  avg_util_l3m_all_tot=f5.util_l3m_total,
                  avg_util_l6m_all_tot=f5.util_l6m_total,
                  avg_util_l12m_all_tot=f5.util_l12m_total,   
                     
                  avg_util_l3m_pl_num=np.where(f5.account_type_cd.isin(['123']),f5.util_l3m_total,np.nan),   
                  avg_util_l3m_pl_den=np.where(f5.account_type_cd.isin(['123']),1,0),
                  avg_util_l6m_pl_num=np.where(f5.account_type_cd.isin(['123']),f5.util_l6m_total,np.nan),   
                  avg_util_l6m_pl_den=np.where(f5.account_type_cd.isin(['123']),1,0),   
                  avg_util_l12m_pl_num=np.where(f5.account_type_cd.isin(['123']),f5.util_l12m_total,np.nan),   
                  avg_util_l12m_pl_den=np.where(f5.account_type_cd.isin(['123']),1,0),   
                     
                  avg_util_l3m_cc_num=np.where(f5.account_type_cd.isin(['5','220','214']),f5.util_l3m_total,np.nan),   
                  avg_util_l3m_cc_den=np.where(f5.account_type_cd.isin(['5','220','214']),1,0),
                  avg_util_l6m_cc_num=np.where(f5.account_type_cd.isin(['5','220','214']),f5.util_l6m_total,np.nan),   
                  avg_util_l6m_cc_den=np.where(f5.account_type_cd.isin(['5','220','214']),1,0),   
                  avg_util_l12m_cc_num=np.where(f5.account_type_cd.isin(['5','220','214']),f5.util_l12m_total,np.nan),   
                  avg_util_l12m_cc_den=np.where(f5.account_type_cd.isin(['5','220','214']),1,0),
                     
                  avg_util_l3m_exc_cc_num=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.util_l3m_total,np.nan),   
                  avg_util_l3m_exc_cc_den=np.where(~f5.account_type_cd.isin(['5','220','214']),1,0),
                  avg_util_l6m_exc_cc_num=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.util_l6m_total,np.nan),   
                  avg_util_l6m_exc_cc_den=np.where(~f5.account_type_cd.isin(['5','220','214']),1,0),   
                  avg_util_l12m_exc_cc_num=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.util_l12m_total,np.nan),   
                  avg_util_l12m_exc_cc_den=np.where(~f5.account_type_cd.isin(['5','220','214']),1,0),   
                     
                  util_curr_cc_tot_num=np.where(f5.account_type_cd.isin(['5','220','214']),f5.curr_bal_total,np.nan),   
                  util_curr_cc_tot_den=np.where(f5.account_type_cd.isin(['5','220','214']),f5.modified_limit,0),
                  util_l3m_cc_tot_num=np.where(f5.account_type_cd.isin(['5','220','214']),f5.bal_l3m_total,np.nan),   
                  util_l3m_cc_tot_den=np.where(f5.account_type_cd.isin(['5','220','214']),f5.modified_limit,0),
                  util_l6m_cc_tot_num=np.where(f5.account_type_cd.isin(['5','220','214']),f5.bal_l6m_total,np.nan),   
                  util_l6m_cc_tot_den=np.where(f5.account_type_cd.isin(['5','220','214']),f5.modified_limit,0),   
                  util_l12m_cc_tot_num=np.where(f5.account_type_cd.isin(['5','220','214']),f5.bal_l12m_total,np.nan),   
                  util_l12m_cc_tot_den=np.where(f5.account_type_cd.isin(['5','220','214']),f5.modified_limit,0),   
                   
                     
                  util_curr_pl_tot_num=np.where(f5.account_type_cd.isin(['123']),f5.curr_bal_total,np.nan),   
                  util_curr_pl_tot_den=np.where(f5.account_type_cd.isin(['123']),f5.modified_limit,0),
                  util_l3m_pl_tot_num=np.where(f5.account_type_cd.isin(['123']),f5.bal_l3m_total,np.nan),   
                  util_l3m_pl_tot_den=np.where(f5.account_type_cd.isin(['123']),f5.modified_limit,0),
                  util_l6m_pl_tot_num=np.where(f5.account_type_cd.isin(['123']),f5.bal_l6m_total,np.nan),   
                  util_l6m_pl_tot_den=np.where(f5.account_type_cd.isin(['123']),f5.modified_limit,0),   
                  util_l12m_pl_tot_num=np.where(f5.account_type_cd.isin(['123']),f5.bal_l12m_total,np.nan),   
                  util_l12m_pl_tot_den=np.where(f5.account_type_cd.isin(['123']),f5.modified_limit,0),      
                
                  util_curr_exc_cc_tot_num=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.curr_bal_total,np.nan),   
                  util_curr_exc_cc_tot_den=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.modified_limit,0),
                  util_l3m_exc_cc_tot_num=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.bal_l3m_total,np.nan),   
                  util_l3m_exc_cc_tot_den=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.modified_limit,0),
                  util_l6m_exc_cc_tot_num=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.bal_l6m_total,np.nan),   
                  util_l6m_exc_cc_tot_den=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.modified_limit,0),   
                  util_l12m_exc_cc_tot_num=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.bal_l12m_total,np.nan),   
                  util_l12m_exc_cc_tot_den=np.where(~f5.account_type_cd.isin(['5','220','214']),f5.modified_limit,0),     
                     
                     
                  cust_loan_amt_live=np.where(f5.closed_with_0_bal_flag_2 <= 0,f5.modified_limit,0),  
                  cust_loan_amt_live_unsec=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product==1),f5.modified_limit,0),
                  cust_loan_amt_live_sec=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product==2),f5.modified_limit,0),   
                  cust_loan_amt_live_oth=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.sec_uns_product==3),f5.modified_limit,0),   
                  cust_loan_amt_live_com=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product==1),f5.modified_limit,0),
                  cust_loan_amt_live_con=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.con_com_product==2),f5.modified_limit,0),      
                  cust_loan_amt_live_com_agri=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product==1),f5.modified_limit,0),   
                  cust_loan_amt_live_com_oth=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.bus_product==2),f5.modified_limit,0),      
                  
                  cust_loan_amt_live_al=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['47','221'])),f5.modified_limit,0),   
                  cust_loan_amt_live_hl=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['58','195'])),f5.modified_limit,0),
                  cust_loan_amt_live_pl=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['123'])),f5.modified_limit,0),   
                  cust_loan_amt_live_cc=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['5','220','214'])),f5.modified_limit,0),   
                  cust_loan_amt_live_gl=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['191'])),f5.modified_limit,0),   
                  cust_loan_amt_live_tw=np.where((f5.closed_with_0_bal_flag_2 <= 0) & (f5.account_type_cd.isin(['173'])),f5.modified_limit,0),   
                  cust_loan_amt_active=np.where((f5.mnth_since_last_reported <= 12),f5.modified_limit,0)   
                     
                     
                    ).groupby('cust_id').agg({'mon_since0_tw':'min','mon_since30_tw':'min','mon_since60_tw':'min','mon_since90_tw':'min','mon_since180_tw':'min',
     'mon_since0_uns':'min','mon_since30_uns':'min','mon_since60_uns':'min','mon_since90_uns':'min','mon_since180_uns':'min',
     'mon_since0_sec':'min','mon_since30_sec':'min','mon_since60_sec':'min','mon_since90_sec':'min','mon_since180_sec':'min',
     'mon_since0_oth':'min','mon_since30_oth':'min','mon_since60_oth':'min','mon_since90_oth':'min','mon_since180_oth':'min',
     'mon_since0_com':'min','mon_since30_com':'min','mon_since60_com':'min','mon_since90_com':'min','mon_since180_com':'min',
     'mon_since0_con':'min','mon_since30_con':'min','mon_since60_con':'min','mon_since90_con':'min','mon_since180_con':'min',
     'mon_since0_com_agri':'min','mon_since30_com_agri':'min','mon_since60_com_agri':'min','mon_since90_com_agri':'min','mon_since180_com_agri':'min',
     'mon_since0_com_oth':'min','mon_since30_com_oth':'min','mon_since60_com_oth':'min','mon_since90_com_oth':'min','mon_since180_com_oth':'min',
     'mon_since0_act':'min','mon_since30_act':'min','mon_since60_act':'min','mon_since90_act':'min','mon_since180_act':'min',
     'mon_since0_live':'min','mon_since30_live':'min','mon_since60_live':'min','mon_since90_live':'min','mon_since180_live':'min',
                                             
      'min_write_off_month_life':'min','max_write_off_month_life':'max','count_write_off_life':'count',
     'min_settlement_month_life':'min','max_settlement_month_life':'max','count_settlement_life':'count',
     'min_restructure_month_life':'min','max_restructure_month_life':'max','count_restructure_life':'count'
     ,'min_suitfiled_month_life':'min','max_suitfiled_month_life':'max','count_suitfiled_life':'count',
     'min_wilful_default_month_life':'min','max_wilful_default_month_life':'max','count_wilful_default_life':'count'
     ,'min_suit_wilful_def_month_life':'min','max_suit_wilful_def_month_life':'max','count_suit_wilful_def_life':'count'
     ,'min_write_off_month_12m':'min','max_write_off_month_12m':'max','count_write_off_12m':'count',
     'min_settlement_month_12m':'min','max_settlement_month_12m':'max','count_settlement_12m':'count'
     ,'min_restructure_month_12m':'min','max_restructure_month_12m':'max','count_restructure_12m':'count'
     ,'min_suitfiled_month_12m':'min','max_suitfiled_month_12m':'max','count_suitfiled_12m':'count'
    ,'min_wilful_default_month_12m':'min','max_wilful_default_month_12m':'max','count_wilful_default_12m':'count'
     ,'min_suit_wilful_def_month_12m':'min','max_suit_wilful_def_month_12m':'max','count_suit_wilful_def_12m':'count'
     ,'min_write_off_month_24m':'min','max_write_off_month_24m':'max','count_write_off_24m':'count',
     'min_settlement_month_24m':'min','max_settlement_month_24m':'max','count_settlement_24m':'count',
     'min_restructure_month_24m':'min','max_restructure_month_24m':'max','count_restructure_24m':'count',
     'min_suitfiled_month_24m':'min','max_suitfiled_month_24m':'max','count_suitfiled_24m':'count'
     ,'min_wilful_default_month_24m':'min','max_wilful_default_month_24m':'max','count_wilful_default_24m':'count'
     ,'min_suit_wilful_def_month_24m':'min','max_suit_wilful_def_month_24m':'max','count_suit_wilful_def_24m':'count'
     ,'min_write_off_month_36m':'min','max_write_off_month_36m':'max','count_write_off_36m':'count'
     ,'min_settlement_month_36m':'min','max_settlement_month_36m':'max','count_settlement_36m':'count',
     'min_restructure_month_36m':'min','max_restructure_month_36m':'max','count_restructure_36m':'count',
     'min_suitfiled_month_36m':'min','max_suitfiled_month_36m':'max','count_suitfiled_36m':'count',
     'min_wilful_default_month_36m':'min','max_wilful_default_month_36m':'max','count_wilful_default_36m':'count',
     'min_suit_wilful_def_month_36m':'min','max_suit_wilful_def_month_36m':'max','count_suit_wilful_def_36m':'count'
                                             
                                             
      ,'util_curr_all_act_num':'sum','util_curr_all_act_den':'sum','util_l3m_all_act_num':'sum','util_l3m_all_act_den':'sum'
      ,'util_l6m_all_act_num':'sum','util_l6m_all_act_den':'sum','util_l12m_all_act_num':'sum','util_l12m_all_act_den':'sum'
                                              
      ,'util_curr_all_live_num':'sum','util_curr_all_live_den':'sum','util_l3m_all_live_num':'sum','util_l3m_all_live_den':'sum'
      ,'util_l6m_all_live_num':'sum','util_l6m_all_live_den':'sum','util_l12m_all_live_num':'sum','util_l12m_all_live_den':'sum'                                       
                                             
      ,'util_curr_all_tot_num':'sum','util_curr_all_tot_den':'sum','util_l3m_all_tot_num':'sum','util_l3m_all_tot_den':'sum'
      ,'util_l6m_all_tot_num':'sum','util_l6m_all_tot_den':'sum','util_l12m_all_tot_num':'sum','util_l12m_all_tot_den':'sum'                                       
                                             
      ,'avg_util_curr_all_act':'mean',
     'avg_util_l3m_all_act':'mean','avg_util_l6m_all_act':'mean','avg_util_l12m_all_act':'mean','avg_util_curr_all_live':'mean',
     'avg_util_l3m_all_live':'mean','avg_util_l6m_all_live':'mean','avg_util_l12m_all_live':'mean','avg_util_curr_all_tot':'mean',
     'avg_util_l3m_all_tot':'mean','avg_util_l6m_all_tot':'mean','avg_util_l12m_all_tot':'mean'                                       
                                             
     ,'avg_util_l3m_pl_num':'sum','avg_util_l3m_pl_den':'sum','avg_util_l6m_pl_num':'sum','avg_util_l6m_pl_den':'sum'                                        
      ,'avg_util_l12m_pl_num':'sum','avg_util_l12m_pl_den':'sum'
                                             
      ,'avg_util_l3m_cc_num':'sum','avg_util_l3m_cc_den':'sum','avg_util_l6m_cc_num':'sum','avg_util_l6m_cc_den':'sum'                                        
      ,'avg_util_l12m_cc_num':'sum','avg_util_l12m_cc_den':'sum'
                                       
      ,'avg_util_l3m_exc_cc_num':'sum','avg_util_l3m_exc_cc_den':'sum','avg_util_l6m_exc_cc_num':'sum','avg_util_l6m_exc_cc_den':'sum'                                        
      ,'avg_util_l12m_exc_cc_num':'sum','avg_util_l12m_exc_cc_den':'sum'
                                       
      ,'util_curr_cc_tot_num':'sum','util_curr_cc_tot_den':'sum','util_l3m_cc_tot_num':'sum','util_l3m_cc_tot_den':'sum'                                       
      ,'util_l6m_cc_tot_num':'sum','util_l6m_cc_tot_den':'sum','util_l12m_cc_tot_num':'sum','util_l12m_cc_tot_den':'sum'                                       
       
      ,'util_curr_pl_tot_num':'sum','util_curr_pl_tot_den':'sum','util_l3m_pl_tot_num':'sum','util_l3m_pl_tot_den':'sum'                                       
      ,'util_l6m_pl_tot_num':'sum','util_l6m_pl_tot_den':'sum','util_l12m_pl_tot_num':'sum','util_l12m_pl_tot_den':'sum'                                                                               
                                             
       
      ,'util_curr_exc_cc_tot_num':'sum','util_curr_exc_cc_tot_den':'sum','util_l3m_exc_cc_tot_num':'sum','util_l3m_exc_cc_tot_den':'sum'                                       
      ,'util_l6m_exc_cc_tot_num':'sum','util_l6m_exc_cc_tot_den':'sum','util_l12m_exc_cc_tot_num':'sum','util_l12m_exc_cc_tot_den':'sum'                                                                               
                                              
      ,'cust_loan_amt_live':'sum','cust_loan_amt_live_unsec':'sum','cust_loan_amt_live_sec':'sum',                                        
      'cust_loan_amt_live_oth':'sum','cust_loan_amt_live_com':'sum','cust_loan_amt_live_con':'sum',                                        
      'cust_loan_amt_live_com_agri':'sum','cust_loan_amt_live_com_oth':'sum',                                       
       'cust_loan_amt_live_al':'sum','cust_loan_amt_live_hl':'sum','cust_loan_amt_live_pl':'sum',                                       
       'cust_loan_amt_live_cc':'sum','cust_loan_amt_live_gl':'sum','cust_loan_amt_live_tw':'sum',
       'cust_loan_amt_active':'sum'                                       
                                             })

In [111]:
factor_63['util_curr_all_act']=factor_63['util_curr_all_act_num']/factor_63['util_curr_all_act_den']
factor_63['util_l3m_all_act']=factor_63['util_l3m_all_act_num']/(3*factor_63['util_l3m_all_act_den'])
factor_63['util_l6m_all_act']=factor_63['util_l6m_all_act_num']/(6*factor_63['util_l6m_all_act_den'])
factor_63['util_l12m_all_act']=factor_63['util_l12m_all_act_num']/(12*factor_63['util_l12m_all_act_den'])
                                              
factor_63['util_curr_all_live']=factor_63['util_curr_all_live_num']/factor_63['util_curr_all_live_den']
factor_63['util_l3m_all_live']=factor_63['util_l3m_all_live_num']/(3*factor_63['util_l3m_all_live_den'])
factor_63['util_l6m_all_live']=factor_63['util_l6m_all_live_num']/(6*factor_63['util_l6m_all_live_den'])
factor_63['util_l12m_all_live']=factor_63['util_l12m_all_live_num']/(12*factor_63['util_l12m_all_live_den'])                                       
                                             
factor_63['util_curr_all_tot']=factor_63['util_curr_all_tot_num']/factor_63['util_curr_all_tot_den']
factor_63['util_l3m_all_tot']=factor_63['util_l3m_all_tot_num']/(3*factor_63['util_l3m_all_tot_den'])
factor_63['util_l6m_all_tot']=factor_63['util_l6m_all_tot_num']/(6*factor_63['util_l6m_all_tot_den'])
factor_63['util_l12m_all_tot']=factor_63['util_l12m_all_tot_num']/(12*factor_63['util_l12m_all_tot_den'])


factor_63['avg_util_l3m_pl']=factor_63['avg_util_l3m_pl_num']/factor_63['avg_util_l3m_pl_den']
factor_63['avg_util_l6m_pl']=factor_63['avg_util_l6m_pl_num']/factor_63['avg_util_l6m_pl_den']                                        
factor_63['avg_util_l12m_pl']=factor_63['avg_util_l12m_pl_num']/factor_63['avg_util_l12m_pl_den']
                                             
factor_63['avg_util_l3m_cc']=factor_63['avg_util_l3m_cc_num']/factor_63['avg_util_l3m_cc_den']
factor_63['avg_util_l6m_cc']=factor_63['avg_util_l6m_cc_num']/factor_63['avg_util_l6m_cc_den']                                        
factor_63['avg_util_l12m_cc']=factor_63['avg_util_l12m_cc_num']/factor_63['avg_util_l12m_cc_den']
                                       
factor_63['avg_util_l3m_exc_cc']=factor_63['avg_util_l3m_exc_cc_num']/factor_63['avg_util_l3m_exc_cc_den']
factor_63['avg_util_l6m_exc_cc']=factor_63['avg_util_l6m_exc_cc_num']/factor_63['avg_util_l6m_exc_cc_den']
factor_63['avg_util_l12m_exc_cc']=factor_63['avg_util_l12m_exc_cc_num']/factor_63['avg_util_l12m_exc_cc_den']
                                       
factor_63['util_curr_cc_tot']=factor_63['util_curr_cc_tot_num']/factor_63['util_curr_cc_tot_den']
factor_63['util_l3m_cc_tot']=factor_63['util_l3m_cc_tot_num']/factor_63['util_l3m_cc_tot_den']                                       
factor_63['util_l6m_cc_tot']=factor_63['util_l6m_cc_tot_num']/factor_63['util_l6m_cc_tot_den']
factor_63['util_l12m_cc_tot']=factor_63['util_l12m_cc_tot_num']/factor_63['util_l12m_cc_tot_den']                                       
       
factor_63['util_curr_pl_tot']=factor_63['util_curr_pl_tot_num']/factor_63['util_curr_pl_tot_den']
factor_63['util_l3m_pl_tot']=factor_63['util_l3m_pl_tot_num']/factor_63['util_l3m_pl_tot_den']                                       
factor_63['util_l6m_pl_tot']=factor_63['util_l6m_pl_tot_num']/factor_63['util_l6m_pl_tot_den']
factor_63['util_l12m_pl_tot']=factor_63['util_l12m_pl_tot_num']/factor_63['util_l12m_pl_tot_den']                                                                               
                                             
       
factor_63['util_curr_exc_cc_tot']=factor_63['util_curr_exc_cc_tot_num']/factor_63['util_curr_exc_cc_tot_den']
factor_63['util_l3m_exc_cc_tot']=factor_63['util_l3m_exc_cc_tot_num']/factor_63['util_l3m_exc_cc_tot_den']                                       
factor_63['util_l6m_exc_cc_tot']=factor_63['util_l6m_exc_cc_tot_num']/factor_63['util_l6m_exc_cc_tot_den']
factor_63['util_l12m_exc_cc_tot']=factor_63['util_l12m_exc_cc_tot_num']/factor_63['util_l12m_exc_cc_tot_den']                                                                               







In [112]:
factor_61=factor_61.reset_index()
factor_62=factor_62.reset_index()
factor_63=factor_63.reset_index()

In [113]:
factor_63.shape

(1388, 249)

In [114]:
factor_63=factor_63.drop(['util_curr_all_act_num','util_curr_all_act_den','util_l3m_all_act_num','util_l3m_all_act_den'
      ,'util_l6m_all_act_num','util_l6m_all_act_den','util_l12m_all_act_num','util_l12m_all_act_den'
                                              
      ,'util_curr_all_live_num','util_curr_all_live_den','util_l3m_all_live_num','util_l3m_all_live_den'
      ,'util_l6m_all_live_num','util_l6m_all_live_den','util_l12m_all_live_num','util_l12m_all_live_den'                                       
                                             
      ,'util_curr_all_tot_num','util_curr_all_tot_den','util_l3m_all_tot_num','util_l3m_all_tot_den'
      ,'util_l6m_all_tot_num','util_l6m_all_tot_den','util_l12m_all_tot_num','util_l12m_all_tot_den'                                       
     ,'avg_util_l3m_pl_num','avg_util_l3m_pl_den','avg_util_l6m_pl_num','avg_util_l6m_pl_den'                                        
      ,'avg_util_l12m_pl_num','avg_util_l12m_pl_den'
                                             
      ,'avg_util_l3m_cc_num','avg_util_l3m_cc_den','avg_util_l6m_cc_num','avg_util_l6m_cc_den'                                        
      ,'avg_util_l12m_cc_num','avg_util_l12m_cc_den'
                                       
      ,'avg_util_l3m_exc_cc_num','avg_util_l3m_exc_cc_den','avg_util_l6m_exc_cc_num','avg_util_l6m_exc_cc_den'                                        
      ,'avg_util_l12m_exc_cc_num','avg_util_l12m_exc_cc_den'
                                       
      ,'util_curr_cc_tot_num','util_curr_cc_tot_den','util_l3m_cc_tot_num','util_l3m_cc_tot_den'                                       
      ,'util_l6m_cc_tot_num','util_l6m_cc_tot_den','util_l12m_cc_tot_num','util_l12m_cc_tot_den'                                       
       
      ,'util_curr_pl_tot_num','util_curr_pl_tot_den','util_l3m_pl_tot_num','util_l3m_pl_tot_den'                                       
      ,'util_l6m_pl_tot_num','util_l6m_pl_tot_den','util_l12m_pl_tot_num','util_l12m_pl_tot_den'                                                                               
                                             
       
      ,'util_curr_exc_cc_tot_num','util_curr_exc_cc_tot_den','util_l3m_exc_cc_tot_num','util_l3m_exc_cc_tot_den'                                       
      ,'util_l6m_exc_cc_tot_num','util_l6m_exc_cc_tot_den','util_l12m_exc_cc_tot_num','util_l12m_exc_cc_tot_den'],axis=1)

In [115]:
factor_611 = pd.merge(factor_61,factor_62,how='left',on='cust_id')

In [116]:
factor_611.shape

(1388, 506)

In [117]:
#factor_611.head()

In [118]:
factor_6=pd.merge(factor_611,factor_63,how='left',on='cust_id')

In [119]:
factor_6.shape

(1388, 688)

In [120]:
#factor_6.head(50)

In [121]:
bad_table_temp=copy.deepcopy(bureau_base_0.loc[((bureau_base_0['monthly_dt']>=pd.to_datetime('2019-4-30'))&(bureau_base_0['monthly_dt']<=pd.to_datetime('2020-4-30'))
                                          &(bureau_base_0['balance_dt1']==pd.to_datetime('2020-4-30'))&(bureau_base_0['open_date']<=pd.to_datetime('2019-04-30')))])

target=bad_table_temp.assign(bad_flag=np.where(((bad_table_temp['dpd_new_2']>=90)&(bad_table_temp['account_type_cd']=='189')),1,0)
                                          ).groupby('cust_id').agg({'bad_flag':'max'})

target=target.reset_index()
#target=bureau_base_0.assign(bad_flag=np.where((bureau_base_0['dpd_new_2']>=90) & (bureau_base_0['account_type_cd'].isin(['123']))  & (bureau_base_0['monthly_dt'] >= pd.to_datetime('2019-11-30')) & (bureau_base_0['monthly_dt']<= pd.to_datetime('2020-04-30')) & (bureau_base_0['balance_dt1'] == pd.to_datetime('2020-04-30')) ,1,0)).groupby('cust_id').agg({'bad_flag':'max'})                         


In [122]:
target.bad_flag.value_counts()

0    1197
1     184
Name: bad_flag, dtype: int64

In [131]:
#iv_input=pd.merge(factor_6,target,how='left',on='cust_id')

In [125]:
#iv_input=iv_input.fillna(value=-555555)
#iv_input=iv_input.replace([np.inf,-np.inf],value=-555555)

In [126]:
#iv_input1=copy.deepcopy(iv_input)

In [127]:
#from xverse.transformer import MonotonicBinning

In [128]:
#pd.set_option("display.max_rows",1400)
#q=iv_input['cust_id']

In [129]:
#iv_input1.min_suit_wilful_def_month_36m.value_counts()

In [123]:
target.shape

(1381, 2)

In [124]:
factor_6.shape

(1388, 688)

In [125]:
bureau_vars_1=pd.merge(factor_6,base_4,how='left',on='cust_id')

In [126]:
bureau_vars_1=bureau_vars_1.assign(ratio_amt_uns=np.where(bureau_vars_1['cust_loan_amt_live'] > 0,round(bureau_vars_1['cust_loan_amt_live_unsec']*100/bureau_vars_1['cust_loan_amt_live'],1),
                                                          np.nan),
                                  ratio_amt_sec=np.where(bureau_vars_1['cust_loan_amt_live'] > 0,round(bureau_vars_1['cust_loan_amt_live_sec']*100/bureau_vars_1['cust_loan_amt_live'],1),
                                                         np.nan),
                                 ratio_amt_con=np.where(bureau_vars_1['cust_loan_amt_live'] > 0,round(bureau_vars_1['cust_loan_amt_live_con']*100/bureau_vars_1['cust_loan_amt_live'],1),np.nan),
                                 ratio_amt_com=np.where(bureau_vars_1['cust_loan_amt_live'] > 0,round(bureau_vars_1['cust_loan_amt_live_com']*100/bureau_vars_1['cust_loan_amt_live'],1),np.nan),
                                  
                                ratio_amt_al=np.where(bureau_vars_1['cust_loan_amt_live'] > 0,round(bureau_vars_1['cust_loan_amt_live_al']*100/bureau_vars_1['cust_loan_amt_live'],1),np.nan),
                                ratio_amt_hl=np.where(bureau_vars_1['cust_loan_amt_live'] > 0,round(bureau_vars_1['cust_loan_amt_live_hl']*100/bureau_vars_1['cust_loan_amt_live'],1),np.nan),  
                                ratio_amt_pl=np.where(bureau_vars_1['cust_loan_amt_live'] > 0,round(bureau_vars_1['cust_loan_amt_live_pl']*100/bureau_vars_1['cust_loan_amt_live'],1),np.nan),
                                ratio_amt_tw=np.where(bureau_vars_1['cust_loan_amt_live'] > 0,round(bureau_vars_1['cust_loan_amt_live_tw']*100/bureau_vars_1['cust_loan_amt_live'],1),np.nan),
                                ratio_amt_cc=np.where(bureau_vars_1['cust_loan_amt_live'] > 0,round(bureau_vars_1['cust_loan_amt_live_cc']*100/bureau_vars_1['cust_loan_amt_live'],1),np.nan), 
                                  
                                  
#                                 Customer Clean accounts variables  
                                pct_live_act_bal_cc=np.where(bureau_vars_1['nbr_cc_live_accts'] > 0,bureau_vars_1['nbr_cc_live_active_accts']/bureau_vars_1['nbr_cc_live_accts'],np.nan),
                                pct_live_clean_accts=np.where(bureau_vars_1['nbr_live_accts'].isin([0,np.nan]),np.nan,np.where(bureau_vars_1['nbr_clean_live_accts'].isin([0,np.nan]),0,bureau_vars_1['nbr_clean_live_accts']/bureau_vars_1['nbr_live_accts'])),
                                ratio_avg_util_c0toc1to3=round(100*bureau_vars_1['avg_util_curr_all_act']/bureau_vars_1['avg_util_l3m_all_act'],1),
                                ratio_avg_util_c0toc1to6=round(100*bureau_vars_1['avg_util_curr_all_act']/bureau_vars_1['avg_util_l6m_all_act'],1),
                                ratio_avg_util_c0toc1to12=round(100*bureau_vars_1['avg_util_curr_all_act']/bureau_vars_1['avg_util_l12m_all_act'],1),
                                ratio_avg_util_C1to3toC1to6=round(100*bureau_vars_1['avg_util_l3m_all_act']/bureau_vars_1['avg_util_l6m_all_act'],1),
                                ratio_avg_util_C1to3toC1to12=round(100*bureau_vars_1['avg_util_l3m_all_act']/bureau_vars_1['avg_util_l12m_all_act'],1)) 

In [127]:
bureau_vars_1=bureau_vars_1.drop(['min_write_off_month_life',
'max_write_off_month_life',
'min_settlement_month_life',
'max_settlement_month_life',
'min_restructure_month_life',
'max_restructure_month_life',
'min_suitfiled_month_life',
'max_suitfiled_month_life',
'min_wilful_default_month_life',
'max_wilful_default_month_life',
'min_suit_wilful_def_month_life',
'max_suit_wilful_def_month_life',
'min_write_off_month_12m',
'max_write_off_month_12m',
'min_settlement_month_12m',
'max_settlement_month_12m',
'min_restructure_month_12m',
'max_restructure_month_12m',
'min_suitfiled_month_12m',
'max_suitfiled_month_12m',
'min_wilful_default_month_12m',
'max_wilful_default_month_12m',
'min_suit_wilful_def_month_12m',
'max_suit_wilful_def_month_12m',
'min_write_off_month_24m',
'max_write_off_month_24m',
'min_settlement_month_24m',
'max_settlement_month_24m',
'min_restructure_month_24m',
'max_restructure_month_24m',
'min_suitfiled_month_24m',
'max_suitfiled_month_24m',
'min_wilful_default_month_24m',
'max_wilful_default_month_24m',
'min_suit_wilful_def_month_24m',
'max_suit_wilful_def_month_24m',
'min_write_off_month_36m',
'max_write_off_month_36m',
'min_settlement_month_36m',
'max_settlement_month_36m',
'min_restructure_month_36m',
'max_restructure_month_36m',
'min_suitfiled_month_36m',
'max_suitfiled_month_36m',
'min_wilful_default_month_36m',
'max_wilful_default_month_36m',
'min_suit_wilful_def_month_36m',
'max_suit_wilful_def_month_36m'
],axis=1)


In [128]:
bureau_vars_1.shape

(1388, 658)

In [129]:
iv_input=pd.merge(bureau_vars_1,target,how='left',on='cust_id')
iv_input=iv_input.fillna(value=-555555)
iv_input=iv_input.replace([np.inf,-np.inf],value=-555555)
iv_input

,cust_id,nbr_live_accts,nbr_closed_accts,nbr_tot_accts,nbr_hl_live_accts,nbr_hl_closed_accts,nbr_hl_tot_accts,nbr_pl_live_accts,nbr_pl_closed_accts,nbr_pl_tot_accts,...,ratio_amt_tw,ratio_amt_cc,pct_live_act_bal_cc,pct_live_clean_accts,ratio_avg_util_c0toc1to3,ratio_avg_util_c0toc1to6,ratio_avg_util_c0toc1to12,ratio_avg_util_C1to3toC1to6,ratio_avg_util_C1to3toC1to12,bad_flag
0,1000009,1.0,2.0,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,36.7,1324.5,0.0
1,1000027,9.0,2.0,11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,59.5,1994.9,0.0
2,1000028,2.0,2.0,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,57.5,1050.0,0.0
3,1000033,2.0,8.0,10,0.0,0.0,0.0,0.0,0.0,0.0,...,7.2,0.0,-555555.0,0.0,0.0,0.0,0.0,54.1,950.8,0.0
4,1000037,2.0,2.0,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,20.0,711.4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1010163,13.0,6.0,19,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,51.1,1766.4,0.0
1384,1010176,3.0,2.0,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,40.5,1471.2,0.0
1385,1010180,2.0,7.0,9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,27.4,786.3,0.0
1386,1010183,7.0,3.0,10,0.0,0.0,0.0,2.0,1.0,3.0,...,40.9,0.0,-555555.0,0.0,0.0,0.0,0.0,54.2,1995.7,0.0


In [130]:
missing = []
for i in iv_input.columns:
    num = len(iv_input[(iv_input[i]==-555555)])/len(iv_input)*100
    if num>95:
        #missing.append(num)
        missing.append(i)        

In [131]:
len(missing)

67

In [132]:
missing

['max_live_mob_oth',
 'max_mob_oth',
 'min_mob_oth',
 'max_dpd_0m_all',
 'max_dpd_0m_hl',
 'max_dpd_0m_pl',
 'max_dpd_0m_cc',
 'max_dpd_0m_al',
 'max_dpd_0m_gl',
 'max_dpd_0m_tw',
 'max_dpd_0m_uns',
 'max_dpd_0m_sec',
 'max_dpd_0m_oth',
 'max_dpd_3m_oth',
 'max_dpd_6m_oth',
 'max_dpd_12m_oth',
 'max_dpd_24m_oth',
 'max_dpd_0m_com',
 'max_dpd_0m_con',
 'max_dpd_0m_com_agri',
 'max_dpd_0m_com_oth',
 'max_dpd_0m_act',
 'max_dpd_0m_live',
 'nbr_prod_0_3m_oth',
 'nbr_prod_0_6m_oth',
 'nbr_prod_0_12m_oth',
 'nbr_prod_0_24m_oth',
 'nbr_prod_30_3m_oth',
 'nbr_prod_30_6m_oth',
 'nbr_prod_30_12m_oth',
 'nbr_prod_30_24m_oth',
 'nbr_prod_90_3m_oth',
 'nbr_prod_90_6m_oth',
 'nbr_prod_90_12m_oth',
 'nbr_prod_90_24m_oth',
 'mon_since0_hl',
 'mon_since30_hl',
 'mon_since60_hl',
 'mon_since90_hl',
 'mon_since180_hl',
 'mon_since180_pl',
 'mon_since60_al',
 'mon_since90_al',
 'mon_since180_al',
 'mon_since30_gl',
 'mon_since60_gl',
 'mon_since90_gl',
 'mon_since180_gl',
 'mon_since90_tw',
 'mon_since180

In [133]:
iv_input.drop(missing, axis=1,inplace=True)


In [134]:
iv_input.shape

(1388, 592)

In [144]:
# iv_input = iv_input[iv_input['bad_flag'] >= 0]
# print(iv_input.shape)

In [135]:
iv_input['bad_flag']=np.where(iv_input['bad_flag'] < 0,0,iv_input['bad_flag'])
iv_input.shape

(1388, 592)

In [136]:
iv_input.bad_flag.value_counts()
iv_input['bad_flag'].value_counts()

0.0    1204
1.0     184
Name: bad_flag, dtype: int64

In [137]:
def bad_rate(df):
    br = ((df['bad_flag'].sum())/(df['bad_flag'].count()))*100
    print(br)
bad_rate(iv_input)

13.256484149855908


In [138]:
iv_input

,cust_id,nbr_live_accts,nbr_closed_accts,nbr_tot_accts,nbr_hl_live_accts,nbr_hl_closed_accts,nbr_hl_tot_accts,nbr_pl_live_accts,nbr_pl_closed_accts,nbr_pl_tot_accts,...,ratio_amt_tw,ratio_amt_cc,pct_live_act_bal_cc,pct_live_clean_accts,ratio_avg_util_c0toc1to3,ratio_avg_util_c0toc1to6,ratio_avg_util_c0toc1to12,ratio_avg_util_C1to3toC1to6,ratio_avg_util_C1to3toC1to12,bad_flag
0,1000009,1.0,2.0,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,36.7,1324.5,0.0
1,1000027,9.0,2.0,11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,59.5,1994.9,0.0
2,1000028,2.0,2.0,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,57.5,1050.0,0.0
3,1000033,2.0,8.0,10,0.0,0.0,0.0,0.0,0.0,0.0,...,7.2,0.0,-555555.0,0.0,0.0,0.0,0.0,54.1,950.8,0.0
4,1000037,2.0,2.0,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,20.0,711.4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1010163,13.0,6.0,19,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,51.1,1766.4,0.0
1384,1010176,3.0,2.0,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,40.5,1471.2,0.0
1385,1010180,2.0,7.0,9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-555555.0,0.0,0.0,0.0,0.0,27.4,786.3,0.0
1386,1010183,7.0,3.0,10,0.0,0.0,0.0,2.0,1.0,3.0,...,40.9,0.0,-555555.0,0.0,0.0,0.0,0.0,54.2,1995.7,0.0


In [139]:
def iv_woe(data, target, IDcol, bins=10, show_woe=True):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target,IDcol])]:
        #if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
        if (data[ivars].dtype.kind in 'bifc'):
            #binned_x = pd.qcut(data[ivars], bins,  duplicates='drop', labels=False)
            d0 = pd.DataFrame({'x':data[ivars], 'decile': pd.qcut(data[ivars], bins, duplicates='drop') , 'y': data[target]})
              
        else:
            d0 = pd.DataFrame({'x':data[ivars], 'decile': data[ivars], 'y': data[target]})
        
        #d0['decile'] = np.where(d0['x'].isna()== True, 'missing',d0['decile'])
        d0['decile'] = np.where(d0['x']== -555555, '-555555',d0['decile'])
        d = d0.groupby('decile', as_index= False).agg({"y": ["count", "sum"]})
        d3 = d0.assign(Min=d0.x,Max=d0.x).groupby("decile",as_index=False).agg({"Min":'min',"Max":'max'})
        
        #d = d0.groupby("bucket", as_index=True)
        #r, p = stats.spearmanr(d.mean().x, d.mean().y)
        #n = n - 1
        d.columns = ['decile', 'N', 'bad']
        #d['min'] = d['decile'].min()
        #d['max'] = d['decile'].max()
        d['pop_perc'] = (d['N']/d['N'].sum())*100
        d['bad_perc'] = np.maximum(d['bad'], 0.005) / d['bad'].sum()
        d['good'] = d['N'] - d['bad']
        d['good_perc'] = np.maximum(d['good'], 0.005) / d['good'].sum()
        d['bad_rate'] =  (d['bad']/d['N'])*100
        d['woe'] = np.log(d['good_perc']/d['bad_perc'])
        d['contribution'] = d['good_perc'] - d['bad_perc']
        d['IV'] = d['woe'] * d['contribution']
        d.insert(loc=0, column='Variable', value=ivars)
        d4 = pd.merge(left=d, right=d3,on = 'decile', how='left')
        #print("Information value of " + ivars + " is " + str(round(d4['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d4['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d4], axis=0)
        #woeDF= woeDF[woeDF['Variable'] == str(column)]
        #woeDF.rename(columns= {('x', 'min'): 'min'} , inplace = True)
        #woeDF.rename(columns= {('x', 'max'): 'max'} , inplace = True)
        woeDF.sort_values(by=["Variable","Max"],inplace=True)
        



       #Show WOE Table
        if show_woe == True:
            #d4 = pd.merge(left=d, right=d3,on = 'decile', how='left')
            #d4 = d.join(pd.concat([d3], axis=1, keys=['decile']))
            print(d4)
    return newDF, woeDF

iv, woe = iv_woe(data = iv_input , target ='bad_flag', IDcol='cust_id', bins=10, show_woe = False)
print(iv)
print(woe)

                        Variable        IV
0                 nbr_live_accts  0.023878
0               nbr_closed_accts  0.133289
0                  nbr_tot_accts  0.115931
0              nbr_hl_live_accts  0.000000
0            nbr_hl_closed_accts  0.000000
..                           ...       ...
0       ratio_avg_util_c0toc1to3  0.028320
0       ratio_avg_util_c0toc1to6  0.006422
0      ratio_avg_util_c0toc1to12  0.006422
0    ratio_avg_util_C1to3toC1to6  0.170771
0   ratio_avg_util_C1to3toC1to12  0.177093

[590 rows x 2 columns]
                 Variable                  decile    N   bad   pop_perc  \
10  avg_util_l12m_all_act                 -555555    3   1.0   0.216138   
0   avg_util_l12m_all_act  (-555555.001, 0.00816]  136  12.0   9.798271   
1   avg_util_l12m_all_act       (0.00816, 0.0173]  139  12.0  10.014409   
2   avg_util_l12m_all_act        (0.0173, 0.0244]  139  11.0  10.014409   
3   avg_util_l12m_all_act        (0.0244, 0.0314]  138   8.0   9.942363   
..        

In [140]:
#iv['Variable']
iv_sort = iv.sort_values('IV', ascending=False)


#iv_sort.dtypes
iv_sort = iv_sort.reset_index(drop=True)
iv_sort = iv_sort[(iv_sort['IV']>0.02)]
iv_sort

,Variable,IV
0,target_90,3.521354
1,target_end_90,2.471502
2,mon_since0_all,2.248250
3,mon_since90_uns,2.155416
4,mon_since60_uns,2.085549
...,...,...
449,mon_since30_al,0.022146
450,nbr_prod_90_6m_com_agri,0.020704
451,nbr_prod_0_6m_com_agri,0.020704
452,nbr_prod_30_6m_com_agri,0.020704


In [141]:
len(iv_sort)

454

In [142]:
iv_not_reqd = iv[(iv['IV']<=0.02)]
col_name = (iv_not_reqd['Variable'])
iv_input_new = copy.deepcopy(iv_input)
iv_input_new.drop(col_name, axis=1, inplace=True)
iv1, woe1=iv_woe(data = iv_input_new , target ='bad_flag', IDcol='cust_id', bins=10, show_woe = False)

In [143]:
iv_input_new

,cust_id,nbr_live_accts,nbr_closed_accts,nbr_tot_accts,nbr_pl_live_accts,nbr_pl_tot_accts,nbr_al_live_accts,nbr_cc_live_accts,nbr_cc_tot_accts,nbr_cl_live_accts,...,ratio_amt_al,ratio_amt_hl,ratio_amt_pl,ratio_amt_tw,ratio_amt_cc,pct_live_act_bal_cc,ratio_avg_util_c0toc1to3,ratio_avg_util_C1to3toC1to6,ratio_avg_util_C1to3toC1to12,bad_flag
0,1000009,1.0,2.0,3,0.0,0.0,1.0,0.0,0.0,0.0,...,100.0,0.0,0.0,0.0,0.0,-555555.0,0.0,36.7,1324.5,0.0
1,1000027,9.0,2.0,11,0.0,0.0,1.0,0.0,0.0,4.0,...,0.2,0.0,0.0,0.0,0.0,-555555.0,0.0,59.5,1994.9,0.0
2,1000028,2.0,2.0,4,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,-555555.0,0.0,57.5,1050.0,0.0
3,1000033,2.0,8.0,10,0.0,0.0,1.0,0.0,0.0,0.0,...,92.8,0.0,0.0,7.2,0.0,-555555.0,0.0,54.1,950.8,0.0
4,1000037,2.0,2.0,4,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,-555555.0,0.0,20.0,711.4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1010163,13.0,6.0,19,0.0,0.0,2.0,0.0,0.0,9.0,...,32.4,0.0,0.0,0.0,0.0,-555555.0,0.0,51.1,1766.4,0.0
1384,1010176,3.0,2.0,5,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,-555555.0,0.0,40.5,1471.2,0.0
1385,1010180,2.0,7.0,9,0.0,0.0,1.0,0.0,0.0,1.0,...,83.1,0.0,0.0,0.0,0.0,-555555.0,0.0,27.4,786.3,0.0
1386,1010183,7.0,3.0,10,2.0,3.0,0.0,0.0,0.0,2.0,...,0.0,0.0,44.3,40.9,0.0,-555555.0,0.0,54.2,1995.7,0.0


In [144]:
woe1

,Variable,decile,N,bad,pop_perc,bad_perc,good,good_perc,bad_rate,woe,contribution,IV,Min,Max
10,avg_util_l12m_all_act,-555555,3,1.0,0.216138,0.005435,2.0,0.001661,33.333333,-1.185322,-0.003774,0.004473,-555555.000000,-555555.000000
0,avg_util_l12m_all_act,"(-555555.001, 0.00816]",136,12.0,9.798271,0.065217,124.0,0.102990,8.823529,0.456906,0.037773,0.017259,-0.672283,0.008023
1,avg_util_l12m_all_act,"(0.00816, 0.0173]",139,12.0,10.014409,0.065217,127.0,0.105482,8.633094,0.480812,0.040264,0.019360,0.008220,0.017297
2,avg_util_l12m_all_act,"(0.0173, 0.0244]",139,11.0,10.014409,0.059783,128.0,0.106312,7.913669,0.575666,0.046530,0.026786,0.017356,0.024407
3,avg_util_l12m_all_act,"(0.0244, 0.0314]",138,8.0,9.942363,0.043478,130.0,0.107973,5.797101,0.909624,0.064495,0.058666,0.024431,0.031333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,util_l6m_pl_tot,"(-555555.001, 0.0]",161,13.0,11.599424,0.070652,148.0,0.122924,8.074534,0.553794,0.052271,0.028948,-0.517255,0.000000
1,util_l6m_pl_tot,"(0.0, 0.496]",51,9.0,3.674352,0.048913,42.0,0.034884,17.647059,-0.338024,-0.014029,0.004742,0.011023,0.490589
2,util_l6m_pl_tot,"(0.496, 1.997]",139,14.0,10.014409,0.076087,125.0,0.103821,10.071942,0.310788,0.027734,0.008619,0.496539,1.992029
3,util_l6m_pl_tot,"(1.997, 3.455]",139,11.0,10.014409,0.059783,128.0,0.106312,7.913669,0.575666,0.046530,0.026786,2.000000,3.453688


In [145]:
bin_cut = woe1.groupby('Variable').apply(lambda x: [-np.inf] + list(x.Max.iloc[:-1].values) + [np.inf])
bin_cut['avg_util_l12m_all_act']

[-inf,
 -555555.0,
 0.008022579597141,
 0.017296562917699234,
 0.02440726462373588,
 0.03133278993940062,
 0.03676869917660836,
 0.042490800809098284,
 0.049433265779324866,
 0.05740599661748942,
 0.07103195305497498,
 inf]

In [146]:
woe_val=woe1.groupby('Variable').apply(lambda x:list(x.woe.values))
woe_val['avg_util_l12m_all_act']


[-1.1853216876998354,
 0.45690604755725583,
 0.4808115684108102,
 0.5756661228614658,
 0.9096240405159659,
 0.09233883241725971,
 0.5756661228614658,
 0.15296345423369453,
 -0.31159057024447623,
 -0.5452843324538386,
 -1.075320792485507]

In [147]:
bin_cut

Variable
avg_util_l12m_all_act     [-inf, -555555.0, 0.008022579597141, 0.0172965...
avg_util_l12m_all_live    [-inf, 0.17853333333333332, 0.5903964061672617...
avg_util_l12m_all_tot     [-inf, 0.5, 0.9894036151802201, 1.334855555555...
avg_util_l12m_cc          [-inf, -555555.0, 0.3363695652173913, 2.716408...
avg_util_l12m_exc_cc      [-inf, 0.43333333333333335, 0.882984059156893,...
                                                ...                        
util_l6m_all_live         [-inf, -555555.0, 0.018969159826965384, 0.1456...
util_l6m_all_tot          [-inf, 0.015906100311082565, 0.065670245889556...
util_l6m_cc_tot           [-inf, -555555.0, 0.10666177130502183, 1.64617...
util_l6m_exc_cc_tot       [-inf, 0.06292333563355465, 0.3197728511576726...
util_l6m_pl_tot           [-inf, -555555.0, 0.0, 0.49058852508716083, 1....
Length: 454, dtype: object

In [148]:
df_new_woe = pd.DataFrame()
df_new_woe['cust_id'] = iv_input_new['cust_id']
for i in bin_cut.index:
    df_new_woe[i]=pd.cut(iv_input[i],bin_cut[i],labels=woe_val[i],right=True,ordered=False)
    
df_new_woe

,cust_id,avg_util_l12m_all_act,avg_util_l12m_all_live,avg_util_l12m_all_tot,avg_util_l12m_cc,avg_util_l12m_exc_cc,avg_util_l12m_pl,avg_util_l3m_all_act,avg_util_l3m_all_live,avg_util_l3m_all_tot,...,util_l3m_all_tot,util_l3m_cc_tot,util_l3m_exc_cc_tot,util_l3m_pl_tot,util_l6m_all_act,util_l6m_all_live,util_l6m_all_tot,util_l6m_cc_tot,util_l6m_exc_cc_tot,util_l6m_pl_tot
0,1000009,0.152963,-0.501143,-1.206828,-0.15433,-1.108782,-0.284779,-0.095078,-0.260373,-0.095078,...,-0.095078,-0.143415,-0.152307,-0.283193,-0.455807,0.092339,-0.311591,-0.143415,-0.207337,-0.283193
1,1000027,-0.545284,-0.035416,-0.035416,-0.15433,-0.035416,-0.284779,-0.750869,-0.750869,-0.750869,...,-0.095078,-0.143415,-0.152307,-0.283193,-0.095078,0.092339,-0.311591,-0.143415,-0.207337,-0.283193
2,1000028,-0.311591,-0.095078,0.026950,-0.15433,0.575666,-0.284779,0.233762,-0.260373,0.233762,...,-0.095078,-0.143415,0.392864,-0.283193,-0.285838,-0.215921,-0.207337,-0.143415,-0.361146,-0.283193
3,1000033,0.092339,1.640512,1.050818,-0.15433,0.575666,-0.284779,-0.152307,-0.035416,-0.095078,...,-0.260373,-0.143415,-0.152307,-0.283193,-0.095078,-0.361146,-0.207337,-0.143415,-0.152307,-0.283193
4,1000037,0.575666,0.480812,-0.311591,-0.15433,-1.108782,-0.284779,-0.152307,-0.035416,-0.095078,...,-0.095078,-0.143415,-0.152307,-0.283193,0.026950,-0.035416,-0.152307,-0.143415,-0.035416,-0.283193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1010163,-0.311591,-0.035416,0.152963,-0.15433,0.152963,-0.284779,0.026950,-0.207337,0.026950,...,0.018651,-0.143415,-0.103517,-0.283193,-0.129269,0.092339,-0.152307,-0.143415,0.092339,-0.283193
1384,1010176,0.152963,-0.095078,0.152963,-0.15433,0.152963,-0.284779,-0.160817,0.310788,-0.160817,...,-0.095078,-0.143415,-0.207337,-0.283193,0.392864,-0.215921,0.480812,-0.143415,0.233762,-0.283193
1385,1010180,0.480812,0.392864,0.026950,-0.15433,0.575666,-0.284779,0.392864,0.392864,0.392864,...,-0.095078,-0.143415,-0.152307,-0.283193,0.026950,-0.035416,-0.152307,-0.143415,-0.035416,-0.283193
1386,1010183,0.152963,-0.095078,0.152963,-0.15433,0.152963,0.526974,0.026950,-0.260373,0.026950,...,0.161127,-0.143415,0.392864,0.369722,0.392864,0.092339,0.480812,-0.143415,0.233762,0.553794


In [149]:
for i in bin_cut.index:
    df_new_woe[i]=df_new_woe[i].astype("float")

In [150]:
###Pearson correlation on data

#df_woe = copy.deepcopy(df_new_woe)
df_woe1 = df_new_woe.drop(['cust_id','target_90','target_end_90'], axis=1)  
corrM = df_woe1.corr(method = 'pearson')
#corr_matrix1.to_excel(".xlsx",sheet_name='Pearson_Correlation')
#corrM = df_woe.corr()
corrM

,avg_util_l12m_all_act,avg_util_l12m_all_live,avg_util_l12m_all_tot,avg_util_l12m_cc,avg_util_l12m_exc_cc,avg_util_l12m_pl,avg_util_l3m_all_act,avg_util_l3m_all_live,avg_util_l3m_all_tot,avg_util_l3m_cc,...,util_l3m_all_tot,util_l3m_cc_tot,util_l3m_exc_cc_tot,util_l3m_pl_tot,util_l6m_all_act,util_l6m_all_live,util_l6m_all_tot,util_l6m_cc_tot,util_l6m_exc_cc_tot,util_l6m_pl_tot
avg_util_l12m_all_act,1.000000,0.631615,0.630253,0.169500,0.602393,0.133265,0.518019,0.550127,0.550688,0.150230,...,0.294916,0.148540,0.147426,0.064928,0.502453,0.304788,0.355987,0.159560,0.243313,0.044200
avg_util_l12m_all_live,0.631615,1.000000,0.692063,0.133056,0.630705,0.080201,0.448570,0.516324,0.492067,0.063945,...,0.290202,0.063387,0.165083,0.007316,0.445862,0.307955,0.327913,0.092643,0.250231,-0.029228
avg_util_l12m_all_tot,0.630253,0.692063,1.000000,0.171896,0.775420,0.135480,0.386983,0.363969,0.419444,0.130102,...,0.174687,0.129507,0.063970,0.065472,0.381471,0.203393,0.290764,0.137324,0.183151,0.033658
avg_util_l12m_cc,0.169500,0.133056,0.171896,1.000000,0.018981,0.156444,0.049537,0.058770,0.074462,0.770604,...,-0.058111,0.718325,-0.053259,0.181148,-0.009683,0.018683,-0.004288,0.816987,-0.039887,0.157451
avg_util_l12m_exc_cc,0.602393,0.630705,0.775420,0.018981,1.000000,0.136434,0.416293,0.409757,0.447094,0.059758,...,0.255792,0.061738,0.151659,0.057928,0.416206,0.242703,0.331082,0.040931,0.266352,0.031368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
util_l6m_all_live,0.304788,0.307955,0.203393,0.018683,0.242703,-0.063979,0.273177,0.510476,0.334783,-0.032090,...,0.649138,-0.031105,0.495319,-0.115982,0.491626,1.000000,0.453064,-0.014351,0.444555,-0.114598
util_l6m_all_tot,0.355987,0.327913,0.290764,-0.004288,0.331082,-0.068749,0.260680,0.398526,0.311403,-0.016598,...,0.528906,0.000647,0.365560,-0.115051,0.545385,0.453064,1.000000,-0.010279,0.607602,-0.131708
util_l6m_cc_tot,0.159560,0.092643,0.137324,0.816987,0.040931,0.132367,0.042690,0.042646,0.057706,0.826156,...,-0.075822,0.859665,-0.050723,0.170052,-0.014873,-0.014351,-0.010279,1.000000,-0.053526,0.147644
util_l6m_exc_cc_tot,0.243313,0.250231,0.183151,-0.039887,0.266352,-0.095652,0.171746,0.346115,0.229706,-0.039215,...,0.498749,-0.036882,0.460162,-0.137294,0.452022,0.444555,0.607602,-0.053526,1.000000,-0.123365


In [194]:
'''
corr_val = corrM.abs().unstack().sort_values(kind="quicksort", ascending = False)
corr_val = corr_val.to_frame()
corr_val = corr_val[corr_val[0] != 1]
corr_val = corr_val.drop_duplicates(subset=[0])
corr_val.reset_index(inplace=True)
corr_val
'''

,level_0,level_1,0
0,nbr_prod_0_3m_act,nbr_prod_0_3m_all,9.994777e-01
1,mon_since60_act,mon_since60_all,9.990165e-01
2,nbr_prod_30_3m_all,nbr_prod_30_3m_act,9.989521e-01
3,nbr_prod_90_3m_all,nbr_prod_90_3m_act,9.989392e-01
4,nbr_90_3m_all,nbr_prod_90_3m_act,9.980793e-01
...,...,...,...
68243,util_l12m_all_tot,nbr_prod_90_6m_gl,1.123389e-05
68244,nbr_prod_0_24m_hl,mon_since0_live,9.493733e-06
68245,nbr_cl_live_accts,nbr_prod_30_12m_cc,8.595290e-06
68246,max_mob_pl,max_dpd_3m_al,1.527187e-06


In [151]:
def correlation_iv_sort(corr_mat, old_list,cutoff):
    new_list=['mon_since0_all']
    for i in range(1,len(old_list)):
        flag = 1
        for j in range(len(new_list)):
            corr = corr_mat.loc[old_list[i],new_list[j]]
            if(abs(corr) > (cutoff)):
                flag = 0
                break
        if(flag==1):
            new_list.append(old_list[i])
        i = i+1
    return new_list

iv1.sort_values(by='IV',ascending = False, inplace = True, ignore_index = True)
iv1.drop(index=[0,1],inplace=True)
iv1.reset_index(drop=True,inplace=True)
old_cols = list(iv1['Variable'])
old_cols

['mon_since0_all',
 'mon_since90_uns',
 'mon_since60_uns',
 'mon_since30_uns',
 'max_dpd_3m_uns',
 'max_dpd_6m_uns',
 'mon_since90_con',
 'nbr_prod_0_3m_uns',
 'max_dpd_12m_uns',
 'nbr_prod_30_3m_uns',
 'mon_since90_live',
 'max_dpd_12m_live',
 'nbr_prod_90_3m_uns',
 'mon_since90_act',
 'mon_since0_uns',
 'mon_since90_all',
 'nbr_prod_30_6m_uns',
 'nbr_prod_90_6m_uns',
 'nbr_prod_90_12m_uns',
 'nbr_prod_90_24m_uns',
 'mon_since180_uns',
 'mon_since60_live',
 'max_dpd_3m_live',
 'mon_since60_con',
 'nbr_prod_30_12m_uns',
 'nbr_prod_30_24m_uns',
 'nbr_prod_0_6m_uns',
 'max_dpd_24m_live',
 'nbr_prod_90_3m_con',
 'max_nbr_0_3m_act',
 'max_nbr_0_3m_all',
 'mon_since60_act',
 'mon_since30_live',
 'nbr_prod_90_3m_act',
 'mon_since60_all',
 'nbr_prod_90_3m_all',
 'max_dpd_6m_live',
 'nbr_90_3m_all',
 'nbr_90_3m_act',
 'nbr_prod_0_12m_live',
 'nbr_prod_0_24m_live',
 'nbr_prod_0_3m_live',
 'mon_since30_con',
 'mon_since180_con',
 'nbr_prod_0_12m_uns',
 'nbr_prod_0_24m_uns',
 'mon_since30_act',
 

In [152]:
iv1.shape

(452, 2)

In [153]:
filtered_cols = correlation_iv_sort(corrM,old_cols,0.6)
filtered_cols

['mon_since0_all',
 'mon_since90_uns',
 'nbr_prod_0_6m_act',
 'min_mob_uns',
 'max_live_mob_cc',
 'count_write_off_36m',
 'max_dpd_24m_cc',
 'avg_util_l12m_all_live',
 'max_dpd_24m_pl',
 'min_mob_pl',
 'nbr_accts_open_l6m_con',
 'max_live_mob_uns',
 'nbr_accts_open_l12m',
 'ratio_amt_uns',
 'cust_loan_amt_live_unsec',
 'nbr_prod_0_3m_sec',
 'cust_loan_amt_live_con',
 'mon_since90_sec',
 'max_dpd_6m_tw',
 'avg_util_l3m_all_act',
 'nbr_closed_active_accts',
 'mon_since60_tw',
 'avg_util_l6m_cc',
 'nbr_prod_90_24m_sec',
 'nbr_accts_open_l3m',
 'nbr_accts_open_l6m_cc',
 'ratio_amt_pl',
 'max_length_hist',
 'ratio_avg_util_C1to3toC1to12',
 'nbr_tot_active_accts',
 'util_l12m_all_live',
 'nbr_prod_30_3m_hl',
 'cust_loan_amt_active',
 'max_mob_com_agri',
 'nbr_uns_tot_accts',
 'util_l3m_all_live',
 'mon_since180_cc',
 'nbr_cl_closed_accts',
 'ratio_amt_sec',
 'mon_since90_pl',
 'nbr_prod_0_24m_al',
 'max_live_mob_sec',
 'ratio_amt_cc',
 'util_l12m_all_act',
 'util_l6m_all_act',
 'max_dpd_24m_

In [154]:
len(filtered_cols)

80

In [155]:
filtered_cols.sort()
filtered_cols

['avg_util_l12m_all_live',
 'avg_util_l3m_all_act',
 'avg_util_l6m_cc',
 'count_write_off_36m',
 'cust_loan_amt_active',
 'cust_loan_amt_live_con',
 'cust_loan_amt_live_sec',
 'cust_loan_amt_live_unsec',
 'max_dpd_24m_cc',
 'max_dpd_24m_com_agri',
 'max_dpd_24m_pl',
 'max_dpd_6m_tw',
 'max_length_hist',
 'max_live_mob_cc',
 'max_live_mob_sec',
 'max_live_mob_uns',
 'max_mob_all',
 'max_mob_com_agri',
 'max_mob_com_oth',
 'max_mob_con',
 'max_mob_sec',
 'max_mob_tw',
 'max_mob_uns',
 'min_mob_pl',
 'min_mob_tw',
 'min_mob_uns',
 'mon_since0_all',
 'mon_since180_cc',
 'mon_since30_al',
 'mon_since60_tw',
 'mon_since90_pl',
 'mon_since90_sec',
 'mon_since90_uns',
 'nbr_accts_open_l12m',
 'nbr_accts_open_l12m_pl',
 'nbr_accts_open_l3m',
 'nbr_accts_open_l3m_pl',
 'nbr_accts_open_l6m_cc',
 'nbr_accts_open_l6m_con',
 'nbr_accts_open_l6m_sec',
 'nbr_al_live_accts',
 'nbr_cc_live_active_accts',
 'nbr_cc_tot_accts',
 'nbr_cl_closed_accts',
 'nbr_cl_live_accts',
 'nbr_cl_tot_accts',
 'nbr_closed

In [156]:
iv_input_reqd = iv_input.loc[:,['avg_util_l12m_all_live',
 'avg_util_l3m_all_act',
 'avg_util_l6m_cc',
 'count_write_off_36m',
 'cust_loan_amt_active',
 'cust_loan_amt_live_con',
 'cust_loan_amt_live_sec',
 'cust_loan_amt_live_unsec',
 'max_dpd_24m_cc',
 'max_dpd_24m_com_agri',
 'max_dpd_24m_pl',
 'max_dpd_6m_tw',
 'max_length_hist',
 'max_live_mob_cc',
 'max_live_mob_sec',
 'max_live_mob_uns',
 'max_mob_all',
 'max_mob_com_agri',
 'max_mob_com_oth',
 'max_mob_con',
 'max_mob_sec',
 'max_mob_tw',
 'max_mob_uns',
 'min_mob_pl',
 'min_mob_tw',
 'min_mob_uns',
 'mon_since0_all',
 'mon_since180_cc',
 'mon_since30_al',
 'mon_since60_tw',
 'mon_since90_pl',
 'mon_since90_sec',
 'mon_since90_uns',
 'nbr_accts_open_l12m',
 'nbr_accts_open_l12m_pl',
 'nbr_accts_open_l3m',
 'nbr_accts_open_l3m_pl',
 'nbr_accts_open_l6m_cc',
 'nbr_accts_open_l6m_con',
 'nbr_accts_open_l6m_sec',
 'nbr_al_live_accts',
 'nbr_cc_live_active_accts',
 'nbr_cc_tot_accts',
 'nbr_cl_closed_accts',
 'nbr_cl_live_accts',
 'nbr_cl_tot_accts',
 'nbr_closed_active_accts',
 'nbr_com_agri_tot_accts',
 'nbr_con_live_accts',
 'nbr_pl_live_accts',
 'nbr_pl_tot_accts',
 'nbr_prod_0_24m_al',
 'nbr_prod_0_3m_sec',
 'nbr_prod_0_6m_act',
 'nbr_prod_30_3m_hl',
 'nbr_prod_30_6m_com',
 'nbr_prod_30_6m_com_oth',
 'nbr_prod_90_12m_gl',
 'nbr_prod_90_24m_sec',
 'nbr_sec_tot_accts',
 'nbr_tot_active_accts',
 'nbr_uns_live_accts',
 'nbr_uns_tot_accts',
 'ratio_amt_al',
 'ratio_amt_cc',
 'ratio_amt_com',
 'ratio_amt_con',
 'ratio_amt_pl',
 'ratio_amt_sec',
 'ratio_amt_tw',
 'ratio_amt_uns',
 'ratio_avg_util_C1to3toC1to12',
 'util_l12m_all_act',
 'util_l12m_all_live',
 'util_l12m_all_tot',
 'util_l12m_exc_cc_tot',
 'util_l3m_all_live',
 'util_l3m_exc_cc_tot',
 'util_l6m_all_act',
 'util_l6m_all_tot','bad_flag', 'cust_id']]
iv_input_reqd


,avg_util_l12m_all_live,avg_util_l3m_all_act,avg_util_l6m_cc,count_write_off_36m,cust_loan_amt_active,cust_loan_amt_live_con,cust_loan_amt_live_sec,cust_loan_amt_live_unsec,max_dpd_24m_cc,max_dpd_24m_com_agri,...,util_l12m_all_act,util_l12m_all_live,util_l12m_all_tot,util_l12m_exc_cc_tot,util_l3m_all_live,util_l3m_exc_cc_tot,util_l6m_all_act,util_l6m_all_tot,bad_flag,cust_id
0,3.336302,0.576196,-555555.0,0,569750.0,525000.0,525000.0,0.0,-555555.0,-555555.0,...,0.772653,0.834076,0.772653,9.271832,0.576196,1.592820,0.677556,0.677556,0.0,1000009
1,2.460591,1.269778,-555555.0,0,1225820.0,390791.0,1003401.0,80579.0,-555555.0,0.0,...,0.391342,0.393450,0.378722,4.544662,0.583878,1.644344,0.623249,0.603150,0.0,1000027
2,1.655150,0.547755,-555555.0,0,329757.0,329757.0,104757.0,225000.0,-555555.0,-555555.0,...,0.317395,0.317395,0.203043,2.436515,0.378515,0.726427,0.315005,0.201514,0.0,1000028
3,0.722391,0.306929,-555555.0,0,963600.0,806800.0,806800.0,0.0,-555555.0,-555555.0,...,0.322495,0.385171,0.265241,3.182887,0.876258,1.810254,0.629438,0.517691,0.0,1000033
4,1.385173,0.276863,-555555.0,1,267733.0,241067.0,221002.0,20065.0,-555555.0,-555555.0,...,0.063117,0.038431,0.063117,0.757400,0.030726,0.082997,0.050116,0.050116,1.0,1000037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,2.279840,0.950130,-555555.0,0,989051.0,359764.0,181427.0,174738.0,-555555.0,29.0,...,0.252561,0.446129,0.240407,2.884888,0.600779,0.971233,0.381080,0.362742,0.0,1010163
1384,1.559695,0.679852,-555555.0,0,60519.0,41571.0,0.0,41571.0,-555555.0,-555555.0,...,0.237045,0.265925,0.237045,2.844545,0.255346,0.624498,0.262551,0.262551,0.0,1010176
1385,0.504780,0.090818,-555555.0,0,230996.0,26500.0,130000.0,26500.0,-555555.0,-555555.0,...,0.084841,0.064105,0.074774,0.897285,0.046134,0.082642,0.043064,0.037954,0.0,1010180
1386,1.821387,0.864098,-555555.0,0,269551.0,258618.0,105780.0,150989.0,-555555.0,-555555.0,...,0.202963,0.208217,0.202963,2.435554,0.284825,0.834428,0.264580,0.264580,0.0,1010183


In [157]:
iv2, woe2 = iv_woe(data = iv_input_reqd , target ='bad_flag', IDcol='cust_id', bins=10, show_woe = False)


In [158]:
woe2

,Variable,decile,N,bad,pop_perc,bad_perc,good,good_perc,bad_rate,woe,contribution,IV,Min,Max
0,avg_util_l12m_all_live,"(-28.439, 0.184]",139,10.0,10.014409,0.054348,129.0,0.107143,7.194245,0.678758,0.052795,0.035835,-28.437786,0.178533
1,avg_util_l12m_all_live,"(0.184, 0.591]",139,13.0,10.014409,0.070652,126.0,0.104651,9.352518,0.392864,0.033999,0.013357,0.185714,0.590396
2,avg_util_l12m_all_live,"(0.591, 0.858]",139,4.0,10.014409,0.021739,135.0,0.112126,2.877698,1.640512,0.090387,0.148281,0.591749,0.857872
3,avg_util_l12m_all_live,"(0.858, 1.17]",138,13.0,9.942363,0.070652,125.0,0.103821,9.420290,0.384896,0.033168,0.012766,0.861056,1.166667
4,avg_util_l12m_all_live,"(1.17, 1.496]",139,12.0,10.014409,0.065217,127.0,0.105482,8.633094,0.480812,0.040264,0.019360,1.170520,1.493539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,util_l6m_all_tot,"(0.238, 0.294]",139,12.0,10.014409,0.065217,127.0,0.105482,8.633094,0.480812,0.040264,0.019360,0.238470,0.294351
6,util_l6m_all_tot,"(0.294, 0.361]",138,21.0,9.942363,0.114130,117.0,0.097176,15.217391,-0.160817,-0.016954,0.002727,0.294611,0.360494
7,util_l6m_all_tot,"(0.361, 0.455]",139,21.0,10.014409,0.114130,118.0,0.098007,15.107914,-0.152307,-0.016124,0.002456,0.360577,0.455378
8,util_l6m_all_tot,"(0.455, 0.593]",139,22.0,10.014409,0.119565,117.0,0.097176,15.827338,-0.207337,-0.022389,0.004642,0.455528,0.592760


In [212]:
#dict = {'Reqd_Vars': filtered_cols}  
#df = pd.DataFrame(dict) 
woe2.to_csv('Reqd_Vars.csv') 

In [399]:
'''
corr_val2 = pd.DataFrame(corr_val.index.values)
corr_val2
corr_val2[0] = corr_val2[0].astype(str)
corr_val3 =corr_val2[0].str.split(',', expand=True)
corr_val3

'''

,0,1
0,('nbr_live_accts','nbr_live_accts')
1,('nbr_prod_0_3m_pl','nbr_prod_0_3m_pl')
2,('nbr_prod_0_24m_hl','nbr_prod_0_12m_hl')
3,('nbr_prod_0_12m_hl','nbr_prod_0_24m_hl')
4,('nbr_prod_0_12m_hl','nbr_prod_0_12m_hl')
...,...,...
345739,('ratio_avg_util_C1to3toC1to12','avg_util_curr_all_tot')
345740,('ratio_avg_util_C1to3toC1to12','cust_loan_amt_live_oth')
345741,('ratio_avg_util_C1to3toC1to12','util_curr_all_act')
345742,('ratio_avg_util_C1to3toC1to12','util_curr_all_tot')


In [ ]:
'''
#1-90
none

#91-203
month_since90_pl
month_since60_tw
month_since30_al
min_mob_tw

#203-254
nbr al live acts
Nbr acts open l6m sec
Nbr acts open l6m cc

#Last
util_l3m_exc_cc_tot
ratio_amt_com 
nbr_uns_live_accts
nbr_sec_tot_accts
nbr_prod_90_12m_gl
'''

In [161]:
bureau_factor_7 = copy.deepcopy(iv_input_reqd)

In [170]:
bureau_factor_7['b_max_live_mob_sec']=np.select([(bureau_factor_7['max_live_mob_sec']< 0) ,
( 0 <= bureau_factor_7['max_live_mob_sec']) & (bureau_factor_7['max_live_mob_sec']< 3) ,
( 3 <= bureau_factor_7['max_live_mob_sec']) & (bureau_factor_7['max_live_mob_sec']< 31) ,
( 31 <= bureau_factor_7['max_live_mob_sec'])],['1. Less than 0','2. GE  0 and LT 3','3. GE  3 and LT 31','4. GT31'])

bureau_factor_7['w_max_live_mob_sec']=np.select([(bureau_factor_7['max_live_mob_sec']< 0) ,
( 0 <= bureau_factor_7['max_live_mob_sec']) & (bureau_factor_7['max_live_mob_sec']< 3) ,
( 3 <= bureau_factor_7['max_live_mob_sec']) & (bureau_factor_7['max_live_mob_sec']< 31) ,
( 31 <= bureau_factor_7['max_live_mob_sec'])],[-0.137852757141913,1.99273214264811,0.206392740889394,-0.0950776487022423])

bureau_factor_7['b_max_live_mob_cc']=np.select([(bureau_factor_7['max_live_mob_cc']< 0) ,
( 0 <= bureau_factor_7['max_live_mob_cc']) & (bureau_factor_7['max_live_mob_cc']< 19) ,
( 19 <= bureau_factor_7['max_live_mob_cc']) & (bureau_factor_7['max_live_mob_cc']< 41) ,
( 41 <= bureau_factor_7['max_live_mob_cc'])],['1. Less than 0','2. GE  0 and LT 19','3. GE  19 and LT 41','4. GT41'])

bureau_factor_7['w_max_live_mob_cc']=np.select([(bureau_factor_7['max_live_mob_cc']< 0) ,
( 0 <= bureau_factor_7['max_live_mob_cc']) & (bureau_factor_7['max_live_mob_cc']< 19) ,
( 19 <= bureau_factor_7['max_live_mob_cc']) & (bureau_factor_7['max_live_mob_cc']< 41) ,
( 41 <= bureau_factor_7['max_live_mob_cc'])],[-0.176321557721413,1.20242121363452,0.46493821925452,-0.26903095582568])

bureau_factor_7['b_max_length_hist']=np.select([(bureau_factor_7['max_length_hist']< 37) ,
( 37 <= bureau_factor_7['max_length_hist'])],['1. Less than 37','2. GT37'])

bureau_factor_7['w_max_length_hist']=np.select([(bureau_factor_7['max_length_hist']< 37) ,
( 37 <= bureau_factor_7['max_length_hist'])],[-0.6932421067549,0.267770154343571])

bureau_factor_7['b_max_dpd_6m_tw']=np.select([(bureau_factor_7['max_dpd_6m_tw']< 0) ,
( 0 <= bureau_factor_7['max_dpd_6m_tw']) & (bureau_factor_7['max_dpd_6m_tw']< 29) ,
( 29 <= bureau_factor_7['max_dpd_6m_tw'])],['1. Less than 0','2. GE  0 and LT 29','3. GT29'])

bureau_factor_7['w_max_dpd_6m_tw']=np.select([(bureau_factor_7['max_dpd_6m_tw']< 0) ,
( 0 <= bureau_factor_7['max_dpd_6m_tw']) & (bureau_factor_7['max_dpd_6m_tw']< 29) ,
( 29 <= bureau_factor_7['max_dpd_6m_tw'])],[0.0564856021345575,0.381556610315469,-1.54199663163857])

bureau_factor_7['b_max_dpd_24m_pl']=np.select([(bureau_factor_7['max_dpd_24m_pl']< 0) ,
( 0 <= bureau_factor_7['max_dpd_24m_pl']) & (bureau_factor_7['max_dpd_24m_pl']< 29) ,
( 29 <= bureau_factor_7['max_dpd_24m_pl']) & (bureau_factor_7['max_dpd_24m_pl']< 30) ,
( 30 <= bureau_factor_7['max_dpd_24m_pl'])],['1. Less than 0','2. GE  0 and LT 29','3. GE  29 and LT 30','4. GT30'])

bureau_factor_7['w_max_dpd_24m_pl']=np.select([(bureau_factor_7['max_dpd_24m_pl']< 0) ,
( 0 <= bureau_factor_7['max_dpd_24m_pl']) & (bureau_factor_7['max_dpd_24m_pl']< 29) ,
( 29 <= bureau_factor_7['max_dpd_24m_pl']) & (bureau_factor_7['max_dpd_24m_pl']< 30) ,
( 30 <= bureau_factor_7['max_dpd_24m_pl'])],[-0.284779312793819,1.10413411385242,0.107446615409232,-0.902458901684003])

bureau_factor_7['b_max_dpd_24m_com_agri']=np.select([(bureau_factor_7['max_dpd_24m_com_agri']< 0) ,
( 0 <= bureau_factor_7['max_dpd_24m_com_agri']) & (bureau_factor_7['max_dpd_24m_com_agri']< 29) ,
( 29 <= bureau_factor_7['max_dpd_24m_com_agri'])],['1. Less than 0','2. GE  0 and LT 29','3. GT29'])

bureau_factor_7['w_max_dpd_24m_com_agri']=np.select([(bureau_factor_7['max_dpd_24m_com_agri']< 0) ,
( 0 <= bureau_factor_7['max_dpd_24m_com_agri']) & (bureau_factor_7['max_dpd_24m_com_agri']< 29) ,
( 29 <= bureau_factor_7['max_dpd_24m_com_agri'])],[-0.0743341816040721,1.3986758647324,0.0433437292164722])

bureau_factor_7['b_max_dpd_24m_cc']=np.select([(bureau_factor_7['max_dpd_24m_cc']< 0) ,
( 0 <= bureau_factor_7['max_dpd_24m_cc']) & (bureau_factor_7['max_dpd_24m_cc']< 29) ,
( 29 <= bureau_factor_7['max_dpd_24m_cc'])],['1. Less than 0','2. GE  0 and LT 29','3. GT29'])

bureau_factor_7['w_max_dpd_24m_cc']=np.select([(bureau_factor_7['max_dpd_24m_cc']< 0) ,
( 0 <= bureau_factor_7['max_dpd_24m_cc']) & (bureau_factor_7['max_dpd_24m_cc']< 29) ,
( 29 <= bureau_factor_7['max_dpd_24m_cc'])],[-0.154329586706705,1.9752185601554,-0.49217450713989])

bureau_factor_7['b_cust_loan_amt_live_unsec']=np.select([(bureau_factor_7['cust_loan_amt_live_unsec']< 100980) ,
( 100980 <= bureau_factor_7['cust_loan_amt_live_unsec']) & (bureau_factor_7['cust_loan_amt_live_unsec']< 413440) ,
( 413440 <= bureau_factor_7['cust_loan_amt_live_unsec'])],['1. Less than 100980','2. GE  100980 and LT 413440','3. GT413440'])

bureau_factor_7['w_cust_loan_amt_live_unsec']=np.select([(bureau_factor_7['cust_loan_amt_live_unsec']< 100980) ,
( 100980 <= bureau_factor_7['cust_loan_amt_live_unsec']) & (bureau_factor_7['cust_loan_amt_live_unsec']< 413440) ,
( 413440 <= bureau_factor_7['cust_loan_amt_live_unsec'])],[-0.18150291646341,0.206392740889394,0.791841004859582])

bureau_factor_7['b_cust_loan_amt_live_sec']=np.select([(bureau_factor_7['cust_loan_amt_live_sec']< 80000) ,
( 80000 <= bureau_factor_7['cust_loan_amt_live_sec']) & (bureau_factor_7['cust_loan_amt_live_sec']< 1115520) ,
( 1115520 <= bureau_factor_7['cust_loan_amt_live_sec'])],['1. Less than 80000','2. GE  80000 and LT 1115520','3. GT1115520'])

bureau_factor_7['w_cust_loan_amt_live_sec']=np.select([(bureau_factor_7['cust_loan_amt_live_sec']< 80000) ,
( 80000 <= bureau_factor_7['cust_loan_amt_live_sec']) & (bureau_factor_7['cust_loan_amt_live_sec']< 1115520) ,
( 1115520 <= bureau_factor_7['cust_loan_amt_live_sec'])],[-0.154730925129045,0.310787539427262,0.791841004859582])

bureau_factor_7['b_cust_loan_amt_live_con']=np.select([(bureau_factor_7['cust_loan_amt_live_con']< 82999) ,
( 82999 <= bureau_factor_7['cust_loan_amt_live_con']) & (bureau_factor_7['cust_loan_amt_live_con']< 607686) ,
( 607686 <= bureau_factor_7['cust_loan_amt_live_con'])],['1. Less than 82999','2. GE  82999 and LT 607686','3. GT607686'])

bureau_factor_7['w_cust_loan_amt_live_con']=np.select([(bureau_factor_7['cust_loan_amt_live_con']< 82999) ,
( 82999 <= bureau_factor_7['cust_loan_amt_live_con']) & (bureau_factor_7['cust_loan_amt_live_con']< 607686) ,
( 607686 <= bureau_factor_7['cust_loan_amt_live_con'])],[-0.195908690996718,-0.00666669135818935,0.527300460969496])

bureau_factor_7['b_cust_loan_amt_active']=np.select([(bureau_factor_7['cust_loan_amt_active']< 72645) ,
( 72645 <= bureau_factor_7['cust_loan_amt_active']) & (bureau_factor_7['cust_loan_amt_active']< 181762) ,
( 181762 <= bureau_factor_7['cust_loan_amt_active']) & (bureau_factor_7['cust_loan_amt_active']< 435015) ,
( 435015 <= bureau_factor_7['cust_loan_amt_active'])],['1. Less than 72645','2. GE  72645 and LT 181762','3. GE  181762 and LT 435015','4. GT435015'])

bureau_factor_7['w_cust_loan_amt_active']=np.select([(bureau_factor_7['cust_loan_amt_active']< 72645) ,
( 72645 <= bureau_factor_7['cust_loan_amt_active']) & (bureau_factor_7['cust_loan_amt_active']< 181762) ,
( 181762 <= bureau_factor_7['cust_loan_amt_active']) & (bureau_factor_7['cust_loan_amt_active']< 435015) ,
( 435015 <= bureau_factor_7['cust_loan_amt_active'])],[-0.385344483502704,-0.264692554227082,-0.0354161046441752,0.456247502917058])

bureau_factor_7['b_count_write_off_36m']=np.select([(bureau_factor_7['count_write_off_36m']< 2) ,
( 2 <= bureau_factor_7['count_write_off_36m'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_count_write_off_36m']=np.select([(bureau_factor_7['count_write_off_36m']< 2) ,
( 2 <= bureau_factor_7['count_write_off_36m'])],[0.238384604959768,-2.14316142248686])

bureau_factor_7['b_avg_util_l6m_cc']=np.select([(bureau_factor_7['avg_util_l6m_cc']< -0.596682157773987) ,
( -0.596682157773987 <= bureau_factor_7['avg_util_l6m_cc']) & (bureau_factor_7['avg_util_l6m_cc']< 3.32350692129564) ,
( 3.32350692129564 <= bureau_factor_7['avg_util_l6m_cc'])],['1. Less than -0.6','2. GE  -0.6 and LT 3.32','3. GT3.32'])

bureau_factor_7['w_avg_util_l6m_cc']=np.select([(bureau_factor_7['avg_util_l6m_cc']< -0.596682157773987) ,
( -0.596682157773987 <= bureau_factor_7['avg_util_l6m_cc']) & (bureau_factor_7['avg_util_l6m_cc']< 3.32350692129564) ,
( 3.32350692129564 <= bureau_factor_7['avg_util_l6m_cc'])],[-0.154329586706705,0.80255266045451,-0.545284332453839])

bureau_factor_7['b_avg_util_l3m_all_act']=np.select([(bureau_factor_7['avg_util_l3m_all_act']< -8.46641249222535) ,
( -8.46641249222535 <= bureau_factor_7['avg_util_l3m_all_act']) & (bureau_factor_7['avg_util_l3m_all_act']< 0.473328166528978) ,
( 0.473328166528978 <= bureau_factor_7['avg_util_l3m_all_act']) & (bureau_factor_7['avg_util_l3m_all_act']< 1.25221813617) ,
( 1.25221813617 <= bureau_factor_7['avg_util_l3m_all_act'])],['1. Less than -8.47','2. GE  -8.47 and LT 0.47','3. GE  0.47 and LT 1.25','4. GT1.25'])

bureau_factor_7['w_avg_util_l3m_all_act']=np.select([(bureau_factor_7['avg_util_l3m_all_act']< -8.46641249222535) ,
( -8.46641249222535 <= bureau_factor_7['avg_util_l3m_all_act']) & (bureau_factor_7['avg_util_l3m_all_act']< 0.473328166528978) ,
( 0.473328166528978 <= bureau_factor_7['avg_util_l3m_all_act']) & (bureau_factor_7['avg_util_l3m_all_act']< 1.25221813617) ,
( 1.25221813617 <= bureau_factor_7['avg_util_l3m_all_act'])],[-1.18532168769984,0.406311600100758,-0.0731956798650029,-0.750869042718419])

bureau_factor_7['b_avg_util_l12m_all_live']=np.select([(bureau_factor_7['avg_util_l12m_all_live']< 0.185714285714286) ,
( 0.185714285714286 <= bureau_factor_7['avg_util_l12m_all_live']) & (bureau_factor_7['avg_util_l12m_all_live']< 1.49837525970849) ,
( 1.49837525970849 <= bureau_factor_7['avg_util_l12m_all_live']) & (bureau_factor_7['avg_util_l12m_all_live']< 2.72186824835309) ,
( 2.72186824835309 <= bureau_factor_7['avg_util_l12m_all_live'])],['1. Less than 0.19','2. GE  0.19 and LT 1.5','3. GE  1.5 and LT 2.72','4. GT2.72'])

bureau_factor_7['w_avg_util_l12m_all_live']=np.select([(bureau_factor_7['avg_util_l12m_all_live']< 0.185714285714286) ,
( 0.185714285714286 <= bureau_factor_7['avg_util_l12m_all_live']) & (bureau_factor_7['avg_util_l12m_all_live']< 1.49837525970849) ,
( 1.49837525970849 <= bureau_factor_7['avg_util_l12m_all_live']) & (bureau_factor_7['avg_util_l12m_all_live']< 2.72186824835309) ,
( 2.72186824835309 <= bureau_factor_7['avg_util_l12m_all_live'])],[0.678758443107846,0.624137358627621,-0.136306340171098,-0.789426030607822])



########################



bureau_factor_7['b_nbr_accts_open_l12m_pl']=np.select([(bureau_factor_7['nbr_accts_open_l12m_pl']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l12m_pl'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_accts_open_l12m_pl']=np.select([(bureau_factor_7['nbr_accts_open_l12m_pl']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l12m_pl'])],[-0.0691136371413461,0.560975407451462])

bureau_factor_7['b_nbr_accts_open_l12m']=np.select([(bureau_factor_7['nbr_accts_open_l12m']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l12m'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_accts_open_l12m']=np.select([(bureau_factor_7['nbr_accts_open_l12m']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l12m'])],[-0.683628547497117,0.455386867622086])

bureau_factor_7['b_mon_since90_uns']=np.select([(bureau_factor_7['mon_since90_uns']< 0) ,
( 0 <= bureau_factor_7['mon_since90_uns']) & (bureau_factor_7['mon_since90_uns']< 2) ,
( 2 <= bureau_factor_7['mon_since90_uns'])],['1. Less than 0','2. GE  0 and LT 2','3. GT2'])

bureau_factor_7['w_mon_since90_uns']=np.select([(bureau_factor_7['mon_since90_uns']< 0) ,
( 0 <= bureau_factor_7['mon_since90_uns']) & (bureau_factor_7['mon_since90_uns']< 2) ,
( 2 <= bureau_factor_7['mon_since90_uns'])],[1.4999213770859,-1.78690167473429,0.585384372330387])

bureau_factor_7['b_mon_since90_sec']=np.select([(bureau_factor_7['mon_since90_sec']< 0) ,
( 0 <= bureau_factor_7['mon_since90_sec']) & (bureau_factor_7['mon_since90_sec']< 2) ,
( 2 <= bureau_factor_7['mon_since90_sec'])],['1. Less than 0','2. GE  0 and LT 2','3. GT2'])

bureau_factor_7['w_mon_since90_sec']=np.select([(bureau_factor_7['mon_since90_sec']< 0) ,
( 0 <= bureau_factor_7['mon_since90_sec']) & (bureau_factor_7['mon_since90_sec']< 2) ,
( 2 <= bureau_factor_7['mon_since90_sec'])],[0.213758235717027,-1.41310561857055,-0.845453862077484])

bureau_factor_7['b_mon_since180_cc']=np.select([(bureau_factor_7['mon_since180_cc']< 0) ,
( 0 <= bureau_factor_7['mon_since180_cc']) & (bureau_factor_7['mon_since180_cc']< 2) ,
( 2 <= bureau_factor_7['mon_since180_cc'])],['1. Less than 0','2. GE  0 and LT 2','3. GT2'])

bureau_factor_7['w_mon_since180_cc']=np.select([(bureau_factor_7['mon_since180_cc']< 0) ,
( 0 <= bureau_factor_7['mon_since180_cc']) & (bureau_factor_7['mon_since180_cc']< 2) ,
( 2 <= bureau_factor_7['mon_since180_cc'])],[0.143307031570246,-0.927492578397735,-0.443384342970458])

bureau_factor_7['b_mon_since0_all']=np.select([(bureau_factor_7['mon_since0_all']< 0) ,
( 0 <= bureau_factor_7['mon_since0_all']) & (bureau_factor_7['mon_since0_all']< 2) ,
( 2 <= bureau_factor_7['mon_since0_all'])],['1. Less than 0','2. GE  0 and LT 2','3. GT2'])

bureau_factor_7['w_mon_since0_all']=np.select([(bureau_factor_7['mon_since0_all']< 0) ,
( 0 <= bureau_factor_7['mon_since0_all']) & (bureau_factor_7['mon_since0_all']< 2) ,
( 2 <= bureau_factor_7['mon_since0_all'])],[1.99462429080015,-0.97328250711986,1.42474810504217])

bureau_factor_7['b_min_mob_uns']=np.select([(bureau_factor_7['min_mob_uns']< 0) ,
( 0 <= bureau_factor_7['min_mob_uns']) & (bureau_factor_7['min_mob_uns']< 3) ,
( 3 <= bureau_factor_7['min_mob_uns']) & (bureau_factor_7['min_mob_uns']< 24) ,
( 24 <= bureau_factor_7['min_mob_uns'])],['1. Less than 0','2. GE  0 and LT 3','3. GE  3 and LT 24','4. GT24'])

#bureau_factor_7['w_min_mob_uns']=np.select([(bureau_factor_7['min_mob_uns']< 0) ,
#( 0 <= bureau_factor_7['min_mob_uns']) & (bureau_factor_7['min_mob_uns']< 3) ,
#( 3 <= bureau_factor_7['min_mob_uns']) & (bureau_factor_7['min_mob_uns']< 24) ,
#( 24 <= bureau_factor_7['min_mob_uns'])],[,0.945791864524322,-0.0814043468020326,-1.56481130940474])

bureau_factor_7['b_min_mob_pl']=np.select([(bureau_factor_7['min_mob_pl']< 0) ,
( 0 <= bureau_factor_7['min_mob_pl']) & (bureau_factor_7['min_mob_pl']< 4) ,
( 4 <= bureau_factor_7['min_mob_pl']) & (bureau_factor_7['min_mob_pl']< 19) ,
( 19 <= bureau_factor_7['min_mob_pl'])],['1. Less than 0','2. GE  0 and LT 4','3. GE  4 and LT 19','4. GT19'])

bureau_factor_7['w_min_mob_pl']=np.select([(bureau_factor_7['min_mob_pl']< 0) ,
( 0 <= bureau_factor_7['min_mob_pl']) & (bureau_factor_7['min_mob_pl']< 4) ,
( 4 <= bureau_factor_7['min_mob_pl']) & (bureau_factor_7['min_mob_pl']< 19) ,
( 19 <= bureau_factor_7['min_mob_pl'])],[-0.284779312793819,1.43773530056898,0.342623190483082,-0.320324250213231])

bureau_factor_7['b_max_mob_uns']=np.select([(bureau_factor_7['max_mob_uns']< 0) ,
( 0 <= bureau_factor_7['max_mob_uns']) & (bureau_factor_7['max_mob_uns']< 27) ,
( 27 <= bureau_factor_7['max_mob_uns']) & (bureau_factor_7['max_mob_uns']< 54) ,
( 54 <= bureau_factor_7['max_mob_uns'])],['1. Less than 0','2. GE  0 and LT 27','3. GE  27 and LT 54','4. GT54'])

#bureau_factor_7['w_max_mob_uns']=np.select([(bureau_factor_7['max_mob_uns']< 0) ,
#( 0 <= bureau_factor_7['max_mob_uns']) & (bureau_factor_7['max_mob_uns']< 27) ,
#( 27 <= bureau_factor_7['max_mob_uns']) & (bureau_factor_7['max_mob_uns']< 54) ,
#( 54 <= bureau_factor_7['max_mob_uns'])],[,0.207362471518826,-0.134270810230665,-0.273485306175061])

bureau_factor_7['b_max_mob_tw']=np.select([(bureau_factor_7['max_mob_tw']< 0) ,
( 0 <= bureau_factor_7['max_mob_tw']) & (bureau_factor_7['max_mob_tw']< 22) ,
( 22 <= bureau_factor_7['max_mob_tw'])],['1. Less than 0','2. GE  0 and LT 22','3. GT22'])

bureau_factor_7['w_max_mob_tw']=np.select([(bureau_factor_7['max_mob_tw']< 0) ,
( 0 <= bureau_factor_7['max_mob_tw']) & (bureau_factor_7['max_mob_tw']< 22) ,
( 22 <= bureau_factor_7['max_mob_tw'])],[0.0564856021345575,-0.374391471483507,0.0882320504646063])

bureau_factor_7['b_max_mob_sec']=np.select([(bureau_factor_7['max_mob_sec']< 0) ,
( 0 <= bureau_factor_7['max_mob_sec']) & (bureau_factor_7['max_mob_sec']< 26) ,
( 26 <= bureau_factor_7['max_mob_sec']) & (bureau_factor_7['max_mob_sec']< 49) ,
( 49 <= bureau_factor_7['max_mob_sec'])],['1. Less than 0','2. GE  0 and LT 26','3. GE  26 and LT 49','4. GT49'])

bureau_factor_7['w_max_mob_sec']=np.select([(bureau_factor_7['max_mob_sec']< 0) ,
( 0 <= bureau_factor_7['max_mob_sec']) & (bureau_factor_7['max_mob_sec']< 26) ,
( 26 <= bureau_factor_7['max_mob_sec']) & (bureau_factor_7['max_mob_sec']< 49) ,
( 49 <= bureau_factor_7['max_mob_sec'])],[-0.100132419363866,0.186986431445315,0.0474005299120865,-0.0556647773497656])

bureau_factor_7['b_max_mob_con']=np.select([(bureau_factor_7['max_mob_con']< 23) ,
( 23 <= bureau_factor_7['max_mob_con']) & (bureau_factor_7['max_mob_con']< 54) ,
( 54 <= bureau_factor_7['max_mob_con'])],['1. Less than 23','2. GE  23 and LT 54','3. GT54'])

bureau_factor_7['w_max_mob_con']=np.select([(bureau_factor_7['max_mob_con']< 23) ,
( 23 <= bureau_factor_7['max_mob_con']) & (bureau_factor_7['max_mob_con']< 54) ,
( 54 <= bureau_factor_7['max_mob_con'])],[0.0864894757662271,0.0694513317235567,-0.175212200608025])

bureau_factor_7['b_max_mob_com_oth']=np.select([(bureau_factor_7['max_mob_com_oth']< 0) ,
( 0 <= bureau_factor_7['max_mob_com_oth']) & (bureau_factor_7['max_mob_com_oth']< 16) ,
( 16 <= bureau_factor_7['max_mob_com_oth'])],['1. Less than 0','2. GE  0 and LT 16','3. GT16'])

bureau_factor_7['w_max_mob_com_oth']=np.select([(bureau_factor_7['max_mob_com_oth']< 0) ,
( 0 <= bureau_factor_7['max_mob_com_oth']) & (bureau_factor_7['max_mob_com_oth']< 16) ,
( 16 <= bureau_factor_7['max_mob_com_oth'])],[-0.0267015662313288,0.829581332842429,-0.120610950707407])

bureau_factor_7['b_max_mob_com_agri']=np.select([(bureau_factor_7['max_mob_com_agri']< 0) ,
( 0 <= bureau_factor_7['max_mob_com_agri'])],['1. Less than 0','2. GT0'])

bureau_factor_7['w_max_mob_com_agri']=np.select([(bureau_factor_7['max_mob_com_agri']< 0) ,
( 0 <= bureau_factor_7['max_mob_com_agri'])],[-0.0743341816040721,0.80937862552491])

bureau_factor_7['b_max_mob_all']=np.select([(bureau_factor_7['max_mob_all']< 25) ,
( 25 <= bureau_factor_7['max_mob_all']) & (bureau_factor_7['max_mob_all']< 54) ,
( 54 <= bureau_factor_7['max_mob_all'])],['1. Less than 25','2. GE  25 and LT 54','3. GT54'])

bureau_factor_7['w_max_mob_all']=np.select([(bureau_factor_7['max_mob_all']< 25) ,
( 25 <= bureau_factor_7['max_mob_all']) & (bureau_factor_7['max_mob_all']< 54) ,
( 54 <= bureau_factor_7['max_mob_all'])],[0.0887754232705237,0.0405523805792014,-0.137140252658617])

bureau_factor_7['b_max_live_mob_uns']=np.select([(bureau_factor_7['max_live_mob_uns']< 0) ,
( 0 <= bureau_factor_7['max_live_mob_uns']) & (bureau_factor_7['max_live_mob_uns']< 7) ,
( 7 <= bureau_factor_7['max_live_mob_uns']) & (bureau_factor_7['max_live_mob_uns']< 15) ,
( 15 <= bureau_factor_7['max_live_mob_uns']) & (bureau_factor_7['max_live_mob_uns']< 31) ,
( 31 <= bureau_factor_7['max_live_mob_uns'])],['1. Less than 0','2. GE  0 and LT 7','3. GE  7 and LT 15','4. GE  15 and LT 31','5. GT31'])

bureau_factor_7['w_max_live_mob_uns']=np.select([(bureau_factor_7['max_live_mob_uns']< 0) ,
( 0 <= bureau_factor_7['max_live_mob_uns']) & (bureau_factor_7['max_live_mob_uns']< 7) ,
( 7 <= bureau_factor_7['max_live_mob_uns']) & (bureau_factor_7['max_live_mob_uns']< 15) ,
( 15 <= bureau_factor_7['max_live_mob_uns']) & (bureau_factor_7['max_live_mob_uns']< 31) ,
( 31 <= bureau_factor_7['max_live_mob_uns'])],[1.32427757467854,0.573861611093607,0.186986431445315,-0.364776012784065,-0.634965225177209])



########################


bureau_factor_7['b_nbr_com_agri_tot_accts']=np.select([(bureau_factor_7['nbr_com_agri_tot_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_com_agri_tot_accts'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_com_agri_tot_accts']=np.select([(bureau_factor_7['nbr_com_agri_tot_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_com_agri_tot_accts'])],[-0.0408174057750704,1.22761146246308])

bureau_factor_7['b_nbr_closed_active_accts']=np.select([(bureau_factor_7['nbr_closed_active_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_closed_active_accts']) & (bureau_factor_7['nbr_closed_active_accts']< 9) ,
( 9 <= bureau_factor_7['nbr_closed_active_accts'])],['1. Less than 2','2. GE  2 and LT 9','3. GT9'])

bureau_factor_7['w_nbr_closed_active_accts']=np.select([(bureau_factor_7['nbr_closed_active_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_closed_active_accts']) & (bureau_factor_7['nbr_closed_active_accts']< 9) ,
( 9 <= bureau_factor_7['nbr_closed_active_accts'])],[-0.432156497413637,0.249174775237933,1.04827053380726])

bureau_factor_7['b_nbr_cl_tot_accts']=np.select([(bureau_factor_7['nbr_cl_tot_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_cl_tot_accts']) & (bureau_factor_7['nbr_cl_tot_accts']< 6) ,
( 6 <= bureau_factor_7['nbr_cl_tot_accts'])],['1. Less than 4','2. GE  4 and LT 6','3. GT6'])

bureau_factor_7['w_nbr_cl_tot_accts']=np.select([(bureau_factor_7['nbr_cl_tot_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_cl_tot_accts']) & (bureau_factor_7['nbr_cl_tot_accts']< 6) ,
( 6 <= bureau_factor_7['nbr_cl_tot_accts'])],[-0.106681532218641,0.0402902917295817,0.27192440939234])

bureau_factor_7['b_nbr_cl_live_accts']=np.select([(bureau_factor_7['nbr_cl_live_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_cl_live_accts'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_cl_live_accts']=np.select([(bureau_factor_7['nbr_cl_live_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_cl_live_accts'])],[0.269586685269969,-0.333569476963252])

bureau_factor_7['b_nbr_cl_closed_accts']=np.select([(bureau_factor_7['nbr_cl_closed_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_cl_closed_accts']) & (bureau_factor_7['nbr_cl_closed_accts']< 3) ,
( 3 <= bureau_factor_7['nbr_cl_closed_accts'])],['1. Less than 2','2. GE  2 and LT 3','3. GT3'])

bureau_factor_7['w_nbr_cl_closed_accts']=np.select([(bureau_factor_7['nbr_cl_closed_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_cl_closed_accts']) & (bureau_factor_7['nbr_cl_closed_accts']< 3) ,
( 3 <= bureau_factor_7['nbr_cl_closed_accts'])],[-0.307700096984834,0.200972673420055,0.455438695863306])

bureau_factor_7['b_nbr_cc_tot_accts']=np.select([(bureau_factor_7['nbr_cc_tot_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_cc_tot_accts'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_cc_tot_accts']=np.select([(bureau_factor_7['nbr_cc_tot_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_cc_tot_accts'])],[-0.0961881429862695,0.459483368571556])

bureau_factor_7['b_nbr_cc_live_active_accts']=np.select([(bureau_factor_7['nbr_cc_live_active_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_cc_live_active_accts'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_cc_live_active_accts']=np.select([(bureau_factor_7['nbr_cc_live_active_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_cc_live_active_accts'])],[-0.0559968123441959,0.540188395987019])

bureau_factor_7['b_nbr_accts_open_l6m_con']=np.select([(bureau_factor_7['nbr_accts_open_l6m_con']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l6m_con'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_accts_open_l6m_con']=np.select([(bureau_factor_7['nbr_accts_open_l6m_con']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l6m_con'])],[-0.399749407073588,0.748612270308762])

bureau_factor_7['b_nbr_accts_open_l3m_pl']=np.select([(bureau_factor_7['nbr_accts_open_l3m_pl']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l3m_pl'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_accts_open_l3m_pl']=np.select([(bureau_factor_7['nbr_accts_open_l3m_pl']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l3m_pl'])],[-0.0428152054745004,1.57151867757181])

bureau_factor_7['b_nbr_accts_open_l3m']=np.select([(bureau_factor_7['nbr_accts_open_l3m']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l3m'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_accts_open_l3m']=np.select([(bureau_factor_7['nbr_accts_open_l3m']< 2) ,
( 2 <= bureau_factor_7['nbr_accts_open_l3m'])],[-0.205629444475184,0.960917334903516])



########################


bureau_factor_7['b_nbr_con_live_accts']=np.select([(bureau_factor_7['nbr_con_live_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_con_live_accts']) & (bureau_factor_7['nbr_con_live_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_con_live_accts'])],['1. Less than 2','2. GE  2 and LT 4','3. GT4'])

bureau_factor_7['w_nbr_con_live_accts']=np.select([(bureau_factor_7['nbr_con_live_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_con_live_accts']) & (bureau_factor_7['nbr_con_live_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_con_live_accts'])],[-0.184347868595642,-0.0867093990317257,0.185705201289267])

bureau_factor_7['b_nbr_pl_tot_accts']=np.select([(bureau_factor_7['nbr_pl_tot_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_pl_tot_accts']) & (bureau_factor_7['nbr_pl_tot_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_pl_tot_accts'])],['1. Less than 2','2. GE  2 and LT 4','3. GT4'])

bureau_factor_7['w_nbr_pl_tot_accts']=np.select([(bureau_factor_7['nbr_pl_tot_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_pl_tot_accts']) & (bureau_factor_7['nbr_pl_tot_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_pl_tot_accts'])],[-0.120834889796911,0.424116224734265,0.528476240058508])

bureau_factor_7['b_nbr_pl_live_accts']=np.select([(bureau_factor_7['nbr_pl_live_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_pl_live_accts'])],['1. Less than 2','2. GT2'])

bureau_factor_7['w_nbr_pl_live_accts']=np.select([(bureau_factor_7['nbr_pl_live_accts']< 2) ,
( 2 <= bureau_factor_7['nbr_pl_live_accts'])],[-0.092540319342519,0.818408032244304])

bureau_factor_7['b_nbr_prod_0_24m_al']=np.select([(bureau_factor_7['nbr_prod_0_24m_al']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_0_24m_al']) & (bureau_factor_7['nbr_prod_0_24m_al']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_0_24m_al'])],['1. Less than 0','2. GE  0 and LT 1','3. GT1'])

bureau_factor_7['w_nbr_prod_0_24m_al']=np.select([(bureau_factor_7['nbr_prod_0_24m_al']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_0_24m_al']) & (bureau_factor_7['nbr_prod_0_24m_al']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_0_24m_al'])],[-0.0775490296330615,1.03025202830458,-0.412131799466354])

bureau_factor_7['b_nbr_prod_0_3m_sec']=np.select([(bureau_factor_7['nbr_prod_0_3m_sec']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_0_3m_sec']) & (bureau_factor_7['nbr_prod_0_3m_sec']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_0_3m_sec'])],['1. Less than 0','2. GE  0 and LT 1','3. GT1'])

bureau_factor_7['w_nbr_prod_0_3m_sec']=np.select([(bureau_factor_7['nbr_prod_0_3m_sec']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_0_3m_sec']) & (bureau_factor_7['nbr_prod_0_3m_sec']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_0_3m_sec'])],[-0.11545186789738,0.686480489201756,-0.862923178290285])

bureau_factor_7['b_nbr_prod_0_6m_act']=np.select([(bureau_factor_7['nbr_prod_0_6m_act']< 2) ,
( 2 <= bureau_factor_7['nbr_prod_0_6m_act']) & (bureau_factor_7['nbr_prod_0_6m_act']< 3) ,
( 3 <= bureau_factor_7['nbr_prod_0_6m_act']) & (bureau_factor_7['nbr_prod_0_6m_act']< 4) ,
( 4 <= bureau_factor_7['nbr_prod_0_6m_act'])],['1. Less than 2','2. GE  2 and LT 3','3. GE  3 and LT 4','4. GT4'])

bureau_factor_7['w_nbr_prod_0_6m_act']=np.select([(bureau_factor_7['nbr_prod_0_6m_act']< 2) ,
( 2 <= bureau_factor_7['nbr_prod_0_6m_act']) & (bureau_factor_7['nbr_prod_0_6m_act']< 3) ,
( 3 <= bureau_factor_7['nbr_prod_0_6m_act']) & (bureau_factor_7['nbr_prod_0_6m_act']< 4) ,
( 4 <= bureau_factor_7['nbr_prod_0_6m_act'])],[0.770598554950578,-1.03449879796525,-1.14895404352896,-1.85547935003508])

bureau_factor_7['b_nbr_prod_30_3m_hl']=np.select([(bureau_factor_7['nbr_prod_30_3m_hl']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_30_3m_hl']) & (bureau_factor_7['nbr_prod_30_3m_hl']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_30_3m_hl'])],['1. Less than 0','2. GE  0 and LT 1','3. GT1'])

bureau_factor_7['w_nbr_prod_30_3m_hl']=np.select([(bureau_factor_7['nbr_prod_30_3m_hl']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_30_3m_hl']) & (bureau_factor_7['nbr_prod_30_3m_hl']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_30_3m_hl'])],[-0.0762877035693116,1.41736799774455,-1.18532168769984])

bureau_factor_7['b_nbr_prod_30_6m_com']=np.select([(bureau_factor_7['nbr_prod_30_6m_com']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_30_6m_com']) & (bureau_factor_7['nbr_prod_30_6m_com']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_30_6m_com'])],['1. Less than 0','2. GE  0 and LT 1','3. GT1'])

bureau_factor_7['w_nbr_prod_30_6m_com']=np.select([(bureau_factor_7['nbr_prod_30_6m_com']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_30_6m_com']) & (bureau_factor_7['nbr_prod_30_6m_com']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_30_6m_com'])],[-0.0361785903845096,0.693449158517849,-0.856817620727799])

bureau_factor_7['b_nbr_prod_30_6m_com_oth']=np.select([(bureau_factor_7['nbr_prod_30_6m_com_oth']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_30_6m_com_oth']) & (bureau_factor_7['nbr_prod_30_6m_com_oth']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_30_6m_com_oth'])],['1. Less than 0','2. GE  0 and LT 1','3. GT1'])

bureau_factor_7['w_nbr_prod_30_6m_com_oth']=np.select([(bureau_factor_7['nbr_prod_30_6m_com_oth']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_30_6m_com_oth']) & (bureau_factor_7['nbr_prod_30_6m_com_oth']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_30_6m_com_oth'])],[-0.0233218467532309,0.60643778152822,-1.18532168769984])

bureau_factor_7['b_nbr_prod_90_24m_sec']=np.select([(bureau_factor_7['nbr_prod_90_24m_sec']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_90_24m_sec']) & (bureau_factor_7['nbr_prod_90_24m_sec']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_90_24m_sec'])],['1. Less than 0','2. GE  0 and LT 1','3. GT1'])

bureau_factor_7['w_nbr_prod_90_24m_sec']=np.select([(bureau_factor_7['nbr_prod_90_24m_sec']< 0) ,
( 0 <= bureau_factor_7['nbr_prod_90_24m_sec']) & (bureau_factor_7['nbr_prod_90_24m_sec']< 1) ,
( 1 <= bureau_factor_7['nbr_prod_90_24m_sec'])],[-0.132048079620788,0.401643368882206,-1.28248543615348])

bureau_factor_7['b_nbr_tot_active_accts']=np.select([(bureau_factor_7['nbr_tot_active_accts']< 3) ,
( 3 <= bureau_factor_7['nbr_tot_active_accts']) & (bureau_factor_7['nbr_tot_active_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_tot_active_accts']) & (bureau_factor_7['nbr_tot_active_accts']< 5) ,
( 5 <= bureau_factor_7['nbr_tot_active_accts']) & (bureau_factor_7['nbr_tot_active_accts']< 8) ,
( 8 <= bureau_factor_7['nbr_tot_active_accts'])],['1. Less than 3','2. GE  3 and LT 4','3. GE  4 and LT 5','4. GE  5 and LT 8','5. GT8'])

bureau_factor_7['w_nbr_tot_active_accts']=np.select([(bureau_factor_7['nbr_tot_active_accts']< 3) ,
( 3 <= bureau_factor_7['nbr_tot_active_accts']) & (bureau_factor_7['nbr_tot_active_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_tot_active_accts']) & (bureau_factor_7['nbr_tot_active_accts']< 5) ,
( 5 <= bureau_factor_7['nbr_tot_active_accts']) & (bureau_factor_7['nbr_tot_active_accts']< 8) ,
( 8 <= bureau_factor_7['nbr_tot_active_accts'])],[-0.514647362991949,-0.409482791294777,-0.16549027688484,0.258323969891521,0.333602217193433])

bureau_factor_7['b_nbr_uns_tot_accts']=np.select([(bureau_factor_7['nbr_uns_tot_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_uns_tot_accts']) & (bureau_factor_7['nbr_uns_tot_accts']< 6) ,
( 6 <= bureau_factor_7['nbr_uns_tot_accts']) & (bureau_factor_7['nbr_uns_tot_accts']< 10) ,
( 10 <= bureau_factor_7['nbr_uns_tot_accts'])],['1. Less than 4','2. GE  4 and LT 6','3. GE  6 and LT 10','4. GT10'])

bureau_factor_7['w_nbr_uns_tot_accts']=np.select([(bureau_factor_7['nbr_uns_tot_accts']< 4) ,
( 4 <= bureau_factor_7['nbr_uns_tot_accts']) & (bureau_factor_7['nbr_uns_tot_accts']< 6) ,
( 6 <= bureau_factor_7['nbr_uns_tot_accts']) & (bureau_factor_7['nbr_uns_tot_accts']< 10) ,
( 10 <= bureau_factor_7['nbr_uns_tot_accts'])],[-0.262235912692852,-0.152958784572926,0.338756375783108,0.419940853323784])

bureau_factor_7['b_ratio_amt_al']=np.select([(bureau_factor_7['ratio_amt_al']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_al']) & (bureau_factor_7['ratio_amt_al']< 0.1) ,
( 0.1 <= bureau_factor_7['ratio_amt_al']) & (bureau_factor_7['ratio_amt_al']< 28.7) ,
( 28.7 <= bureau_factor_7['ratio_amt_al'])],['1. Less than 0','2. GE  0 and LT 0.1','3. GE  0.1 and LT 28.7','4. GT28.7'])

bureau_factor_7['w_ratio_amt_al']=np.select([(bureau_factor_7['ratio_amt_al']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_al']) & (bureau_factor_7['ratio_amt_al']< 0.1) ,
( 0.1 <= bureau_factor_7['ratio_amt_al']) & (bureau_factor_7['ratio_amt_al']< 28.7) ,
( 28.7 <= bureau_factor_7['ratio_amt_al'])],[0.563878167109424,-0.0826277607120772,0.403913517416746,0.48081156841081])



########################


bureau_factor_7['b_ratio_amt_cc']=np.select([(bureau_factor_7['ratio_amt_cc']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_cc']) & (bureau_factor_7['ratio_amt_cc']< 0.1) ,
( 0.1 <= bureau_factor_7['ratio_amt_cc'])],['1. Less than 0','2. GE  0 and LT 0.1','3. GT0.1'])

bureau_factor_7['w_ratio_amt_cc']=np.select([(bureau_factor_7['ratio_amt_cc']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_cc']) & (bureau_factor_7['ratio_amt_cc']< 0.1) ,
( 0.1 <= bureau_factor_7['ratio_amt_cc'])],[0.563878167109424,-0.204952099141158,0.408079225824448])

bureau_factor_7['b_ratio_amt_con']=np.select([(bureau_factor_7['ratio_amt_con']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_con']) & (bureau_factor_7['ratio_amt_con']< 100) ,
( 100 <= bureau_factor_7['ratio_amt_con'])],['1. Less than 0','2. GE  0 and LT 100','3. GT100'])

bureau_factor_7['w_ratio_amt_con']=np.select([(bureau_factor_7['ratio_amt_con']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_con']) & (bureau_factor_7['ratio_amt_con']< 100) ,
( 100 <= bureau_factor_7['ratio_amt_con'])],[0.563878167109424,0.26159729523649,-0.0676612040610312])

bureau_factor_7['b_ratio_amt_pl']=np.select([(bureau_factor_7['ratio_amt_pl']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_pl']) & (bureau_factor_7['ratio_amt_pl']< 0.1) ,
( 0.1 <= bureau_factor_7['ratio_amt_pl']) & (bureau_factor_7['ratio_amt_pl']< 10.4) ,
( 10.4 <= bureau_factor_7['ratio_amt_pl']) & (bureau_factor_7['ratio_amt_pl']< 36.1) ,
( 36.1 <= bureau_factor_7['ratio_amt_pl']) & (bureau_factor_7['ratio_amt_pl']< 66.7) ,
( 66.7 <= bureau_factor_7['ratio_amt_pl'])],['1. Less than 0','2. GE  0 and LT 0.1','3. GE  0.1 and LT 10.4','4. GE  10.4 and LT 36.1','5. GE  36.1 and LT 66.7','6. GT66.7'])

bureau_factor_7['w_ratio_amt_pl']=np.select([(bureau_factor_7['ratio_amt_pl']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_pl']) & (bureau_factor_7['ratio_amt_pl']< 0.1) ,
( 0.1 <= bureau_factor_7['ratio_amt_pl']) & (bureau_factor_7['ratio_amt_pl']< 10.4) ,
( 10.4 <= bureau_factor_7['ratio_amt_pl']) & (bureau_factor_7['ratio_amt_pl']< 36.1) ,
( 36.1 <= bureau_factor_7['ratio_amt_pl']) & (bureau_factor_7['ratio_amt_pl']< 66.7) ,
( 66.7 <= bureau_factor_7['ratio_amt_pl'])],[0.563878167109424,-0.277639330362281,0.0674412807955325,0.472906388903697,0.575666122861466,1.22012079073392])

bureau_factor_7['b_ratio_amt_sec']=np.select([(bureau_factor_7['ratio_amt_sec']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_sec']) & (bureau_factor_7['ratio_amt_sec']< 1) ,
( 1 <= bureau_factor_7['ratio_amt_sec']) & (bureau_factor_7['ratio_amt_sec']< 93.5) ,
( 93.5 <= bureau_factor_7['ratio_amt_sec'])],['1. Less than 0','2. GE  0 and LT 1','3. GE  1 and LT 93.5','4. GT93.5'])

bureau_factor_7['w_ratio_amt_sec']=np.select([(bureau_factor_7['ratio_amt_sec']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_sec']) & (bureau_factor_7['ratio_amt_sec']< 1) ,
( 1 <= bureau_factor_7['ratio_amt_sec']) & (bureau_factor_7['ratio_amt_sec']< 93.5) ,
( 93.5 <= bureau_factor_7['ratio_amt_sec'])],[0.563878167109424,-0.174525605038643,0.0699013066363949,0.791841004859582])

bureau_factor_7['b_ratio_amt_tw']=np.select([(bureau_factor_7['ratio_amt_tw']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_tw']) & (bureau_factor_7['ratio_amt_tw']< 0.7) ,
( 0.7 <= bureau_factor_7['ratio_amt_tw']) & (bureau_factor_7['ratio_amt_tw']< 28.9) ,
( 28.9 <= bureau_factor_7['ratio_amt_tw'])],['1. Less than 0','2. GE  0 and LT 0.7','3. GE  0.7 and LT 28.9','4. GT28.9'])

bureau_factor_7['w_ratio_amt_tw']=np.select([(bureau_factor_7['ratio_amt_tw']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_tw']) & (bureau_factor_7['ratio_amt_tw']< 0.7) ,
( 0.7 <= bureau_factor_7['ratio_amt_tw']) & (bureau_factor_7['ratio_amt_tw']< 28.9) ,
( 28.9 <= bureau_factor_7['ratio_amt_tw'])],[0.563878167109424,0.062213877360381,-0.26903095582568,-0.361146244733486])

bureau_factor_7['b_ratio_amt_uns']=np.select([(bureau_factor_7['ratio_amt_uns']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_uns']) & (bureau_factor_7['ratio_amt_uns']< 24) ,
( 24 <= bureau_factor_7['ratio_amt_uns']) & (bureau_factor_7['ratio_amt_uns']< 97.3) ,
( 97.3 <= bureau_factor_7['ratio_amt_uns'])],['1. Less than 0','2. GE  0 and LT 24','3. GE  24 and LT 97.3','4. GT97.3'])

bureau_factor_7['w_ratio_amt_uns']=np.select([(bureau_factor_7['ratio_amt_uns']< 0) ,
( 0 <= bureau_factor_7['ratio_amt_uns']) & (bureau_factor_7['ratio_amt_uns']< 24) ,
( 24 <= bureau_factor_7['ratio_amt_uns']) & (bureau_factor_7['ratio_amt_uns']< 97.3) ,
( 97.3 <= bureau_factor_7['ratio_amt_uns'])],[0.563878167109424,0.343147162200598,0.132344167030959,-0.301119270377181])

bureau_factor_7['b_ratio_avg_util_C1to3toC1to12']=np.select([(bureau_factor_7['ratio_avg_util_C1to3toC1to12']< -718.4) ,
( -718.4 <= bureau_factor_7['ratio_avg_util_C1to3toC1to12']) & (bureau_factor_7['ratio_avg_util_C1to3toC1to12']< 1685.6) ,
( 1685.6 <= bureau_factor_7['ratio_avg_util_C1to3toC1to12']) & (bureau_factor_7['ratio_avg_util_C1to3toC1to12']< 2453.4) ,
( 2453.4 <= bureau_factor_7['ratio_avg_util_C1to3toC1to12'])],['1. Less than -718.4','2. GE  -718.4 and LT 1685.6','3. GE  1685.6 and LT 2453.4','4. GT2453.4'])

bureau_factor_7['w_ratio_avg_util_C1to3toC1to12']=np.select([(bureau_factor_7['ratio_avg_util_C1to3toC1to12']< -718.4) ,
( -718.4 <= bureau_factor_7['ratio_avg_util_C1to3toC1to12']) & (bureau_factor_7['ratio_avg_util_C1to3toC1to12']< 1685.6) ,
( 1685.6 <= bureau_factor_7['ratio_avg_util_C1to3toC1to12']) & (bureau_factor_7['ratio_avg_util_C1to3toC1to12']< 2453.4) ,
( 2453.4 <= bureau_factor_7['ratio_avg_util_C1to3toC1to12'])],[0.365275724711331,-0.224151629648247,0.231070704577335,0.678758443107846])

bureau_factor_7['b_util_l12m_all_act']=np.select([(bureau_factor_7['util_l12m_all_act']< 0.0274702198398962) ,
( 0.0274702198398962 <= bureau_factor_7['util_l12m_all_act']) & (bureau_factor_7['util_l12m_all_act']< 0.174942303871542) ,
( 0.174942303871542 <= bureau_factor_7['util_l12m_all_act']) & (bureau_factor_7['util_l12m_all_act']< 0.432727798296256) ,
( 0.432727798296256 <= bureau_factor_7['util_l12m_all_act'])],['1. Less than 0.03','2. GE  0.03 and LT 0.17','3. GE  0.17 and LT 0.43','4. GT0.43'])

bureau_factor_7['w_util_l12m_all_act']=np.select([(bureau_factor_7['util_l12m_all_act']< 0.0274702198398962) ,
( 0.0274702198398962 <= bureau_factor_7['util_l12m_all_act']) & (bureau_factor_7['util_l12m_all_act']< 0.174942303871542) ,
( 0.174942303871542 <= bureau_factor_7['util_l12m_all_act']) & (bureau_factor_7['util_l12m_all_act']< 0.432727798296256) ,
( 0.432727798296256 <= bureau_factor_7['util_l12m_all_act'])],[0.392863681230161,0.231070704577335,-0.0375016111351966,-0.361146244733486])

bureau_factor_7['b_util_l12m_all_live']=np.select([(bureau_factor_7['util_l12m_all_live']< -0.0825833333333333) ,
( -0.0825833333333333 <= bureau_factor_7['util_l12m_all_live']) & (bureau_factor_7['util_l12m_all_live']< 0.0157709941894257) ,
( 0.0157709941894257 <= bureau_factor_7['util_l12m_all_live']) & (bureau_factor_7['util_l12m_all_live']< 0.161234112341123) ,
( 0.161234112341123 <= bureau_factor_7['util_l12m_all_live']) & (bureau_factor_7['util_l12m_all_live']< 0.429569139678596) ,
( 0.429569139678596 <= bureau_factor_7['util_l12m_all_live'])],['1. Less than -0.08','2. GE  -0.08 and LT 0.02','3. GE  0.02 and LT 0.16','4. GE  0.16 and LT 0.43','5. GT0.43'])

bureau_factor_7['w_util_l12m_all_live']=np.select([(bureau_factor_7['util_l12m_all_live']< -0.0825833333333333) ,
( -0.0825833333333333 <= bureau_factor_7['util_l12m_all_live']) & (bureau_factor_7['util_l12m_all_live']< 0.0157709941894257) ,
( 0.0157709941894257 <= bureau_factor_7['util_l12m_all_live']) & (bureau_factor_7['util_l12m_all_live']< 0.161234112341123) ,
( 0.161234112341123 <= bureau_factor_7['util_l12m_all_live']) & (bureau_factor_7['util_l12m_all_live']< 0.429569139678596) ,
( 0.429569139678596 <= bureau_factor_7['util_l12m_all_live'])],[0.563878167109424,1.17788802711065,0.233762496243046,-0.0395914760546558,-0.277639330362281])

bureau_factor_7['b_util_l12m_all_tot']=np.select([(bureau_factor_7['util_l12m_all_tot']< 0.0227922918095149) ,
( 0.0227922918095149 <= bureau_factor_7['util_l12m_all_tot']) & (bureau_factor_7['util_l12m_all_tot']< 0.387404888381406) ,
( 0.387404888381406 <= bureau_factor_7['util_l12m_all_tot'])],['1. Less than 0.02','2. GE  0.02 and LT 0.39','3. GT0.39'])

bureau_factor_7['w_util_l12m_all_tot']=np.select([(bureau_factor_7['util_l12m_all_tot']< 0.0227922918095149) ,
( 0.0227922918095149 <= bureau_factor_7['util_l12m_all_tot']) & (bureau_factor_7['util_l12m_all_tot']< 0.387404888381406) ,
( 0.387404888381406 <= bureau_factor_7['util_l12m_all_tot'])],[0.575666122861466,0.0429502607872368,-0.336567162816168])

bureau_factor_7['b_util_l12m_exc_cc_tot']=np.select([(bureau_factor_7['util_l12m_exc_cc_tot']< 0.21378490169044) ,
( 0.21378490169044 <= bureau_factor_7['util_l12m_exc_cc_tot']) & (bureau_factor_7['util_l12m_exc_cc_tot']< 2.21036450985783) ,
( 2.21036450985783 <= bureau_factor_7['util_l12m_exc_cc_tot']) & (bureau_factor_7['util_l12m_exc_cc_tot']< 3.52188741053198) ,
( 3.52188741053198 <= bureau_factor_7['util_l12m_exc_cc_tot'])],['1. Less than 0.21','2. GE  0.21 and LT 2.21','3. GE  2.21 and LT 3.52','4. GT3.52'])

bureau_factor_7['w_util_l12m_exc_cc_tot']=np.select([(bureau_factor_7['util_l12m_exc_cc_tot']< 0.21378490169044) ,
( 0.21378490169044 <= bureau_factor_7['util_l12m_exc_cc_tot']) & (bureau_factor_7['util_l12m_exc_cc_tot']< 2.21036450985783) ,
( 2.21036450985783 <= bureau_factor_7['util_l12m_exc_cc_tot']) & (bureau_factor_7['util_l12m_exc_cc_tot']< 3.52188741053198) ,
( 3.52188741053198 <= bureau_factor_7['util_l12m_exc_cc_tot'])],[0.575666122861466,0.0571847806283434,-0.0697598407179525,-0.170883894686156])

bureau_factor_7['b_util_l3m_all_live']=np.select([(bureau_factor_7['util_l3m_all_live']< -0.14585) ,
( -0.14585 <= bureau_factor_7['util_l3m_all_live']) & (bureau_factor_7['util_l3m_all_live']< 0.139364091506791) ,
( 0.139364091506791 <= bureau_factor_7['util_l3m_all_live']) & (bureau_factor_7['util_l3m_all_live']< 0.409644234890698) ,
( 0.409644234890698 <= bureau_factor_7['util_l3m_all_live']) & (bureau_factor_7['util_l3m_all_live']< 0.765459188885483) ,
( 0.765459188885483 <= bureau_factor_7['util_l3m_all_live'])],['1. Less than -0.15','2. GE  -0.15 and LT 0.14','3. GE  0.14 and LT 0.41','4. GE  0.41 and LT 0.77','5. GT0.77'])

bureau_factor_7['w_util_l3m_all_live']=np.select([(bureau_factor_7['util_l3m_all_live']< -0.14585) ,
( -0.14585 <= bureau_factor_7['util_l3m_all_live']) & (bureau_factor_7['util_l3m_all_live']< 0.139364091506791) ,
( 0.139364091506791 <= bureau_factor_7['util_l3m_all_live']) & (bureau_factor_7['util_l3m_all_live']< 0.409644234890698) ,
( 0.409644234890698 <= bureau_factor_7['util_l3m_all_live']) & (bureau_factor_7['util_l3m_all_live']< 0.765459188885483) ,
( 0.765459188885483 <= bureau_factor_7['util_l3m_all_live'])],[0.563878167109424,0.309046894949478,0.00307590484062925,-0.097180698899021,-0.20733738682034])

bureau_factor_7['b_util_l6m_all_act']=np.select([(bureau_factor_7['util_l6m_all_act']< 0.0215088929411453) ,
( 0.0215088929411453 <= bureau_factor_7['util_l6m_all_act']) & (bureau_factor_7['util_l6m_all_act']< 0.0818660190244889) ,
( 0.0818660190244889 <= bureau_factor_7['util_l6m_all_act']) & (bureau_factor_7['util_l6m_all_act']< 0.638920038625673) ,
( 0.638920038625673 <= bureau_factor_7['util_l6m_all_act'])],['1. Less than 0.02','2. GE  0.02 and LT 0.08','3. GE  0.08 and LT 0.64','4. GT0.64'])

bureau_factor_7['w_util_l6m_all_act']=np.select([(bureau_factor_7['util_l6m_all_act']< 0.0215088929411453) ,
( 0.0215088929411453 <= bureau_factor_7['util_l6m_all_act']) & (bureau_factor_7['util_l6m_all_act']< 0.0818660190244889) ,
( 0.0818660190244889 <= bureau_factor_7['util_l6m_all_act']) & (bureau_factor_7['util_l6m_all_act']< 0.638920038625673) ,
( 0.638920038625673 <= bureau_factor_7['util_l6m_all_act'])],[0.48081156841081,0.0269499194407957,0.0154965398775853,-0.455806862969015])

bureau_factor_7['b_util_l6m_all_tot']=np.select([(bureau_factor_7['util_l6m_all_tot']< 0.294610874206387) ,
( 0.294610874206387 <= bureau_factor_7['util_l6m_all_tot'])],['1. Less than 0.29','2. GT0.29'])

bureau_factor_7['w_util_l6m_all_tot']=np.select([(bureau_factor_7['util_l6m_all_tot']< 0.294610874206387) ,
( 0.294610874206387 <= bureau_factor_7['util_l6m_all_tot'])],[0.15977083246172,-0.20947642506909])



In [ ]:
#bureau_factor_6 = copy.deepcopy(iv_input)

In [133]:
'''
mod=bureau_factor_6.assign(b_nbr_sec_tot_accts = np.where((bureau_factor_6.nbr_sec_tot_accts< 1), '1. Less than 1',
np.where(((bureau_factor_6.nbr_sec_tot_accts <5) & (bureau_factor_6.nbr_sec_tot_accts >1)),'2. GE 1& LT 5',
'3. GE 5' )),

w_nbr_sec_tot_accts= np.where((bureau_factor_6.nbr_sec_tot_accts< 1), -0.189174802307698,
np.where(((bureau_factor_6.nbr_sec_tot_accts <5) & (bureau_factor_6.nbr_sec_tot_accts >1)),0.0906875551553483,
0.388546577650916 )),


b_util_curr_all_tot= np.where((bureau_factor_6.util_curr_all_tot< 0.047778369), '1. Less than 0.047778369',
np.where(((bureau_factor_6.util_curr_all_tot <0.390120897) & (bureau_factor_6.util_curr_all_tot >0.047778369)),'2. GE 0.047778369& LT 0.390120897',
np.where(((bureau_factor_6.util_curr_all_tot <0.457991852) & (bureau_factor_6.util_curr_all_tot >0.390120897)),'3. GE 0.390120897& LT 0.457991852',
np.where(((bureau_factor_6.util_curr_all_tot <0.742100071) & (bureau_factor_6.util_curr_all_tot >0.457991852)),'4. GE 0.457991852& LT 0.742100071',
'5. GE 0.742100071' )))),

w_util_curr_all_tot= np.where((bureau_factor_6.util_curr_all_tot< 0.047778369), -0.398895333289767,
np.where(((bureau_factor_6.util_curr_all_tot <0.390120897) & (bureau_factor_6.util_curr_all_tot >0.047778369)),0.101510860658368,
np.where(((bureau_factor_6.util_curr_all_tot <0.457991852) & (bureau_factor_6.util_curr_all_tot >0.390120897)),0.0468721538424196,
np.where(((bureau_factor_6.util_curr_all_tot <0.742100071) & (bureau_factor_6.util_curr_all_tot >0.457991852)),0.0320301751314615,
-0.0916119073179508 )))),


b_mon_since0_sec= np.where((bureau_factor_6.mon_since0_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.mon_since0_sec <8) & (bureau_factor_6.mon_since0_sec >0)),'2. GE 0& LT 8',
'3. GE 8' )),

w_mon_since0_sec= np.where((bureau_factor_6.mon_since0_sec< 0), 0.171820432482413,
np.where(((bureau_factor_6.mon_since0_sec <8) & (bureau_factor_6.mon_since0_sec >0)),-0.451204077100904,
-0.321186348962451 )),


b_nbr_uns_live_accts= np.where((bureau_factor_6.nbr_uns_live_accts< 4), '1. Less than 4',
np.where(((bureau_factor_6.nbr_uns_live_accts <8) & (bureau_factor_6.nbr_uns_live_accts >4)),'2. GE 4& LT 8',
'3. GE 8' )),

w_nbr_uns_live_accts= np.where((bureau_factor_6.nbr_uns_live_accts< 4), -0.0907626503666953,
np.where(((bureau_factor_6.nbr_uns_live_accts <8) & (bureau_factor_6.nbr_uns_live_accts >4)),0.0993987981643098,
0.407238710663069 )),


b_cust_loan_amt_live_sec= np.where((bureau_factor_6.cust_loan_amt_live_sec< 71340), '1. Less than 71340',
np.where(((bureau_factor_6.cust_loan_amt_live_sec <500000) & (bureau_factor_6.cust_loan_amt_live_sec >71340)),'2. GE 71340& LT 500000',
'3. GE 500000' )),

w_cust_loan_amt_live_sec= np.where((bureau_factor_6.cust_loan_amt_live_sec< 71340), -0.120222330780352,
np.where(((bureau_factor_6.cust_loan_amt_live_sec <500000) & (bureau_factor_6.cust_loan_amt_live_sec >71340)),0.167758740029165,
0.489636438559571 )),


b_ratio_amt_al= np.where((bureau_factor_6.ratio_amt_al< 0), '1. Less than 0',
np.where(((bureau_factor_6.ratio_amt_al <21.9) & (bureau_factor_6.ratio_amt_al >0)),'2. GE 0& LT 21.9',
'3. GE 21.9' )),

w_ratio_amt_al= np.where((bureau_factor_6.ratio_amt_al< 0), -0.318852625616231,
np.where(((bureau_factor_6.ratio_amt_al <21.9) & (bureau_factor_6.ratio_amt_al >0)),-0.0339187513901666,
0.387197959779622 )),


b_nbr_closed_active_accts= np.where((bureau_factor_6.nbr_closed_active_accts< 4), '1. Less than 4',
np.where(((bureau_factor_6.nbr_closed_active_accts <9) & (bureau_factor_6.nbr_closed_active_accts >4)),'2. GE 4& LT 9',
'3. GE 9' )),

w_nbr_closed_active_accts= np.where((bureau_factor_6.nbr_closed_active_accts< 4), -0.0768837918294025,
np.where(((bureau_factor_6.nbr_closed_active_accts <9) & (bureau_factor_6.nbr_closed_active_accts >4)),0.105934769144095,
0.116465445641615 )),


b_util_l12m_all_live= np.where((bureau_factor_6.util_l12m_all_live< 0.247073677), '1. Less than 0.247073677',
np.where(((bureau_factor_6.util_l12m_all_live <0.463808037) & (bureau_factor_6.util_l12m_all_live >0.247073677)),'2. GE 0.247073677& LT 0.463808037',
np.where(((bureau_factor_6.util_l12m_all_live <0.765684489) & (bureau_factor_6.util_l12m_all_live >0.463808037)),'3. GE 0.463808037& LT 0.765684489',
'4. GE 0.765684489' ))),

w_util_l12m_all_live= np.where((bureau_factor_6.util_l12m_all_live< 0.247073677), 0.0438646327784642,
np.where(((bureau_factor_6.util_l12m_all_live <0.463808037) & (bureau_factor_6.util_l12m_all_live >0.247073677)),0.0202136461820288,
np.where(((bureau_factor_6.util_l12m_all_live <0.765684489) & (bureau_factor_6.util_l12m_all_live >0.463808037)),-0.0291297361295969,
-0.212423162769499 ))),


b_min_mob_com= np.where((bureau_factor_6.min_mob_com< 0), '1. Less than 0',
np.where(((bureau_factor_6.min_mob_com <32) & (bureau_factor_6.min_mob_com >0)),'2. GE 0& LT 32',
'3. GE 32' )),

w_min_mob_com= np.where((bureau_factor_6.min_mob_com< 0), -0.0826288504399251,
np.where(((bureau_factor_6.min_mob_com <32) & (bureau_factor_6.min_mob_com >0)),0.319806369659645,
1.20324327666272 )),


b_max_mob_com= np.where((bureau_factor_6.max_mob_com< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_mob_com <32) & (bureau_factor_6.max_mob_com >0)),'2. GE 0& LT 32',
'3. GE 32' )),

w_max_mob_com= np.where((bureau_factor_6.max_mob_com< 0), -0.0826288504399251,
np.where(((bureau_factor_6.max_mob_com <32) & (bureau_factor_6.max_mob_com >0)),0.319806369659645,
1.20324327666272 )),


b_nbr_closed_accts= np.where((bureau_factor_6.nbr_closed_accts< 6), '1. Less than 6',
np.where(((bureau_factor_6.nbr_closed_accts <10) & (bureau_factor_6.nbr_closed_accts >6)),'2. GE 6& LT 10',
'3. GE 10' )),

w_nbr_closed_accts= np.where((bureau_factor_6.nbr_closed_accts< 6), -0.101149836064909,
np.where(((bureau_factor_6.nbr_closed_accts <10) & (bureau_factor_6.nbr_closed_accts >6)),0.129948795978902,
0.213026407347592 )),


b_nbr_uns_tot_accts= np.where((bureau_factor_6.nbr_uns_tot_accts< 5), '1. Less than 5',
np.where(((bureau_factor_6.nbr_uns_tot_accts <9) & (bureau_factor_6.nbr_uns_tot_accts >5)),'2. GE 5& LT 9',
np.where(((bureau_factor_6.nbr_uns_tot_accts <12) & (bureau_factor_6.nbr_uns_tot_accts >9)),'3. GE 9& LT 12',
'4. GE 12' ))),

w_nbr_uns_tot_accts= np.where((bureau_factor_6.nbr_uns_tot_accts< 5), -0.182799163558998,
np.where(((bureau_factor_6.nbr_uns_tot_accts <9) & (bureau_factor_6.nbr_uns_tot_accts >5)),0.0689129053925328,
np.where(((bureau_factor_6.nbr_uns_tot_accts <12) & (bureau_factor_6.nbr_uns_tot_accts >9)),0.144636322608311,
0.175718101084678 ))),


b_nbr_pl_tot_accts= np.where((bureau_factor_6.nbr_pl_tot_accts< 3), '1. Less than 3',
np.where(((bureau_factor_6.nbr_pl_tot_accts <8) & (bureau_factor_6.nbr_pl_tot_accts >3)),'2. GE 3& LT 8',
'3. GE 8' )),

w_nbr_pl_tot_accts= np.where((bureau_factor_6.nbr_pl_tot_accts< 3), 0.162375146945693,
np.where(((bureau_factor_6.nbr_pl_tot_accts <8) & (bureau_factor_6.nbr_pl_tot_accts >3)),-0.13956597251743,
-0.390065253841495 )),


b_ratio_amt_tw= np.where((bureau_factor_6.ratio_amt_tw< 0), '1. Less than 0',
np.where(((bureau_factor_6.ratio_amt_tw <23.2) & (bureau_factor_6.ratio_amt_tw >0)),'2. GE 0& LT 23.2',
'3. GE 23.2' )),

w_ratio_amt_tw= np.where((bureau_factor_6.ratio_amt_tw< 0), -0.318852625616231,
np.where(((bureau_factor_6.ratio_amt_tw <23.2) & (bureau_factor_6.ratio_amt_tw >0)),0.0548281655107938,
-0.398895333289767 )),


b_util_curr_exc_cc_tot= np.where((bureau_factor_6.util_curr_exc_cc_tot< 0.001152222), '1. Less than 0.001152222',
np.where(((bureau_factor_6.util_curr_exc_cc_tot <0.246093169) & (bureau_factor_6.util_curr_exc_cc_tot >0.001152222)),'2. GE 0.001152222& LT 0.246093169',
np.where(((bureau_factor_6.util_curr_exc_cc_tot <0.488379253) & (bureau_factor_6.util_curr_exc_cc_tot >0.246093169)),'3. GE 0.246093169& LT 0.488379253',
np.where(((bureau_factor_6.util_curr_exc_cc_tot <0.694313776) & (bureau_factor_6.util_curr_exc_cc_tot >0.488379253)),'4. GE 0.488379253& LT 0.694313776',
'5. GE 0.694313776' )))),

w_util_curr_exc_cc_tot= np.where((bureau_factor_6.util_curr_exc_cc_tot< 0.001152222), -0.100505246565514,
np.where(((bureau_factor_6.util_curr_exc_cc_tot <0.246093169) & (bureau_factor_6.util_curr_exc_cc_tot >0.001152222)),-0.132953247474392,
np.where(((bureau_factor_6.util_curr_exc_cc_tot <0.488379253) & (bureau_factor_6.util_curr_exc_cc_tot >0.246093169)),0.0173175418288448,
np.where(((bureau_factor_6.util_curr_exc_cc_tot <0.694313776) & (bureau_factor_6.util_curr_exc_cc_tot >0.488379253)),0.0648906593450981,
0.200292106563725 )))),


b_nbr_pl_closed_accts= np.where((bureau_factor_6.nbr_pl_closed_accts< 3), '1. Less than 3',
np.where(((bureau_factor_6.nbr_pl_closed_accts <10) & (bureau_factor_6.nbr_pl_closed_accts >3)),'2. GE 3& LT 10',
'3. GE 10' )),

w_nbr_pl_closed_accts= np.where((bureau_factor_6.nbr_pl_closed_accts< 3), 0.0763660597699107,
np.where(((bureau_factor_6.nbr_pl_closed_accts <10) & (bureau_factor_6.nbr_pl_closed_accts >3)),-0.244036521720419,
-0.349624284282984 )),
                           
b_max_dpd_24m_com= np.where((bureau_factor_6.max_dpd_24m_com< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_24m_com <30) & (bureau_factor_6.max_dpd_24m_com >0)),'2. GE 0& LT 30',
'3. GE 30' )),

w_max_dpd_24m_com= np.where((bureau_factor_6.max_dpd_24m_com< 0), -0.0826288504399251,
np.where(((bureau_factor_6.max_dpd_24m_com <30) & (bureau_factor_6.max_dpd_24m_com >0)),1.08133084766433,
-0.341842143840929 )),


b_max_dpd_0m_tw= np.where((bureau_factor_6.max_dpd_0m_tw< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_0m_tw <29) & (bureau_factor_6.max_dpd_0m_tw >0)),'2. GE 0& LT 29',
'3. GE 29' )),

w_max_dpd_0m_tw= np.where((bureau_factor_6.max_dpd_0m_tw< 0), -0.00899669567012763,
np.where(((bureau_factor_6.max_dpd_0m_tw <29) & (bureau_factor_6.max_dpd_0m_tw >0)),0.414389012145593,
-1.16055450049931 )),


b_avg_util_l3m_exc_cc= np.where((bureau_factor_6.avg_util_l3m_exc_cc< 0.498646795), '1. Less than 0.498646795',
np.where(((bureau_factor_6.avg_util_l3m_exc_cc <0.874213421) & (bureau_factor_6.avg_util_l3m_exc_cc >0.498646795)),'2. GE 0.498646795& LT 0.874213421',
np.where(((bureau_factor_6.avg_util_l3m_exc_cc <1.330265384) & (bureau_factor_6.avg_util_l3m_exc_cc >0.874213421)),'3. GE 0.874213421& LT 1.330265384',
'4. GE 1.330265384' ))),

w_avg_util_l3m_exc_cc= np.where((bureau_factor_6.avg_util_l3m_exc_cc< 0.498646795), 0.291406895137947,
np.where(((bureau_factor_6.avg_util_l3m_exc_cc <0.874213421) & (bureau_factor_6.avg_util_l3m_exc_cc >0.498646795)),-0.0260762986427066,
np.where(((bureau_factor_6.avg_util_l3m_exc_cc <1.330265384) & (bureau_factor_6.avg_util_l3m_exc_cc >0.874213421)),-0.16825014584507,
-0.307348801134746 ))),


b_nbr_prod_0_6m_tw= np.where((bureau_factor_6.nbr_prod_0_6m_tw< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_0_6m_tw <1) & (bureau_factor_6.nbr_prod_0_6m_tw >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_0_6m_tw= np.where((bureau_factor_6.nbr_prod_0_6m_tw< 0), -0.0193118985541645,
np.where(((bureau_factor_6.nbr_prod_0_6m_tw <1) & (bureau_factor_6.nbr_prod_0_6m_tw >0)),0.53681348444149,
-0.842750330949574 )),


b_max_dpd_12m_tw= np.where((bureau_factor_6.max_dpd_12m_tw< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_12m_tw <30) & (bureau_factor_6.max_dpd_12m_tw >0)),'2. GE 0& LT 30',
'3. GE 30' )),

w_max_dpd_12m_tw= np.where((bureau_factor_6.max_dpd_12m_tw< 0), -0.0316392665547979,
np.where(((bureau_factor_6.max_dpd_12m_tw <30) & (bureau_factor_6.max_dpd_12m_tw >0)),0.242205123636093,
-0.84494572151301 )),


b_nbr_live_accts= np.where((bureau_factor_6.nbr_live_accts< 3), '1. Less than 3',
np.where(((bureau_factor_6.nbr_live_accts <8) & (bureau_factor_6.nbr_live_accts >3)),'2. GE 3& LT 8',
'3. GE 8' )),

w_nbr_live_accts= np.where((bureau_factor_6.nbr_live_accts< 3), -0.349624284282984,
np.where(((bureau_factor_6.nbr_live_accts <8) & (bureau_factor_6.nbr_live_accts >3)),0.153726665066301,
0.255861957718536 )),


b_nbr_live_active_accts= np.where((bureau_factor_6.nbr_live_active_accts< 3), '1. Less than 3',
np.where(((bureau_factor_6.nbr_live_active_accts <8) & (bureau_factor_6.nbr_live_active_accts >3)),'2. GE 3& LT 8',
'3. GE 8' )),

w_nbr_live_active_accts= np.where((bureau_factor_6.nbr_live_active_accts< 3), -0.336552202715632,
np.where(((bureau_factor_6.nbr_live_active_accts <8) & (bureau_factor_6.nbr_live_active_accts >3)),0.141824677788625,
0.27898437513939 )),


b_nbr_cl_closed_accts= np.where((bureau_factor_6.nbr_cl_closed_accts< 1), '1. Less than 1',
np.where(((bureau_factor_6.nbr_cl_closed_accts <5) & (bureau_factor_6.nbr_cl_closed_accts >1)),'2. GE 1& LT 5',
'3. GE 5' )),

w_nbr_cl_closed_accts= np.where((bureau_factor_6.nbr_cl_closed_accts< 1), -0.296701882828642,
np.where(((bureau_factor_6.nbr_cl_closed_accts <5) & (bureau_factor_6.nbr_cl_closed_accts >1)),0.107881701950224,
0.489476808900041 )),


b_nbr_accts_open_l12m_pl= np.where((bureau_factor_6.nbr_accts_open_l12m_pl< 2), '1. Less than 2',
np.where(((bureau_factor_6.nbr_accts_open_l12m_pl <10) & (bureau_factor_6.nbr_accts_open_l12m_pl >2)),'2. GE 2& LT 10',
'3. GE 10' )),

w_nbr_accts_open_l12m_pl= np.where((bureau_factor_6.nbr_accts_open_l12m_pl< 2), 0.179379271052184,
np.where(((bureau_factor_6.nbr_accts_open_l12m_pl <10) & (bureau_factor_6.nbr_accts_open_l12m_pl >2)),-0.214449505914732,
-0.470252272071599 )),


b_nbr_tot_accts= np.where((bureau_factor_6.nbr_tot_accts< 5), '1. Less than 5',
np.where(((bureau_factor_6.nbr_tot_accts <11) & (bureau_factor_6.nbr_tot_accts >5)),'2. GE 5& LT 11',
'3. GE 11' )),

w_nbr_tot_accts= np.where((bureau_factor_6.nbr_tot_accts< 5), -0.412059755223777,
np.where(((bureau_factor_6.nbr_tot_accts <11) & (bureau_factor_6.nbr_tot_accts >5)),0.118815622986568,
0.186407299107239 )),


b_nbr_cc_live_active_accts= np.where((bureau_factor_6.nbr_cc_live_active_accts< 1), '1. Less than 1',
'2. GE 1' ),

w_nbr_cc_live_active_accts= np.where((bureau_factor_6.nbr_cc_live_active_accts< 1), -0.206523440642311,
0.468526179906831 ),


b_ratio_amt_sec= np.where((bureau_factor_6.ratio_amt_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.ratio_amt_sec <32.3) & (bureau_factor_6.ratio_amt_sec >0)),'2. GE 0& LT 32.3',
'3. GE 32.3' )),

w_ratio_amt_sec= np.where((bureau_factor_6.ratio_amt_sec< 0), -0.318852625616231,
np.where(((bureau_factor_6.ratio_amt_sec <32.3) & (bureau_factor_6.ratio_amt_sec >0)),-0.0307271924832789,
0.0862383010095511 )),


b_nbr_cl_tot_accts= np.where((bureau_factor_6.nbr_cl_tot_accts< 3), '1. Less than 3',
np.where(((bureau_factor_6.nbr_cl_tot_accts <9) & (bureau_factor_6.nbr_cl_tot_accts >3)),'2. GE 3& LT 9',
'3. GE 9' )),

w_nbr_cl_tot_accts= np.where((bureau_factor_6.nbr_cl_tot_accts< 3), -0.12157475419992,
np.where(((bureau_factor_6.nbr_cl_tot_accts <9) & (bureau_factor_6.nbr_cl_tot_accts >3)),0.0446675232270548,
0.68836338256869 )),


b_nbr_sec_live_accts= np.where((bureau_factor_6.nbr_sec_live_accts< 1), '1. Less than 1',
'2. GE 1' ),

w_nbr_sec_live_accts= np.where((bureau_factor_6.nbr_sec_live_accts< 1), -0.13254302764432,
0.189950752768992 ),


b_max_mob_sec= np.where((bureau_factor_6.max_mob_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_mob_sec <15) & (bureau_factor_6.max_mob_sec >0)),'2. GE 0& LT 15',
np.where(((bureau_factor_6.max_mob_sec <36) & (bureau_factor_6.max_mob_sec >15)),'3. GE 15& LT 36',
'4. GE 36' ))),

w_max_mob_sec= np.where((bureau_factor_6.max_mob_sec< 0), -0.189174802307698,
np.where(((bureau_factor_6.max_mob_sec <15) & (bureau_factor_6.max_mob_sec >0)),0.0558408238251802,
np.where(((bureau_factor_6.max_mob_sec <36) & (bureau_factor_6.max_mob_sec >15)),0.132801864961308,
0.159276411073422 ))),


b_nbr_con_live_accts= np.where((bureau_factor_6.nbr_con_live_accts< 3), '1. Less than 3',
np.where(((bureau_factor_6.nbr_con_live_accts <7) & (bureau_factor_6.nbr_con_live_accts >3)),'2. GE 3& LT 7',
'3. GE 7' )),

w_nbr_con_live_accts= np.where((bureau_factor_6.nbr_con_live_accts< 3), -0.324125707514389,
np.where(((bureau_factor_6.nbr_con_live_accts <7) & (bureau_factor_6.nbr_con_live_accts >3)),0.152232669500246,
0.260204480487794 )),


b_avg_util_curr_all_tot= np.where((bureau_factor_6.avg_util_curr_all_tot< 0.192975053), '1. Less than 0.192975053',
np.where(((bureau_factor_6.avg_util_curr_all_tot <0.395315197) & (bureau_factor_6.avg_util_curr_all_tot >0.192975053)),'2. GE 0.192975053& LT 0.395315197',
'3. GE 0.395315197' )),

w_avg_util_curr_all_tot= np.where((bureau_factor_6.avg_util_curr_all_tot< 0.192975053), 0.176145928914273,
np.where(((bureau_factor_6.avg_util_curr_all_tot <0.395315197) & (bureau_factor_6.avg_util_curr_all_tot >0.192975053)),-0.0319347870572055,
-0.173345041944992 )),


b_util_l3m_all_tot= np.where((bureau_factor_6.util_l3m_all_tot< 0.065752921), '1. Less than 0.065752921',
np.where(((bureau_factor_6.util_l3m_all_tot <0.185010539) & (bureau_factor_6.util_l3m_all_tot >0.065752921)),'2. GE 0.065752921& LT 0.185010539',
np.where(((bureau_factor_6.util_l3m_all_tot <0.424280508) & (bureau_factor_6.util_l3m_all_tot >0.185010539)),'3. GE 0.185010539& LT 0.424280508',
np.where(((bureau_factor_6.util_l3m_all_tot <0.699433979) & (bureau_factor_6.util_l3m_all_tot >0.424280508)),'4. GE 0.424280508& LT 0.699433979',
'5. GE 0.699433979' )))),

w_util_l3m_all_tot= np.where((bureau_factor_6.util_l3m_all_tot< 0.065752921), -0.398895333289767,
np.where(((bureau_factor_6.util_l3m_all_tot <0.185010539) & (bureau_factor_6.util_l3m_all_tot >0.065752921)),0.135883531498717,
np.where(((bureau_factor_6.util_l3m_all_tot <0.424280508) & (bureau_factor_6.util_l3m_all_tot >0.185010539)),0.124108794954183,
np.where(((bureau_factor_6.util_l3m_all_tot <0.699433979) & (bureau_factor_6.util_l3m_all_tot >0.424280508)),-0.011732601007545,
-0.13275034598237 )))),


b_mon_since30_sec= np.where((bureau_factor_6.mon_since30_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.mon_since30_sec <2) & (bureau_factor_6.mon_since30_sec >0)),'2. GE 0& LT 2',
'3. GE 2' )),

w_mon_since30_sec= np.where((bureau_factor_6.mon_since30_sec< 0), 0.150498333096165,
np.where(((bureau_factor_6.mon_since30_sec <2) & (bureau_factor_6.mon_since30_sec >0)),-0.612613744144394,
-0.479677412531182 )),


b_nbr_accts_open_l6m_pl= np.where((bureau_factor_6.nbr_accts_open_l6m_pl< 2), '1. Less than 2',
np.where(((bureau_factor_6.nbr_accts_open_l6m_pl <5) & (bureau_factor_6.nbr_accts_open_l6m_pl >2)),'2. GE 2& LT 5',
'3. GE 5' )),

w_nbr_accts_open_l6m_pl= np.where((bureau_factor_6.nbr_accts_open_l6m_pl< 2), 0.101188643803639,
np.where(((bureau_factor_6.nbr_accts_open_l6m_pl <5) & (bureau_factor_6.nbr_accts_open_l6m_pl >2)),-0.198504828359009,
-0.316398636654664 )),


b_avg_util_l6m_exc_cc= np.where((bureau_factor_6.avg_util_l6m_exc_cc< 0.222041667), '1. Less than 0.222041667',
np.where(((bureau_factor_6.avg_util_l6m_exc_cc <1.543094762) & (bureau_factor_6.avg_util_l6m_exc_cc >0.222041667)),'2. GE 0.222041667& LT 1.543094762',
'3. GE 1.543094762' )),

w_avg_util_l6m_exc_cc= np.where((bureau_factor_6.avg_util_l6m_exc_cc< 0.222041667), -0.13275034598237,
np.where(((bureau_factor_6.avg_util_l6m_exc_cc <1.543094762) & (bureau_factor_6.avg_util_l6m_exc_cc >0.222041667)),0.113872371397365,
-0.161882660019691 )),


b_nbr_sec_closed_accts= np.where((bureau_factor_6.nbr_sec_closed_accts< 1), '1. Less than 1',
np.where(((bureau_factor_6.nbr_sec_closed_accts <6) & (bureau_factor_6.nbr_sec_closed_accts >1)),'2. GE 1& LT 6',
'3. GE 6' )),

w_nbr_sec_closed_accts= np.where((bureau_factor_6.nbr_sec_closed_accts< 1), -0.14267330405542,
np.where(((bureau_factor_6.nbr_sec_closed_accts <6) & (bureau_factor_6.nbr_sec_closed_accts >1)),0.218075622052397,
0.384344890797216 )),


b_ratio_amt_hl= np.where((bureau_factor_6.ratio_amt_hl< 0), '1. Less than 0',
np.where(((bureau_factor_6.ratio_amt_hl <2.6) & (bureau_factor_6.ratio_amt_hl >0)),'2. GE 0& LT 2.6',
'3. GE 2.6' )),

w_ratio_amt_hl= np.where((bureau_factor_6.ratio_amt_hl< 0), -0.318852625616231,
np.where(((bureau_factor_6.ratio_amt_hl <2.6) & (bureau_factor_6.ratio_amt_hl >0)),-0.0423438537625749,
0.569786574927414 )),


b_nbr_prod_30_24m_al= np.where((bureau_factor_6.nbr_prod_30_24m_al< 0), '1. Less than 0',
'2. GE 0' ),

w_nbr_prod_30_24m_al= np.where((bureau_factor_6.nbr_prod_30_24m_al< 0), -0.0793339545430727,
0.52260645779714 ),


b_mon_since30_tw= np.where((bureau_factor_6.mon_since30_tw< 0), '1. Less than 0',
'2. GE 0' ),

w_mon_since30_tw= np.where((bureau_factor_6.mon_since30_tw< 0), 0.0881807318030329,
-0.899670621202256 ),


b_nbr_prod_90_24m_sec= np.where((bureau_factor_6.nbr_prod_90_24m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_90_24m_sec <1) & (bureau_factor_6.nbr_prod_90_24m_sec >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_90_24m_sec= np.where((bureau_factor_6.nbr_prod_90_24m_sec< 0), -0.219341085862369,
np.where(((bureau_factor_6.nbr_prod_90_24m_sec <1) & (bureau_factor_6.nbr_prod_90_24m_sec >0)),0.302137680114107,
-0.497544414359607 )),


b_nbr_prod_90_12m_sec= np.where((bureau_factor_6.nbr_prod_90_12m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_90_12m_sec <1) & (bureau_factor_6.nbr_prod_90_12m_sec >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_90_12m_sec= np.where((bureau_factor_6.nbr_prod_90_12m_sec< 0), -0.219341085862369,
np.where(((bureau_factor_6.nbr_prod_90_12m_sec <1) & (bureau_factor_6.nbr_prod_90_12m_sec >0)),0.302137680114107,
-0.497544414359607 )),
                 
b_avg_util_l12m_all_tot= np.where((bureau_factor_6.avg_util_l12m_all_tot< 2.340334138), '1. Less than 2.340334138',
'2. GE 2.340334138' ),

w_avg_util_l12m_all_tot= np.where((bureau_factor_6.avg_util_l12m_all_tot< 2.340334138), 0.0689989084026911,
-0.080778579),


b_nbr_prod_30_24m_sec= np.where((bureau_factor_6.nbr_prod_30_24m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_30_24m_sec <1) & (bureau_factor_6.nbr_prod_30_24m_sec >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_30_24m_sec= np.where((bureau_factor_6.nbr_prod_30_24m_sec< 0), -0.219341085862369,
np.where(((bureau_factor_6.nbr_prod_30_24m_sec <1) & (bureau_factor_6.nbr_prod_30_24m_sec >0)),0.457595352536809,
-0.713292263712687 )),


b_nbr_prod_30_12m_sec= np.where((bureau_factor_6.nbr_prod_30_12m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_30_12m_sec <1) & (bureau_factor_6.nbr_prod_30_12m_sec >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_30_12m_sec= np.where((bureau_factor_6.nbr_prod_30_12m_sec< 0), -0.219341085862369,
np.where(((bureau_factor_6.nbr_prod_30_12m_sec <1) & (bureau_factor_6.nbr_prod_30_12m_sec >0)),0.457595352536809,
-0.713292263712687 )),


b_max_live_mob_con= np.where((bureau_factor_6.max_live_mob_con< 5), '1. Less than 5',
'2. GE 5' ),

w_max_live_mob_con= np.where((bureau_factor_6.max_live_mob_con< 5), -0.435069490239869,
0.0845204315197096 ),


b_nbr_accts_open_l9m= np.where((bureau_factor_6.nbr_accts_open_l9m< 10), '1. Less than 10',
'2. GE 10' ),

w_nbr_accts_open_l9m= np.where((bureau_factor_6.nbr_accts_open_l9m< 10), 0.0341222998705372,
-0.302937170310331 ),


b_nbr_cc_tot_accts= np.where((bureau_factor_6.nbr_cc_tot_accts< 2), '1. Less than 2',
np.where(((bureau_factor_6.nbr_cc_tot_accts <6) & (bureau_factor_6.nbr_cc_tot_accts >2)),'2. GE 2& LT 6',
'3. GE 6' )),

w_nbr_cc_tot_accts= np.where((bureau_factor_6.nbr_cc_tot_accts< 2), -0.183248693389736,
np.where(((bureau_factor_6.nbr_cc_tot_accts <6) & (bureau_factor_6.nbr_cc_tot_accts >2)),0.508330870578207,
1.29058528681787 )),


b_max_dpd_12m_sec= np.where((bureau_factor_6.max_dpd_12m_sec< 60), '1. Less than 60',
'2. GE 60' ),

w_max_dpd_12m_sec= np.where((bureau_factor_6.max_dpd_12m_sec< 60), 0.0806313844052473,
-0.713292263712687 ),




b_avg_util_l6m_all_act= np.where((bureau_factor_6.avg_util_l6m_all_act< 1.233336721), '1. Less than 1.233336721',
np.where(((bureau_factor_6.avg_util_l6m_all_act <1.97032) & (bureau_factor_6.avg_util_l6m_all_act >1.233336721)),'2. GE 1.233336721& LT 1.97032',
'3. GE 1.97032' )),

w_avg_util_l6m_all_act= np.where((bureau_factor_6.avg_util_l6m_all_act< 1.233336721), 0.200292106563725,
np.where(((bureau_factor_6.avg_util_l6m_all_act <1.97032) & (bureau_factor_6.avg_util_l6m_all_act >1.233336721)),-0.043365826258268,
-0.266130322105233 )),


b_max_dpd_6m_sec= np.where((bureau_factor_6.max_dpd_6m_sec< 29), '1. Less than 29',
np.where(((bureau_factor_6.max_dpd_6m_sec <30) & (bureau_factor_6.max_dpd_6m_sec >29)),'2. GE 29& LT 30',
'3. GE 30' )),

w_max_dpd_6m_sec= np.where((bureau_factor_6.max_dpd_6m_sec< 29), 0.105048611721709,
np.where(((bureau_factor_6.max_dpd_6m_sec <30) & (bureau_factor_6.max_dpd_6m_sec >29)),-0.231841248626601,
-0.671991914722655 )),


b_count_settlement_life= np.where((bureau_factor_6.count_settlement_life< 1), '1. Less than 1',
'2. GE 1' ),

w_count_settlement_life= np.where((bureau_factor_6.count_settlement_life< 1), 0.0960376229515169,
-1.42282685742573 ),


b_count_settlement_36m= np.where((bureau_factor_6.count_settlement_36m< 1), '1. Less than 1',
'2. GE 1' ),

w_count_settlement_36m= np.where((bureau_factor_6.count_settlement_36m< 1), 0.0960376229515169,
-1.42282685742573 ),


b_max_dpd_3m_sec= np.where((bureau_factor_6.max_dpd_3m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_3m_sec <29) & (bureau_factor_6.max_dpd_3m_sec >0)),'2. GE 0& LT 29',
np.where(((bureau_factor_6.max_dpd_3m_sec <60) & (bureau_factor_6.max_dpd_3m_sec >29)),'3. GE 29& LT 60',
'4. GE 60' ))),

w_max_dpd_3m_sec= np.where((bureau_factor_6.max_dpd_3m_sec< 0), -0.189174802307698,
np.where(((bureau_factor_6.max_dpd_3m_sec <29) & (bureau_factor_6.max_dpd_3m_sec >0)),0.376041280278656,
np.where(((bureau_factor_6.max_dpd_3m_sec <60) & (bureau_factor_6.max_dpd_3m_sec >29)),-0.406782698122933,
-0.805929069170558 ))),


b_nbr_accts_open_l9m_pl= np.where((bureau_factor_6.nbr_accts_open_l9m_pl< 3), '1. Less than 3',
'2. GE 3' ),

w_nbr_accts_open_l9m_pl= np.where((bureau_factor_6.nbr_accts_open_l9m_pl< 3), 0.107374473940363,
-0.339010083041211 ),


b_ratio_avg_util_C1to3toC1to12= np.where((bureau_factor_6.ratio_avg_util_C1to3toC1to12< 1608), '1. Less than 1608',
np.where(((bureau_factor_6.ratio_avg_util_C1to3toC1to12 <2575.8) & (bureau_factor_6.ratio_avg_util_C1to3toC1to12 >1608)),'2. GE 1608& LT 2575.8',
'3. GE 2575.8' )),

w_ratio_avg_util_C1to3toC1to12= np.where((bureau_factor_6.ratio_avg_util_C1to3toC1to12< 1608), 0.148499654307885,
np.where(((bureau_factor_6.ratio_avg_util_C1to3toC1to12 <2575.8) & (bureau_factor_6.ratio_avg_util_C1to3toC1to12 >1608)),0.0272674513811242,
-0.604516533911774 )), 

b_ratio_avg_util_c0toc1to6= np.where((bureau_factor_6.ratio_avg_util_c0toc1to6< 1.4), '1. Less than 1.4',
np.where(((bureau_factor_6.ratio_avg_util_c0toc1to6 <20.2) & (bureau_factor_6.ratio_avg_util_c0toc1to6 >1.4)),'2. GE 1.4& LT 20.2',
'3. GE 20.2' )),

w_ratio_avg_util_c0toc1to6= np.where((bureau_factor_6.ratio_avg_util_c0toc1to6< 1.4), -0.362869511033005,
np.where(((bureau_factor_6.ratio_avg_util_c0toc1to6 <20.2) & (bureau_factor_6.ratio_avg_util_c0toc1to6 >1.4)),-0.0357263697003104,
0.124833695312131 )),


b_nbr_accts_open_l3m= np.where((bureau_factor_6.nbr_accts_open_l3m< 3), '1. Less than 3',
'2. GE 3' ),

w_nbr_accts_open_l3m= np.where((bureau_factor_6.nbr_accts_open_l3m< 3), -0.105525367792786,
0.547920404770115 ),


b_util_l3m_all_live= np.where((bureau_factor_6.util_l3m_all_live< 0.032004938), '1. Less than 0.032004938',
np.where(((bureau_factor_6.util_l3m_all_live <0.380980539) & (bureau_factor_6.util_l3m_all_live >0.032004938)),'2. GE 0.032004938& LT 0.380980539',
np.where(((bureau_factor_6.util_l3m_all_live <0.632876988) & (bureau_factor_6.util_l3m_all_live >0.380980539)),'3. GE 0.380980539& LT 0.632876988',
np.where(((bureau_factor_6.util_l3m_all_live <0.836286822) & (bureau_factor_6.util_l3m_all_live >0.632876988)),'4. GE 0.632876988& LT 0.836286822',
'5. GE 0.836286822' )))),

w_util_l3m_all_live= np.where((bureau_factor_6.util_l3m_all_live< 0.032004938), -0.288999662466549,
np.where(((bureau_factor_6.util_l3m_all_live <0.380980539) & (bureau_factor_6.util_l3m_all_live >0.032004938)),0.15825582925325,
np.where(((bureau_factor_6.util_l3m_all_live <0.632876988) & (bureau_factor_6.util_l3m_all_live >0.380980539)),0.123144505721287,
np.where(((bureau_factor_6.util_l3m_all_live <0.836286822) & (bureau_factor_6.util_l3m_all_live >0.632876988)),0.0142918209123078,
-0.331924707183583 )))),


b_max_dpd_24m_sec= np.where((bureau_factor_6.max_dpd_24m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_24m_sec <29) & (bureau_factor_6.max_dpd_24m_sec >0)),'2. GE 0& LT 29',
np.where(((bureau_factor_6.max_dpd_24m_sec <60) & (bureau_factor_6.max_dpd_24m_sec >29)),'3. GE 29& LT 60',
'4. GE 60' ))),

w_max_dpd_24m_sec= np.where((bureau_factor_6.max_dpd_24m_sec< 0), -0.189174802307698,
np.where(((bureau_factor_6.max_dpd_24m_sec <29) & (bureau_factor_6.max_dpd_24m_sec >0)),0.615456611760603,
np.where(((bureau_factor_6.max_dpd_24m_sec <60) & (bureau_factor_6.max_dpd_24m_sec >29)),-0.0541600713891484,
-0.85171622808022 ))),


b_nbr_accts_open_l9m_cc= np.where((bureau_factor_6.nbr_accts_open_l9m_cc< 1), '1. Less than 1',
'2. GE 1' ),

w_nbr_accts_open_l9m_cc= np.where((bureau_factor_6.nbr_accts_open_l9m_cc< 1), -0.182463367719711,
1.17914572508366 ),


b_nbr_accts_open_l3m_con= np.where((bureau_factor_6.nbr_accts_open_l3m_con< 3), '1. Less than 3',
'2. GE 3' ),

w_nbr_accts_open_l3m_con= np.where((bureau_factor_6.nbr_accts_open_l3m_con< 3), -0.0952100900809537,
0.507825947568237 ),


b_nbr_prod_0_24m_sec= np.where((bureau_factor_6.nbr_prod_0_24m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_0_24m_sec <1) & (bureau_factor_6.nbr_prod_0_24m_sec >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_0_24m_sec= np.where((bureau_factor_6.nbr_prod_0_24m_sec< 0), -0.219341085862369,
np.where(((bureau_factor_6.nbr_prod_0_24m_sec <1) & (bureau_factor_6.nbr_prod_0_24m_sec >0)),0.67984118016136,
-0.422415947177276 )),


b_nbr_prod_0_12m_sec= np.where((bureau_factor_6.nbr_prod_0_12m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_0_12m_sec <1) & (bureau_factor_6.nbr_prod_0_12m_sec >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_0_12m_sec= np.where((bureau_factor_6.nbr_prod_0_12m_sec< 0), -0.219341085862369,
np.where(((bureau_factor_6.nbr_prod_0_12m_sec <1) & (bureau_factor_6.nbr_prod_0_12m_sec >0)),0.67984118016136,
-0.422415947177276 )),


b_nbr_accts_open_l3m_uns= np.where((bureau_factor_6.nbr_accts_open_l3m_uns< 3), '1. Less than 3',
'2. GE 3' ),

w_nbr_accts_open_l3m_uns= np.where((bureau_factor_6.nbr_accts_open_l3m_uns< 3), -0.0771391488791116,
0.482583330380629 ),


b_nbr_prod_0_6m_sec= np.where((bureau_factor_6.nbr_prod_0_6m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_0_6m_sec <1) & (bureau_factor_6.nbr_prod_0_6m_sec >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_0_6m_sec= np.where((bureau_factor_6.nbr_prod_0_6m_sec< 0), -0.186423768935808,
np.where(((bureau_factor_6.nbr_prod_0_6m_sec <1) & (bureau_factor_6.nbr_prod_0_6m_sec >0)),0.613186463226064,
-0.52311398920911 )),


b_max_mob_uns= np.where((bureau_factor_6.max_mob_uns< 4), '1. Less than 4',
np.where(((bureau_factor_6.max_mob_uns <20) & (bureau_factor_6.max_mob_uns >4)),'2. GE 4& LT 20',
np.where(((bureau_factor_6.max_mob_uns <43) & (bureau_factor_6.max_mob_uns >20)),'3. GE 20& LT 43',
'4. GE 43' ))),

w_max_mob_uns= np.where((bureau_factor_6.max_mob_uns< 4), -0.483155676907507,
np.where(((bureau_factor_6.max_mob_uns <20) & (bureau_factor_6.max_mob_uns >4)),-0.372198606321523,
np.where(((bureau_factor_6.max_mob_uns <43) & (bureau_factor_6.max_mob_uns >20)),0.210831486902163,
0.299071133706127 ))),


b_max_mob_pl= np.where((bureau_factor_6.max_mob_pl< 1), '1. Less than 1',
np.where(((bureau_factor_6.max_mob_pl <6) & (bureau_factor_6.max_mob_pl >1)),'2. GE 1& LT 6',
np.where(((bureau_factor_6.max_mob_pl <25) & (bureau_factor_6.max_mob_pl >6)),'3. GE 6& LT 25',
'4. GE 25' ))),

w_max_mob_pl= np.where((bureau_factor_6.max_mob_pl< 1), 1.90822491486967,
np.where(((bureau_factor_6.max_mob_pl <6) & (bureau_factor_6.max_mob_pl >1)),-0.188612514075781,
np.where(((bureau_factor_6.max_mob_pl <25) & (bureau_factor_6.max_mob_pl >6)),-0.103092265882293,
0.19148147688157 ))),


b_max_dpd_24m_tw= np.where((bureau_factor_6.max_dpd_24m_tw< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_24m_tw <29) & (bureau_factor_6.max_dpd_24m_tw >0)),'2. GE 0& LT 29',
np.where(((bureau_factor_6.max_dpd_24m_tw <60) & (bureau_factor_6.max_dpd_24m_tw >29)),'3. GE 29& LT 60',
'4. GE 60' ))),

w_max_dpd_24m_tw= np.where((bureau_factor_6.max_dpd_24m_tw< 0), -0.0316392665547979,
np.where(((bureau_factor_6.max_dpd_24m_tw <29) & (bureau_factor_6.max_dpd_24m_tw >0)),0.608789920402414,
np.where(((bureau_factor_6.max_dpd_24m_tw <60) & (bureau_factor_6.max_dpd_24m_tw >29)),-0.57903744861079,
-1.13629752285375 ))),


b_ratio_avg_util_C1to3toC1to6= np.where((bureau_factor_6.ratio_avg_util_C1to3toC1to6< 44.1), '1. Less than 44.1',
np.where(((bureau_factor_6.ratio_avg_util_C1to3toC1to6 <61.5) & (bureau_factor_6.ratio_avg_util_C1to3toC1to6 >44.1)),'2. GE 44.1& LT 61.5',
np.where(((bureau_factor_6.ratio_avg_util_C1to3toC1to6 <85.6) & (bureau_factor_6.ratio_avg_util_C1to3toC1to6 >61.5)),'3. GE 61.5& LT 85.6',
'4. GE 85.6' ))),

w_ratio_avg_util_C1to3toC1to6= np.where((bureau_factor_6.ratio_avg_util_C1to3toC1to6< 44.1), 0.326330495736588,
np.where(((bureau_factor_6.ratio_avg_util_C1to3toC1to6 <61.5) & (bureau_factor_6.ratio_avg_util_C1to3toC1to6 >44.1)),0.0483725292176542,
np.where(((bureau_factor_6.ratio_avg_util_C1to3toC1to6 <85.6) & (bureau_factor_6.ratio_avg_util_C1to3toC1to6 >61.5)),-0.0869498942121394,
-0.888620785015671 ))),


b_max_mob_con= np.where((bureau_factor_6.max_mob_con< 18), '1. Less than 18',
np.where(((bureau_factor_6.max_mob_con <59) & (bureau_factor_6.max_mob_con >18)),'2. GE 18& LT 59',
'3. GE 59' )),

w_max_mob_con= np.where((bureau_factor_6.max_mob_con< 18), -0.409047704753785,
np.where(((bureau_factor_6.max_mob_con <59) & (bureau_factor_6.max_mob_con >18)),0.0909321436364501,
0.171912995077934 )),


b_nbr_prod_0_3m_sec= np.where((bureau_factor_6.nbr_prod_0_3m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_0_3m_sec <1) & (bureau_factor_6.nbr_prod_0_3m_sec >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_0_3m_sec= np.where((bureau_factor_6.nbr_prod_0_3m_sec< 0), -0.170283355627168,
np.where(((bureau_factor_6.nbr_prod_0_3m_sec <1) & (bureau_factor_6.nbr_prod_0_3m_sec >0)),0.575716283111089,
-0.593631293232104 )),


b_max_dpd_0m_sec= np.where((bureau_factor_6.max_dpd_0m_sec< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_0m_sec <29) & (bureau_factor_6.max_dpd_0m_sec >0)),'2. GE 0& LT 29',
np.where(((bureau_factor_6.max_dpd_0m_sec <60) & (bureau_factor_6.max_dpd_0m_sec >29)),'3. GE 29& LT 60',
'4. GE 60' ))),

w_max_dpd_0m_sec= np.where((bureau_factor_6.max_dpd_0m_sec< 0), -0.165841806849355,
np.where(((bureau_factor_6.max_dpd_0m_sec <29) & (bureau_factor_6.max_dpd_0m_sec >0)),0.531549976558179,
np.where(((bureau_factor_6.max_dpd_0m_sec <60) & (bureau_factor_6.max_dpd_0m_sec >29)),-0.470252272071599,
-0.899670621202256 ))),


b_pct_live_act_bal_cc= np.where((bureau_factor_6.pct_live_act_bal_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.pct_live_act_bal_cc <0.533333333) & (bureau_factor_6.pct_live_act_bal_cc >0)),'2. GE 0& LT 0.533333333',
'3. GE 0.533333333' )),

w_pct_live_act_bal_cc= np.where((bureau_factor_6.pct_live_act_bal_cc< 0), -0.273135307086566,
np.where(((bureau_factor_6.pct_live_act_bal_cc <0.533333333) & (bureau_factor_6.pct_live_act_bal_cc >0)),0.72260124921193,
0.33410367436047 )),


b_max_mob_all= np.where((bureau_factor_6.max_mob_all< 16), '1. Less than 16',
np.where(((bureau_factor_6.max_mob_all <43) & (bureau_factor_6.max_mob_all >16)),'2. GE 16& LT 43',
np.where(((bureau_factor_6.max_mob_all <56) & (bureau_factor_6.max_mob_all >43)),'3. GE 43& LT 56',
'4. GE 56' ))),

w_max_mob_all= np.where((bureau_factor_6.max_mob_all< 16), -0.456308426871319,
np.where(((bureau_factor_6.max_mob_all <43) & (bureau_factor_6.max_mob_all >16)),0.0266580570681202,
np.where(((bureau_factor_6.max_mob_all <56) & (bureau_factor_6.max_mob_all >43)),0.117972604932186,
0.183940826452496 ))),


b_max_nbr_90_24m_all= np.where((bureau_factor_6.max_nbr_90_24m_all< 0.083333333), '1. Less than 0.083333333',
np.where(((bureau_factor_6.max_nbr_90_24m_all <0.916666667) & (bureau_factor_6.max_nbr_90_24m_all >0.083333333)),'2. GE 0.083333333& LT 0.916666667',
'3. GE 0.916666667' )),

w_max_nbr_90_24m_all= np.where((bureau_factor_6.max_nbr_90_24m_all< 0.083333333), 0.63596461598173,
np.where(((bureau_factor_6.max_nbr_90_24m_all <0.916666667) & (bureau_factor_6.max_nbr_90_24m_all >0.083333333)),-1.20415981197042,
-1.37830955847235 )),


b_max_nbr_90_24m_act= np.where((bureau_factor_6.max_nbr_90_24m_act< 0.083333333), '1. Less than 0.083333333',
np.where(((bureau_factor_6.max_nbr_90_24m_act <0.916666667) & (bureau_factor_6.max_nbr_90_24m_act >0.083333333)),'2. GE 0.083333333& LT 0.916666667',
'3. GE 0.916666667' )),

w_max_nbr_90_24m_act= np.where((bureau_factor_6.max_nbr_90_24m_act< 0.083333333), 0.63596461598173,
np.where(((bureau_factor_6.max_nbr_90_24m_act <0.916666667) & (bureau_factor_6.max_nbr_90_24m_act >0.083333333)),-1.20415981197042,
-1.37830955847235 )),


b_max_nbr_90_12m_all= np.where((bureau_factor_6.max_nbr_90_12m_all< 0.083333333), '1. Less than 0.083333333',
np.where(((bureau_factor_6.max_nbr_90_12m_all <0.916666667) & (bureau_factor_6.max_nbr_90_12m_all >0.083333333)),'2. GE 0.083333333& LT 0.916666667',
'3. GE 0.916666667' )),

w_max_nbr_90_12m_all= np.where((bureau_factor_6.max_nbr_90_12m_all< 0.083333333), 0.63596461598173,
np.where(((bureau_factor_6.max_nbr_90_12m_all <0.916666667) & (bureau_factor_6.max_nbr_90_12m_all >0.083333333)),-1.20415981197042,
-1.37830955847235 )),


b_max_nbr_90_12m_act= np.where((bureau_factor_6.max_nbr_90_12m_act< 0.083333333), '1. Less than 0.083333333',
np.where(((bureau_factor_6.max_nbr_90_12m_act <0.916666667) & (bureau_factor_6.max_nbr_90_12m_act >0.083333333)),'2. GE 0.083333333& LT 0.916666667',
'3. GE 0.916666667' )),

w_max_nbr_90_12m_act= np.where((bureau_factor_6.max_nbr_90_12m_act< 0.083333333), 0.63596461598173,
np.where(((bureau_factor_6.max_nbr_90_12m_act <0.916666667) & (bureau_factor_6.max_nbr_90_12m_act >0.083333333)),-1.20415981197042,
-1.37830955847235 )),


b_max_dpd_0m_cc= np.where((bureau_factor_6.max_dpd_0m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_0m_cc <29) & (bureau_factor_6.max_dpd_0m_cc >0)),'2. GE 0& LT 29',
np.where(((bureau_factor_6.max_dpd_0m_cc <180) & (bureau_factor_6.max_dpd_0m_cc >29)),'3. GE 29& LT 180',
'4. GE 180' ))),

w_max_dpd_0m_cc= np.where((bureau_factor_6.max_dpd_0m_cc< 0), -0.288411600114995,
np.where(((bureau_factor_6.max_dpd_0m_cc <29) & (bureau_factor_6.max_dpd_0m_cc >0)),1.40198811693166,
np.where(((bureau_factor_6.max_dpd_0m_cc <180) & (bureau_factor_6.max_dpd_0m_cc >29)),0.10802657699575,
-1.19692214467019 ))),


b_max_dpd_3m_cc= np.where((bureau_factor_6.max_dpd_3m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_3m_cc <29) & (bureau_factor_6.max_dpd_3m_cc >0)),'2. GE 0& LT 29',
np.where(((bureau_factor_6.max_dpd_3m_cc <180) & (bureau_factor_6.max_dpd_3m_cc >29)),'3. GE 29& LT 180',
'4. GE 180' ))),

w_max_dpd_3m_cc= np.where((bureau_factor_6.max_dpd_3m_cc< 0), -0.277066346631537,
np.where(((bureau_factor_6.max_dpd_3m_cc <29) & (bureau_factor_6.max_dpd_3m_cc >0)),1.39084189055752,
np.where(((bureau_factor_6.max_dpd_3m_cc <180) & (bureau_factor_6.max_dpd_3m_cc >29)),0.27898437513939,
-1.14813198050076 ))),


b_max_dpd_12m_cc= np.where((bureau_factor_6.max_dpd_12m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_12m_cc <29) & (bureau_factor_6.max_dpd_12m_cc >0)),'2. GE 0& LT 29',
np.where(((bureau_factor_6.max_dpd_12m_cc <60) & (bureau_factor_6.max_dpd_12m_cc >29)),'3. GE 29& LT 60',
'4. GE 60' ))),

w_max_dpd_12m_cc= np.where((bureau_factor_6.max_dpd_12m_cc< 0), -0.277066346631537,
np.where(((bureau_factor_6.max_dpd_12m_cc <29) & (bureau_factor_6.max_dpd_12m_cc >0)),1.34519323975284,
np.where(((bureau_factor_6.max_dpd_12m_cc <60) & (bureau_factor_6.max_dpd_12m_cc >29)),0.697694709997575,
-0.889897109375276 ))),


b_max_dpd_6m_cc= np.where((bureau_factor_6.max_dpd_6m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_6m_cc <29) & (bureau_factor_6.max_dpd_6m_cc >0)),'2. GE 0& LT 29',
np.where(((bureau_factor_6.max_dpd_6m_cc <60) & (bureau_factor_6.max_dpd_6m_cc >29)),'3. GE 29& LT 60',
'4. GE 60' ))),

w_max_dpd_6m_cc= np.where((bureau_factor_6.max_dpd_6m_cc< 0), -0.277066346631537,
np.where(((bureau_factor_6.max_dpd_6m_cc <29) & (bureau_factor_6.max_dpd_6m_cc >0)),1.31996755097086,
np.where(((bureau_factor_6.max_dpd_6m_cc <60) & (bureau_factor_6.max_dpd_6m_cc >29)),0.502127926453599,
-0.906296296174394 ))),


b_nbr_prod_0_3m_cc= np.where((bureau_factor_6.nbr_prod_0_3m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_0_3m_cc <1) & (bureau_factor_6.nbr_prod_0_3m_cc >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_0_3m_cc= np.where((bureau_factor_6.nbr_prod_0_3m_cc< 0), -0.28516428381602,
np.where(((bureau_factor_6.nbr_prod_0_3m_cc <1) & (bureau_factor_6.nbr_prod_0_3m_cc >0)),1.50809315273687,
-0.521134924972723 )),


b_nbr_prod_30_3m_cc= np.where((bureau_factor_6.nbr_prod_30_3m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_30_3m_cc <1) & (bureau_factor_6.nbr_prod_30_3m_cc >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_30_3m_cc= np.where((bureau_factor_6.nbr_prod_30_3m_cc< 0), -0.28516428381602,
np.where(((bureau_factor_6.nbr_prod_30_3m_cc <1) & (bureau_factor_6.nbr_prod_30_3m_cc >0)),1.13721130605878,
-1.01901737883482 )),


b_nbr_prod_90_24m_cc= np.where((bureau_factor_6.nbr_prod_90_24m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_90_24m_cc <1) & (bureau_factor_6.nbr_prod_90_24m_cc >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_90_24m_cc= np.where((bureau_factor_6.nbr_prod_90_24m_cc< 0), -0.283666316491187,
np.where(((bureau_factor_6.nbr_prod_90_24m_cc <1) & (bureau_factor_6.nbr_prod_90_24m_cc >0)),1.13503502663619,
-0.996680357642663 )),


b_nbr_prod_90_12m_cc= np.where((bureau_factor_6.nbr_prod_90_12m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_90_12m_cc <1) & (bureau_factor_6.nbr_prod_90_12m_cc >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_90_12m_cc= np.where((bureau_factor_6.nbr_prod_90_12m_cc< 0), -0.283666316491187,
np.where(((bureau_factor_6.nbr_prod_90_12m_cc <1) & (bureau_factor_6.nbr_prod_90_12m_cc >0)),1.13503502663619,
-0.996680357642663 )),


b_nbr_prod_90_3m_cc= np.where((bureau_factor_6.nbr_prod_90_3m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_90_3m_cc <1) & (bureau_factor_6.nbr_prod_90_3m_cc >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_90_3m_cc= np.where((bureau_factor_6.nbr_prod_90_3m_cc< 0), -0.28516428381602,
np.where(((bureau_factor_6.nbr_prod_90_3m_cc <1) & (bureau_factor_6.nbr_prod_90_3m_cc >0)),1.10230365366988,
-1.08359345936318 )),


b_nbr_prod_30_24m_cc= np.where((bureau_factor_6.nbr_prod_30_24m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_30_24m_cc <1) & (bureau_factor_6.nbr_prod_30_24m_cc >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_30_24m_cc= np.where((bureau_factor_6.nbr_prod_30_24m_cc< 0), -0.283666316491187,
np.where(((bureau_factor_6.nbr_prod_30_24m_cc <1) & (bureau_factor_6.nbr_prod_30_24m_cc >0)),1.18634647626924,
-0.889897109375276 )),


b_nbr_prod_30_12m_cc= np.where((bureau_factor_6.nbr_prod_30_12m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_30_12m_cc <1) & (bureau_factor_6.nbr_prod_30_12m_cc >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_30_12m_cc= np.where((bureau_factor_6.nbr_prod_30_12m_cc< 0), -0.283666316491187,
np.where(((bureau_factor_6.nbr_prod_30_12m_cc <1) & (bureau_factor_6.nbr_prod_30_12m_cc >0)),1.18634647626924,
-0.889897109375276 )),


b_max_dpd_24m_cc= np.where((bureau_factor_6.max_dpd_24m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.max_dpd_24m_cc <29) & (bureau_factor_6.max_dpd_24m_cc >0)),'2. GE 0& LT 29',
np.where(((bureau_factor_6.max_dpd_24m_cc <120) & (bureau_factor_6.max_dpd_24m_cc >29)),'3. GE 29& LT 120',
'4. GE 120' ))),

w_max_dpd_24m_cc= np.where((bureau_factor_6.max_dpd_24m_cc< 0), -0.277066346631537,
np.where(((bureau_factor_6.max_dpd_24m_cc <29) & (bureau_factor_6.max_dpd_24m_cc >0)),1.35187177256906,
np.where(((bureau_factor_6.max_dpd_24m_cc <120) & (bureau_factor_6.max_dpd_24m_cc >29)),0.643627488727299,
-0.869928652003519 ))),


b_nbr_prod_90_6m_cc= np.where((bureau_factor_6.nbr_prod_90_6m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_90_6m_cc <1) & (bureau_factor_6.nbr_prod_90_6m_cc >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_90_6m_cc= np.where((bureau_factor_6.nbr_prod_90_6m_cc< 0), -0.286932630872762,
np.where(((bureau_factor_6.nbr_prod_90_6m_cc <1) & (bureau_factor_6.nbr_prod_90_6m_cc >0)),1.11189349807449,
-0.985613051002981 )),


b_nbr_prod_30_6m_cc= np.where((bureau_factor_6.nbr_prod_30_6m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_30_6m_cc <1) & (bureau_factor_6.nbr_prod_30_6m_cc >0)),'2. GE 0& LT 1',
'3. GE 1' )),

w_nbr_prod_30_6m_cc= np.where((bureau_factor_6.nbr_prod_30_6m_cc< 0), -0.286932630872762,
np.where(((bureau_factor_6.nbr_prod_30_6m_cc <1) & (bureau_factor_6.nbr_prod_30_6m_cc >0)),1.14342764248158,
-0.906296296174394 )),


b_count_write_off_life= np.where((bureau_factor_6.count_write_off_life< 1), '1. Less than 1',
'2. GE 1' ),

w_count_write_off_life= np.where((bureau_factor_6.count_write_off_life< 1), 0.297994452364363,
-1.37490529986554 ),


b_count_write_off_36m= np.where((bureau_factor_6.count_write_off_36m< 1), '1. Less than 1',
'2. GE 1' ),

w_count_write_off_36m= np.where((bureau_factor_6.count_write_off_36m< 1), 0.297994452364363,
-1.37490529986554 ),


b_count_write_off_24m= np.where((bureau_factor_6.count_write_off_24m< 1), '1. Less than 1',
'2. GE 1' ),

w_count_write_off_24m= np.where((bureau_factor_6.count_write_off_24m< 1), 0.288934705992558,
-1.36435508897039 ),


b_count_write_off_12m= np.where((bureau_factor_6.count_write_off_12m< 1), '1. Less than 1',
'2. GE 1' ),

w_count_write_off_12m= np.where((bureau_factor_6.count_write_off_12m< 1), 0.280962615151496,
-1.3651390952826 ),


b_util_l3m_cc_tot= np.where((bureau_factor_6.util_l3m_cc_tot< -0.25696), '1. Less than -0.25696',
np.where(((bureau_factor_6.util_l3m_cc_tot <1.21514398) & (bureau_factor_6.util_l3m_cc_tot >-0.25696)),'2. GE -0.25696& LT 1.21514398',
np.where(((bureau_factor_6.util_l3m_cc_tot <1.932728959) & (bureau_factor_6.util_l3m_cc_tot >1.21514398)),'3. GE 1.21514398& LT 1.932728959',
'4. GE 1.932728959' ))),

w_util_l3m_cc_tot= np.where((bureau_factor_6.util_l3m_cc_tot< -0.25696), -0.276599149268094,
np.where(((bureau_factor_6.util_l3m_cc_tot <1.21514398) & (bureau_factor_6.util_l3m_cc_tot >-0.25696)),1.32435214928869,
np.where(((bureau_factor_6.util_l3m_cc_tot <1.932728959) & (bureau_factor_6.util_l3m_cc_tot >1.21514398)),0.709142095837926,
-0.380261253744874 ))),


b_nbr_prod_0_6m_cc= np.where((bureau_factor_6.nbr_prod_0_6m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_0_6m_cc <1) & (bureau_factor_6.nbr_prod_0_6m_cc >0)),'2. GE 0& LT 1',
np.where(((bureau_factor_6.nbr_prod_0_6m_cc <2) & (bureau_factor_6.nbr_prod_0_6m_cc >1)),'3. GE 1& LT 2',
'4. GE 2' ))),

w_nbr_prod_0_6m_cc= np.where((bureau_factor_6.nbr_prod_0_6m_cc< 0), -0.286932630872762,
np.where(((bureau_factor_6.nbr_prod_0_6m_cc <1) & (bureau_factor_6.nbr_prod_0_6m_cc >0)),1.43929023281284,
np.where(((bureau_factor_6.nbr_prod_0_6m_cc <2) & (bureau_factor_6.nbr_prod_0_6m_cc >1)),-0.278361264261494,
-0.54199617693044 ))),


b_util_l6m_cc_tot= np.where((bureau_factor_6.util_l6m_cc_tot< -0.57624), '1. Less than -0.57624',
np.where(((bureau_factor_6.util_l6m_cc_tot <2.822374572) & (bureau_factor_6.util_l6m_cc_tot >-0.57624)),'2. GE -0.57624& LT 2.822374572',
np.where(((bureau_factor_6.util_l6m_cc_tot <4.497621346) & (bureau_factor_6.util_l6m_cc_tot >2.822374572)),'3. GE 2.822374572& LT 4.497621346',
'4. GE 4.497621346' ))),

w_util_l6m_cc_tot= np.where((bureau_factor_6.util_l6m_cc_tot< -0.57624), -0.276599149268094,
np.where(((bureau_factor_6.util_l6m_cc_tot <2.822374572) & (bureau_factor_6.util_l6m_cc_tot >-0.57624)),1.25981362815112,
np.where(((bureau_factor_6.util_l6m_cc_tot <4.497621346) & (bureau_factor_6.util_l6m_cc_tot >2.822374572)),0.186869086231584,
-0.537776761387732 ))),


b_avg_util_l3m_cc= np.where((bureau_factor_6.avg_util_l3m_cc< -0.25696), '1. Less than -0.25696',
np.where(((bureau_factor_6.avg_util_l3m_cc <0.898315044) & (bureau_factor_6.avg_util_l3m_cc >-0.25696)),'2. GE -0.25696& LT 0.898315044',
np.where(((bureau_factor_6.avg_util_l3m_cc <2.373400115) & (bureau_factor_6.avg_util_l3m_cc >0.898315044)),'3. GE 0.898315044& LT 2.373400115',
'4. GE 2.373400115' ))),

w_avg_util_l3m_cc= np.where((bureau_factor_6.avg_util_l3m_cc< -0.25696), -0.277066346631537,
np.where(((bureau_factor_6.avg_util_l3m_cc <0.898315044) & (bureau_factor_6.avg_util_l3m_cc >-0.25696)),1.1801614692035,
np.where(((bureau_factor_6.avg_util_l3m_cc <2.373400115) & (bureau_factor_6.avg_util_l3m_cc >0.898315044)),0.607259844556424,
-0.537776761387732 ))),


b_avg_util_l12m_cc= np.where((bureau_factor_6.avg_util_l12m_cc< -1.06752), '1. Less than -1.06752',
np.where(((bureau_factor_6.avg_util_l12m_cc <3.279578987) & (bureau_factor_6.avg_util_l12m_cc >-1.06752)),'2. GE -1.06752& LT 3.279578987',
np.where(((bureau_factor_6.avg_util_l12m_cc <7.184201044) & (bureau_factor_6.avg_util_l12m_cc >3.279578987)),'3. GE 3.279578987& LT 7.184201044',
'4. GE 7.184201044' ))),

w_avg_util_l12m_cc= np.where((bureau_factor_6.avg_util_l12m_cc< -1.06752), -0.277066346631537,
np.where(((bureau_factor_6.avg_util_l12m_cc <3.279578987) & (bureau_factor_6.avg_util_l12m_cc >-1.06752)),1.2421586929124,
np.where(((bureau_factor_6.avg_util_l12m_cc <7.184201044) & (bureau_factor_6.avg_util_l12m_cc >3.279578987)),0.343522896276961,
-0.469314625495636 ))),


b_util_curr_cc_tot= np.where((bureau_factor_6.util_curr_cc_tot< -0.15185167), '1. Less than -0.15185167',
np.where(((bureau_factor_6.util_curr_cc_tot <0.470980799) & (bureau_factor_6.util_curr_cc_tot >-0.15185167)),'2. GE -0.15185167& LT 0.470980799',
np.where(((bureau_factor_6.util_curr_cc_tot <0.889032073) & (bureau_factor_6.util_curr_cc_tot >0.470980799)),'3. GE 0.470980799& LT 0.889032073',
'4. GE 0.889032073' ))),

w_util_curr_cc_tot= np.where((bureau_factor_6.util_curr_cc_tot< -0.15185167), -0.276599149268094,
np.where(((bureau_factor_6.util_curr_cc_tot <0.470980799) & (bureau_factor_6.util_curr_cc_tot >-0.15185167)),1.15841351370939,
np.where(((bureau_factor_6.util_curr_cc_tot <0.889032073) & (bureau_factor_6.util_curr_cc_tot >0.470980799)),0.495207483609026,
-0.503774964110243 ))),


b_nbr_prod_0_24m_cc= np.where((bureau_factor_6.nbr_prod_0_24m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_0_24m_cc <1) & (bureau_factor_6.nbr_prod_0_24m_cc >0)),'2. GE 0& LT 1',
np.where(((bureau_factor_6.nbr_prod_0_24m_cc <2) & (bureau_factor_6.nbr_prod_0_24m_cc >1)),'3. GE 1& LT 2',
'4. GE 2' ))),

w_nbr_prod_0_24m_cc= np.where((bureau_factor_6.nbr_prod_0_24m_cc< 0), -0.283666316491187,
np.where(((bureau_factor_6.nbr_prod_0_24m_cc <1) & (bureau_factor_6.nbr_prod_0_24m_cc >0)),1.43015899389835,
np.where(((bureau_factor_6.nbr_prod_0_24m_cc <2) & (bureau_factor_6.nbr_prod_0_24m_cc >1)),-0.217135101236905,
-0.429666991956521 ))),


b_nbr_prod_0_12m_cc= np.where((bureau_factor_6.nbr_prod_0_12m_cc< 0), '1. Less than 0',
np.where(((bureau_factor_6.nbr_prod_0_12m_cc <1) & (bureau_factor_6.nbr_prod_0_12m_cc >0)),'2. GE 0& LT 1',
np.where(((bureau_factor_6.nbr_prod_0_12m_cc <2) & (bureau_factor_6.nbr_prod_0_12m_cc >1)),'3. GE 1& LT 2',
'4. GE 2' ))),

w_nbr_prod_0_12m_cc= np.where((bureau_factor_6.nbr_prod_0_12m_cc< 0), -0.283666316491187,
np.where(((bureau_factor_6.nbr_prod_0_12m_cc <1) & (bureau_factor_6.nbr_prod_0_12m_cc >0)),1.43015899389835,
np.where(((bureau_factor_6.nbr_prod_0_12m_cc <2) & (bureau_factor_6.nbr_prod_0_12m_cc >1)),-0.217135101236905,
-0.429666991956521 ))),


b_util_l12m_cc_tot= np.where((bureau_factor_6.util_l12m_cc_tot< -1.06752), '1. Less than -1.06752',
np.where(((bureau_factor_6.util_l12m_cc_tot <3.488284114) & (bureau_factor_6.util_l12m_cc_tot >-1.06752)),'2. GE -1.06752& LT 3.488284114',
np.where(((bureau_factor_6.util_l12m_cc_tot <5.6698125) & (bureau_factor_6.util_l12m_cc_tot >3.488284114)),'3. GE 3.488284114& LT 5.6698125',
'4. GE 5.6698125' ))),

w_util_l12m_cc_tot= np.where((bureau_factor_6.util_l12m_cc_tot< -1.06752), -0.276599149268094,
np.where(((bureau_factor_6.util_l12m_cc_tot <3.488284114) & (bureau_factor_6.util_l12m_cc_tot >-1.06752)),1.26637102869727,
np.where(((bureau_factor_6.util_l12m_cc_tot <5.6698125) & (bureau_factor_6.util_l12m_cc_tot >3.488284114)),0.709142095837926,
-0.356224968314336 ))),


b_avg_util_l6m_cc= np.where((bureau_factor_6.avg_util_l6m_cc< -0.57624), '1. Less than -0.57624',
np.where(((bureau_factor_6.avg_util_l6m_cc <2.713228462) & (bureau_factor_6.avg_util_l6m_cc >-0.57624)),'2. GE -0.57624& LT 2.713228462',
np.where(((bureau_factor_6.avg_util_l6m_cc <4.318028543) & (bureau_factor_6.avg_util_l6m_cc >2.713228462)),'3. GE 2.713228462& LT 4.318028543',
'4. GE 4.318028543' ))),

w_avg_util_l6m_cc= np.where((bureau_factor_6.avg_util_l6m_cc< -0.57624), -0.277066346631537,
np.where(((bureau_factor_6.avg_util_l6m_cc <2.713228462) & (bureau_factor_6.avg_util_l6m_cc >-0.57624)),1.11808546832241,
np.where(((bureau_factor_6.avg_util_l6m_cc <4.318028543) & (bureau_factor_6.avg_util_l6m_cc >2.713228462)),0.236424760720594,
-0.503774964110243 ))),


b_util_curr_all_live= np.where((bureau_factor_6.util_curr_all_live< 0.001210626), '1. Less than 0.001210626',
np.where(((bureau_factor_6.util_curr_all_live <0.239912343) & (bureau_factor_6.util_curr_all_live >0.001210626)),'2. GE 0.001210626& LT 0.239912343',
np.where(((bureau_factor_6.util_curr_all_live <0.819938931) & (bureau_factor_6.util_curr_all_live >0.239912343)),'3. GE 0.239912343& LT 0.819938931',
'4. GE 0.819938931' ))),

w_util_curr_all_live= np.where((bureau_factor_6.util_curr_all_live< 0.001210626), -0.367642789785663,
np.where(((bureau_factor_6.util_curr_all_live <0.239912343) & (bureau_factor_6.util_curr_all_live >0.001210626)),0.225400129005283,
np.where(((bureau_factor_6.util_curr_all_live <0.819938931) & (bureau_factor_6.util_curr_all_live >0.239912343)),0.112387590129786,
-0.235680024933766 )))
)

'''





In [171]:
def keep_cols(DataFrame, keep_these):
    """Keep only the columns [keep_these] in a DataFrame, delete
    all other columns. 
    """
    drop_these = list(set(list(DataFrame)) - set(keep_these))
    return DataFrame.drop(drop_these, axis = 1)

In [172]:
mod = copy.deepcopy(bureau_factor_7)

In [181]:
lst = bureau_factor_7.columns
lst[lst.str.startswith('w_')]

Index(['w_max_live_mob_sec', 'w_max_live_mob_cc', 'w_max_length_hist',
       'w_max_dpd_6m_tw', 'w_max_dpd_24m_pl', 'w_max_dpd_24m_com_agri',
       'w_max_dpd_24m_cc', 'w_cust_loan_amt_live_unsec',
       'w_cust_loan_amt_live_sec', 'w_cust_loan_amt_live_con',
       'w_cust_loan_amt_active', 'w_count_write_off_36m', 'w_avg_util_l6m_cc',
       'w_avg_util_l3m_all_act', 'w_avg_util_l12m_all_live',
       'w_nbr_accts_open_l12m_pl', 'w_nbr_accts_open_l12m',
       'w_mon_since90_uns', 'w_mon_since90_sec', 'w_mon_since180_cc',
       'w_mon_since0_all', 'w_min_mob_pl', 'w_max_mob_tw', 'w_max_mob_sec',
       'w_max_mob_con', 'w_max_mob_com_oth', 'w_max_mob_com_agri',
       'w_max_mob_all', 'w_max_live_mob_uns', 'w_nbr_com_agri_tot_accts',
       'w_nbr_closed_active_accts', 'w_nbr_cl_tot_accts',
       'w_nbr_cl_live_accts', 'w_nbr_cl_closed_accts', 'w_nbr_cc_tot_accts',
       'w_nbr_cc_live_active_accts', 'w_nbr_accts_open_l6m_con',
       'w_nbr_accts_open_l3m_pl', 'w_nbr_accts_ope

In [257]:
mod1 = mod.pipe(keep_cols, ["cust_id","bad_flag", 'w_max_live_mob_sec', 'w_max_live_mob_cc', 'w_max_length_hist',
       'w_max_dpd_6m_tw', 'w_max_dpd_24m_pl', 'w_max_dpd_24m_com_agri',
       'w_max_dpd_24m_cc', 'w_cust_loan_amt_live_unsec',
       'w_cust_loan_amt_live_sec', 'w_cust_loan_amt_live_con',
       'w_cust_loan_amt_active', 'w_count_write_off_36m', 'w_avg_util_l6m_cc',
       'w_avg_util_l3m_all_act', 'w_avg_util_l12m_all_live',
       'w_nbr_accts_open_l12m_pl', 'w_nbr_accts_open_l12m',
       'w_mon_since90_uns', 'w_mon_since90_sec', 'w_mon_since180_cc',
       'w_mon_since0_all', 'w_min_mob_pl', 'w_max_mob_tw', 'w_max_mob_sec',
       'w_max_mob_con', 'w_max_mob_com_oth', 'w_max_mob_com_agri',
       'w_max_mob_all', 'w_max_live_mob_uns', 'w_nbr_com_agri_tot_accts',
       'w_nbr_closed_active_accts', 'w_nbr_cl_tot_accts',
       'w_nbr_cl_live_accts', 'w_nbr_cl_closed_accts', 'w_nbr_cc_tot_accts',
       'w_nbr_cc_live_active_accts', 'w_nbr_accts_open_l6m_con',
       'w_nbr_accts_open_l3m_pl', 'w_nbr_accts_open_l3m',
       'w_nbr_con_live_accts', 'w_nbr_pl_tot_accts', 'w_nbr_pl_live_accts',
       'w_nbr_prod_0_24m_al', 'w_nbr_prod_0_3m_sec', 'w_nbr_prod_0_6m_act',
       'w_nbr_prod_30_3m_hl', 'w_nbr_prod_30_6m_com',
       'w_nbr_prod_30_6m_com_oth', 'w_nbr_prod_90_24m_sec',
       'w_nbr_tot_active_accts', 'w_nbr_uns_tot_accts', 'w_ratio_amt_al',
       'w_ratio_amt_cc', 'w_ratio_amt_con', 'w_ratio_amt_pl',
       'w_ratio_amt_sec', 'w_ratio_amt_tw', 'w_ratio_amt_uns',
       'w_ratio_avg_util_C1to3toC1to12', 'w_util_l12m_all_act',
       'w_util_l12m_all_live', 'w_util_l12m_all_tot', 'w_util_l12m_exc_cc_tot',
       'w_util_l3m_all_live', 'w_util_l6m_all_act', 'w_util_l6m_all_tot'])


In [258]:
X=mod1.drop(['bad_flag','cust_id'],axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                          for i in range(len(X.columns))] 
  
print(vif_data)

                           feature        VIF
0               w_max_live_mob_sec   1.597246
1                w_max_live_mob_cc   3.005291
2                w_max_length_hist   1.662359
3                  w_max_dpd_6m_tw   1.722667
4                 w_max_dpd_24m_pl   2.424255
5           w_max_dpd_24m_com_agri   4.472317
6                 w_max_dpd_24m_cc   2.650093
7       w_cust_loan_amt_live_unsec   3.728465
8         w_cust_loan_amt_live_sec   4.158130
9         w_cust_loan_amt_live_con   4.297703
10          w_cust_loan_amt_active   3.880589
11           w_count_write_off_36m   1.629951
12               w_avg_util_l6m_cc   2.173946
13          w_avg_util_l3m_all_act   2.653656
14        w_avg_util_l12m_all_live   3.290279
15        w_nbr_accts_open_l12m_pl   3.061192
16           w_nbr_accts_open_l12m   2.258393
17               w_mon_since90_uns   3.779529
18               w_mon_since90_sec   2.648063
19               w_mon_since180_cc   2.537097
20                w_mon_since0_all

In [281]:
vif_data['VIF']=vif_data['VIF'].astype(float)
vif_data_cut2=vif_data[vif_data['VIF']<=4]
#vif_data_cut2.shape[0]
vif_data_cut2.reset_index(drop=True,inplace=True)
vif_data_cut2

,feature,VIF
0,w_max_live_mob_sec,1.597246
1,w_max_live_mob_cc,3.005291
2,w_max_length_hist,1.662359
3,w_max_dpd_6m_tw,1.722667
4,w_max_dpd_24m_pl,2.424255
5,w_max_dpd_24m_cc,2.650093
6,w_cust_loan_amt_live_unsec,3.728465
7,w_cust_loan_amt_active,3.880589
8,w_count_write_off_36m,1.629951
9,w_avg_util_l6m_cc,2.173946


In [265]:
from sklearn.datasets import load_boston
import statsmodels.api as sm

def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out = 0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit(disp = False)
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            #best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))



       # backward step
        #print(included)
        model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included]))).fit(disp = False)
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.index[pvalues.argmax()]
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [301]:
mod2 = pd.Series(['cust_id','bad_flag'])
s2 = vif_data_cut2['feature'].copy()
mod2 = mod2.append(s2)

mod2.reset_index(drop=True,inplace=True)
mod2

0                            cust_id
1                           bad_flag
2                 w_max_live_mob_sec
3                  w_max_live_mob_cc
4                  w_max_length_hist
5                    w_max_dpd_6m_tw
6                   w_max_dpd_24m_pl
7                   w_max_dpd_24m_cc
8         w_cust_loan_amt_live_unsec
9             w_cust_loan_amt_active
10             w_count_write_off_36m
11                 w_avg_util_l6m_cc
12            w_avg_util_l3m_all_act
13          w_avg_util_l12m_all_live
14          w_nbr_accts_open_l12m_pl
15             w_nbr_accts_open_l12m
16                 w_mon_since90_uns
17                 w_mon_since90_sec
18                 w_mon_since180_cc
19                  w_mon_since0_all
20                      w_min_mob_pl
21                      w_max_mob_tw
22                     w_max_mob_sec
23                 w_max_mob_com_oth
24                w_max_live_mob_uns
25          w_nbr_com_agri_tot_accts
26         w_nbr_closed_active_accts
2

In [302]:
mod3 = mod1.pipe(keep_cols, mod2)
mod3.append(mod1["cust_id"])
mod3

,bad_flag,cust_id,w_max_live_mob_sec,w_max_live_mob_cc,w_max_length_hist,w_max_dpd_6m_tw,w_max_dpd_24m_pl,w_max_dpd_24m_cc,w_cust_loan_amt_live_unsec,w_cust_loan_amt_active,w_count_write_off_36m,w_avg_util_l6m_cc,w_avg_util_l3m_all_act,w_avg_util_l12m_all_live,w_nbr_accts_open_l12m_pl,w_nbr_accts_open_l12m,w_mon_since90_uns,w_mon_since90_sec,w_mon_since180_cc,w_mon_since0_all,w_min_mob_pl,w_max_mob_tw,w_max_mob_sec,w_max_mob_com_oth,w_max_live_mob_uns,w_nbr_com_agri_tot_accts,w_nbr_closed_active_accts,w_nbr_cl_live_accts,w_nbr_cl_closed_accts,w_nbr_cc_tot_accts,w_nbr_cc_live_active_accts,w_nbr_accts_open_l6m_con,w_nbr_accts_open_l3m_pl,w_nbr_accts_open_l3m,w_nbr_pl_tot_accts,w_nbr_pl_live_accts,w_nbr_prod_0_24m_al,w_nbr_prod_0_3m_sec,w_nbr_prod_0_6m_act,w_nbr_prod_30_3m_hl,w_nbr_prod_30_6m_com,w_nbr_prod_30_6m_com_oth,w_nbr_prod_90_24m_sec,w_ratio_amt_al,w_ratio_amt_con,w_ratio_amt_pl,w_ratio_amt_sec,w_ratio_amt_tw,w_ratio_avg_util_C1to3toC1to12,w_util_l12m_all_live,w_util_l12m_exc_cc_tot,w_util_l3m_all_live,w_util_l6m_all_act,w_util_l6m_all_tot
0,0.0,1000009,0.206393,-0.176322,0.267770,0.056486,-0.284779,-0.15433,-0.181503,0.456248,0.238385,-0.15433,-0.073196,-0.789426,-0.069114,-0.683629,1.499921,0.213758,0.143307,1.994624,-0.284779,0.056486,0.186986,-0.026702,1.324278,-0.040817,0.249175,0.269587,0.200973,-0.096188,-0.055997,-0.399749,-0.042815,-0.205629,-0.120835,-0.092540,1.030252,0.686480,0.770599,-0.076288,-0.036179,-0.023322,0.401643,0.480812,-0.067661,-0.277639,0.791841,0.062214,-0.224152,-0.277639,-0.170884,-0.097181,-0.455807,-0.209476
1,0.0,1000027,0.206393,-0.176322,0.267770,0.056486,-0.284779,-0.15433,-0.181503,0.456248,0.238385,-0.15433,-0.750869,-0.136306,-0.069114,0.455387,1.499921,0.213758,0.143307,-0.973283,-0.284779,0.056486,0.047401,-0.120611,0.573862,-0.040817,-0.432156,-0.333569,-0.307700,-0.096188,-0.055997,0.748612,-0.042815,-0.205629,-0.120835,-0.092540,1.030252,-0.862923,0.770599,-0.076288,0.693449,0.606438,0.401643,0.403914,0.261597,-0.277639,0.069901,0.062214,0.231071,-0.039591,-0.170884,-0.097181,0.015497,-0.209476
2,0.0,1000028,0.206393,-0.176322,0.267770,0.056486,-0.284779,-0.15433,0.206393,-0.035416,0.238385,-0.15433,-0.073196,-0.136306,-0.069114,-0.683629,1.499921,0.213758,0.143307,1.424748,-0.284779,0.056486,0.047401,-0.026702,-0.634965,-0.040817,-0.432156,0.269587,-0.307700,-0.096188,-0.055997,-0.399749,-0.042815,-0.205629,-0.120835,-0.092540,-0.077549,0.686480,0.770599,-0.076288,-0.036179,-0.023322,0.401643,-0.082628,-0.067661,-0.277639,0.069901,0.062214,-0.224152,-0.039591,-0.069760,0.003076,0.015497,0.159771
3,0.0,1000033,-0.095078,-0.176322,-0.693242,0.381557,-0.284779,-0.15433,-0.181503,0.456248,0.238385,-0.15433,0.406312,0.624137,-0.069114,-0.683629,1.499921,-0.845454,0.143307,-0.973283,-0.284779,0.088232,-0.055665,-0.026702,1.324278,-0.040817,0.249175,0.269587,0.455439,-0.096188,-0.055997,-0.399749,-0.042815,-0.205629,-0.120835,-0.092540,-0.412132,-0.862923,0.770599,-0.076288,-0.036179,-0.023322,0.401643,0.480812,-0.067661,-0.277639,0.791841,-0.269031,-0.224152,-0.039591,-0.069760,-0.207337,0.015497,-0.209476
4,1.0,1000037,0.206393,-0.176322,0.267770,0.056486,-0.284779,-0.15433,-0.181503,-0.035416,0.238385,-0.15433,0.406312,0.624137,-0.069114,0.455387,-1.786902,0.213758,0.143307,-0.973283,-0.284779,0.056486,0.186986,-0.026702,0.186986,-0.040817,0.249175,0.269587,0.200973,-0.096188,-0.055997,-0.399749,-0.042815,-0.205629,-0.120835,-0.092540,-0.077549,0.686480,0.770599,-0.076288,-0.036179,-0.023322,0.401643,-0.082628,-0.067661,-0.277639,0.069901,0.062214,-0.224152,0.233762,0.057185,0.309047,0.026950,0.159771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,0.0,1010163,0.206393,-0.176322,0.267770,0.056486,-0.284779,-0.15433,0.206393,0.456248,0.238385,-0.15433,-0.073196,-0.136306,-0.069114,0.455387,1.499921,0.213758,

In [303]:
X=mod3.iloc[:,2:]
y=mod3.iloc[:,0]

login = sm.Logit(y,X).fit()
login.summary2()

Optimization terminated successfully.
         Current function value: 0.289376
         Iterations 8


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                Results: Logit
===============================================================================
Model:                    Logit                Pseudo R-squared:     0.260     
Dependent Variable:       bad_flag             AIC:                  907.3078  
Date:                     2022-11-04 12:38     BIC:                  1179.5600 
No. Observations:         1388                 Log-Likelihood:       -401.65   
Df Model:                 51                   LL-Null:              -543.03   
Df Residuals:             1336                 LLR p-value:          7.5337e-34
Converged:                1.0000               Scale:                1.0000    
No. Iterations:           8.0000                                               
-------------------------------------------------------------------------------
                                Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-------------------------------------------------------------------------------
w_max_live_mob_sec             -1.0175   0.3779  -2.6928 0.0071 -1.7581 -0.2769
w_max_live_mob_cc              -0.7333   0.2601  -2.8194 0.0048 -1.2430 -0.2235
w_max_length_hist              -1.1799   0.2624  -4.4960 0.0000 -1.6943 -0.6656
w_max_dpd_6m_tw                -0.4065   0.2713  -1.4984 0.1340 -0.9382  0.1252
w_max_dpd_24m_pl                0.8413   0.1948   4.3185 0.0000  0.4595  1.2231
w_max_dpd_24m_cc               -0.1429   0.1691  -0.8453 0.3980 -0.4744  0.1885
w_cust_loan_amt_live_unsec     -0.9104   0.5587  -1.6296 0.1032 -2.0054  0.1846
w_cust_loan_amt_active         -0.3962   0.4739  -0.8359 0.4032 -1.3250  0.5327
w_count_write_off_36m          -0.4226   0.1965  -2.1509 0.0315 -0.8077 -0.0375
w_avg_util_l6m_cc              -0.1323   0.2793  -0.4738 0.6356 -0.6798  0.4151
w_avg_util_l3m_all_act         -0.1822   0.3867  -0.4712 0.6375 -0.9402  0.5758
w_avg_util_l12m_all_live       -0.1785   0.2719  -0.6567 0.5114 -0.7114  0.3543
w_nbr_accts_open_l12m_pl        0.3571   0.8347   0.4279 0.6687 -1.2788  1.9930
w_nbr_accts_open_l12m          -0.2726   0.2465  -1.1056 0.2689 -0.7558  0.2106
w_mon_since90_uns              -1.1434   0.1055 -10.8357 0.0000 -1.3502 -0.9366
w_mon_since90_sec              -0.6820   0.3137  -2.1743 0.0297 -1.2967 -0.0672
w_mon_since180_cc               2.8925   0.3990   7.2491 0.0000  2.1104  3.6745
w_mon_since0_all               -0.3302   0.0961  -3.4347 0.0006 -0.5187 -0.1418
w_min_mob_pl                   -0.9694   0.2772  -3.4971 0.0005 -1.5127 -0.4261
w_max_mob_tw                   -2.0744   0.8973  -2.3118 0.0208 -3.8331 -0.3157
w_max_mob_sec                  -0.9397   1.1913  -0.7888 0.4302 -3.2745  1.3952
w_max_mob_com_oth              -0.8992   0.6095  -1.4753 0.1401 -2.0939  0.2955
w_max_live_mob_uns             -0.7643   0.2556  -2.9903 0.0028 -1.2653 -0.2634
w_nbr_com_agri_tot_accts       -0.9647   0.4606  -2.0945 0.0362 -1.8673 -0.0620
w_nbr_closed_active_accts      -0.1591   0.3571  -0.4454 0.6560 -0.8589  0.5408
w_nbr_cl_live_accts            -0.6217   0.3937  -1.5793 0.1143 -1.3932  0.1499
w_nbr_cl_closed_accts          -1.1640   0.4026  -2.8915 0.0038 -1.9530 -0.3750
w_nbr_cc_tot_accts             -0.3302   0.7805  -0.4231 0.6722 -1.8600  1.1995
w_nbr_cc_live_active_accts      0.3345   0.7680   0.4355 0.6632 -1.1708  1.8399
w_nbr_accts_open_l6m_con        0.3031   0.2762   1.0975 0.2724 -0.2382  0.8445
w_nbr_accts_open_l3m_pl         0.1835   0.4079   0.4498 0.6528 -0.6159  0.9829
w_nbr_accts_open_l3m            0.4160   0.2723   1.5275 0.1266 -0.1178  0.9497
w_nbr_pl_tot_accts              0.8338   0.6982   1.1943 0.2324 -0.5346  2.2023
w_nbr_pl_live_accts            -0.3153   0.5273  -0.5978 0.5500 -1.3488  0.7183
w_nbr_prod_0_24m_al            -0.3070   0.3600  -0.8527 0.3938 -1.0125  0.3986
w_nbr_prod_0_3m_sec             0.5229   0.2828   1.8488 0.0645 -0.0314  1.0771
w_nbr_prod_0_6m_act            -0.5821   0.1541  -3.7786 0.0002 

In [304]:
result = stepwise_selection(X, y, X,threshold_in=0.005, threshold_out = 0.005)

print('resulting features:')
print(result)

Drop w_ratio_amt_tw                 with p-value 0.952993
Drop w_nbr_cc_live_active_accts     with p-value 0.947282
Drop w_nbr_accts_open_l12m          with p-value 0.946692
Drop w_nbr_prod_0_24m_al            with p-value 0.928062
Drop w_max_mob_sec                  with p-value 0.910699
Drop w_avg_util_l3m_all_act         with p-value 0.852713
Drop w_util_l3m_all_live            with p-value 0.730405
Drop w_nbr_accts_open_l12m_pl       with p-value 0.728842
Drop w_nbr_accts_open_l3m_pl        with p-value 0.667489
Drop w_avg_util_l12m_all_live       with p-value 0.664341
Drop w_nbr_prod_30_6m_com           with p-value 0.594362
Drop w_ratio_avg_util_C1to3toC1to12 with p-value 0.592196
Drop w_avg_util_l6m_cc              with p-value 0.542303
Drop w_nbr_accts_open_l6m_con       with p-value 0.567317
Drop w_util_l12m_exc_cc_tot         with p-value 0.54689
Drop w_max_mob_com_oth              with p-value 0.569341
Drop w_nbr_prod_90_24m_sec          with p-value 0.519542
Drop w_util_l6m

In [173]:
'''
mod1 = mod.pipe(keep_cols, ["cust_id","bad_flag","w_max_nbr_90_24m_all",
"w_max_nbr_90_24m_act",
"w_max_nbr_90_12m_all",
"w_max_nbr_90_12m_act",
"w_max_dpd_0m_cc",
"w_max_dpd_3m_cc",
"w_nbr_prod_0_3m_cc",
"w_nbr_prod_30_3m_cc",
"w_max_dpd_12m_cc",
"w_nbr_prod_90_3m_cc",
"w_nbr_prod_90_24m_cc",
"w_nbr_prod_90_12m_cc",
"w_nbr_prod_30_24m_cc",
"w_nbr_prod_30_12m_cc",
"w_max_dpd_6m_cc",
"w_nbr_prod_90_6m_cc",
"w_nbr_prod_30_6m_cc",
"w_max_dpd_24m_cc",
"w_nbr_prod_0_6m_cc",
"w_nbr_prod_0_24m_cc",
"w_nbr_prod_0_12m_cc",
"w_count_write_off_life",
"w_count_write_off_36m",
"w_count_write_off_24m",
"w_count_write_off_12m",
"w_util_l6m_cc_tot",
"w_util_l3m_cc_tot",
"w_util_l12m_cc_tot",
"w_avg_util_l6m_cc",
"w_avg_util_l12m_cc",
"w_util_curr_cc_tot",
"w_avg_util_l3m_cc",
"w_max_dpd_24m_sec",
"w_nbr_accts_open_l9m_cc",
"w_nbr_prod_0_24m_sec",
"w_nbr_prod_0_12m_sec",
"w_nbr_prod_0_6m_sec",
"w_max_dpd_24m_tw",
"w_nbr_prod_0_3m_sec",
"w_max_dpd_0m_sec",
"w_nbr_prod_30_24m_sec",
"w_nbr_prod_30_12m_sec",
"w_pct_live_act_bal_cc",
"w_count_settlement_life",
"w_count_settlement_36m",
"w_nbr_cc_tot_accts",
"w_max_dpd_24m_com",
"w_max_dpd_0m_tw",
"w_nbr_prod_0_6m_tw",
"w_max_mob_pl",
"w_max_mob_uns",
"w_nbr_cc_live_active_accts",
"w_max_dpd_3m_sec",
"w_mon_since30_sec",
"w_mon_since30_tw",
"w_nbr_cl_closed_accts",
"w_nbr_prod_90_24m_sec",
"w_nbr_prod_90_12m_sec",
"w_mon_since0_sec",
"w_ratio_avg_util_C1to3toC1to6",
"w_min_mob_com",
"w_max_mob_com",
"w_nbr_tot_accts",
"w_max_dpd_6m_sec",
"w_nbr_live_accts",
"w_nbr_con_live_accts",
"w_nbr_live_active_accts",
"w_nbr_accts_open_l3m",
"w_max_dpd_12m_sec",
"w_max_dpd_12m_tw",
"w_ratio_avg_util_C1to3toC1to12"])
'''

In [136]:
'''
model2=mod1.pipe(keep_cols,["cust_id","w_max_nbr_90_24m_all",
# "w_nbr_prod_30_12m_cc",
"w_max_dpd_6m_cc",
"w_count_write_off_24m",
# "w_util_l6m_cc_tot",
# "w_util_l3m_cc_tot",
# "w_util_l12m_cc_tot",
"w_avg_util_l6m_cc",
# "w_avg_util_l12m_cc",
"w_util_curr_cc_tot",
# "w_avg_util_l3m_cc",
"w_nbr_accts_open_l9m_cc",
"w_nbr_prod_30_24m_sec",
"w_pct_live_act_bal_cc",
"w_count_settlement_life",
"w_nbr_cc_tot_accts",
"w_max_dpd_24m_com",
# "w_max_dpd_0m_tw",
"w_nbr_prod_0_6m_tw",
"w_max_mob_pl",
"w_max_mob_uns",
"w_nbr_cc_live_active_accts",
"w_mon_since30_sec",
"w_mon_since30_tw",
"w_nbr_cl_closed_accts",
"w_mon_since0_sec",
"w_ratio_avg_util_C1to3toC1to6",
"w_min_mob_com",
"w_nbr_tot_accts",
"w_nbr_con_live_accts",
"w_nbr_live_active_accts",
"w_nbr_accts_open_l3m",
"w_max_dpd_12m_sec",
"w_max_dpd_12m_tw",
"w_ratio_avg_util_C1to3toC1to12",
"bad_flag"])
'''

In [150]:
# model_final=model2.pipe(keep_cols,["w_mon_since0_sec",
# "w_min_mob_com",
# # "w_max_dpd_24m_com",
# # "w_nbr_prod_0_6m_tw",
# # "w_max_dpd_12m_tw",
# "w_nbr_cl_closed_accts",
# # "w_nbr_tot_accts",
# # "w_nbr_cc_live_active_accts",
# # "w_mon_since30_sec",
# # "w_mon_since30_tw",
# "w_nbr_cc_tot_accts",
# # "w_max_dpd_12m_sec",
# "w_count_settlement_life",
# "w_ratio_avg_util_C1to3toC1to12",
# # "w_nbr_accts_open_l3m",
# "w_nbr_accts_open_l9m_cc",
# "w_max_mob_uns",
# "w_max_mob_pl",
# "w_ratio_avg_util_C1to3toC1to6",
# "w_max_nbr_90_24m_all",
# # "w_max_dpd_6m_cc",
# "w_count_write_off_24m",
# "w_util_curr_cc_tot",
# "w_avg_util_l6m_cc","bad_flag"
# ])

In [140]:
'''
model_final=model2.pipe(keep_cols,[
#"w_mon_since0_sec",
# "w_min_mob_com",
# "w_max_dpd_24m_com",
# "w_nbr_prod_0_6m_tw",
# "w_max_dpd_12m_tw",
 "w_nbr_cl_closed_accts",
# "w_nbr_tot_accts",
# "w_nbr_cc_live_active_accts",
 #"w_mon_since30_sec",
# "w_mon_since30_tw",
# "w_nbr_cc_tot_accts",
# "w_max_dpd_12m_sec",
"w_count_settlement_life",
 "w_ratio_avg_util_C1to3toC1to12",
# "w_nbr_accts_open_l3m",
# "w_nbr_accts_open_l9m_cc",
 "w_max_mob_uns",
# "w_max_mob_pl",
#"w_pct_live_act_bal_cc",
#"w_ratio_avg_util_C1to3toC1to6",
"w_max_nbr_90_24m_all",
# "w_max_dpd_6m_cc",
# "w_count_write_off_24m",
"w_util_curr_cc_tot",
# "w_avg_util_l6m_cc",
"bad_flag"
])
'''

In [308]:

model_final

,w_max_live_mob_cc,w_max_length_hist,w_mon_since90_uns,w_mon_since90_sec,w_mon_since180_cc,w_max_mob_tw,w_nbr_cl_closed_accts,w_nbr_prod_0_6m_act,w_ratio_amt_pl,w_ratio_amt_sec
0,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,0.200973,0.770599,-0.277639,0.791841
1,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,-0.307700,0.770599,-0.277639,0.069901
2,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,-0.307700,0.770599,-0.277639,0.069901
3,-0.176322,-0.693242,1.499921,-0.845454,0.143307,0.088232,0.455439,0.770599,-0.277639,0.791841
4,-0.176322,0.267770,-1.786902,0.213758,0.143307,0.056486,0.200973,0.770599,-0.277639,0.069901
...,...,...,...,...,...,...,...,...,...,...
1383,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,0.200973,-1.034499,-0.277639,0.069901
1384,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,0.200973,0.770599,-0.277639,-0.174526
1385,-0.176322,0.267770,-1.786902,0.213758,0.143307,0.056486,0.455439,0.770599,-0.277639,0.069901
1386,-0.176322,0.267770,1.499921,0.213758,0.143307,-0.374391,-0.307700,0.770599,0.575666,0.069901


In [309]:
x = mod3.pipe(keep_cols,result)
x

,w_max_live_mob_cc,w_max_length_hist,w_mon_since90_uns,w_mon_since90_sec,w_mon_since180_cc,w_max_mob_tw,w_nbr_cl_closed_accts,w_nbr_prod_0_6m_act,w_ratio_amt_pl,w_ratio_amt_sec
0,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,0.200973,0.770599,-0.277639,0.791841
1,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,-0.307700,0.770599,-0.277639,0.069901
2,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,-0.307700,0.770599,-0.277639,0.069901
3,-0.176322,-0.693242,1.499921,-0.845454,0.143307,0.088232,0.455439,0.770599,-0.277639,0.791841
4,-0.176322,0.267770,-1.786902,0.213758,0.143307,0.056486,0.200973,0.770599,-0.277639,0.069901
...,...,...,...,...,...,...,...,...,...,...
1383,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,0.200973,-1.034499,-0.277639,0.069901
1384,-0.176322,0.267770,1.499921,0.213758,0.143307,0.056486,0.200973,0.770599,-0.277639,-0.174526
1385,-0.176322,0.267770,-1.786902,0.213758,0.143307,0.056486,0.455439,0.770599,-0.277639,0.069901
1386,-0.176322,0.267770,1.499921,0.213758,0.143307,-0.374391,-0.307700,0.770599,0.575666,0.069901


In [311]:
y=mod3.pipe(keep_cols,['bad_flag'])

In [312]:
pm=LogisticRegression().fit(x,y)

In [313]:
logit_model=sm.Logit(y,x)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.328865
         Iterations 7
                            Results: Logit
Model:                 Logit             Pseudo R-squared:  0.159     
Dependent Variable:    bad_flag          AIC:               932.9286  
Date:                  2022-11-04 13:15  BIC:               985.2848  
No. Observations:      1388              Log-Likelihood:    -456.46   
Df Model:              9                 LL-Null:           -543.03   
Df Residuals:          1378              LLR p-value:       1.3707e-32
Converged:             1.0000            Scale:             1.0000    
No. Iterations:        7.0000                                         
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
w_max_live_mob_cc     -0.6867   0.2043  -3.3607 0.0008 -1.0871 -0.2862
w_m

In [314]:
pred=result.predict(x)
pred1=np.where(pred>0.5,1,0)
fpr, tpr, thresholds = metrics.roc_curve(y, pred1)
auc=metrics.auc(fpr, tpr)
gini=2*auc-1
gini,auc



(0.7192510472338585, 0.8596255236169292)

In [315]:
pred=pd.DataFrame(pred,columns=['prob'])


In [316]:
gini_data=np.hstack((y,pred))

In [317]:
gini_data=pd.DataFrame(gini_data, columns=['bad_flag','prob'])


In [318]:
metrics.accuracy_score(y,pred1)

0.8962536023054755

In [319]:
def make_gini_table(data):
    data.sort_values(['prob'],ascending=False,inplace =True)
    data['group']=pd.qcut(data['prob'].rank(method='first'), 10, labels=[1,2,3,4,5,6,7,8,9,10])
    
    gini_table=data.assign(
    n_goods = np.where(data['bad_flag']==0,1,0),
    n_bads = np.where(data['bad_flag']==1,1,0),
    n_total = data['bad_flag']).groupby('group').agg({'n_goods':'sum','n_bads':'sum','n_total':'count'})
    
    gini_table.sort_values(by = ['n_bads'], ascending=False, inplace=True)
    gini_table=gini_table.reset_index()
    
    for i in range(len(gini_table)):
        if(i==0):
            gini_table.loc[i,'cum_goods'] = gini_table.loc[i,'n_goods']
            gini_table.loc[i,'cum_bads'] = gini_table.loc[i,'n_bads']
        else:
            gini_table.loc[i,'cum_goods'] = gini_table.loc[i,'n_goods'] + gini_table.loc[i-1,'cum_goods']
            gini_table.loc[i,'cum_bads'] = gini_table.loc[i,'n_bads'] + gini_table.loc[i-1,'cum_bads']
        
    gini_table = gini_table.assign(
        percent_cum_good = 100*gini_table['cum_goods']/np.sum(gini_table['n_goods']),
        percent_cum_bad = 100*gini_table['cum_bads']/np.sum(gini_table['n_bads']),
        percent_bad = 100*gini_table['n_bads']/np.sum(gini_table['n_bads'])
    )  

    return gini_table

In [320]:
final_gini_table = make_gini_table(gini_data)
final_gini_table


,group,n_goods,n_bads,n_total,cum_goods,cum_bads,percent_cum_good,percent_cum_bad,percent_bad
0,10,38,101,139,38.0,101.0,3.156146,54.891304,54.891304
1,9,87,52,139,125.0,153.0,10.382060,83.152174,28.260870
2,7,129,9,138,254.0,162.0,21.096346,88.043478,4.891304
3,8,131,8,139,385.0,170.0,31.976744,92.391304,4.347826
4,6,132,7,139,517.0,177.0,42.940199,96.195652,3.804348
5,4,132,6,138,649.0,183.0,53.903654,99.456522,3.260870
6,2,138,1,139,787.0,184.0,65.365449,100.000000,0.543478
7,1,139,0,139,926.0,184.0,76.910299,100.000000,0.000000
8,3,139,0,139,1065.0,184.0,88.455150,100.000000,0.000000
9,5,139,0,139,1204.0,184.0,100.000000,100.000000,0.000000


In [154]:
# writer1 = pd.ExcelWriter('ginidata1.xlsx')
# data_sem.to_excel(writer1, 'Sheet1')
# writer1.save()

final_gini_table.to_csv(r"gini_table.csv")

In [ ]:
#KS Stat Test
#RankOrder